In [1]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
en_data=pd.read_csv("../input/eng-data/train (1).csv")
en1_data=pd.read_csv("../input/eng-data/test (3).csv")
data=pd.concat([en_data, en1_data], ignore_index=False)

In [3]:
data

,"text,label"
0,"""Much of today's business is conducted across ..."
1,Addressing someone
2,When discussing this topic in a training cours...
3,This stark difference in opinion over somethin...
4,Smiling
...,...
3528,"""David lost his yellow pencil. He could not fi..."
3529,"""There are many law enforcement agencies in th..."
3530,NaN
3531,"The CHP, as it is usually called, is a police ..."


In [4]:
d=[]
for i in range(150):
    d.append(data.iloc[i])

In [5]:
d[0]

text,label    "Much of today's business is conducted across ...
Name: 0, dtype: object

In [6]:
text=[]

In [7]:
level=[]

****

In [8]:
eng_data=pd.read_csv("../input/eng-data/cefr_leveled_texts.csv")

In [9]:
eng_data.head()

,text,label
0,Hi!\nI've been meaning to write for ages and f...,B2
1,﻿It was not so much how hard people found the ...,B2
2,Keith recently came back from a trip to Chicag...,B2
3,"The Griffith Observatory is a planetarium, and...",B2
4,-LRB- The Hollywood Reporter -RRB- It's offici...,B2


In [10]:
eng_data.label.value_counts()

A1    288
B2    286
A2    272
C1    241
B1    205
C2    202
Name: label, dtype: int64

In [11]:
L=[]
for i in range(150):
    L.append(eng_data.loc[i,"text"])

In [12]:
ch=""
for i in range(150):
     ch=ch+L[i]+",,,"

In [13]:
ch2="""Salut\xa0!\nJe voulais écrire depuis des lustres et enfin aujourd\'hui, je fais quelque chose à ce sujet. Non pas que j\'essaie de me trouver des excuses, ça a été vraiment difficile de s\'asseoir et d\'écrire, car j\'ai tellement bougé. Depuis la dernière fois que nous nous sommes vus, j\'ai posé mes valises dans quatre villes différentes. Ce travail s\'est avéré être plus un tourbillon que ce à quoi je m\'attendais, mais tout va bien\xa0! \nJe suis allé de Londres à Prague pour y installer un nouveau bureau régional. Vous savez, j\'ai toujours voulu y aller, mais peut-être que j\'imaginais Prague au printemps quand j\'en parlais. L\'hiver a été vraiment dur, avec moins 15 degrés le matin et très sombre très tôt le soir. Mais au moins c\'était du ciel bleu et de la neige blanche et non des journées entières de ciel gris et de pluie, comme à la maison. C\'est difficile d\'être loin de chez soi à Noël, cependant, et Skype le jour de Noël n\'était pas vraiment la même chose que d\'être avec tout le monde.\nDe là, j\'ai participé à une autre mission de trois mois pour superviser la mise en place du bureau à New York. Aimé, aimé, aimé New York ! C\'est comme être dans une grande émission de télévision, car partout semble un peu familier. J\'ai fait toutes les choses touristiques auxquelles vous pouvez penser quand je ne travaillais pas et j\'ai dû dépenser la majeure partie de mon salaire pour manger au restaurant. C\'était vraiment difficile de partir pour le prochain travail, d\'autant plus que j\'ai en quelque sorte rencontré quelqu\'un (!) Plus sur Michael plus tard...\nAlors j\'ai été affecté à LA, qui ressemblait à un tout autre pays par rapport à la côte Est. Je pourrais certainement m\'habituer à ce genre de vie en plein air, à la plage, mais je n\'ai pas passé autant de temps à découvrir la Californie que j\'aurais pu parce que je revenais voir Michael tous les week-ends. Il est venu me voir quand il le pouvait, mais son travail fait qu\'il travaille souvent le week-end, donc il ne pouvait pas prendre l\'avion très souvent. Ces trois mois ont filé et puis je suis reparti, à Francfort, où je suis maintenant. Et … Michael aussi ! Il a pris un mois de congé et nous essayons de trouver comment nous pouvons être au même endroit au même moment pendant un certain temps. Nous pensons que le premier pas dans cette direction est de se marier, c\'est aussi pourquoi je voulais écrire - je ne peux pas me marier sans mon plus vieil ami là-bas\xa0! Le mariage aura lieu chez nous à Londres en septembre et j\'espère que vous pourrez venir\xa0!\nQuoi qu\'il en soit, dites-moi toutes vos nouvelles et je promets de ne pas le laisser si longtemps cette fois\xa0!\nBeaucoup d\'amour,\nKath,, ,\ufeffCe n\'est pas tant la difficulté avec laquelle les gens ont relevé le défi, mais jusqu\'où ils iraient pour l\'éviter qui a laissé les chercheurs bouche bée. La tâche? S\'asseoir sur une chaise et ne rien faire d\'autre que penser. \nCertains l\'ont trouvé tellement insupportable qu\'ils ont saisi l\'occasion sûre mais alarmante de se donner de légères décharges électriques pour tenter de briser l\'ennui. \nDeux tiers des hommes ont appuyé sur un bouton pour délivrer une secousse douloureuse pendant une période de solitude de 15 minutes. \nDans les mêmes conditions, un quart des femmes ont appuyé sur le bouton de choc. La différence, soupçonnent les scientifiques, est que les hommes ont tendance à être plus à la recherche de sensations que les femmes. \nLe rapport des psychologues des universités de Virginie et de Harvard est l\'un des rares à aborder la question de savoir pourquoi la plupart d\'entre nous trouvent si difficile de ne rien faire. \nDans plus de 11\xa0études distinctes, les chercheurs ont montré que les gens détestaient qu\'on les laisse réfléchir, quels que soient leur âge, leur éducation, leurs revenus ou le temps qu\'ils passaient à utiliser des smartphones ou des réseaux sociaux. \nTimothy Wilson, qui a dirigé les travaux, a déclaré que les résultats ne reflétaient pas nécessairement le rythme de la vie moderne ou la diffusion des appareils mobiles et des médias sociaux. Au lieu de cela, ces choses pourraient être populaires en raison de notre envie constante de faire quelque chose plutôt que rien. \nLa première série d\'expériences a commencé avec des étudiants introduits - seuls, sans téléphones, livres ou quoi que ce soit pour écrire - dans une pièce sans fioritures et invités à réfléchir. Les seules règles étaient qu\'ils devaient rester assis et ne pas s\'endormir. Ils ont été informés - précisément ou vaguement - qu\'ils auraient six à 15 minutes seuls. \nLes élèves ont été interrogés une fois le temps écoulé. En moyenne, ils n\'ont pas apprécié l\'expérience. Ils avaient du mal à se concentrer. Leurs esprits vagabondaient même sans rien pour les distraire. Même leur donner le temps de réfléchir à quoi penser n\'a pas aidé. \nAu cas où l\'environnement inconnu entraverait la capacité de réflexion, les chercheurs ont recommencé l\'expérience avec des personnes à la maison. \nIls ont obtenu à peu près les mêmes résultats, seuls les gens ont trouvé l\'expérience encore plus misérable et ont triché en se levant de leur chaise ou en vérifiant leur téléphone. \nPour voir si l\'effet n\'a été constaté que chez les étudiants, les scientifiques ont recruté plus de 100\xa0personnes, âgées de 18 à 77\xa0ans, dans une église et un marché de producteurs. Eux aussi n\'aimaient pas être abandonnés à leurs pensées. \nMais, le résultat le plus stupéfiant était encore à venir. Pour vérifier si les gens pouvaient réellement préférer quelque chose de mal à rien du tout, les étudiants ont eu la possibilité d\'administrer un léger choc électrique. \nOn leur avait demandé plus tôt d\'évaluer à quel point les chocs étaient désagréables, ainsi que d\'autres options, telles que regarder des photos de cafards ou entendre le bruit d\'un couteau frottant contre une bouteille. \nTous les étudiants sélectionnés pour le test ont déclaré qu\'ils paieraient pour éviter de légers chocs électriques après avoir reçu une démonstration. \nÀ la surprise des chercheurs, 12 hommes sur 18 se sont donné jusqu\'à quatre décharges électriques, tout comme six femmes sur 24. \n"Ce qui est frappant, c\'est que le simple fait d\'être seuls avec leurs pensées était apparemment si aversif que cela a poussé de nombreux participants à s\'auto-administrer un choc électrique qu\'ils avaient précédemment déclaré qu\'ils paieraient pour l\'éviter", écrivent les scientifiques dans Science \nJessica Andrews -Hanna de l\'Université du Colorado a déclaré que de nombreux étudiants se zapperaient probablement pour égayer une conférence fastidieuse. Mais, dit-elle, il faut en savoir plus sur la motivation des choquants dans l\'étude de Wilson. \n"Imaginez la configuration\xa0: une personne est invitée à s\'asseoir sur une chaise avec des fils attachés à sa peau et un bouton qui délivrera un choc inoffensif mais inconfortable, et on lui dit de s\'asseoir là et de se divertir avec ses pensées", dit-elle. \n"Alors qu\'ils sont assis là, attachés à cette machine, leur esprit commence à vagabonder et cela va naturellement vers ce choc - était-ce vraiment si grave\xa0? \n"Qu\'est-ce qui intéresse vraiment les expérimentateurs\xa0? C\'est peut-être un cas où la curiosité a tué le chat.”,,,Keith est récemment revenu d\'un voyage à Chicago, Illinois. Cette métropole du Midwest se trouve le long des rives du lac Michigan. Au cours de sa visite, Keith a passé beaucoup de temps à explorer la ville pour visiter des sites et des monuments importants.\nKeith adore le baseball et il s\'est assuré de visiter le Wrigley Field. Non seulement il a fait le tour de ce stade spectaculaire, mais il a également pu assister à un match des Chicago Cubs. Dans le stade, Keith et les autres fans ont applaudi les Cubs. Keith était heureux que les Cubs aient gagné avec un score de 5-4.\nChicago a de nombreux lieux historiques à visiter. Keith a trouvé le Chicago Water Tower impressionnant car c\'est l\'un des rares monuments restants à avoir survécu au grand incendie de Chicago de 1871. Keith s\'est également promené dans Jackson Park, un grand espace extérieur qui a accueilli l\'Exposition universelle de 1892. Le parc est idéal pour une promenade tranquille, et il présente encore une partie de l\'architecture originale et des répliques de monuments qui ont été présentés à l\'Exposition universelle.\nAu cours de la dernière partie de sa visite, Keith a réussi à monter les escaliers à l\'intérieur de la Willis Tower, un 110 gratte-ciel à étages. Malgré le défi de monter les nombreux escaliers, Keith a estimé qu\'atteindre le sommet en valait la peine. Depuis le toit, Keith a eu une vue magnifique sur les toits de la ville avec le lac Michigan en arrière-plan.,,,L\'observatoire Griffith est un planétarium et une salle d\'exposition située dans le parc Griffith de Los Angeles. Il présente plusieurs expositions astronomiques et télescopes dans les collines d Hollywood dans le quartier de Los Feliz de la ville. L Observatoire a été construit sur un terrain donné par Griffith J. Griffith en 1896. Griffith était un philanthrope et magnat des mines d\'origine galloise, qui voulait donner au public l\'accès à un observatoire ultramoderne, donc c\'était gratuit. aux visiteurs. Il pensait que les gens ordinaires devraient pouvoir profiter des merveilles de l\'univers, et pas uniquement pour la communauté scientifique.\nGriffith a fait don de 3\xa0015\xa0acres de terrain spécifiquement pour abriter l\'observatoire qui a ouvert ses portes le 14\xa0mai\xa01935. Au cours des cinq\xa0premiers jours qui ont suivi l\'ouverture, le planétarium comptait en moyenne plus de 13 000 visiteurs par jour. L\'une des principales attractions était le pendule de Foucault conçu pour montrer la rotation de la terre et le télescope à réflexion Zeiss. L\'observatoire offre une vue saisissante sur Los Angeles et Hollywood, et l\'océan Pacifique peut être vu par temps clair à partir de là.\nEn 2002, le planétarium a fait l\'objet d\'une rénovation et d\'un agrandissement de 93\xa0millions de dollars, qui ont fermé le musée pendant quatre ans. Après sa réouverture en 2006, l\'installation s\'est agrandie sous terre avec de nouvelles expositions, un café, une boutique de cadeaux et un théâtre. La section Wilder Hall of the Eye se concentre sur l\'aspect astronomique de l\'observatoire. C\'est là que les gens peuvent voir les étoiles à travers le télescope de l\'observatoire et propose une exposition interactive très populaire. L\'Ahmanson Hall of the Sky est une autre attraction populaire pour la visualisation d\'images dans le ciel nocturne visibles à l\'œil nu. L\'observatoire organise fréquemment des soirées d\'observation où le public est invité à apporter ses propres télescopes. Il est ouvert tous les jours et reste gratuit.,,,-LRB- The Hollywood Reporter -RRB- C\'est officiel\xa0: la série compagnon The Walking Dead d\'AMC a un titre.\nLe producteur exécutif Robert Kirkman a annoncé vendredi que la série compagnon, qui commence comme une préquelle de l\'original, sera intitulée Fear The Walking Dead. \nLa nouvelle survient alors que le compagnon traversait la saison de développement – \u200b\u200bet a été repris dans la série et renouvelé de manière préventive pour une deuxième saison – sous des noms de code tels que Cobalt et Fear The Walking Dead. \nLe 9 mars, AMC a annoncé le ramassage et le renouvellement de la série pour l\'ancien tled.\nKirkman, qui a créé la série de bandes dessinées The Walking Dead, a co-écrit le pilote avec Dave Erickson de Sons of Anarchy. Kirkman et Erickson seront les producteurs exécutifs aux côtés de Gale Anne Hurd et Dave Alpert de Walking Dead. Erickson servira de showrunner et Adam Davidson dirigera le pilote. Le gourou des effets visuels de Walking Dead, Greg Nicotero, sera également le producteur exécutif de la série.\nAlors qu\'AMC a été discret sur les prémisses de la série, des sources ont déclaré au Hollywood Reporter que le drame est une préquelle qui se déroule à Los Angeles au début de l\'épidémie de zombies. . AMC a confirmé que la série compagnon se déroule à Los Angeles, mais n\'a rien révélé de plus au-delà du fait qu\'elle se concentrera sur de nouveaux personnages et scénarios.\nCliff Curtis -LRB- Gang Related -RRB- met en vedette Sean Cabrera, un enseignant qui partage un fils avec son ex-femme. Kim Dickens de Sons of Anarchy devrait jouer le rôle de Nancy, une conseillère d\'orientation qui travaille à l\'école avec Sean et le voit de manière romantique. Frank Dillane -LRB- Harry Potter et le Prince de sang-mêlé -RRB- joue le rôle du fils de Nancy, Nick, qui a lutté contre un problème de drogue. Et Alycia Debnam Carey -LRB- Into the Woods -RRB- jouera la fille ambitieuse de Nancy, Ashley, qui est à l\'opposé de Nick et rêve de quitter Los Angeles pour Berkeley lorsque l\'apocalypse frappera.\n Fear The Walking Dead sera présenté à la fin de l\'été avec la saison deux prévue pour 2016. Une date de première officielle n\'a pas encore été annoncée. La finale de la saison 5 de la série phare est diffusée dimanche à 21h\n© 2015 The Hollywood Reporter. Tous droits réservés.\n,,,Il peut y avoir différents types de divertissement lors d\'une réception de mariage. Habituellement, il y aura un maître de cérémonie, ou MC, qui aide à faire avancer la réception en annonçant ce qui va se passer ensuite. Par exemple, le MC annoncera quand le couple nouvellement marié entrera dans la salle de réception, et tous les invités se lèveront et applaudiront. Une réception de mariage a généralement de la musique et les invités doivent danser. La musique peut être assurée par un DJ ou par un groupe live. Parfois, le DJ ou le groupe live acceptera les demandes des invités pour jouer une certaine chanson. Lorsque le couple nouvellement marié entrera dans la réception, ils partageront une danse seuls sur une chanson qu\'ils auront choisie au préalable. Habituellement, la chanson est quelque chose de romantique et représente l\'amour du couple l\'un pour l\'autre. Parfois, la mariée partagera une danse avec son père et le marié partagera une danse avec sa mère. Après ces danses spéciales, les invités sont invités sur la piste de danse.\n \nLa nourriture lors d\'une réception de mariage peut être sous forme de buffet, où chacun se sert soi-même, ou il peut s\'agir d\'un repas assis servi par des serveurs. Les clients ont le choix entre une offre de viande, de poulet ou de poisson. Les invités font généralement leur choix lorsqu\'ils répondent aux premières invitations de mariage.\nLors d\'un mariage, les invités ne doivent pas s\'attendre à passer trop de temps avec les mariés. Le nouveau couple est généralement très occupé à accueillir tous ses invités. Parfois, le couple n\'a même pas la chance de manger. Ce que presque tous les couples nouvellement mariés mangent à leur réception de mariage, c\'est le gâteau de mariage. La mariée et le marié coupent traditionnellement le gâteau de mariage ensemble, puis nourrissent l\'autre personne.,,,\ufeffFacebook a perdu des millions d\'utilisateurs par mois sur ses plus grands marchés, selon des données indépendantes, alors que les réseaux sociaux alternatifs attirent l\'attention de ceux qui recherchent de nouveaux terrains de jeux en ligne. \nAlors que Facebook se prépare à informer les investisseurs de ses performances au cours des trois premiers mois de l\'année, les analystes prévoyant des revenus en hausse de 36\xa0% par rapport à l\'année dernière, des études suggèrent que son expansion aux États-Unis, au Royaume-Uni et dans d\'autres grands pays européens a atteint son apogée. Au cours du mois dernier, le plus grand réseau social du monde a perdu 6 millions de visiteurs américains, soit une baisse de 4 %, selon le cabinet d\'analyse Socialbakers. Au Royaume-Uni, 1,4 million d\'utilisateurs en moins se sont enregistrés en mars, soit une baisse de 4,5 %. Les baisses sont soutenues. Au cours des six derniers mois, Facebook a perdu près de 9 millions de visiteurs mensuels aux États-Unis et 2 millions au Royaume-Uni. \nLes utilisateurs s\'éteignent également au Canada, en Espagne, en France, en Allemagne et au Japon, où Facebook compte certains de ses plus grands abonnés. Une porte-parole de Facebook a refusé de commenter. \n"Le problème est qu\'aux États-Unis et au Royaume-Uni, la plupart des personnes qui souhaitent s\'inscrire sur Facebook l\'ont déjà fait", a déclaré le spécialiste des nouveaux médias Ian Maude chez Enders Analysis. "Il y a un facteur d\'ennui où les gens aiment essayer quelque chose de nouveau. Facebook va-t-il suivre le chemin de MySpace ? Le risque est relativement faible, mais cela ne veut pas dire qu\'il n\'existe pas. Les réseaux sociaux alternatifs tels qu\'Instagram, le site de partage de photos qui a gagné 30 millions d\'utilisateurs en 18 mois avant l\'acquisition de l\'entreprise par Facebook, ont connu une popularité croissante auprès des groupes d\'âge plus jeunes. \nPath, le réseau social basé sur la téléphonie mobile fondé par l\'ancien employé de Facebook Dave Morin, qui limite ses utilisateurs à 150 amis, gagne 1 million d\'utilisateurs par semaine. Il a récemment dépassé les 9 millions d\'utilisateurs, avec 500 000 Vénézuéliens téléchargeant l\'application en un seul e week-end. \nFacebook continue de croître rapidement en Amérique du Sud. Les visiteurs mensuels au Brésil ont augmenté de 6 % le mois dernier pour atteindre 70 millions, selon Socialbakers, dont les informations sont utilisées par les annonceurs Facebook. L\'Inde a connu une augmentation de 4\xa0% pour atteindre 64\xa0millions d\'habitants, ce qui représente toujours une fraction de la population du pays, ce qui laisse de la place pour une croissance supplémentaire. \nMais sur les marchés développés, d\'autres trackers Facebook signalent des baisses. Les analystes de la banque Jefferies ont développé un algorithme qui s\'interface directement avec le logiciel Facebook et il "suggère que les niveaux d\'utilisateurs au [premier trimestre] pourraient avoir diminué par rapport au pic". \nJefferies a vu ses chiffres mondiaux culminer à 1,05\xa0milliard par mois en janvier, avant de chuter de 20\xa0millions en février. Les chiffres ont encore augmenté en avril. Le réseau a maintenant perdu près de 2 millions de visiteurs au Royaume-Uni depuis décembre, selon le cabinet d\'études Nielsen, avec son total de 27 millions de visiteurs un an auparavant. \nLe nombre de minutes que les Américains passent sur Facebook semble également diminuer. Le total était de 121 milliards de minutes en décembre 2012, mais il est tombé à 115 milliards de minutes en février, selon comScore. \nComme Facebook lui-même l\'a averti, le temps passé sur ses pages par ceux qui sont assis devant des ordinateurs personnels diminue rapidement car nous passons notre temps d\'écran aux smartphones et aux tablettes. \nBien que les minutes d\'utilisation des smartphones aient doublé en un an, pour atteindre 69 par mois, il n\'est pas garanti que cette croissance compense la diminution de l\'utilisation des ordinateurs de bureau. \nFacebook est la source la plus fiable sur ses propres numéros d\'utilisateurs, et la société informera les investisseurs de ses performances pour le trimestre. Wall Street prévoit des revenus d\'environ 1,44\xa0milliard de dollars, contre 1,06\xa0milliard de dollars en 2012. \nLes actionnaires seront particulièrement désireux de savoir à quelle vitesse la base d\'utilisateurs mobiles de Facebook augmente et si les revenus publicitaires augmentent au même rythme. \nL\'utilisation mobile représentait près d\'un quart des revenus publicitaires de Facebook à la fin de 2012, et le réseau comptait 680 millions d\'utilisateurs mobiles par mois en décembre. \nL\'entreprise a averti dans de récentes déclarations boursières qu\'elle pourrait perdre des "utilisateurs plus jeunes" au profit d\'"autres produits et services similaires ou en remplacement de Facebook". \nMéfiant face à la concurrence des services qui ont été inventés pour le téléphone mobile plutôt que pour le PC, le fondateur Mark Zuckerberg a récemment lancé une série de nouvelles initiatives conçues pour plaire aux utilisateurs de smartphones. Le plus important est Facebook Home, un logiciel qui peut être téléchargé sur certains téléphones Android pour envoyer des nouvelles et des photos d\'amis - et de la publicité - directement sur l\'écran d\'accueil verrouillé du propriétaire.\n,,,La Pâque est une fête juive qui se souvient et célèbre leur libération de l\'ancien esclavage égyptien tel que décrit dans l\'Ancien Testament de la Bible. Elle est traditionnellement célébrée au printemps et dure environ une semaine.\n \nSelon l\'histoire de la Bible, Dieu a puni l\'Égypte ancienne avec 10 plaies. La dernière peste a tué des mâles premiers-nés égyptiens. On a dit aux Juifs de marquer leurs portes avec du sang d\'agneau afin que Dieu sache passer au-dessus de ces maisons et ne pas tuer de mâles premiers-nés à l\'intérieur. C\'est de là que vient le nom de la fête, la Pâque. Après ce dernier fléau, le roi égyptien, connu sous le nom de Pharaon, a accepté de libérer les anciens Juifs de l\'esclavage. L\'histoire biblique dit aussi que le peuple juif a quitté ses maisons dans une telle précipitation après avoir été libéré qu\'il n\'a pas eu le temps d\'attendre que le pain lève. C\'est pourquoi lors du repas commémoratif de la Pâque, les Juifs mangent de la matzo, du pain sans levain, également appelé pain sans levain. La matzo et d\'autres aliments sont consommés lors d\'un repas spécial appelé seder.\nDes rituels très spécifiques ont lieu lors du repas du Seder. Ces rituels sont tous dans un livre appelé la Haggadah. À l\'intérieur de la Haggadah se trouve également l\'histoire de l\'exode juif d\'Égypte. Pendant le repas du seder, des parties de la Haggadah sont lues à haute voix. La première partie du repas est une bénédiction sur le vin. La deuxième partie du repas est le lavage des mains. La troisième partie du repas est lorsque quelqu\'un trempe une sorte de légume dans de l\'eau salée et mange ensuite le légume. Le légume symbolise les humbles origines du peuple juif. L\'eau symbolise leurs larmes. La quatrième partie du repas est lorsque trois morceaux de matzo sont cassés.,,,Stephanie a récemment fait un week-end à Los Angeles, en Californie. Los Angeles est une ville côtière située le long de l\'océan Pacifique. De nombreuses célébrités ont gagné leur titre de gloire ici. Bien que la ville offre de nombreuses attractions centrées sur la culture hollywoodienne, il y a beaucoup à voir et à visiter à Los Angeles.\nBien sûr, toutes les choses liées à Hollywood sont des attractions touristiques populaires. Le panneau Hollywood, situé sur les collines d\'Hollywood des montagnes de Santa Monica, est un repère célèbre pour cette ville étoilée. Une fois dans le centre-ville d\'Hollywood, il est possible de se promener le long du Hollywood Walk of Fame. Ce point de repère contient plus de 2500 étoiles en laiton dédiées aux célébrités qui ont eu un impact significatif sur l\'industrie du divertissement. À Hollywood, les visiteurs peuvent même faire des visites de postudios de cinéma. Stéphanie a eu la chance de visiter Universal Studios, qui produit ses films préférés.\nLos Angeles est également une ville balnéaire populaire, offrant de nombreuses possibilités pour bronzer et surfer. Une destination de plage commune est la jetée de Santa Monica, qui propose des manèges et des attractions à ses visiteurs. Ici, Stéphanie a monté la grande roue emblématique, qui lui a offert une vue spectaculaire sur la ville et la côte.\nCe ne sont là que quelques-unes des choses que Stéphanie a vécues lors de sa visite à Los Angeles. Elle a adoré son voyage et espère revenir un jour bientôt. Royaume - avec l\'Angleterre, le Pays de Galles et l\'Irlande du Nord - à la suite d\'un vote référendaire historique. Une majorité d\'électeurs a rejeté la possibilité que l\'Écosse se sépare et devienne une nation indépendante.\nAvec 31\xa0des 32\xa0conseils déclarés, la campagne du Non a remporté le référendum sur l\'indépendance de l\'Écosse.\nLe Premier ministre écossais Alex Salmond a reconnu sa défaite dans l\'indépendance de l\'Écosse. référendum vendredi - et a exhorté le reste de l\'Écosse à faire de même.\nDans une déclaration télévisée, il a remercié l\'Écosse pour 1,6 million de votes pour l\'indépendance de l\'Écosse. Un taux de participation de 86\xa0% est l\'un des plus élevés du monde démocratique pour un tel vote, a-t-il déclaré.\n-LSB- Récit original publié à 12h37. ET -RSB-\nLes électeurs écossais ont fait leur choix jeudi\xa0: rester au Royaume-Uni ou former leur propre nation indépendante. Ils ont choisi de rester.\nAvec 31\xa0des 32\xa0conseils faisant rapport, la campagne du Non avait recueilli suffisamment de voix pour garantir leur victoire, et avec elle la place de l\'Écosse au sein du syndicat.\nLes premiers conseils à déclarer tous sont allés au Non campagne, tout comme la capitale, Édimbourg, qui a voté massivement pour rester dans l\'union avec 123 927 pour le Oui et 194 628 contre. Argyll and Bute et Aberdeenshire ont également voté Non. \nGlasgow a remporté une solide victoire pour le camp de l\'indépendance avec 194\xa0779\xa0votes pour le Oui et 169\xa0347 pour le Non, mais l\'avance n\'était pas assez importante pour renverser la majorité globale du camp du Non jusqu\'à présent, ce qui maintient une avance de 8\xa0% -LRB- 54\xa0% à 46\xa0% -RRB- sur le camp indépendantiste.\nLa majorité des premiers résultats vont dans le sens du camp anti-indépendantiste. La campagne Non a prolongé l\'avance avec les résultats des conseils d\'Angus et d\'East Dunbartonshire, parallèlement à un glissement de terrain à Dumfries et Galloway. La ville d\'Aberdeen, riche en pétrole, a également fortement voté pour le camp unioniste.\nLa ville de Dundee a été la première grande avancée de la campagne pour l\'indépendance, votant Oui avec 53\xa0620\xa0votes pour, 39\xa0880 contre.\nLes décomptes individuels par les autorités locales sont finalement sans importance. car c\'est le nombre total de votes dans tout le pays qui déterminera le résultat.\nLes sondages dans plus de 5 500 bureaux de vote dans tous les districts, des hauts plateaux et des îles éloignés aux grandes villes d\'Édimbourg et de Glasgow, ont fermé à 22 heures. heure locale -LRB- 17h00 ET -RRB-. Les personnes encore en ligne pour voter à ce moment-là ont été autorisées à voter.\nParticipation élevée\nLes premiers districts à signaler une participation élevée - bien plus de 80\xa0% dans la plupart des cas - a annoncé Mary Pitcaithly, responsable du dépouillement, à Édimbourg. \nLa participation à Glasgow, l\'un des cœurs de la campagne du Oui, a été relativement décevante de 75\xa0%.\nPlus de 4,2\xa0millions de personnes se sont inscrites pour voter sur la question\xa0: l\'Écosse doit-elle être un pays indépendant\xa0? \nUn vote pour l\'indépendance signifierait que l\'Écosse, avec sa population d\'environ 5,3\xa0millions d\'habitants, se séparerait du reste du Royaume-Uni, composé de l\'Angleterre, du Pays de Galles et de l\'Irlande du Nord, en plus de l\'Écosse.\n Le peuple écossais s\'est engagé dans cette conversation, a déclaré Phil MacHugh, porte-parole du camp du Oui, à CNN jeudi soir. Ils sont vraiment allés là-bas, ont trouvé les informations qu\'ils voulaient savoir et ont fait ce choix aujourd\'hui. \nDiscours du Premier ministre\nLe Premier ministre britannique David Cameron prononcera un discours vendredi matin. Le contenu dépendra du résultat du vote, mais au cas où les électeurs écossais décideraient de rester dans l\'union, Downing Street dit qu\'il se référera aux plans de redistribution des pouvoirs du gouvernement central au Parlement écossais, signés par Westminster. dirigeants plus tôt cette semaine.\nL\'officier de comptage d\'Édimbourg, Sue Bruce, a expliqué juste après 22 heures. que chaque bulletin de vote sera revérifié, pour vérifier la décision de chaque électeur.\nS\'adressant à des dizaines de travailleurs électoraux, Bruce a ajouté, j\'ai hâte de travailler avec vous au cours de la soirée pour fournir un décompte d\'Édimbourg précis et transparent dans à laquelle vous et les électeurs d\'Édimbourg pouvez avoir toute confiance. \nLe vote a été vif dans un bureau de vote de Glasgow à l\'ouverture des bureaux de vote jeudi, de nombreuses personnes votant sur le chemin du travail ou avant d\'emmener leurs enfants à l\'école.\nLe mauvais temps ou le volume considérable de votes exprimés pourraient ralentir le processus de dépouillement. Cependant, les prévisions météorologiques semblent bonnes jusqu\'à présent -- important lorsque certaines urnes must être récupéré par hélicoptère, avion ou bateau dans des bureaux de vote situés sur des îles éloignées.\nDes allégations de fraude électorale ont été signalées par le conseil municipal de Glasgow, avec plusieurs cas de personnes se présentant aux isoloirs de la plus grande ville d\'Écosse pour voter, seulement pour se faire dire qu\'ils avaient déjà voté.\nLe Conseil a souligné qu\'il n\'y a jamais eu de situation de double vote et que les bulletins de vote concernés sont maintenant recherchés et emportés.\nLe Premier ministre écossais Alex Salmond, qui a dirigé le pro -l\'indépendance de la campagne Yes Scotland, a voté jeudi matin dans le village de Strichen, Aberdeenshire. tandis que l\'ancien Premier ministre Gordon Brown, un autre militant pro-syndical, a voté dans la ville de Kirkcaldy.\nPour la première fois, le vote a été étendu aux jeunes de 16 et 17\xa0ans vivant en Écosse. Près de 110\xa0000\xa0personnes de moins de 18\xa0ans se sont inscrites pour voter.\nIl n\'était pas nécessaire que les électeurs du référendum soient citoyens britanniques\xa0; Les citoyens du Commonwealth, irlandais et de l\'UE qui vivent en Écosse et sont inscrits pour y voter peuvent voter. Cependant, les Écossais vivant en dehors de l\'Écosse n\'ont pas leur mot à dire.\nPrès de 790\xa0000\xa0personnes ont demandé un vote par correspondance, soit le plus grand volume d\'inscriptions pour les votes par correspondance jamais enregistré en Écosse.\nAprès la fermeture des bureaux de vote, Salmond a tweeté\xa0: "Ce fut une journée remarquable". . L\'avenir de l\'Écosse est vraiment entre les mains de l\'Écosse. Nic Robertson de \nCNN a rapporté de Glasgow, Laura Smith-Spark a écrit et rapporté à Londres et Euan McKirdy a écrit et rapporté à Hong Kong. Richard Allen Greene, Lindsay Isaac et Greg Botelho de CNN ont contribué à ce rapport.\n,,,Les pays font rarement défaut, du moins pas selon la compréhension de base du terme.\nSi vous ou moi ne pouvons pas payer notre les factures et les créanciers nous appellent, nous pouvons éventuellement déclarer faillite et passer péniblement à autre chose. La plupart de nos créanciers obtiennent peu ou rien en retour. Et les entreprises le font tout le temps\xa0– en effet, cela semble aller de soi pour les compagnies aériennes américaines.\nMais les pays ne font pas vraiment faillite. Ils choisissent d\'arrêter de payer, d\'une manière ou d\'une autre.\nLa différence\xa0? Les pays reçoivent chaque jour de nouveaux contribuables. Ils naissent ou ils migrent. Les pays ne peuvent pas devenir insolvables. Ils arrêtent de payer pour économiser de l\'argent.\nEN SAVOIR PLUS\xa0: Neuf choses que nous avons manquées pendant l\'arrêt\nComment choisissent-ils la valeur par défaut\xa0? Les politiciens peuvent flirter avec le suicide financier, comme certains membres du Congrès américain aiment le faire. Mais soyons clairs, les États-Unis n\'allaient pas faire défaut. La confrontation n\'était que de la politique.\n Les États-Unis ont une énorme crédibilité et ne courent aucun risque de défaillance, m\'a dit Anne Pettifor, de Prime Economics. À mon avis, il n\'y a aucune possibilité que les États-Unis fassent défaut sur leur dette. \nAinsi, les États-Unis ne peuvent pas vraiment faire défaut, en partie grâce à une Réserve fédérale forte et à des créanciers internationaux qui veulent toujours acheter ses obligations, quelle que soit la baisse des taux d\'intérêt.\nMais d\'autres pays ont choisi de faire défaut, du moins en théorie . De l\'Argentine à la Russie en passant par la Corée du Nord, les pays ont cessé de payer les intérêts ou le principal sur les obligations nationales ou internationales.\nDans le cas de la Russie, elle a pu rembourser ses créanciers en quelques années, grâce à la hausse des prix de l\'énergie. La Russie n\'a pas abandonné ses créanciers.\nEN SAVOIR PLUS\xa0: Cinq choses les plus surprenantes dans l\'accord sur la dette\nL\'Argentine a été un cas beaucoup plus difficile. Lorsqu\'il a cessé de payer ses créanciers à la fin de 2001, il a causé beaucoup de douleur à ses citoyens en 2002-2003. Ses efforts pour régler avec ses créanciers bouillonnent toujours devant les tribunaux de New York. Personne ne peut dire que l\'Argentine a entièrement retrouvé sa réputation sur les marchés internationaux. Certains détenteurs d\'obligations refusent simplement de prendre une partie de ce qui leur est dû et de passer à autre chose. Ils veulent récupérer leur argent.\nNéanmoins, son économie a connu une croissance rapide en quelques années de défaut. Est-ce à dire que faire défaut n\'est pas une étape terrible ? Après tout, cela signifie qu\'un pays peut gérer ses obligations.\n Les dettes deviennent plus gérables et donc le pays devient plus solvable, a déclaré Pettifor. Ensuite, cependant, les créanciers s\'accumulent. \nLIRE LA SUITE\xa0: Les conservateurs défient face aux problèmes du GOP\nL\'Allemagne nazie a fait défaut en 1933. Le nouveau gouvernement n\'allait pas honorer les obligations des dirigeants précédents. Nous savons tous comment cela s\'est passé. Il a fallu des décennies avant qu\'une partie de la dette allemande ne soit remboursée de manière symbolique.\nAucun pays de la zone euro n\'a refusé unilatéralement de payer ses factures. Le marché a rendu impossible pour des pays comme l\'Irlande et la Grèce d\'émettre de nouvelles obligations à un taux d\'intérêt durable. De nouveaux créanciers sont intervenus, au lieu de s\'en aller. Nous les connaissons sous le nom de troïka - composée de la Banque centrale européenne, de l\'Union européenne et du Fonds monétaire international. L\'effondrement de la Grèce a conduit les détenteurs d\'obligations à se couper les cheveux, mais le pays est sur le point de prendre un autre paquet d\'aide. Mais il ne peut pas gonfler sa devise -LRB- il a perdu cette option lorsqu\'il a rejoint l\'euro -RRB- et peut n \'t par défaut -LRB- pour la même raison -RRB-.\nEN SAVOIR PLUS\xa0: Les États-Unis doivent se débarrasser de cette dépendance à la dette - ou tuer la reprise\nLe dernier plan n\'est pas le renflouement, mais le bail-in. Regardez Chypre. Les détenteurs d\'obligations attendent, les déposants paient -LRB- ceux qui sont forcés de renflouer -RRB- et tout le monde retient son souffle en espérant que le pays puisse reconstruire son économie et son secteur bancaire et éventuellement rembourser quelque chose.\nAprès tout, la troïka s\'attend à être remboursé. Sinon, ce serait un défaut. Tout cela vous fait vous demander pourquoi les États-Unis voudraient même être mentionnés dans le même souffle ou paragraphe avec ces pays.\n,,,A\nEn 2010, l\'équipe de défense planétaire de la NASA avait identifié et enregistré 90\xa0% des astéroïdes près de la Terre mesurant 1 km de large. Ces «\xa0objets géocroiseurs\xa0», ou objets géocroiseurs, ont la taille de montagnes et comprennent tout ce qui se trouve à moins de 50\xa0millions de kilomètres de l\'orbite terrestre. Avec environ 50 restants à enregistrer, la NASA affirme qu\'aucun des 887 dont elle a connaissance ne représente un danger significatif pour la planète.\nB\nMaintenant, la NASA travaille à l\'enregistrement de certains des plus petits astéroïdes, ceux mesurant 140\xa0mètres de large ou plus. Sur les 25 000 astéroïdes estimés de cette taille, jusqu\'à présent, environ 8 000 ont été enregistrés, laissant 17 000 disparus. Considérant qu\'un astéroïde de 19\xa0mètres qui a explosé au-dessus de la ville de Tcheliabinsk en Russie en 2013 a blessé 1\xa0200\xa0personnes, ces astéroïdes de taille moyenne constitueraient un grave danger s\'ils entraient dans l\'orbite terrestre.\nC\nSi la NASA peut trouver le reste les NEO de taille moyenne dépendent de l\'obtention de l\'argent pour construire NEOCam, un télescope spatial de 0,5 mètre qui utiliserait la lumière infrarouge pour localiser les astéroïdes. S\'il obtenait l\'argent, il pourrait probablement atteindre son objectif en dix ans. Une fois connecté, l\'équipe de défense planétaire aurait encore besoin de trouver comment défendre la planète contre les astéroïdes vraiment inquiétants\xa0: les PHA.\nD\n\'Les astéroïdes potentiellement dangereux\' sont des roches suffisamment proches pour passer à moins de 7,5\xa0millions kilomètres de l\'orbite terrestre. La NASA a créé une carte de 1 400 PHA, dont aucune ne devrait constituer une menace au cours des cent prochaines années. Grâce à la technologie déjà disponible, la NASA peut suivre ces objets et faire des prédictions sur un éventuel impact, auquel cas deux solutions de défense pourraient être lancées.\nE\nLa première est DART - le test de redirection des doubles astéroïdes. Des plans sont prévus pour tester DART sur la lune d\'un astéroïde appelé Didymos. \'Didymoon\' mesure 150\xa0mètres de large, en orbite autour de sa mère à 800\xa0mètres, et nous espérons que l\'impact de DART le fera sortir suffisamment de son orbite pour que les télescopes terrestres le détectent.\nF\nUne autre suggestion de défense contre un PHA sur sûr pour frapper la Terre est de le faire exploser à l\'aide d\'une arme nucléaire. Cela peut ressembler à une intrigue tirée d\'un film, et c\'était le sujet du film Armageddon de 1998, mais la mission d\'atténuation des astéroïdes à hypervitesse pour l\'intervention d\'urgence (HAMMER) est une véritable proposition de la NASA. Les roquettes de huit tonnes seraient tirées sur un astéroïde en approche dans l\'espoir de le faire dévier de sa trajectoire. Si l\'astéroïde était trop proche de la Terre pour que ce plan fonctionne, les fusées transporteraient des bombes nucléaires pour le faire exploser à la place. chaleur record ce week-end.\nQuatre des décès ont impliqué deux accidents de la route dans le Kentucky. Trois personnes se sont noyées dans la rivière Rolling Fork près de New Hope lorsqu\'une voiture est tombée dans l\'eau. Deux personnes se sont échappées mais ont été hospitalisées pour hypothermie.\n L\'eau était hors des berges, considérablement jusqu\'à la zone de la chaussée. Ils ont fait tomber leur véhicule dans l\'eau. Deux des personnes sortaient du véhicule lorsque l\'eau rapide a commencé à pousser le véhicule en aval. Les trois autres occupants du véhicule n\'ont pas pu sortir, a déclaré Joe Prewitt, de la gestion des urgences du comté de Nelson.\nDans l\'autre accident, un conducteur de VTT s\'est renversé dans un ruisseau près de Carrollton et a été coincé en dessous.\nLa gestion des urgences du Kentucky a rapporté un cinquième décès à cause des eaux de crue, dans le comté de Ballard, mais n\'a pas fourni de détails.\nLes autorités n\'ont pas dévoilé l\'identité des victimes.\nPlusieurs avertissements d\'inondation sont restés en vigueur dimanche le long de certaines rivières et ruisseaux du Kentucky.\nDeux décès sont survenus dans le Comtés de Coahoma et Jasper, dans le Mississippi, où des intempéries ont endommagé des maisons et abattu des arbres, selon l\'agence de gestion des urgences du Mississippi.\nLe huitième décès est survenu dans un accident de la circulation près de Kansas City, a déclaré Chris Redline du ministère des Transports du Missouri.\n Nous avons également dû fermer de nombreuses routes dans le sud-est du Missouri à cause des inondations, a déclaré Redline.\nDes inondations, un froid extrême et une chaleur inhabituelle ont couvert d\'autres parties de t e pays. Pour donner un sens à tout cela, laissez-nous vous emmener dans une tournée nord-américaine.\nSud-est\xa0: orages, tornades et averses torrentielles\nDe fortes pluies, des vents destructeurs et des éclairs ont continué. Ces tempêtes devaient se propager, entraînant des averses en Géorgie, en Caroline du Sud et sur la côte Est jusqu\'à lundi.\nBien que plus frais, le temps devrait être beaucoup plus calme et plus sec la veille de Noël et le jour de Noël.\nSamedi soir, le service météorologique national a déclaré qu\'une tornade était probablement responsable des dégâts dans le centre du Mississippi, dont quatre semi-camions renversés et cinq maisons fortement endommagées.\nLe Le principal déclencheur du temps violent est les températures supérieures à la moyenne plus au nord.\nMidwest\xa0: neige d\'effet de lac\nSara Hadley a envoyé dimanche à CNN iReport des photographies des conséquences d\'une tempête à Lansing, Michigan.\nElle n\'a pas commencé tant pis jusqu\'à ce que nous ayons perdu le courant la nuit dernière. À ce moment-là, nous savions que ça allait mal. La dernière fois que nous avons eu de la glace comme celle-ci, c\'était en 1998, a-t-elle écrit.\nSes images montraient des branches et des pommes de pin recouvertes de fines couches de glace.\nDe la neige modérée à lourde renforcée par le lac était attendue pour l\'extrême ouest et le centre-nord du haut Michigan jusqu\'à lundi soir. \nPlaines centrales\xa0: recouvertes de glace et de neige\nLes températures seront de 15 à 25\xa0degrés inférieures à la normale lundi avec une modération progressive jusqu\'au jour de Noël.\nDans le Kansas et le sud-est du Nebraska, les chutes de neige totaliseront 3 à 6\xa0pouces. Et ces températures froides feront frissonner les gens.\nWhitney Eichinger de Southwest Airlines a déclaré que la compagnie essayait d\'offrir des options aux clients.\nNous permettons aux clients de réserver à nouveau leur voyage sans pénalité via -LRB- dimanche -RRB-, a-t-elle déclaré. Cela a plus à voir avec le voyage à l\'aéroport qu\'avec les annulations de vol. \nMid-Atlantic/Northeast\xa0: étrangement chaud\nS\'il y a un cadeau distribué au début de l\'hiver, ce sont les températures bien supérieures à la moyenne dans le Mid-Atlantic.\nNew York a battu un record dimanche, selon Ashley Sears, météorologue au Service météorologique national.\n -LRB- Le record -RRB- était auparavant de 63\xa0degrés en 1998. Et nous sommes actuellement assis à 70\xa0degrés, ce à quoi nous sommes depuis quelques heures, dit-elle.\nPlus au nord, cependant, la situation est moins confortable. Dans le Maine et dans le New Hampshire, le Vermont et dans le nord de l\'État de New York, des avertissements hivernaux, y compris des avertissements de tempête de verglas et de pluie verglaçante, étaient en vigueur.\nLe gouverneur de New York, Andrew Cuomo, a déclaré une urgence de glace hivernale et activé l\'état centre des opérations d\'urgence.\nLe Service météorologique national de Buffalo a déclaré qu\'une accumulation de glace pouvant atteindre un pouce est attendue, et les zones proches du Saint-Laurent et des rivières Noires pourraient en voir encore plus.\nCanada\xa0:\xa0Froid inhabituel\nLe maire de Toronto, Rob\xa0Ford, a décrit l\'hiver Météo dimanche comme l\'une des pires tempêtes de l\'histoire de la ville. Jeff Zammit a déclaré qu\'il a fait plus froid que la normale en décembre et que la pluie verglaçante et les arbres tombés ont fait tomber de nombreuses lignes électriques.\n C\'est vraiment une tempête de verglas catastrophique que nous avons eue ici, probablement l\'une des pires que nous ayons connues \'ai jamais eu, Anthony Haines, PDG de Toronto Hydro, a déclaré à CTV Network. , Morgan Winsor, Nick Valencia, Janet DiGiacomo, Todd Borek et Matt Daniel ont contribué à ce rapport.\n,,,La Saint-Valentin, le 14 février, est une fête qui célèbre l\'amour et l\'amitié aux États-Unis. La plupart des gens le considèrent comme une journée romantique pour montrer à celui que vous aimez à quel point vous tenez à lui en achetant des cartes et des cadeaux. La Saint-Valentin est aussi une fête extrêmement commerciale. Les gens dépensent de l\'argent pour acheter des fleurs, des ballons, du chocolat, des peluches et des bijoux. Les gens sortent aussi pour des dîners coûteux. La Saint-Valentin est même célébrée par les écoliers, qui remettent des cartes de remerciement à leurs professeurs. Certaines écoles secondaires ont des danses de la Saint-Valentin. Les camarades de classe se donnent des petites cartes. Ces cartes peuvent être achetées dans n\'importe quel magasin avec des personnages de dessins animés célèbres. Il est considéré comme beaucoup plus réfléchi de demander aux enfants de créer des cartes faites à la main pour les donner à des amis et à des enseignants.\n \nLa Saint-Valentin est également une excellente journée pour annoncer votre amour à quelqu\'un comme un béguin secret. Certaines personnes se marient même le jour de la Saint-Valentin. Cependant, vous n\'avez pas besoin d\'être amoureux pour partager les festivités. Vous pouvez dire "Joyeuse Saint-Valentin" à vos amis et collègues.\nLa Saint-Valentin n\'est pas un jour férié officiel aux États-Unis, ce qui signifie que les bureaux gouvernementaux et les écoles sont ouverts comme d\'habitude. La date était la célébration d\'un saint nommé Valentin qui aurait sauvé la vie d\'une jeune fille. Ce n\'est qu\'au Moyen Âge que la romance s\'est en quelque sorte mêlée à la date et que les amoureux s\'envoyaient des notes d\'amour manuscrites. À l\'époque, vous ne pouviez pas simplement aller au magasin et acheter une carte comme vous le pouvez maintenant.\nVous savez que la Saint-Valentin approche à grands pas lorsque les magasins commencent à décorer avec des cœurs rouges et roses et le symbole de Cupidon, un ancien dieu romain ailé censé frapper le cœur des gens Maintenant, il semble que Cupidon s\'attaque directement à votre portefeuille, car vous verrez également de nombreuses autres annonces de bijoux à l\'approche de la Saint-Valentin.,,,Six Les Ouïghours qui ont été emprisonnés dans le centre de détention de Guantanamo Bay, à Cuba, ont été transférés samedi dans l\'État insulaire du Pacifique Sud de Palau, a annoncé le ministère de la Justice.\nAhmad Tourson, Abdul Ghappar Abdul Rahman, Edham Mamet, Anwar Hassan, Dawut Abdurehim et Adel Noori a été réinstallé à Palau, a annoncé samedi le ministère de la Justice dans un communiqué.\nSept Ouïghours sont toujours à Guantanamo Bay, où 215 prisonniers sont toujours détenus. Les Ouïghours sont un groupe ethnique de l\'ouest de la Chine.\nDepuis janvier, 25\xa0détenus ont été transférés vers d\'autres pays.\nLes États-Unis se sont coordonnés avec les Palaos pour garantir que les transferts se déroulent dans le cadre de mesures de sécurité appropriées et continueront de consulter la République de Palau concernant les individus, a déclaré le ministère de la Justice.\nLes 13 Ouïghours ont été accusés d\'avoir reçu des armes et un entraînement militaire en Afghanistan. Certains d\'entre eux ont été autorisés à être libérés depuis 2003, et d\'autres Ouïghours ont été libérés dans d\'autres pays. Les États-Unis ont déclaré qu\'ils ne les renverraient pas dans leur pays d\'origine par crainte qu\'ils ne soient torturés par les autorités chinoises.\nLe gouvernement chinois a déclaré qu\'aucun Ouïghour de retour ne serait maltraité et a mis en garde à plusieurs reprises d\'autres pays contre l\'enlèvement des hommes. Cet été, les responsables de Pékin ont de nouveau exhorté les États-Unis à remettre tous les Ouïghours restants au lieu de les envoyer ailleurs.\nLa Chine allègue que les hommes font partie du Mouvement islamique du Turkestan oriental, un groupe que le Département d\'État américain considère comme une organisation terroriste, qui opère dans Région du Xinjiang. Le Turkestan oriental est un autre nom pour le Xinjiang.\nLe 20\xa0octobre, la Cour suprême a provisoirement accepté d\'accepter un appel des 13\xa0Ouïghours qui devaient être libérés aux États-Unis après avoir été détenus par l\'armée américaine.\nCarol Cratty et Bill Mears de CNN ont contribué à ce rapport.\n,,,La plupart des directeurs artistiquespremière journée à la tête d\'une nouvelle compagnie de ballet consiste à rencontrer les danseurs. Mais pour Nacho Duato, c\'est un défi de trouver la salle de répétition.\nLes couloirs labyrinthiques du magnifique théâtre Mikhailovsky de Saint-Pétersbourg s\'avèrent déroutants et inspirent quelques rebondissements non chorégraphiés à Duato avant qu\'il n\'identifie enfin la bonne salle et sa nouvelle compagnie -- les 140 danseurs du Ballet Mikhailovsky. le premier étranger à diriger une compagnie de ballet russe depuis plus d\'un siècle et a été chargé d\'apporter la chorégraphie moderne au répertoire classique de Mikhailovsky. c\'est ce que je pense qu\'ils veulent et c\'est pourquoi je pense qu\'ils m\'ont appelé, a déclaré Duato. Je pense qu\'ils ont assez clairement compris qu\'ils veulent changer; qu\'ils veulent quelque chose de nouveau. \nDanseur de ballet moderne et chorégraphe au pedigree international, on espère qu\'il pourra revitaliser le ballet pour le public russe. Alors que les représentations au Mikhailovsky sont très fréquentées, on craint de plus en plus que les Russes ne se lassent de voir le répertoire standard des ballets classiques.\n Les Russes adorent le ballet. Cela va sans dire, mais l\'attitude qui existe envers le ballet russe en Occident est plus passionnée, a déclaré Vladimir Kekhman, importateur de fruits milliardaire et directeur général du Théâtre Mikhailovsky.\n J\'aimerais qu\'en Russie, nous ayons la même attitude car comme on dit, nous ne traitons pas ce que nous avons jusqu\'à ce qu\'il soit parti. Je considère le ballet comme un trésor national, a-t-il ajouté.\nKekhman est un converti relativement récent au ballet, mais il est devenu une sorte d\'entrepreneur culturel , investissant 45\xa0millions de dollars de son propre argent pour restaurer les bâtiments de Mikhailovsky et, espérons-le, sa réputation.\nKekhman avait besoin de nouveaux talents pour superviser sa vision et pour cela, dit-il, il devait regarder au-delà de la Russie.\n J\'ai regardé dans notre industrie pour voir qui étaient considérés comme des légendes vivantes de la chorégraphie, et quand j\'ai regardé en Russie, j\'ai réalisé que nous n\'avions pas de chorégraphes capables non seulement de chorégraphier, mais aussi de diriger la compagnie, a-t-il déclaré. nIl s\'est approché de Duato, un non- Orateur russe, qui a passé les 20 dernières années à chorégraphier de nouvelles œuvres en tant que directeur artistique de la Compagnie nationale de danse espagnole.\nAu début, Duato était réticent à changer de direction vers le ballet classique. Mais ensuite Mikhailovsky lui a offert la chance d\'apporter une chorégraphie moderne à une compagnie de ballet classique et cela s\'est avéré une offre trop alléchante pour la laisser passer.\n Je connais beaucoup de gens qui pensent qu\'il est -LRB- Kekhman -RRB- fou, que je Je suis fou et que ça ne marchera jamais, a déclaré Duato. Que ce mélange est comme une bombe, une bombe atomique. Mais il m\'a fait sentir que Essayons, faisons-le. Pourquoi pas ? Peut-être que nous pouvons le faire.\nApporter une approche moderne du ballet en Russie, où il est ancré dans la culture, pourrait en décourager certains, mais c\'est une tâche que Duato adore.\n Ce n\'est pas une pression . C\'est un plaisir. C\'est vraiment un plaisir d\'être dans un endroit où le ballet -LRB- fait -RRB- partie de la tradition culturelle depuis -LRB- depuis si longtemps -RRB-, et en même temps ils veulent percer et faire de nouveaux choses, a déclaré Duato.\nDuato s\'est fixé un calendrier difficile, sélectionnant trois danseurs pour créer un ballet si nouveau qu\'il n\'a même pas encore de nom. Ils n\'ont que quelques semaines avant la première mondiale.\nLa danseuse principale Ekaterina Borchenko est le joyau de la couronne impériale de Mikhailovsky. La ballerine vedette de la compagnie a dansé les rôles principaux dans des productions de ballets classiques dont Le Lac des cygnes, Giselle et La Sylphide. \nMaintenant, elle savoure la chance d\'étendre ses compétences. Dans ce répertoire, je peux montrer une autre facette de moi-même, m\'exprimer différemment et trouver quelque chose de nouveau en moi, a-t-elle déclaré.\nLa soirée d\'ouverture sera un test décisif pour savoir à quel point les Russes sont réceptifs à cette nouvelle direction.\nDuato pense que le ballet l\'est. un organisme vivant, qui a besoin d\'évoluer constamment pour maintenir l\'intérêt du public. C\'est une philosophie qu\'il partage avec Kekhman.\n Ensemble, nous allons travailler sur quelque chose, a déclaré Duato. On ne sait pas comment ça va se passer mais je pense que ça va être beau. Si nous y mettons tout notre cœur et toute notre âme, ce sera quelque chose de fantastique. \n,,,L\'un des monuments les plus historiques et les plus importants des États-Unis est le système de téléphérique de San Francisco. Il s\'agit du seul système de téléphérique à commande manuelle au monde. Il est géré par l\'Agence municipale des transports de San Francisco. Le système remonte à 1878, lorsque la ligne California Street a été ouverte pour la première fois. Au total, trois lignes opèrent actuellement dans la ville, et dispose d\'une flotte de 12 téléphériques. Il y avait 23 lignes en service dans toute la ville reliant les diverses communautés qui composent San Francisco. Les voitures à ciel ouvert sont une attraction touristique majeure et sont toujours utilisées quotidiennement par les navetteurs de San Francisco.\n \nLa simple raison pour laquelle les téléphériques ont été utilisés pour la première fois dans la ville était à cause de son paysage. San Francisco est très vallonné, ce qui rend impossible l\'escalade des bus municipaux. La solution à ce problème était le téléphérique. Ces voitures sont littéralement tirées par câble, le long des collines escarpées de la ville, ce qui permet aux gens de se déplacer plus facilement d\'un bout à l\'autre de la ville.\nIl existe aujourd\'hui deux types de voitures. Voitures à une extrémité et voitures à double extrémité. Les voitures à une extrémité ont des sections ouvertes avec des sièges qui font face à la rue. La partie arrière de ces voitures est entourée de sièges faisant face à l\'intérieur de la voiture. Ces voitures peuvent accueillir 29 passagers. Ils se déplacent dans les deux sens en utilisant une plaque tournante à base pivotante qui fait tourner la voiture en fin de ligne. Les voitures à double extrémité sont un peu plus grandes et peuvent accueillir jusqu\'à 68 passagers. Il dispose de 34 sièges et de poignées pour 34 passagers debout supplémentaires. La meilleure caractéristique de ces voitures est le pur plaisir qu\'elles procurent à conduire. Les gens qui empruntent les téléphériques de San Francisco entrent dans une partie de l\'histoire de la Californie.,,,S\'arrêtant sur le parking du plus grand supermarché de la ville, Joe pouvait déjà voir qu\'il allait être difficile de trouver un parking sur ce Samedi matin. Il décide de déposer d\'abord sa femme Melissa, afin qu\'elle puisse avoir une longueur d\'avance sur les achats qu\'ils doivent faire. Pendant ce temps, il se bat pour trouver une place libre parmi la foule d\'autres voitures en attente.\n \nÀ l\'intérieur, Joe trouve enfin Melissa, dont le chariot est déjà chargé de paquets de collations et de boissons bio. Il sait que Melissa est facilement tentée par des mots fantaisistes qui promettent des bienfaits pour la santé. Cela lui fait du bien de savoir qu\'elle prend des décisions saines. Dans le passé, Joe essayait de la raisonner, expliquant que la santé ne consistait pas à consommer des articles spécifiques, mais plutôt à un mode de vie général. Aujourd\'hui, il décide simplement de passer à autre chose. En tant que chef de la famille, Joe attend avec impatience chaque semaine le processus de sélection des viandes et des légumes, imaginant toutes les façons dont il peut les aromatiser et les cuisiner. Des échantillons gratuits sont offerts aux endroits familiers - Melissa s\'assure de frapper chaque station. Ils conviennent que la saucisse de poulet est leur préférée et achètent deux paquets. Les pâtes fraîches à prix unique attirent Joe, tandis que la vente de cheesecake attire Melissa.\nAlors qu\'ils font la queue pour payer, Melissa examine leur panier débordant et se rend compte qu\'ils ont dépassé leur budget. Elle remet les collations et les boissons qui étaient clairement des achats impulsifs et affirme avec joie qu\'elle apprend à être une excellente acheteuse\xa0! Joe ne peut que sourire à l\'expression de son visage. Il y a cinq mois, quand ils se sont mariés pour la première fois, il n\'aurait jamais imaginé qu\'il apprécierait ces courses du samedi matin. Maintenant, il y trouve une satisfaction paisible, un changement bienvenu par rapport à son travail de toute la semaine.,,,Le camion le plus long du monde\nDans l\'arrière-pays australien, Jeremy Hart a fait du stop sur le Centipede.\n\'Si je prends mon pied sur l\'accélérateur, il nous faudra\nenviron quatre kilomètres pour nous arrêter", déclare Matt\nLangley, l\'Australian capitaine du\nCentipéde, qui, à environ 54,5\xa0mètres,\net composé d\'une cabine de conduite et de six remorques,\n prétendait être le camion le plus long du monde. Matt\na un toucher délicat. Ses mains, sur le volant de 60\xa0cm\n, contrôlent un million de dollars de\ntremorques transportant du minerai de zinc.\n Un faux mouvement sur la route étroite de 128\xa0km\nde la mine McArthur River, la plus grande mine\nzinc du monde, à le port de Bing Bong et le\nCentipéde, avec ses 110\xa0roues, s\'effondreraient.\nCependant, en deux\xa0ans de livraison de précieuses\ncharges de minerai de zinc en poudre de la mine à la côte,\nle camion n\'a eu qu\'un seul accident. x1xx7xx\n Je lui ai posé des questions sur le travail. ‘La chose la plus difficile\\est la concentration. La route est vraiment ennuyeuse\net par endroits c\'est un peu accidenté. Un problème est\nle fait que vous tirez six énormes remorques\nderrière vous. x2xx xx Je détesterais que cela m\narrive.\'\n Près de 60\xa0mètres devant la dernière remorque,\nau-dessus de la route dans le\nconfort climatisé de la cabine de Matt, vous pouvez perdre presque toute\nsensation de traction 130 tonnes de matière première.\nx3xx xx Ce mouvement se répète sur\nla longueur du Centipede. Matt est en contrôle, mais\nvous pouvez voir que cela demande un réel effort.\nDans ce coin le plus reculé du nord de\nl\'Australie, la circulation n\'est pas un problème pour Matt et\nson copilote, Mick Webster. "Certains jours, nous\nne voyons pas un autre véhicule courir vers Bing\nBong, mais lorsque nous le voyons, nous essayons d\'être aussi\ncourtois que possible", explique Matt. \'\xa0x4xx xx\nDe toute façon, certains ont trop peur pour essayer.\nD\'autres pensent qu\'il s\'agit d\'un\nvéhicule de taille normale avec deux ou trois remorques attachées.\nQuand ils voient qu\'il est six\xa0heures, ils reculent ou\npassent par nous la bouche grande ouverte.\'\n Le camion innove en\ntechnologie. Le gouvernement\ndu Territoire du Nord a dû modifier la réglementation pour\nlégaliser le mille-pattes pour un usage routier. \' x5xx x\nIls sont scellés avec des couvercles pour empêcher la\npoussière de minerai de polluer l\'environnement, et ils\nont un dispositif spécial de chargement et de déchargement\n.\'\n Cela a pris un peu moins de deux heures, à une\ntempérature de 40 degrés, pour ramper à pas\nplus de 64 km/h de McArthur Mine à Bing\nBong. x6xx xx\n À son arrivée à destination, Matt appuya\n sur un bouton pour décharger le zinc sur le sol\nde l\'immense entrepôt à l\'épreuve des cyclones où il est\nconservé avant d\'être expédié vers l\'Europe et\nl\'Asie. Cela ne ressemblait pas à grand-chose. "Quand vous voyez\nà quel point une charge complète semble petite lorsqu\'elle est déchargée\nonsur le sol de l\'entrepôt, c\'est très\nfrustrant", dit-il. ‘ x7xxx x ’ Je lui ai demandé\ncomment il aimait le travail. "Je ne ferais rien d\'autre, mon pote", a-t-il répondu.,,,Dans une opération audacieuse digne d\'un film, trois hommes se sont échappés d\'une prison canadienne en hélicoptère, provoquant une chasse à l\'homme massive, ont déclaré les autorités.\nLes hommes s\'est évadé d\'un centre de détention samedi soir à Québec.\n Un hélicoptère s\'est brièvement posé dans la cour avant de décoller avec les trois prisonniers, a indiqué Ann Mathieu, porte-parole de la Sûreté provinciale du Québec.\n Nous discutons étroitement avec le centre de détention pour comprendre exactement comment l\'évasion s\'est déroulée, Mathieu a déclaré à CBC, partenaire de CNN. La priorité... est de trouver l\'hélicoptère. \nUne chasse à l\'homme est en cours pour Yves Denis Yvon Lamontagne, 35 ans, Denis Lefebvre, 53 ans, et Serge Pomerleau, 49 ans.\nLes deux premiers hommes ont été arrêtés dans une affaire de drogue en 2010 et attendaient leur procès. La police a trouvé des armes, des dizaines de voitures et un avion pendant le buste, a-t-il ajouté. Aucune information n\'a été fournie sur le troisième suspect.\nIl s\'agit de la deuxième évasion de prison effrontée au Canada en un peu plus d\'un an.\nEn mars\xa02013, deux hommes se faisant passer pour des touristes ont réquisitionné un hélicoptère d\'une agence de voyage canadienne, ont ordonné au pilote pour survoler un centre de détention près de Montréal, a hissé deux détenus dans l\'avion en vol stationnaire à l\'aide de câbles - et s\'est envolé. Jérôme, au nord-ouest de Montréal.\nBien que spectaculaires, les évasions aériennes ne sont pas sans précédent.\nLe détenu albanais Alket Rizaj s\'est évadé d\'une prison à sécurité maximale en Grèce à deux reprises, en 2006 et 2009.\nÀ ces deux occasions, des complices ont fait irruption dans la prison\\ de la cour sur des hélicoptères. Les détenus sont ensuite montés à bord à l\'aide d\'échelles de corde.\nLes deux fois, Rizaj a été repris.\nL\'année dernière, une autre évasion de prison en Grèce a tenté d\'utiliser la même tactique.\nMais le complot a été déjoué lorsque le prisonnier a été abattu et que l\'hélicoptère a été forcé d\'atterrir sur le parking de la prison.\nEn 1971, Joel David Kaplan s\'est évadé d\'une prison mexicaine à bord d\'un hélicoptère. Le vol du New Yorker a inspiré le livre The 10-Second Jailbreak: The Helicopter Escape of Joel David Kaplan et le film Breakout de 1975. \nEt en France, Pascal Payet s\'est échappé non pas une, ni deux, mais trois fois des prisons françaises.\nÀ chaque fois, il a été repris.\nLes évadés de la douche de la prison de l\'Oklahoma ont été capturés\nLe prisonnier de l\'Arizona s\'évade pour rencontrer sa petite amie pour la Saint-Valentin\\ n,,,Qui a dit que les fêtes d\'adultes devaient être ennuyeuses. De plus en plus d\'adultes revivent leur enfance ou créent des souvenirs qu\'ils n\'ont pas avoir comme enfants en organisant des soirées à thème pour leur anniversaire ou d\'autres occasions.\nLes soirées à thème sont basées sur une idée, une émission de télévision, un personnage fictif ou vraiment n\'importe quoi. Parfois, les invités doivent également s\'habiller en fonction du thème. Par exemple, la soirée Toga est un type de soirée à thème où les invités doivent s\'habiller en toges, en réalité de simples draps blancs. Les soirées Toga étaient particulièrement populaires parmi les étudiants. Dans une soirée masquée, tout le monde porte un masque et doit deviner qui se cache derrière. Le mystère fait partie du plaisir. En parlant de mystère, il y a des soirées mystères de meurtre, où les invités doivent résoudre un faux meurtre. Certains adultes organisent des fêtes en fonction des saisons. Une fête estivale sur la plage, par exemple, pourrait mettre en vedette des invités portant leurs maillots de bain. Un autre type populaire de soirée à thème est la soirée jeux. Dans ce type de fête, les gens se réunissent pour jouer à divers jeux de société. Une variante de la soirée jeux est la soirée casino, où les adultes jouent à des jeux que l\'on trouve généralement dans un blackjack ou un poker de type casino.\nUne autre soirée à thème peut se concentrer sur une région ou un pays spécifique. Par exemple, une soirée à thème mexicaine peut comporter des tacos, les couleurs du drapeau mexicain et de la musique mexicaine. Les invités doivent cependant faire attention lorsqu\'ils s\'habillent pour ces types de soirées à thème. S\'habiller comme un certain groupe ethnique ou racial est généralement considéré comme offensant. Il vaut mieux simplement profiter de la nourriture et ne pas dépeindre un stéréotype. ,,, La cérémonie religieuse ou civile qui fait des gens légalement un couple marié n\'est qu\'une partie du mariage proprement dit. Après le mariage vient la réception. Une réception de mariage est une fête organisée immédiatement après la cérémonie de mariage. Habituellement, il y a un peu de temps, environ une heure, entre la cérémonie et la réception pour donner aux gens suffisamment de temps pour se rendre du lieu de la cérémonie au lieu de la réception. La plupart des réceptions de mariage ont lieu dans une salle de restauration, un lieu spécialisé dans l\'organisation de grandes fêtes. Cependant, les réceptions de mariage peuvent avoir lieu dans des restaurants, des parcs et même dans des musées ou des zoos. Une réception de mariage n\'a généralement pas lieu au même endroit que la cérémonie, mais parfois c\'est le cas.\n \nLorsque les invités arrivent à la réception, les mariés ne sont généralement pas là pour les rencontrer. Juste après la cérémonie, le couple nouvellement marié, les demoiselles d\'honneur et les garçons d\'honneur iront quelque part pour se faire photographier. Pendant ce temps, les clients ont généralement droit à un cocktail sur le site de la réception. Pendant ce temps, des apéritifs et des boissons sont servis. Cela donne également aux invités une chance de socialiser. Après l\'heure de l\'apéritif, la salle à manger principale est ouverte. Lors d\'une réception de mariage, les places sont généralement pré-arrangées. Chaque invité recevra une carte lui indiquant où il doit s\'asseoir. Les tableaux sont classés par numéros. Devant la salle de réception, il y a une table spéciale pour les mariés.\nUne fois que tout le monde est assis, les serveurs commencent à servir la salade ou le pain et les boissons. Certaines réceptions ont un bar ouvert, ce qui signifie que l\'alcool est gratuit. D\'autres réceptions ont un bar de cas, ce qui signifie que les boissons non alcoolisées sont gratuites; les clients doivent payer leurs propres boissons alcoolisées. ,,, Il n\'y a peut-être pas de plus grand américain que George Washington. Washington est devenu le premier président des États-Unis en 1789 avec sa victoire unanime aux premières élections américaines. Il a également été élu à l\'unanimité pour un second mandat en 1793, le seul président à recueillir 100 % des voix électorales. Washington était également le commandant en chef de la première armée continentale qui a combattu les Britanniques alors que le pays n\'était pas encore établi.\n \nSon épopée « Traversée du fleuve Delaware » pour récupérer New York est toujours considérée comme l\'une des plus grandes mouvements militaires dans l\'histoire de la guerre. Au moment de la traversée, les Britanniques avaient pris Boston et New York. L\'armée continentale était en fuite et désorganisée, mais Washington avait un plan. Il décida de faire traverser le fleuve Delaware à ses troupes en plein hiver. C\'était une décision que de nombreux esprits militaires pensaient impossible, mais pas Washington. Les Américains ont pu prendre le New Jersey et changer le cours de la guerre.\nUn autre moment marquant pour Washington a été l\'hiver à Valley Forge. En 1777, Washington dirigea une armée de plus de 11 000 hommes à Valley Forge en Pennsylvanie, où l\'armée souffrit d\'un hiver froid qui coûta la vie à 3 000 hommes. Lorsque le printemps est arrivé, au lieu d\'un groupe de soldats démoralisés et battus, l\'armée est sortie prête au combat et capable de poursuivre le combat contre les Anglais à New York. L\'armée a pu survivre à l\'hiver féroce grâce à la formation et à la préparation des officiers de Washington. Washington a gagné le surnom de «\xa0le père de son pays\xa0» en raison de son intelligence militaire et de son intérêt pour le peuple des États-Unis.,,,Docteur\xa0Klein\xa0: Bonjour, Cecilia, comment vous sentez-vous aujourd\'hui\xa0?\nCecilia\xa0: Je ne me sens pas très bien, docteur Klein. J\'espère que tu pourras soigner mon mal lness.\nDocteur\xa0Klein\xa0: Je suis désolé que vous vous sentiez très malade. Dites-moi certains de vos symptômes afin que je puisse vous donner un bon diagnostic.\nCecilia\xa0: Je ne me sens pas bien depuis hier après-midi. Les symptômes ont commencé par un mal de tête et un mal de gorge progressif. J\'ai bu du thé au citron et au miel et je me suis couché tôt. Cependant, je suis extrêmement épuisé et je ne me sens pas mieux aujourd\'hui.\nDocteur\xa0Klein\xa0: J\'ai récemment constaté ces symptômes chez certains de mes autres patients. Je vais vérifier votre température et examiner votre gorge afin de vous donner un diagnostic correct.\nCecilia\xa0: Merci, Docteur.\nDocteur Klein\xa0: Ouvrez et dites "Ahhhhhh..."\nCecilia\xa0: "Ahhhhhh..." \nDocteur\xa0Klein\xa0: Oh, mon Dieu\xa0! Je vois déjà que ta gorge est très rouge. Votre température de 100 degrés indique que vous avez une légère fièvre. J\'ai peur que vous ayez la grippe.\nCecilia\xa0: Quelle est la meilleure façon de soigner mes symptômes\xa0?\nDocteur\xa0Klein\xa0: Vous aurez besoin de beaucoup de repos et vous devriez boire fréquemment afin de rester hydraté. Vous venez également de commencer à présenter des symptômes, je peux donc vous prescrire un médicament qui peut réduire la fièvre et raccourcir la durée de votre maladie.\nCecilia\xa0: Dois-je également rester à la maison\xa0?\nDocteur\xa0Klein\xa0: Oui, vous devriez rester au lit jusqu\'à ce que la fièvre tombe. Vous devez également attendre 24 heures après la chute de la fièvre avant de retourner au travail. Vous ne voulez pas risquer de rendre vos collègues malades également.\nCecilia\xa0: Je suppose que je vais me détendre et me détendre pendant quelques jours. Merci, docteur, pour toute votre aide\xa0!\nDocteur\xa0Klein\xa0: Pas de problème\xa0! Faites de votre mieux pour vous reposer pendant quelques jours. J\'espère que tu te sentiras mieux bientôt\xa0!,,,À\xa0:\xa0Meena\nDe\xa0:\xa0Suki\nObjet\xa0: Que faire pendant notre absence\nSalut Meena,\nMerci beaucoup de nous avoir proposé de garder la maison la semaine prochaine. Je suis seulement désolé que nous ne puissions pas nous rattraper correctement jusqu\'à ce que nous revenions de notre voyage. Quoi qu\'il en soit, voici tout ce que je n\'aurai pas le temps de vous dire samedi matin.\nLe code de l\'alarme est le 7957. N\'oubliez pas de l\'activer lorsque vous sortez et pensez à l\'éteindre à votre retour à la maison car il fait BRUYANT\xa0!\nLes lapins ont besoin d\'être nourris une fois par jour et en aucun cas ils ne doivent être autorisés à sortir de leur cage dans le jardin car les chats du voisin les ont attaqués dans le passé.\nVeuillez utiliser la lessive machine pendant la journée uniquement car elle est vieille et bruyante et les voisins se plaignent du contraire\xa0?\nVoudriez-vous ouvrir toutes les fenêtres à l\'étage pendant une heure le matin car la maison devient humide à cette période de l\'année\xa0?\nC\'est à ce sujet! Vous trouverez tout ce dont vous avez besoin dans la maison et vous servirez de tout dans le réfrigérateur ou les placards. N\'hésitez pas à appeler ou à envoyer un SMS si vous avez des problèmes/questions.\nÀ bientôt pour remettre les clés samedi matin et au plaisir d\'avoir une bonne conversation à notre retour\xa0!\nMerci encore\xa0!\nSuki,,,\ufeffSerial dater Emmanuel Limal en avait assez de rencontrer des femmes qui n\'étaient pas prêtes à fonder une famille, ou du moins qui n\'admettaient pas qu\'elles l\'étaient. L\'acteur de 43 ans, originaire de France, avait passé 20 ans à vivre à Copenhague et à chercher l\'amour dans l\'espoir d\'élever des enfants. Il a récemment pris sa quête en ligne mais a été consterné par les résultats. \n"J\'ai été frustré par le fait que tout le monde essaie de se vendre comme étant vraiment actif, toujours en voyage ou avec une longue liste de passe-temps, mais aucune mention des enfants", a déclaré Limal. "Sur certains sites, il y avait une option sur laquelle cliquer, en disant\xa0:\xa0"J\'aimerais avoir des enfants un jour", mais vous lirez le profil de la personne et penserez\xa0: "Vous n\'aurez jamais le temps\xa0!". Si quelqu\'un va au gym huit fois par semaine et voyageant tous les mois, ils ne font pas passer la famille en premier. \nLimal a une fille de six ans issue d\'une précédente relation, mais issu d\'une famille nombreuse - son père est l\'un des 11\xa0ans - il a toujours voulu avoir plus d\'enfants. «Je n\'arrivais pas à rencontrer quelqu\'un qui était prêt à donner la priorité à fonder une famille et j\'avais du mal à savoir quand mentionner le fait de vouloir des enfants chaque fois que je rencontrais quelqu\'un de nouveau. C\'est le tabou ultime des rencontres », a-t-il déclaré. "Puis un jour, j\'ai lu le profil d\'une femme de 38 ans qui disait qu\'elle savait que c\'était \'vraiment mal à admettre\' mais qu\'elle voulait des enfants. Et j\'ai juste pensé : \'Tu ne devrais pas avoir honte de ça.\' » \nLimal a réhypothéqué son appartement pour financer la création de Babyklar.nu – ou \'baby-ready now\' en anglais. Il fonctionne comme un site de rencontre normal, mais chaque dater potentiel est invité à être honnête quant à son souhait de fonder une famille bientôt. "Nous demandons aux gens s\'ils sont d\'accord avec quelqu\'un qui a déjà des enfants et qui veut un autre bébé", a déclaré Limal. "Mais nous ne leur demandons pas de préciser combien d\'enfants ils aimeraient. Ce serait un peu trop comme l\'épicerie en ligne. \nLa réponse au site a été écrasante, a-t-il déclaré. "Nous avions 50 inscriptions par heure lors de notre lancement en juin et nous entendons déjà des couples qui se sont rencontrés via le site et qui sont maintenant ensemble. J\'attends pleinement le premier bébé Babyklar.nu d\'ici l\'été prochain. Plus d\'hommes que de femmes se sont inscrits (53% contre 47%), avec des témoignages tels que "C\'est tellement beau de pouvoir le dire à haute voix" et "Je fifinalement oser être honnête sur ce que je veux. \nLe site arrive à un moment opportun pour le pays de cinq millions d\'habitants. Les Danois n\'ont pas assez de bébés, selon un rapport de l\'hôpital Rigshospitalet de Copenhague, et le taux actuel de 1,7 enfant par famille n\'est pas suffisant pour maintenir la population du Danemark. Les suspects habituels sont blâmés pour le nouveau creux – les femmes qui le quittent «trop tard» et les couples qui cohabitent et attendent de fonder une famille. \n"Maintenant, j\'espère que les hommes et les femmes qui veulent fonder une famille mais qui n\'ont pas encore rencontré la bonne personne auront une autre option", déclare Limal. Il tient à souligner qu\'il ne s\'agit pas seulement d\'élevage de bébés : « Je veux que ce soit une histoire d\'enfants et d\'amour. Mon objectif est de jumeler des personnes qui veulent vraiment une famille et un partenaire – et qui resteront ensemble. Je suis un romantique dans l\'âme. \nIl est prévu de déployer le site en France et au Royaume-Uni plus tard en 2013, mais pour l\'instant ce sont les Danois qui en récoltent les bénéfices. "Les Danois n\'ont aucun problème à avoir des enfants avant le mariage, donc les choses peuvent aller vite et, parce que le pays est si petit, un Jutlander peut sortir avec un Copenhagener sans trop voyager", a déclaré Limal. De plus, Limal a enfin trouvé l\'amour. "J\'ai rencontré une gentille femme et elle veut aussi un bébé - alors nous verrons."\n,,,-LRB- CNN -RRB- En tant que l\'une des principales capitales financières du monde, Hong Kong est un rêve destination idéale pour les voyageurs d\'affaires.\nAvec plus de gratte-ciel que n\'importe quelle autre ville au monde et certains des restaurants les plus raffinés de la planète, la ville pourrait bien vous obliger à demander l\'annulation d\'un vol afin que vous puissiez profiter d\'une journée supplémentaire.\nPour vous aider profitez de quelques-uns des points forts de la ville, voici un guide pour vous rendre en ville, bien manger/dormir, acheter les meilleurs souvenirs et repartir avec une note de frais dont vous serez fier.\nAéroport le plus simple et le plus confortable transfert\nOubliez tout ce que vous savez sur les métros - le service de train Airport Express de Hong Kong épate même les navetteurs les plus blasés.\nLes trains sont propres, confortables, spacieux, toujours à l\'heure et vous emmènent en ville en 24 minutes chrono, plus rapide que n\'importe quel service de voiture ne peut jamais aspirer.\nMieux encore, les trains sont faciles d\'accès à pied et la procédure simple à comprendre à l\'arrivée - il n\'y a pratiquement aucun stress sur la façon d\'obtenir des billets ou où prendre les trains lorsque vous quittez le terminal d\'arrivée. :28h et intervalles de 12 minutes à partir de 23h28. à 00h48 tous les jours.\nRepas mémorables\nLes restaurants étoilés Michelin ne manquent pas à Hong Kong.\nMais trouver un repas local mémorable auquel vous penserez pendant des mois est bien plus difficile.\nLuk Yu Teahouse a tout pour plaire\xa0: une atmosphère historique inspirée de l\'art déco, des serveurs qui sont restés fidèles à l\'endroit pendant des décennies, une cuisine cantonaise qui tue qu\'on ne trouve nulle part ailleurs, une clientèle glamour et un passé riche en histoires qui comprend un coup de foule dans la salle à manger.\nVous Vous devrez arriver tôt pour un petit-déjeuner composé de dim sum à l\'ancienne, comme une pâtisserie au canard et aux châtaignes.\nLuk Yu Teahouse, Luk Yu Building, 24-26 Stanley St. Central\xa0; +852\xa02523\xa01970\nChambre située au centre avec vue\nPour une véritable sensation de chez-soi loin de chez soi, les suites penthouse de The Upper House sont un bon appel.\nAvec plus de 180\xa0mètres carrés d\'élégance discrète, enveloppant une vue sur Hong Kong et une salle de bains inspirée d\'un spa, c\'est le séjour ultime en ville.\nOffre bonus\xa0: la chance de croiser une célébrité dans les ascenseurs.\nThe Upper House, Pacific Place, 88 Queensway, Admiralty\xa0; +852\xa02918\xa01838\nOù trouver un ajout unique à votre garde-robe\nLa scène du design de mode à Hong Kong est jeune, éclectique et amusante.\nUn endroit pour en avoir un avant-goût est le K11\xa0Mall, surnommé Art\xa0Mall pour son soutien à l\'art contemporain chinois et sa galerie réputée au sous-sol.\nLe centre commercial abrite des marques de créateurs locaux et propose sa propre sélection de produits de créateurs au K11 Design Store.\nK11, 18 Hanoi Road, Tsim Sha Tsui, Kowloon\xa0; +852\xa03118\xa08070\nOù prendre une photo époustouflante\nBien sûr, vous pouvez monter dans l\'historique Peak Tram avec les touristes et prendre une superbe photo de Hong Kong au sommet du Victoria Peak - c\'est infaillible.\nMais vous pouvez également rejoindre les randonneurs locaux -LRB- dont il y a beaucoup -RRB- pour monter Lion Rock Peak pour le panorama le plus incroyable de Kowloon, le tout entouré d\'une nature vierge.\nDécouvrez le cœur de la ville\nPour voir Hong Kong , passé et présent, en un mot accessible à pied, dirigez-vous vers les districts du centre et de Sheung Wan.\nCertaines des parties les plus anciennes de la ville, avec des immeubles d\'habitation en ruine de l\'époque coloniale et des boutiques vieilles de plusieurs décennies, se trouvent dans ces zones.\\ nGentifiant confortablement, Soho et Poho -LRB- le quartier entre Po Hing Fong et Hollywood Road -RRB- offrent des restaurants bien pensés et des boutiques sans prétention.\nInstructions générales\xa0: continuez à marcher vers l\'ouest sur Queen\'s Road Central ou sur la Hollywood Road parallèle. \nGraham Street est l\'endroit où les choses commencent à devenir intéressantes.\nClassique, e souvenir facile à emballer\nCertains des meilleurs souvenirs les visiteurs à emporter de Hong Kong sont centrés sur la nourriture.\nPour ramener un peu de saveur locale à la maison, vous pouvez vous procurer quelques pots de sauce XO.\nChaque restaurant digne de sa réputation aura sa propre recette secrète pour la sauce locale légèrement épicée condiment à base de fruits de mer séchés et souvent précieux, ainsi que de jambon chinois.\nOn adore celui du Mandarin Oriental Cake Shop -LRB- 35$/petit, 45$/grand pot -RRB-.\nMandarin Oriental Cake Shop, H/F , Mandarin Oriental Hong Kong, 5 Connaught Road, Central, +852\xa02825\xa04008\n,,,Un ultra-marathonien tentant de courir de la Floride aux Bermudes dans une bulle gonflable à propulsion humaine a été secouru samedi matin après avoir demandé de l\'aide , a déclaré la Garde côtière.\nReza Baluchi, selon son site Web, s\'est fixé pour objectif de traverser plus de 190 nations reconnues pour encourager la paix dans le monde. CNN a publié un article à son sujet en 2012.\nLes garde-côtes ont rencontré Baluchi pour la première fois mercredi après avoir reçu un rapport sur un homme dans une bulle au large de Miami, désorienté et demandant des directions vers les Bermudes, selon un communiqué de presse des garde-côtes. La date à laquelle il a commencé sa quête n\'était pas claire.\nUn garde-côte a trouvé Baluchi. Les responsables ont décrit l\'engin comme une bulle hydro pod et un homme dans une vidéo YouTube de 2013 intitulée une bulle Baluchi une grande roue de hamster. Il se déplaçait sur le sol alors que Baluchi courait à l\'intérieur.\nDans le communiqué de presse, les garde-côtes ont déclaré que Baluchi avait des barres protéinées, de l\'eau en bouteille, un GPS et un téléphone satellite. Les garde-côtes ont fait part des dangers du voyage et ont demandé à Baluchi d\'arrêter son voyage car il n\'avait pas assez de ravitaillement. Mais il ne quitterait pas son navire, ont déclaré des responsables.\nLes garde-côtes ont surveillé ses progrès et samedi matin, un Baluchi épuisé a activé sa balise de localisation personnelle, ont déclaré les garde-côtes.\nUn avion HC-130 des garde-côtes et un MH- 60 hélicoptères, ainsi que le navire Maersk Montana, ont été envoyés pour secourir Baluchi à 70 milles marins à l\'est de Saint-Augustin, a indiqué la Garde côtière.\nIl était épuisé et emmené à la station aérienne de la Garde côtière à Clearwater, où son état de santé a été évalué. . Aucun blessé n\'a été signalé, a déclaré la Garde côtière. Une partie de ses efforts consistait à rétablir la paix dans le monde, mais il s\'est retrouvé pris dans le Gulf Stream, a déclaré le spécialiste des affaires publiques de la Garde côtière, Mark Barney. Les chances de se sortir du Gulf Stream étaient assez faibles. \nBaluchi n\'a pas pu être joint pour commenter, mais une vidéo publiée sur la page Facebook de Baluchi décrit ses projets de voyager dans la bulle à travers les Caraïbes, ainsi que ses autres exploits sportifs.\nL\'histoire de CNN.com a cité Baluchi disant qu\'il Il traverserait les États-Unis deux fois et ferait le tour de son périmètre une fois. Il a fui l\'Iran pour échapper à la persécution, a-t-il dit. Maintenant, il est citoyen américain.\n Reza n\'écoute personne, a déclaré Davis Hyslop, un homme d\'affaires et l\'un des partisans de Baluchi, dans l\'article de 2012. Il a ces ambitions démesurées auxquelles il se fixe. Il est un succès au-delà des attentes de quiconque. C\'est presque biblique. Mais il faut être un peu fou pour entreprendre une telle entreprise, non ? \n,,,Le baseball est aujourd\'hui le plus ancien sport de ligue majeure aux États-Unis. Avec la popularité croissante du football américain et du basket-ball, le baseball doit désormais partager la vedette avec ces sports. À la fin des années 1990, le baseball était en difficulté. La fréquentation était en baisse. Les sports de haut niveau qui comportaient une action rapide comme le football et le basket-ball volaient le tonnerre du baseball. C\'était avant que Barry Bonds et quelques autres cogneurs ne ramènent le jeu sur le devant de la scène.\n \nBonds a joué la première partie de sa carrière avec les Pirates de Pittsburgh, où il a connu un succès modéré. Après avoir été échangé aux Giants de San Francisco en 1992, Bonds a entamé une deuxième carrière qui l\'a mis sous les projecteurs pour les bonnes et les mauvaises raisons. À un âge où la plupart des joueurs commencent à décliner leurs performances en raison de leur âge avancé, Bonds a fait un home run sans précédent dans l\'histoire du jeu. mais, jouant pour les Giants, il a réussi un nombre impressionnant de 73 circuits en 2001. Cela a brisé un record qui durait depuis 40 ans. Bonds avait 38 ans à la fin de la saison 2001, et beaucoup se sont demandé comment il pouvait afficher de tels chiffres. La réponse à cette question est apparue des années plus tard lorsque Bonds a été accusé de prendre des drogues illégales améliorant la performance. Les obligations ont finalement battu le record d\'Henry Aaron pour la plupart des circuits dans une carrière, mais de nombreux puristes du baseball estiment que le record est entaché à cause de la consommation de drogues illégales. et la Major League Baseball a mis en place une politique de consommation de drogue depuis sa retraite. Il a été inculpé par un grand jury fédéral pour parjure pour avoir menti sur sa consommation de drogue, mais a réussi à conclure un accord pour ne pas aller en prison. De toute façon, son héritage est à jamais sali.,,,Les incendies sont destructeurs. Ils créent également des changements. En 1871, un incendie a brûlé la majeure partie de Chicago au sol. Beaucoup de gens pensent que le feu s\'est déclenché lorsqu\'une vache a renversé une lampe à gaz. La majeure partie de la ville à l\'époque était construite en bois. Ceci, combiné aux fameux vents de Chicago et à une sécheresse, a rapidement fait brûler la ville. L\'incendie a duré trois jours. 100 000 personnes se sont retrouvées sans abri et au moins 300 ont été tuées. Ce qui était étonnant, c\'est la rapidité avec laquelle la ville a été reconstruite, devenant finalement la troisième ville la plus peuplée des États-Unis.\n \nLe grand incendie de Boston de 1872 a provoqué des dégâts matériels de 73,5\xa0millions de dollars, plus que tout autre incendie de l\'histoire des États-Unis. La majeure partie du centre-ville de Boston et du quartier financier a brûlé dans l\'incendie qui a commencé dans le sous-sol d\'un entrepôt. 30 personnes sont mortes et des milliers ont perdu leur emploi et leur maison. Cependant, la ville a été reconstruite en deux ans. Il a commencé à appliquer les réglementations de construction à cause de l\'incendie.\nL\'incendie de l\'usine Triangle Shirtwaist de 1911 à New York a causé le quatrième plus grand nombre de décès dus à un accident industriel dans l\'histoire des États-Unis. 123 femmes et 23 hommes, tous des ouvriers du vêtement et pour la plupart des immigrants pauvres, sont morts de l\'incendie, de l\'inhalation de fumée, de chutes ou de sauts mortels. Le travail des enfants n\'était pas rare. Les plus jeunes victimes étaient deux jeunes filles de 14 ans qui travaillaient à l\'usine. Tant de personnes sont mortes parce que les propriétaires de l\'usine ont verrouillé les portes des escaliers et des sorties. C\'était pour empêcher les travailleurs de prendre des pauses et de voler. À la suite de l\'incendie et des nombreuses pertes en vies humaines, de nombreuses personnes ont commencé à protester contre les mauvaises conditions de travail. Le Congrès américain a adopté des lois améliorant les conditions de sécurité dans les usines. Les travailleuses ont également formé un syndicat pour lutter pour de meilleures conditions de travail.,,,Chaque jour de l\'an, la ville de Pasadena en Californie organise une célébration pour accueillir l\'année avec une série d\'événements qui comprend la Rose Parade. Le défilé a lieu au centre de la ville sur Colorado Blvd et comprend de nombreux chars, fanfares, équipes équestres, célébrités et lauréats. C\'est le défilé le plus populaire du pays. Il a donné naissance à plusieurs autres défilés du Nouvel An dans tout le pays. A Miami, c\'est l\'Orange Bowl Parade ; à la Nouvelle-Orléans, c\'est la Sugar Bowl Parade ; en Arizona, ça s\'appelle la Fiesta Bowl Parade. Cependant, la Rose Parade est la plus ancienne et la meilleure parade. De nombreuses personnes aiment camper dans des tentes ou dans des sacs de couchage la nuit précédente afin d\'obtenir les meilleurs endroits pour voir la parade des roses.\n \nPour braver le froid, certaines personnes apportent des radiateurs portables pour les garder au chaud. Il peut entrer dans les années 40 la nuit à Pasadena pendant l\'hiver, tant de gens s\'habillent chaudement pour l\'occasion. La nourriture est une autre grande caractéristique de la célébration de pré-défilé le soir du Nouvel An. Les gens apportent leurs grils portables pour cuisiner leurs repas extérieurs préférés directement sur le trottoir. L\'arôme riche de la nourriture grillée remplit l\'air nocturne et la plupart des gens sont prêts à partager leur nourriture avec d\'autres campeurs pour la nuit. C\'est amusant de goûter différents plats d\'autres grillades.\nAlors que la nuit se transforme en matin, il y a encore de l\'électricité dans l\'air. Les gens sont excités par le début du défilé, qui commence à 8 heures précises. Les chars sont magnifiquement décorés de fleurs, tandis que les danseurs et autres artistes s\'habillent de costumes colorés, dont certains sont faits juste pour le défilé. Les fanfares viennent des villes voisines, des lycées et des collèges. C\'est considéré comme un grand honneur de participer à la Rose Parade.,,,Bob savait que la maison qu\'il venait d\'acheter se trouvait dans un quartier historiquement difficile. Il y avait des antécédents de violence de gangs, y compris des fusillades, dans le quartier. C\'est probablement pourquoi la maison était si bon marché. Les anciens propriétaires de la maison l\'ont fait saisir. Ensuite, des nageurs immobiliers ont acheté la propriété à un prix très bas, l\'ont rénovée, ont augmenté le prix de vente et l\'ont vendue. Bob pensait que la maison était toujours une bonne affaire.\n \nAprès avoir emménagé, Bob a commencé à voir que la plupart des rénovations de la maison n\'étaient que cosmétiques. Alors que les flippers de la maison ont installé de nouveaux appareils en acier inoxydable dans la cuisine, ils n\'ont pas amélioré les travaux électriques. Cela signifiait que chaque fois que Bob faisait fonctionner le lave-vaisselle et la machine à café, toute l\'électricité de la maison était coupée. Bob a dépensé plus de 1 000 dollars pour faire recâbler la maison. Les nouveaux tapis installés par les nageoires de la maison ont été posés sur les planchers de bois franc d\'origine. Bob a décidé d\'arracher les affreux tapis de couleur gruau et de revernir les vieux planchers. C\'était 3\xa0000\xa0$ de plus.\nLa maison était une vieille maison en stuc qui devenait très chaude en été et froide en hiver. Les nageoires de la maison ont installé des fenêtres bon marché. Bob a embauché un entrepreneur pour installer de nouvelles contre-fenêtres à double vitrage et installer des ventilateurs de plafond. C\'était encore quelques milliers de dollars. Bob voulait rendre la maison éconergétique, alors il a loué des panneaux solaires qui ont été installés sur le toit. Cela a non seulement réduit sa facture d\'électricité mensuelle, mais a ajouté de la valeur à la maison.\nEn deux ans, tous les changements que Bob a apportés à la maison ont totalisé plus de 20\xa0$,000. Au cours de cette période, la valeur de sa maison a augmenté de près de 100 000 $. Bob a estimé qu\'il avait fait un bon investissement. Il savait qu\'en vendant la maison, il gagnerait plus d\'argent qu\'il n\'en aurait payé.\n,,,À 6 heures précises du matin, le son du réveil perce l\'air. Sandy se traîne hors du lit et commence sa routine matinale par une douche rapide. Après sa douche, elle prépare un petit déjeuner simple. Saisissant un bol propre du lave-vaisselle, elle y vide un paquet de flocons d\'avoine instantanés. Elle verse une tasse de lait, remue et chauffe le mélange au micro-ondes. En attendant que ses flocons d\'avoine cuisent, elle s\'habille pour la journée, en tenant compte du temps qu\'il fait.\n \nEntièrement habillée maintenant, elle retire ses flocons d\'avoine du micro-ondes, qui a maintenant juste assez refroidi à son goût. C\'est sa saveur préférée, pomme cannelle. Alors qu\'elle savoure chaque bouchée, elle se souvient de sa mère, qui s\'assurait toujours que Sandy mange un petit-déjeuner chaud avant de quitter la maison le matin.\nVérifiant l\'heure sur son téléphone, elle avale à la hâte les dernières bouchées de gruau avant de partir. . Alors qu\'elle s\'engage dans la rue, une rafale de vent repousse ses cheveux. Elle se couvre la tête avec la capuche de son manteau, serre l\'écharpe autour de son cou et se prépare pour sa marche jusqu\'à la gare.\nAprès s\'être installée dans son siège dans le train, les souvenirs de sa mère se réveillent à nouveau. Elle sort son téléphone et envoie des SMS\xa0: "Hé maman, merci pour l\'écharpe que tu m\'as tricotée. Elle m\'a été très utile ce matin. Je t\'aime toujours.",,,Richard Milhous Nixon était le 37e président des États-Unis et le seul un à démissionner de ses fonctions dans l\'histoire du pays. Né le 9 janvier 1913 à Yorba Linda, en Californie, Nixon était un membre du Parti républicain qui a été vice-président sous le président Dwight David Eisenhower dans les années 1950. Nixon s\'est présenté pour la première fois à la présidence en 1960, perdant face au très charismatique et jeune démocrate John F. Kennedy. L\'élection de 1960 a été l\'une des élections les plus serrées de l\'histoire. Kennedy est devenu l\'un des présidents les plus populaires de tous les temps.\n \nMalgré sa défaite face à Kennedy en 1960, Nixon a rebondi lors des élections de 1968 en battant le démocrate Hubert Humphrey dans une victoire écrasante. Le pays était en ébullition dans les années 60. Le conflit au Vietnam faisait rage, le pays était sous le choc des assassinats de John F. Kennedy et de son frère, le sénateur Robert Kennedy, et la victoire de Martin Luther King Jr. Nixon indiquait que le pays était prêt pour une direction conservatrice pendant ces années difficiles. fois.\nNixon a remporté les élections de 1972 contre le démocrate George McGovern dans une autre victoire écrasante, qui a semblé établir l\'héritage de Nixon en tant que grand président. Parmi ses réalisations figurait l\'ouverture du commerce avec la Chine et sa promesse de mettre fin au conflit au Vietnam. Cependant, Nixon finirait par démissionner de ses fonctions juste un an après sa victoire.\nIl a été découvert que le Parti républicain, à la connaissance de Nixon, avait secrètement enregistré le siège démocrate à l\'hôtel Watergate à Washington D.C. au cours des semaines précédant à l\'élection. Nixon a essayé de dissimuler le scandale, mais la pression était trop grande. Nixon a été contraint de démissionner en disgrâce plutôt que d\'être destitué de ses fonctions, ce qui signifie être démis de ses fonctions par le Congrès américain. Il est devenu le premier président à démissionner alors qu\'il était en poste. ,,, La fête des mères est une fête qui célèbre et honore les mères aux États-Unis. Elle est célébrée le deuxième dimanche de mai. Elle est devenue une fête officielle dans le pays au début du 20e\xa0siècle afin d\'honorer les mères dont les fils sont morts à la guerre.\n \nLa fête est célébrée de plusieurs façons. Les gens offrent aux mères des cadeaux comme des fleurs, des cartes et des bijoux pour les remercier de leur travail acharné. Un symbole populaire de la fête est l\'œillet. En effet, lorsque les vacances ont commencé aux États-Unis, les gens étaient encouragés à porter un œillet rouge si leur mère était vivante ou un œillet blanc si leur mère était décédée. Beaucoup de gens emmènent leur mère pour un repas spécial. En fait, la fête des mères est la date la plus populaire aux États-Unis pour que les gens sortent et mangent et on estime que les gens dépensent des milliards en repas et en cadeaux. La fête des mères est également le jour le plus populaire pour passer des appels interurbains aux États-Unis. C\'est le deuxième jour de cadeaux le plus populaire après Noël.\nDepuis sa création, la fête des mères a été critiquée pour être devenue très commerciale. En fait, la fondatrice de la fête américaine, Anna Jarvis, a commencé à protester contre la fête et a même été arrêtée pour avoir troublé la paix pour avoir manifesté. La propre mère de Jarvis a lancé la campagne pour établir une fête des mères pendant la guerre civile américaine.\nLes mères ne sont pas les seules personnes célébrées ce jour-là. Toutes les figures maternelles, y compris les grands-mères, les arrière-grands-mères, les belles-mères et les mères adoptives, sont honorées pendant les vacances. Dans les écoles, de nombreux élèves fabriquent des cadeaux spéciaux, notamment des cartes faites à la main. Bien que largement reconnue, la fête des mères n\'est pas une fête fédérale. De nombreux autres pays dans le monde ont leur propre version de la fête des mères. ,,, Le principal conseiller du président Bush pour la sécurité intérieure démissionne après 4 ans et demi de travail, a annoncé lundi la Maison Blanche.\nFran Townsend a servi plus de quatre ans en tant que conseillère à la sécurité intérieure.\nLa conseillère à la sécurité intérieure, Fran Townsend, a remis sa lettre de démission au président Bush le 6\xa0novembre et cherchera de nouvelles opportunités en dehors du gouvernement.\n Je vais simplement prendre un autre travail en faisant des journées de 20\xa0heures , mais cette fois dans le secteur privé, a déclaré Townsend, qui a passé 25 ans à travailler dans les forces de l\'ordre et le gouvernement.\nBush a salué le travail de Townsend lundi.\n Fran a toujours fourni des conseils avisés sur la meilleure façon de protéger le peuple américain de la menace terroriste, a déclaré Bush dans un communiqué. Nous sommes plus en sécurité aujourd\'hui grâce à son leadership. Le travail de \nTownsend\\, en tant que principal conseiller du président dans la lutte contre le terrorisme, consistait à identifier les groupes terroristes dans le monde entier, à évaluer leur menace et à trouver des moyens de suivre et de couper leur financement. Elle a déclaré que l\'expérience devrait également être utile dans le secteur privé.\nTownsend, mère de deux enfants, âgée de 6 et 12\xa0ans, a déclaré qu\'elle se pencherait d\'abord sur la prise de parole en public, l\'écriture et le travail au conseil d\'administration avant de poursuivre des opportunités dans la gestion des risques mondiaux. pour une grande multinationale ou une institution financière. Regardez comment Townsend envisage d\'utiliser ses compétences \nElle a dit qu\'elle parlait avec le président de son départ prévu depuis huit mois.\nLe nom de Townsend était apparu lors de la récente recherche d\'un nouvel avocat par le président général, mais elle n\'a pas été considérée. Elle a dit que le fait de ne pas avoir obtenu le poste n\'avait absolument rien à voir avec sa décision de partir. \nElle a dit que Bush voulait qu\'elle reste dans son administration.\n Ce fut une décision difficile car j\'ai aimé et je chérirai chaque minute de mon service, a-t-elle déclaré dans un e-mail. Ma famille a en fait préconisé que je reste et a toujours soutenu mon service, donc c\'était entièrement ma décision. \nTownsend fait partie du comité de recherche pour trouver un remplaçant d\'ici le début de l\'année prochaine. Le président nomme le poste de cadre supérieur, et il n\'a pas besoin de confirmation du Sénat. E-mail à un ami\nSuzanne Malveaux de CNN a contribué à ce reportage.\n,,,Angelina Jolie était sur scène jeudi mais n\'a pas joué. Au lieu de cela, elle a rendu hommage aux millions de personnes déracinées à travers le monde et rendues sans abri par la guerre.\nAngelina Jolie a déclaré que les pays d\'accueil peuvent apprendre de la façon dont les réfugiés survivent à leurs difficultés.\nAmbassadrice de bonne volonté de l\'ONU, Jolie était à Washington pour observer World Refugee Day, un événement annuel qui tombe le samedi cette année. La cérémonie poignante a été parrainée par l\'agence des Nations Unies pour les réfugiés.\nLes larmes ont coulé lorsque les réfugiés et les personnes déplacées ont partagé leurs expériences déchirantes et que les défenseurs ont raconté ce qu\'ils avaient vu dans les camps de réfugiés.\n Je crois que nous devons persuader le monde que les réfugiés ne doivent pas être simplement considéré comme un fardeau, dit-elle. Ce sont les survivants. Et ils peuvent mettre ces qualités au service de leurs communautés et des pays qui les abritent. \nJolie a parlé de ses visites dans des camps de réfugiés en Tanzanie et au Pakistan.\nElle a rencontré un garçon d\'une quinzaine d\'années dans un camp de réfugiés tanzanien qui, sans fauteuil roulant, a dû ramper depuis qu\'une balle dans le dos l\'a paralysé. Sa famille a été tuée, le laissant orphelin. Pourtant, quand il lui a parlé, il l\'a fait avec un sourire.\n Il avait cet esprit vraiment remarquable et incassable, a-t-elle déclaré à Anderson Cooper de CNN après l\'événement au National Geographic Museum.\n Je pense à lui et moi ne peut se plaindre de rien... il était reconnaissant et il n\'avait rien et souffrait de tout. \nLe garçon est décédé quelques mois après l\'avoir rencontré, a déclaré Jolie.\nSelon une enquête du Haut-Commissariat des Nations Unies pour les réfugiés sur les tendances des déplacements en 2008, il y a 42 millions de personnes déracinées - 15,2 millions étaient des réfugiés, 26 millions étaient des personnes déplacées à l\'intérieur du pays et 827\xa0000 étaient des demandeurs d\'asile.\nLes Afghans et les Irakiens constituaient près de la moitié de tous les réfugiés dans le monde. Un réfugié sur quatre venait d\'Afghanistan, et 69\xa0pays ont accepté l\'asile de réfugiés afghans, ont découvert des responsables de l\'agence.\nJolie a raconté sa rencontre avec une réfugiée afghane enceinte.\nElle vivait dans une petite maison en terre sans toit dans un camp de réfugiés abandonné au Pakistan. et n\'a pas pu partir avec les autres parce qu\'elle était trop avancée dans sa grossesse.\nLe Pakistan est aux prises avec la plus grande population de réfugiés au monde, environ 1,8 million, selon les Nations Unies. Le pays compte également plus de 2 millions de citoyens déplacés à l\'intérieur du pays, en grande partie à cause des combats entre l\'armée pakistanaise et les talibans depuis avril.\nAntonio Guterres, chef de l\'agence des Nations Unies pour les réfugiés, a expliqué le sort des personnes déplacées au événement.\nIl a déclaré que les personnes déplacées à l\'intérieur du pays sont confrontées au même sort -LSB- que les réfugiés -RSB-. Ils ont perdu la même chose que les réfugiés. Ils n\'ont pas les mêmes protections internationales accordées aux réfugiés. \nConflits comme tLes attaques au Pakistan, au Sri Lanka et en Somalie ont poussé davantage de personnes à fuir leur foyer, bien que le nombre de personnes déplacées ait diminué d\'environ 700\xa0000 en 2008, selon le rapport.\nDes combats récents dans l\'est de la République démocratique du Congo et en Somalie l\'année dernière a porté le déplacement total dans chacun à 1,5 million et 1,3 million, respectivement. Plus de 2 millions de personnes déplacées à l\'intérieur du pays se trouvaient dans la région soudanaise du Darfour, selon le rapport. au milieu. Elle a pris la parole lors de l\'événement et a décrit son calvaire.\nMapendo, son mari et ses sept enfants ont été jetés dans un camp de la mort, où son mari a été exécuté et ses enfants affamés, a-t-elle dit en larmes.\nPendant qu\'elle était en prison, elle a appris qu\'elle était enceinte. Elle a accouché en silence, sur un sol en béton, a déclaré Mapendo. À sa grande surprise, elle a eu des jumeaux, et à la surprise de ses ravisseurs, elle a donné leur nom à ses nouveau-nés. Peu de temps après son geste envers les commandants de la prison, sa famille a été libérée après leur calvaire de 16 mois.\nMapendo, nommée Héros de CNN en octobre, a créé Mapendo International, une organisation à but non lucratif dédiée à l\'identification, au sauvetage et à la réinstallation des réfugiés en danger. Elle a été nommée humanitaire de l\'année lors de l\'événement par l\'agence des Nations Unies.\n Je veux encourager les réfugiés à ne pas abandonner, a-t-elle déclaré. Abandonner n\'est pas une option. \nKhadijah Rentas, Shalina Wadhwani et Paul Courson de CNN ont contribué à ce rapport\n,,,Aucun nom n\'évoque plus de controverse aux États-Unis qu\'O.J. Simpson. Il a été sous les projecteurs américains pendant des décennies depuis qu\'il était un joueur de football vedette à l\'Université de Californie du Sud, une université universitaire de premier plan qui est principalement connue pour sa riche et riche tradition de football.\n \nL\'USC est l\'un des plus grands endroits pour que les futurs footballeurs professionnels puissent débuter. Simpson le savait lorsqu\'il a été transféré à l\'école au milieu des années 1960 du San Francisco Community College. À l\'USC, Simpson a trouvé la gloire. Sa fortune devait venir plus tard. Il a aidé l\'université à atteindre la place convoitée du Rose Bowl en 1968, après une saison 1967 exceptionnelle où il a mené la nation dans les verges au sol. Sa course de 64 verges contre son rival UCLA, dans ce que beaucoup de gens disent être le plus grand match de l\'histoire du football universitaire, est ce dont sont faites les légendes. Simpson a ensuite joué dans la Ligue nationale de football, où il a acquis plus de renommée et de fortune. À la retraite, il s\'est tourné vers une carrière d\'acteur où il a joué dans de nombreuses publicités, et éventuellement des films.\nMalheureusement, la vie de Simpson a pris une tournure tragique des années plus tard. Il a été accusé du meurtre de sa femme et de son amie au début des années 90. Son procès a été l\'un des événements les plus regardés de l\'histoire de la télévision. Simpson a été acquitté du meurtre, en grande partie grâce à sa "Dream Team" d\'avocats très chers. De nombreuses personnes pensaient que Simpson avait commis le meurtre et s\'en était sorti.\nSimpson a ensuite été condamné par un tribunal du Nevada pour plusieurs infractions et purge actuellement une peine de prison. Beaucoup pensent que cette condamnation était une rétribution pour ses crimes antérieurs.,,,\ufeffLa dernière fois qu\'elle est montée sur scène, le prototype du téléphone portable subissait ses premiers essais. Trente-cinq ans plus tard, alors qu\'elle se produit à nouveau, la chanteuse Kate Bush est confrontée à un monde différent. \nAlors que la plupart des concerts sont désormais éclairés par des téléphones et des tablettes, Bush prend position contre les fans qui regardent ses émissions à travers le voile numérique d\'un écran. \nAvant sa série de concerts très attendue au Hammersmith Apollo de Londres, Bush a publié une déclaration appelant ses fans à éteindre leurs téléphones portables lors de ses concerts. \nBush a écrit sur son site Web\xa0: "J\'ai une demande pour vous tous qui venez aux spectacles. Nous avons délibérément choisi un décor de théâtre intimiste plutôt qu\'une grande salle ou un stade. Cela signifierait beaucoup pour moi si vous vouliez bien vous abstenir de prendre des photos ou de filmer pendant les spectacles. \n"Je veux vraiment avoir un contact avec vous en tant que public, pas avec des iPhones, des iPads ou des appareils photo. Je sais que c\'est beaucoup demander, mais cela nous permettrait de partager l\'expérience ensemble. \nAvec son amour du théâtre et des costumes somptueux, la volonté de Bush d\'empêcher les fans de télécharger des images granuleuses sur YouTube pourrait également être une tentative de garder le spectacle une surprise pour les milliers de fans qui ont acheté des billets pour les 22 dates qu\'elle joue. \nBush n\'est pas le premier à dénoncer l\'effet néfaste de la présence de téléphones lors de concerts, de nombreux artistes réprimandant leurs fans pour avoir expérimenté la musique en direct à travers le filtre d\'un écran. \nLe leader des Who, Roger Daltrey, a récemment déclaré qu\'il était "bizarre" que les gens ne pensent pas à la série lorsqu\'ils étaient allés à une représentation et se concentraient sur l\'écran plutôt que sur l\'artiste sur scène. \nIl a dit\xa0: "Je suis désolé pour eux, je suis vraiment désolé pour eux. Regarder la vie à travers un écran et ne pas être dans le moment t totalement - si vous faites cela, vous êtes à 50\xa0%, n\'est-ce pas\xa0? C\'est étrange. Je trouve ça bizarre." \nEn 2013, Beyoncé a réprimandé l\'un de ses fans lors d\'un concert pour le tournage. "Tu ne peux même pas chanter parce que tu es trop occupé à enregistrer", lui a dit Beyoncé. "Je suis juste devant toi, bébé. Tu dois saisir ce moment. Pose cette foutue caméra ! \nLe débat autour de la présence de téléphones lors d\'événements en direct ne se limite pas à la musique, les fans de sport s\'exprimant également sur le sujet. Récemment, les supporters néerlandais du PSV Eindhoven ont lancé une protestation véhémente contre l\'introduction du wi-fi dans leur stade, brandissant des banderoles avec des messages comme « Pas de wi-fi. Soutenez l\'équipe », « Vous pouvez vous asseoir à la maison » et « Restez unis », tandis que Manchester United a également dit aux fans de laisser leurs « gros appareils électroniques » à la maison, interdisant de filmer sur tablettes cette saison. \nJarvis Cocker a déjà critiqué les utilisateurs de téléphones dans le public pour l\'avoir rendu "fou lors des concerts", ajoutant\xa0: "Cela semble stupide d\'avoir quelque chose qui se passe devant vous et de le regarder sur un écran plus petit que la taille d\'une cigarette. paquet." \nJohnny Marr a déclaré en 2013 que cela signifiait que les fans avaient raté l\'expérience sensorielle de la musique live dans leur désespoir de documenter l\'événement pour plus tard. \n"Se tenir debout et simplement le regarder à travers votre téléphone est une opportunité complètement gâchée. Tu sais, je ne veux pas être méchant mais je pense que tu devrais raccrocher ton téléphone parce que tu es juste un idiot, vraiment. Profitez simplement du concert », a-t-il déclaré. \n"C\'est l\'une des choses à propos des concerts - c\'est prendre en compte ce qui se passe avec les gens autour de vous et, le regarder sur un petit écran, c\'est une perte de temps." \nLes Yeah Yeah Yeahs ont eu recours à l\'installation d\'une pancarte dans l\'une de leurs salles, suppliant les fans d\'empocher leur technologie. Il disait: «Veuillez ne pas regarder l\'émission à travers un écran sur votre appareil intelligent / appareil photo. Rangez-le par courtoisie envers la personne derrière vous et envers le groupe. \nIl s\'est même infiltré dans le monde de la musique classique, l\'un des plus grands pianistes du monde ayant surpris les spectateurs en juin 2013 lorsqu\'il a quitté la scène en trombe parce qu\'un fan filmait sa performance sur un smartphone. Krystian Zimerman est revenu quelques instants plus tard et a déclaré : "La destruction de la musique à cause de YouTube est énorme." \nMais Sam Watt de Vyclone, une application pour téléphone qui encourage le public à filmer lors de concerts, puis rassemble les images pour créer une vidéo participative de l\'événement, a déclaré que ces artistes menaient une bataille perdue d\'avance et que filmer lors de concerts améliorait plutôt que nui à l\'expérience. \n"Le tournage des fans fait désormais partie de l\'expérience du concert - c\'est juste un fait - donc nous prenons cette séquence que les gens filment lors des concerts via l\'application, ils la téléchargent sur l\'application, puis elle leur revient mélangée avec tous les autres qui filmaient. Vous vous retrouvez avec un contenu vraiment fantastique », a-t-il déclaré. \n"Notre réflexion générale est que filmer des concerts ajoute à l\'expérience, plutôt que de lui enlever et je pense que si Kate Bush venait prendre une tasse de thé, nous pourrions avoir une discussion vraiment intéressante à ce sujet et nous pourrions être capable de gagner sa manche », a-t-il ajouté. "Savoir que les gens vont filmer et veulent ces souvenirs est vraiment important car cela va probablement les frapper sur la tête à l\'avenir s\'ils disent à tout le monde qu\'ils ne peuvent pas filmer. Vous devez l\'embrasser. ,,,Lorsque les adjoints du shérif ont arrêté Casey Anthony en 2008 pour la mort de sa fille de 2 ans, peu de gens auraient prévu que cela se terminerait ainsi\xa0: quelques jours seulement après son procès, des geôliers organisant une libération clandestine lançant l\'Orlando femme dans un monde où son histoire pourrait la rendre riche ou mettre sa vie en danger.\nMais c\'est précisément le scénario qui se jouera dimanche quand Anthony finira de purger sa peine pour quatre délits de police trompeuse.\nAnthony , 25 ans, a été acquittée le 5 juillet des accusations de meurtre et de négligence envers les enfants dans la mort de sa fille, Caylee, dont les restes squelettiques ont été retrouvés en 2008 dans un champ boisé. Elle a été reconnue coupable de quatre chefs d\'accusation d\'avoir trompé la police et a été condamnée à quatre ans de prison.\nAprès avoir purgé près de trois ans de prison en attendant son procès, elle a été libérée ce week-end.\nLa fin brutale d\'une affaire pénale qui a parfois bouleversé le nation a provoqué de la colère et de la répulsion ainsi que des dons en espèces sur son compte de prison. Cela a également conduit à la spéculation que le sujet polarisant de la page Facebook I Hate Casey Anthony - et la source de colère de ses plus de 40 000 fans - pourrait changer de nom et d\'apparence et déménager quelque part loin.\n Si ses avocats font ce qu\'il faut et font leur travail, ils vont devoir lui expliquer qu\'il y a une vraie haine contre elle, qu\'il y a eu des menaces de mort, qu\'elle ne peut pas se promener parmi la population , a déclaré Sunny Hostin, contributeur juridique de HLN. Cela ne va pas se produire. \nQuoi qu\'elle finisse par faire, elle doit d\'abord sortir de prison.\nPeu de détails Des informations ont été rendues publiques sur la manière dont Anthony sera libéré de la prison du comté d\'Orange, où elle est détenue depuis son arrestation en octobre\xa02008. de la prison tard vendredi ou tôt samedi et leur couper l\'accès extérieur pendant qu\'ils attendent la libération d\'Anthony.\nLa date exacte à laquelle Anthony sera libéré n\'a pas été rendue publique.\nC\'est en partie probable en raison des émotions intenses que la libération d\'Anthony a générées.\n Je sais que c\'est mauvais Dieu me pardonne mais j\'espère que quelqu\'un efface ce sourire narquois de son visage. Avec un camion mack, une affiche sur la page Facebook I Hate Casey Anthony a écrit.\nUne autre affiche, faisant référence à des rumeurs d\'entretiens lucratifs, d\'offres de livres et de films, a déclaré que la seule chose qu\'elle paierait pour voir ou lire serait à propos de Casey Anthony\\ est le cadavre. \nLa page présente la photo d\'Anthony avec une cible dans le mille sur son front sous le titre Hé, Casey, je ferais attention si j\'étais toi, dis juste. \nUne femme de l\'Oklahoma a déclaré qu\'elle avait déjà fait face au genre de colère qu\'Anthony pourrait rencontrer lorsqu\'elle ne serait plus derrière les murs de protection de la prison du comté d\'Orange.\nSammy Blackwell a déclaré à l\'affilié de CNN KOTV qu\'une femme qui l\'avait prise pour Anthony en juillet 8 a percuté sa voiture deux fois, la retournant.\n Elle a dit que j\'essayais de blesser des bébés, que je tuais des bébés et qu\'elle allait l\'arrêter avant que cela ne se reproduise, a déclaré Blackwell à la station.\nEn l\'occurrence, Blackwell a une fille nommée Caylee aussi, mais c\'est la fin des similitudes. Elle dit qu\'elle ne ressemble vraiment pas beaucoup à Anthony et s\'inquiète pour les femmes qui le font. Le shérif Jerry Demings a déclaré mardi que les enquêteurs évaluaient les menaces à la sécurité d\'Anthony. Bien qu\'il ait déclaré que le département n\'était au courant d\'aucune menace crédible contre sa vie, c\'est une préoccupation qu\'il avait clairement à l\'esprit.\n Personne n\'a le droit de se faire justice lui-même, a-t-il déclaré. Casey Anthony a eu sa journée au tribunal et le jury a pris une décision. J\'espère que les gens reculeront et ne sortiront pas et ne commettront pas un autre crime. \nC\'est également une préoccupation pour ses avocats.\n Moi-même et d\'autres membres de l\'équipe sommes inquiets pour sa sécurité, tout à fait, a déclaré l\'un de ses avocats, Dorothy Sims, à HLN.\nCe qu\'Anthony fera après sa libération n\'est pas clair.\n Eh bien, si je savais à ce stade, je suis sûr que vous pouvez comprendre que je ne vous le dirais pas, a déclaré Sims. Je ne crois pas que cela ait été résolu. Mon espoir pour elle serait qu\'elle serait laissée seule et que sa vie privée serait respectée. \nHostin a déclaré sur CNN qu\'elle avait entendu dire qu\'Anthony se cacherait, vivrait sous un nom d\'emprunt ou subirait une chirurgie plastique.\n Mais je pense que nous allons entendre son histoire parce que des gens lui ont déjà offert un million de dollars pour son histoire, a-t-elle dit.\nAnthony a également encore des problèmes juridiques à régler.\nSon équipe criminelle fait appel de ses condamnations pour avoir induit la police en erreur, et elle est poursuivie dans le cadre de deux actions distinctes devant un tribunal civil. L\'un est déposé par une femme portant le même nom qu\'Anthony a donné aux enquêteurs comme nom de la nounou de sa fille. L\'autre implique un groupe de recherche qui veut qu\'Anthony rembourse les dépenses qu\'il a engagées à la recherche de Caylee.\nAnthony peut se voir proposer de l\'argent pour des offres de livres et de films, mais une offre ne sera pas sur la table.\nLe fondateur de Playboy, Hugh Hefner, a déclaré à CNN \'s Piers Morgan que le magazine n\'offrira pas de séance photo à Anthony.\n Je ne récompenserais pas quelqu\'un comme ça pour ce qui s\'est passé, a déclaré Hefner.\nL\'avocat J. Cheney Mason, qui a dit un jour qu\'il pensait qu\'Anthony était une petite-fille, dit qu\'il ne sait pas ce que la vie réserve à son client, mais qu\'il a de l\'espoir.\n Elle n\'a que 25 ans. Dans dix ans, espérons-le, elle aura une certaine stabilité dans sa vie et peut-être un mari, et ils pourront être quelque part dans le Montana et recommencer, a-t-il dit.\n,,,\ufeffDans le style typique de mauvais petit ami, Dan Sullivan était en retard pour le petit-déjeuner avec le Guardian parce qu\'il s\'est fait arrêter sur sa moto. \nSullivan travaille trop, dit-il. Il manque des dates de dîner. Il oublie de faire des cadeaux. \nEt donc, comme apparemment tout le monde dans la Silicon Valley, le capital-risqueur de 27 ans a mis au point une start-up pour y remédier\xa0: BetterBoyfriend.me, un service qui envoie des copines et des épouses un cadeau chaque mois pour environ 70\xa0$. \nSullivan, qui travaille dans la finance, a discrètement testé le programme et a recruté environ 350 petits amis générant environ 17\xa0000\xa0$ de revenus chaque mois. La plupart des membres, dit-il, font partie de sa collection d\'amis de Harvard et d\'investisseurs : des capital-risqueurs, des fondateurs et des employés d\'entreprises comme Apple, Google et Facebook, dont les copines ont reçu des cadeaux de Sullivan l\'année dernière. \nChaque mois, les membres de Sullivan choisissent parmi une liste de sept cadeaux possibles (chocolat, services à thé, manucure). Le colis est envoyé au petit ami sans marque ni étiquette. \nIl teste ses idées en achetant des publicités Facebook juste pour voir quel genre de traction elles obtiennent. BetterBoyfriend.mea décollé. "BetterBoyfriend a frappé quelque chose", a déclaré Sullivan. "C\'est un point douloureux que tout le monde a." \nBetterBoyfriend.me n\'était pas la première idée de Sullivan. Pendant quelques jours, il a pensé à un argumentaire sur le partage de chiens\xa0: "Je vois tous ces riches qui ont l\'air de misérables chiens de promenade et j\'ai réalisé\xa0: un chien n\'a de valeur que 5\xa0% du temps", a-t-il déclaré. "Alors, et si nous avions un gars qui garderait les chiens et que d\'autres personnes pourraient les louer?" \nUne autre start-up, appelée InvisibleBoyfriend, permet aux utilisateurs "de concevoir et de construire un partenaire social crédible qui est tout ce que vous voulez et qui est toujours disponible pour parler, quand vous le souhaitez". Le résultat final est un "petit ami" qui peut envoyer des SMS avec vous toute la journée. \nLe fondateur Kyle Tabor l\'a lancé pour donner aux utilisateurs un faux petit ami pour dissuader les avances indésirables, mais a constaté que les gens se sont attachés à leurs partenaires algorithmiques\xa0: "Beaucoup plus d\'utilisateurs recherchent la compagnie par la conversation plutôt que la "preuve" externe d\'une relation pour obtenir les gens sur leur dos. \nPour Sullivan et Tabor, la surprise a été les vraies relations qui se sont nouées avec leurs clients. Sullivan dit qu\'il a commencé à se voir comme une sorte de consultant en relations pour les petits amis. \nSullivan admet qu\'il a fait des erreurs. Au début, les colis comprenaient des reçus portant son nom, Dan Sullivan. "L\'un des petits amis m\'a écrit et m\'a dit\xa0: "Écoute, mec, elle n\'est pas en colère, mais Cynthia l\'a découvert". \nEnviron 50\xa0% des femmes BetterBoyfriend.me livrent des colis pour connaître son implication dans leurs relations\xa0: "C\'est corrélé avec l\'âge. Je pense qu\'après avoir été marié pendant un certain temps, vous ne gardez pas beaucoup de secrets. \nEt au fil de l\'année, le jeune fondateur dit qu\'il a très bien connu les petits amis. Ils ont même fait une livraison dans une salle d\'accouchement d\'un hôpital. La clé, a-t-il dit, est de se rappeler que sa relation est avec le petit ami. \nLorsqu\'il a lancé son entreprise pour la première fois, il a attaché des étiquettes indiquant BetterBoyfriend.me à des bouquets de fleurs et s\'est rendu à Dolores Park, un lieu de détente populaire dans le quartier de Mission, très fréquenté par la technologie et la jeunesse. \n"J\'ai cherché des couples et j\'ai donné le bouquet à la fille, mais cela n\'a pas plu au petit ami. Comme, pas du tout. Cela avait du sens, cependant. C\'était cool », a-t-il déclaré. "Alors, j\'ai reconsidéré et j\'ai commencé à les distribuer aux petits amis." ,,,La Californie est le plus grand État des États-Unis d\'Amérique. Il abrite plus de 10 % de la population totale du pays. Il possède également le plus grand système éducatif du pays. Le système collégial et universitaire est divisé en quatre parties. Ces parties sont le système de l\'Université de Californie, le système de l\'Université d\'État de Californie, le système des collèges communautaires et les établissements privés.\n \nLe système UC compte plus de 230\xa0000\xa0étudiants sur ses 10\xa0campus. Il est considéré comme le système le plus prestigieux de l\'État. Il comprend les meilleures universités comme UC Berkeley et l\'Université de Californie à Los Angeles (UCLA). Les deux écoles se classent parmi les 25 meilleures universités du pays, avec Berkeley dans le top 10. Un diplôme du système UC est très recherché et les normes académiques sont plus élevées que les autres systèmes. Elles sont également plus chères.\nLes universités d\'État de Californie comptent 23\xa0universités dans leur système. C\'est un système diversifié qui comprend deux universités polytechniques et une université maritime. Ces écoles proposent un enseignement professionnel et maritime, mais proposent également des programmes académiques traditionnels. Le système d\'inscription est d\'environ 450\xa0000\xa0étudiants, avec environ 45\xa0000\xa0membres du corps professoral.\nLe système des collèges communautaires de Californie est le plus grand système d\'enseignement supérieur de l\'État et du monde. Il compte 112 campus et dessert environ 2,4 millions d\'étudiants. Il propose des diplômes de transfert, une formation professionnelle et des programmes de diplômes d\'associés.\nLa Californie abrite également un grand nombre d\'écoles privées. Ces écoles n\'ont aucun lien avec le système scolaire public, mais certaines acceptent des étudiants transférés des collèges communautaires. Les écoles les plus remarquables et les plus prestigieuses de l\'État sont l\'Université de Stanford, à Palo Alto, et l\'Université de Californie du Sud, à Los Angeles.,,,Le président Obama a réuni lundi tous les membres confirmés de son cabinet pour la première fois en tant que président et les a mis au défi de réduire de 100\xa0millions de dollars au cours des 90\xa0prochains jours.\nLe président Obama rencontre lundi son cabinet, y compris le secrétaire à la Défense Robert Gates, à droite.\nLes agences devront indiquer comment elles ont économisé sur les dépenses à la fin de la période.\nLe gouvernement fédéral Le gouvernement a un déficit de confiance en ce qui concerne le peuple américain, a déclaré Obama à la Maison Blanche.\n Nous devons gagner leur confiance. Ils doivent être sûrs que leur argent est dépensé à bon escient. \nLe décret fait partie de l\'engagement d\'Obama d\'aller ligne par ligne dans le budget pour réduire les dépenses et réformer le gouvernement, a déclaré un haut responsable de l\'administration.\nUn officier potentiel du Cabinet était absent de la réunion - la gouverneure du Kansas, Kathleen Sebelius , la santé et l\'humanité n secrétaire désigné des services. Le Sénat n\'a pas voté sur la confirmation de Sebelius au poste.\nObama a fait sa demande d\'économies alors que la Chambre des représentants et le Sénat rentraient de vacances cette semaine, prêts à commencer à concilier leurs versions de la résolution budgétaire de l\'exercice 2010. La demande de budget du président est de 3,67 billions de dollars. Regardez comment l\'administration espère réduire les coûts \nDans le contexte du budget fédéral, 100\xa0millions de dollars d\'économies représentent un petit montant, ont déclaré les critiques.\n Tout montant d\'économies est évidemment le bienvenu, a déclaré le chef de la minorité au Sénat, Mitch McConnell, R-Kentucky. . Mais -LSB- 100 millions de dollars est -RSB- à peu près le montant moyen que nous dépenserons chaque jour pour couvrir les intérêts du plan de relance que nous avons adopté plus tôt cette année. \nLe secrétaire de presse de la Maison Blanche, Robert Gibbs, a insisté sur le fait que les Américains ordinaires apprécieraient néanmoins l\'effort d\'épargne.\n Il n\'y a qu\'à Washington, D.C., 100 millions de dollars, ce n\'est pas beaucoup d\'argent, a déclaré Gibbs. C\'est d\'où je viens. C\'est là que j\'ai grandi. Et je pense que c\'est pour des centaines de millions d\'Américains. \nL\'administration soutient également que l\'ordonnance a signalé une importante démonstration de responsabilité budgétaire.\n Aucune de ces économies ne résoudra à elles seules nos problèmes budgétaires à long terme, a déclaré Obama. Mais pris ensemble, ils peuvent faire une différence et ils envoient le signal que nous prenons au sérieux la façon dont le gouvernement fonctionne. \nLa Maison Blanche a donné des exemples de la façon dont les agences ont déjà commencé à prendre des mesures de réduction des coûts, notamment\xa0:\nSuzanne Malveaux de CNN a contribué au rapport.\n,,,Les restes de la tempête tropicale Lee se sont installés au milieu de l\'Atlantique et Le nord-est mercredi, provoquant des avertissements de fortes pluies et d\'inondations soudaines dans des régions allant de Washington à l\'ouest de la Nouvelle-Angleterre, a déclaré le National Hurricane Center. 2 à 6\xa0pouces supplémentaires prévus - et dans des cas isolés jusqu\'à 10\xa0pouces.\nCela s\'ajoute aux fortes précipitations qui sont déjà tombées dans la région alors que la tempête, qui a explosé depuis le golfe du Mexique, s\'est heurtée à un système de basse pression extratropical centré sur les montagnes centrales des Appalaches et bloqué.\n Ces pluies peuvent provoquer des crues éclair et des glissements de terrain potentiellement mortels, a déclaré le centre de prévision.\nUn avertissement de crue éclair pour le district de Columbia reste en vigueur ti ll minuit ce soir, avec des rafales de vent dommageables et des orages violents également possibles. Un communiqué de presse de l\'Agence de la sécurité intérieure et de la gestion des urgences de D.C. a exhorté les gens à éviter de conduire dans la mesure du possible, à conduire très prudemment lorsque vous devez conduire et à rester attentif à l\'accumulation d\'eau que vous ne pouvez pas traverser. \nPlus tôt mercredi, la tempête a provoqué une série de retards du trafic aérien dans le nord-est. Plusieurs vols ont été retardés de plus d\'une heure dans la région métropolitaine de New\xa0York et jusqu\'à 55\xa0minutes à Boston et à Philadelphie.\nLa prévision était une mauvaise nouvelle pour le Vermont gorgé d\'eau et le nord de New\xa0York, qui pourraient recevoir encore 1 à 3\xa0pouces de pluie avec jusqu\'à 4 pouces possibles à certains endroits d\'ici mercredi soir. Nous sommes vraiment préoccupés par cela. \nDes ordures sont restées entassées à l\'extérieur de la maison de Stewart après que l\'inondation d\'Irene a détruit son niveau inférieur, a déclaré WCBS. Stewart n\'a pas eu d\'électricité depuis plus d\'une semaine, et elle a peur que de nouvelles pluies inondent à nouveau sa maison et la maintiennent dans le noir encore plus longtemps.\n Lorsque tout le monde est sur le point de se rétablir, nous sommes se faire inonder à nouveau, a déclaré Stewart. Mercredi à Becket, Massachusetts -LRB- 8,11 pouces -RRB-\xa0; Rockaway, New Jersey -LRB- 8,43 -RRB-\xa0; Phillipsburg, New Jersey -LRB- 7,9 -RRB-\xa0; et Forks, Bethléem et Douvres, Pennsylvanie -LRB- un peu plus de 7 pouces -RRB-.\nLee a laissé au moins quatre morts en traversant le sud-est, déversant jusqu\'à 12 à 15 pouces de pluie dans de nombreux endroits.\nÀ Gwinnett Comté de Géorgie, juste à l\'extérieur d\'Atlanta, les pompiers ont déclaré mardi avoir trouvé le corps d\'un homme qui s\'était noyé dans un ruisseau gonflé par la pluie près de Norcross.\nDans le comté de Baldwin, en Alabama, la police a déclaré ne plus croire un garçon de 16\xa0ans disparu est vivant. L\'adolescent a été vu pour la dernière fois sur une plage près de Gulf Shores dimanche.\nUn décès par inondation a également été signalé dans le nord-est rural du Mississippi, où une personne s\'est noyée après que les eaux de crue ont emporté un véhicule dans le comté de Tishomingo, ont déclaré des responsables des urgences.\nEt une femme a été frappée. par un arbre et tué à Chattanooga, Tennessee, tôt mardi.\nSarah Dillingham de CNN a contribué à ce rapport.\n,,,-LRB- CNN -RRB- Arsenal a gardé ses minces espoirs de gagner cette saison en anglais Le titre de Premier League en vie en battant la relégation menaçait Burnley 1-0 à Turf Moor.\nUn but en première mi-temps de l\'international gallois Aaron Ramsey a suffi à séparer les deux équipes et à assurer la deuxième place d\'Arsenal.\nPlus important encore, il a amené le club du nord de Londres à moins de quatre points de la première place de Chelsea, les deux clubs devant jouer la semaine prochaine.\nMais Chelsea a deux matchs en main et affrontera les humbles Queens Park Rangers dimanche, une équipe qui lutte elle-même contre la relégation.\\ nBonne forme\nArsenal est en superbe forme depuis le début de l\'année, transformant ce qui semblait être une autre saison médiocre luttant pour assurer la quatrième place - et avec elle la qualification en Ligue des champions - en une où ils ont au moins une chance de gagner le titre.\nAprès avoir pris les devants, Arsenal a rarement semblé en danger de concéder, montrant davantage le pragmatisme du milieu de terrain incarné par Francis Coquelin, qui a également joué un rôle crucial dans le but.\n Il a été absolument constant dans le qualité de son travail défensif, l\'entraîneur d\'Arsenal, Arsène Wenger, a déclaré à Sky Sports après le match lorsqu\'il a été interrogé sur la contribution de Coquelin à la course actuelle d\'Arsenal. a regardé le jeune Français vers une position de milieu de terrain plus défensive.\n C\'était un joueur qui était avec nous depuis sept ans, de 17 ans, il n\'en a plus que 24, a expliqué Wenger.\n Parfois, il faut être patient. Je suis très content pour lui car il a fait preuve d\'une grande force mentale. \nMaintenant, tous les regards seront tournés vers l\'affrontement de la semaine prochaine entre Arsenal et Chelsea, qui décidera probablement du titre.\n Ils ont les matchs en main, a déclaré Wenger, minimisant les aspirations au titre de son club.\n Mais nous Je vais continuer et c\'est pourquoi la victoire était si importante pour nous aujourd\'hui. \nCombat de relégation\nEn attendant, ce fut une bonne journée pour les équipes en bas de la ligue.\nAston Villa a poursuivi sa bonne forme depuis la nomination de l\'entraîneur Tim Sherwood avec une victoire 1-0 sur Tottenham, qui a viré Sherwood la saison dernière.\nL\'international belge Christian Benteke a marqué le seul but du match, son huitième en six matches, pour assurer trois points vitaux pour donner un répit au club des Midlands.\nUn autre club des Midlands regardant par-dessus son épaule est West Brom, qui a concédé un but dans le temps additionnel pour perdre 3 -2 contre Leicester City.\nMais ce fut une journée horrible pour l\'ancien entraîneur international néerlandais de Sunderland, Dick Advocaat, qui a vu son équipe perdre 4-1 à domicile contre l\'équipe en forme Crystal Palace.\nInternational de la République démocratique du Congo Yannick Bolasie a réussi le tout premier tour du chapeau de Crystal Palace en Premier League pour s\'assurer une victoire facile. \nLe premier graphique à secteurs montre comment les gens passent leur temps sur les smartphones, tandis que le second graphique à secteurs illustre le temps passé sur les tablettes. Pour les deux types d\'appareils, l\'utilisation la plus importante concerne les jeux, mais les chiffres diffèrent grandement. 57% du temps passé sur une tablette est consacré à jouer à des jeux, alors que seulement 35% du temps passé sur un smartphone y est consacré. En revanche, les utilisateurs de smartphones passent 29\xa0% de leur temps sur leur gadget à accéder à des sites de réseaux sociaux, contre seulement 15\xa0% du temps passé sur la tablette à la même activité.\nLa troisième utilisation la plus populaire de la tablette est le divertissement, les utilisateurs y consacrant 13\xa0heures. % du temps passé sur leur tablette à regarder des vidéos et à écouter de la musique. Les utilisateurs de smartphones, cependant, ne consacrent que 8\xa0% du temps de leur smartphone à ces divertissements, préférant passer 20\xa0% de leur temps sur leur téléphone à accéder aux utilitaires. Il peut s\'agir de cartes, d\'informations météorologiques et de calculatrices.\nIl existe une nette différence dans la façon dont les gens utilisent leurs smartphones et leurs tablettes. En général, alors que les tablettes sont davantage utilisées pour les jeux et d\'autres formes de divertissement, les smartphones semblent être l\'option préférée pour les tâches ainsi que la communication avec le monde qui nous entoure.,,,Arnold Schwarzenegger est l\'une des célébrités les plus reconnaissables du monde. Les États-Unis d\'Amérique. La star d\'origine autrichienne est arrivée aux États-Unis à la fin des années 1960 alors qu\'il était une star internationale du circuit de musculation. Depuis lors, Schwarzenegger a eu plusieurs carrières en tant qu\'acteur, politicien et philanthrope.\n \nSchwarzenegger a commencé la musculation à l\'âge de 15 ans. À l\'âge de 20 ans, il avait remporté le prestigieux titre de M. Univers. L\'honneur professionnel de la musculation est décerné chaque année, mais ce n\'était pas son seul titre de musculation sur son CV. Il a remporté sept titres de M. Olympia au cours de sa carrière de culturiste. De plus, il a écrit plusieurs livres sur le sujet. Sa carrière de culturiste s\'est étendue de 1965 à 1980.\nSa carrière cinématographique a commencé en 1970 lorsqu\'il a joué dans un film peu connu intitulé Hercules à New York. Cependant, à partir de ces débuts, Schwarzenegger a cultivé une longue et fructueuse carrière en tant que star de cinéma d\'action à Hollywood. Ses films sont parmi les films les plus populaires jamais diffusés aux États-Unis. Son rôle le plus reconnaissable était sans doute celui de Terminator dans la série de films du même nom. Après une longue et fructueuse carrière cinématographique, Schwarzenegger, qui s\'est toujours intéressé à la politique, a commencé une voiture politique eer qui a culminé avec son élection à deux mandats en tant que gouverneur de la Californie.\nSchwarzenegger a été élu pour la première fois gouverneur en octobre 2003, fonctionnant sur une plate-forme du Parti républicain. Il a connu un succès modéré en tant que gouverneur dans ses premières années, mais a beaucoup lutté par la suite. Il avait de nombreux opposants politiques qui ont déclaré qu\'il n\'avait aucune expérience politique et qu\'il avait remporté les élections simplement en raison de sa popularité. Le mandat de Schwarzenegger a été secoué par des allégations d\'inconduite sexuelle qui ont entaché sa carrière politique. Il reste une figure culturelle très populaire aux États-Unis.,,,Van Halen est au Rock ‘n\\’ Roll ce que Cadillac est aux automobiles. C\'est l\'un des groupes les plus populaires à avoir jamais joué dans le pays, et il est toujours aussi fort 40 ans plus tard. Il est connu comme l\'un des meilleurs groupes jamais issus d\'un milieu de groupe de garage. Fondé par les frères Eddie et Alex Van Halen dans leur maison de Pasadena, en Californie, en 1972, le groupe était l\'un des meilleurs groupes de rock du pays et est aussi populaire aujourd\'hui qu\'à l\'époque.\n \nEddie Van Halen est considéré comme l\'un d\'entre eux. des meilleurs guitaristes de l\'histoire de la musique par plusieurs de ses pairs, tandis qu\'Alex était un batteur supérieur. Ensemble, ils ont formé le noyau du groupe populaire qui comprenait également le bassiste Michael Anthony et le chanteur David Lee Roth. Les frères sont nés aux Pays-Bas et ont immigré aux États-Unis lorsqu\'ils étaient enfants. Les deux étaient inséparables et ont développé un intérêt pour la musique au milieu des années 60. Ils ont formé le groupe en 1972, sous le nom de Genesis. Plus tard, ils ont changé le nom en Mammoth quand Eddie a découvert que Genesis était déjà utilisé. Finalement, ils ont renommé le groupe Van Halen.\nSelon la légende populaire, Van Halen a été découvert par Gene Simmons du groupe de rock Kiss, alors qu\'ils se produisaient au Gazzarri\'s on the Strip, un endroit populaire pour les groupes de rock heavy metal dans le années 70. Ils ont continué à jouer lors de concerts dans le sud de la Californie lorsqu\'ils ont sorti leur premier album éponyme Van Halen en 1978. L\'album a culminé au n ° 19 sur la liste Billboard, et le groupe n\'a jamais regardé en arrière. Le plus grand succès commercial de Van Halen a été de 1978 à 1985, mais leurs albums continuent de se vendre. Il a suscité un culte auprès de plusieurs jeunes générations de fans de rock\'n\'roll américain.,,,\ufeffDavid Cameron, Barack Obama et Pamela Anderson ont refusé. George W Bush, Benedict Cumberbatch et Stephen Hawking ont participé. Alors que l\'Ice Bucket Challenge a récolté 100 millions de dollars pour une organisation caritative américaine contre les maladies des motoneurones et 4,5 millions de livres sterling pour une organisation britannique, ainsi que des milliers d\'autres pour des organisations caritatives à Hong Kong et en Australie, l\'eau froide et vivifiante d\'un contrecoup a rapidement suivi . \nDes célébrités narcissiques exhibant des corps toniques, des gens qui s\'amusent sans faire de don, des plaintes concernant le gaspillage d\'eau - les attaques sont venues de commentateurs, de groupes de défense des animaux et d\'écologistes. Et l\'Association américaine de la sclérose latérale amyotrophique (SLA) - qui n\'est pas liée à l\'Association britannique MND - a également été critiquée pour les salaires à six chiffres versés à son personnel supérieur. \nPendant ce temps, le défi continue de croître. Pour quiconque ne sait pas comment cela fonctionne, quelqu\'un prononce un bref discours devant la caméra sur l\'organisme de bienfaisance, puis jette un seau de glaçons dans l\'eau au-dessus de sa tête, ou demande à un ami de le faire, avant de nommer trois personnes pour faire de même ou faire un don. \nIl a commencé aux États-Unis en juillet, bien que ce soit sur un terrain de golf ou un terrain de baseball dépend de la version que vous préférez, et est apparu pour la première fois à la télévision américaine grand public le 15 juillet. Mais l\'ALS Association a maintenant été forcée de retirer une tentative de brevetage de l\'expression "Ice Bucket Challenge" après des critiques. "Nous comprenons l\'inquiétude du public et retirons les demandes de marque", a déclaré la porte-parole Carrie Munk. L\'ALS n\'a pas encore répondu aux critiques concernant ses frais généraux et sa masse salariale élevés. \nLa malheureuse coïncidence selon laquelle du 31 août au 5 septembre était la Semaine mondiale de l\'eau, avec l\'arrivée de délégués internationaux à Stockholm pour discuter de la crise de l\'eau de la planète, n\'a pas échappé à certains. L\'association caritative WaterAid demande aux gens d\'utiliser l\'eau recyclée des baignoires ou des mégots de jardin ou de les asperger d\'eau de mer. \nDouglas Graham, directeur de la collecte de fonds de l\'Association MND, a déclaré\xa0: "Le contrecoup est à prévoir mais, vraiment, ce n\'est qu\'une merveilleuse aubaine et nous en sommes très reconnaissants. Nous ne l\'avons pas vu venir mais, tout à coup, les dons ont commencé. Ce coup de pouce est une aide énorme pour une petite organisation caritative qui s\'occupe des personnes atteintes d\'une maladie débilitante et peu comprise qui n\'a pas de remède et tue cinq personnes par jour au Royaume-Uni. \nL\'ancienne star de Baywatch Pamela Anderson, une militante de longue date des droits des animaux, a écrit une lettre ouverte à l\'ALS Association, disant qu\'elle ne pouvait pas soutenir son bilan sur l\'expérimentation animale. Quelques stars américaines ont rejeté le défi à cause de la sécheresse en Californie. L\'acteur Matt Damon a contourné le problème en tirant de l\'eau de ses toilettes – soulignant tqu\'une grande partie du monde disposait de moins d\'eau potable. L\'acteur Verne Troyer a utilisé du lait, invoquant à nouveau des raisons environnementales. Et le défi a été accusé d\'avoir provoqué une pénurie d\'eau sur l\'île écossaise de Colonsay après que ses 135 habitants ont repris l\'engouement. \nEn Australie, un présentateur de télévision s\'est excusé pour sa réponse ferme "non, merci" à sa nomination. Lincoln Humphries avait déclaré : « Au lieu de se verser de l\'eau fraîche sur la tête et de gaspiller de la glace, voici une liste d\'organisations caritatives aidant les communautés qui ont désespérément besoin d\'argent à travers le monde. J\'aimerais nommer tous ceux, partout, qui ont plus que ce dont ils ont besoin, pour qu\'ils donnent ce qu\'ils peuvent aux personnes qui en ont le plus besoin... parce que c\'est ça la charité, ne pas se mettre dans une légère gêne avec un seau d\'eau glacée .” \nUne autre critique a été que les petits organismes de bienfaisance ne seront pas en mesure de faire face à l\'argent supplémentaire, mais l\'association MND a rejeté cela. "Oh, nous pouvons nous débrouiller ici", a déclaré Graham. «Nous finançons des recherches de classe mondiale sur les causes et, en fin de compte, pour trouver un traitement ou un remède. Nous fournissons des soins et un soutien à 3 500 personnes et elles en ont besoin car il s\'agit d\'une maladie qui évolue si rapidement et dont la gestion est coûteuse. Plus de 50 % meurent dans les deux ans suivant le diagnostic. C\'est navrant de voir le déclin du nombre de personnes avec qui nous travaillons en quelques mois seulement. \nMais, pour de nombreuses personnes ayant un lien avec la maladie, la prise de conscience que le défi a créée est aussi précieuse que l\'argent. Graham dit que cela n\'a pas de prix. Normalement, l\'association MND reçoit environ 300 000 visites par an sur son site Web. En une seule journée récemment, il y en avait 330 000. \n"Nous n\'aurions pas pu créer ceci si nous avions essayé. Les organismes de bienfaisance sont tous des causes valables et je comprends même que certaines personnes pourraient vouloir faire un don à un autre. En 2013, les Britanniques ont donné 62 milliards de livres sterling à des œuvres caritatives - nous devrions en être fiers. C\'est fabuleux pour nous d\'obtenir cette aubaine. Nous allons nous asseoir dans les prochaines semaines pour voir comment le dépenser au mieux mais je vous assure que chaque centime comptera pour de bon. la Loire était autrefois un terrain de jeu pour les rois de la Renaissance. Aujourd\'hui, ses châteaux vantés attirent de jeunes couples entreprenants et des artistes qui les ont transformés en auberges captivantes - et étonnamment abordables.\nEntouré de murs de calcaire de 3 mètres de haut, Le Clos d\'Amboise offre un sanctuaire inattendu dans la ville d\'Amboise.\nCHINON\nHôtel Diderot\n Si vous n\'avez jamais mangé de brioche au Ste. Fromage de chèvre maure, miel et noix au petit déjeuner, alors vous n\'êtes pas allé dans la Loire, dit Laurent Dutheil, qui est à juste titre fier des petits déjeuners simples qu\'il sert dans son hôtel de 23 chambres à l\'angle ouest de la vallée. \nDutheil produit également des dizaines de confitures artisanales parfumées comme la pomme-lavande et le coing-cannelle. -LRB- Malheureusement, ils ne sont pas à vendre, mais vous pouvez acheter le livre de recettes de Dutheil, Jam in the Cupboard. -RRB- Les plats traditionnels cadrent bien avec l\'atmosphère vénérable de l\'hôtel : Diderot est installé dans une vaste maison du XVe siècle que Dutheil, originaire de Chinon, avec ses deux sœurs, Martine et Françoise, a achetée et rénovée il y a six ans. . Voyage à petit budget\xa0: découvrez les hôtels\nDutheil s\'est attaqué aux problèmes structurels, en prenant soin de conserver intactes les poutres apparentes en chêne et les murs en pierre d\'origine, tandis que ses sœurs ont rénové les chambres avec du papier peint à rayures gaies, de la literie en toile et des armoires du XIXe\xa0siècle qu\'elles ont récupérées lors de voyages à Paris. La plus grande chambre, qui peut accueillir quatre personnes, a des doubles fenêtres avec vue sur une cour. Au loin s\'étendent des vignobles remplis de cépages rouges Chinon tant aimés par le romancier et satiriste du XVIe\xa0François Rabelais. .\nTHÉSÉE\nLe Moulin de la Renne\nUn tunnel de sapins majestueux mène à l\'entrée de ce moulin du XIXe\xa0siècle reconverti à Thésée, sur les rives du Cher. Les invités sont accueillis par un énorme bouvier bernois nommé Alpha et ses propriétaires tout aussi sympathiques, Véronique et Christophe Villanfin.\nIl y a 13 chambres d\'hôtes, décorées d\'objets tels que des couettes brodées et des puzzles encadrés de bateaux assemblés par la fanatique de puzzle Véronique. Amateurs de jazz, les Villanfin accueillent ponctuellement des concerts de groupes locaux, dont Les Bras Coeurs, un quatuor qui interprète les standards de Georges Brassens. vignoble des Varennes à côté.\n11 route de Vierzon, 011-33/2 -54 -71 -41 -56, moulindelarenne.com, fermé 10 janv. -- 10 fév. et 10 jours en nov., chambres à partir de 76 $, entrées à partir de 15,75\xa0$.\nCIVRAY-DE-TOURAINE\nChâteau de l\'Isle\nLe Château de l\'Isle est calme jusqu\'à l\'âme - à moins que vous ne comptiez le chœur de charlatans provenant de la mare aux canards sur le terrain de 35\xa0acres . Le manoir du XVIIIe siècle était à l\'abandon depuis 10 ans lorsque Denis Gandon le rachète en 1986 et transforme les lieux en hôtel de 12 chambres. à quoi ressemble une maison privée : un portrait du grand-père de Gandon est suspendu au-dessus d\'une table antique vieille de 100 ans dans la salle à manger, et un aimable Jack Russell terrier divertit les invités avec d\'interminables rondes de récupération. Les chambres élégantes présentent des poutres apparentes et des lits drapés de couvre-lits dans des tons de pourpre et de souci. Voyage à petit budget\xa0: voir les hôtels\nEn été, le petit-déjeuner est servi sur une vaste terrasse vitrée donnant sur le jardin. Un potager -LRB- ou un potager -RRB- à proximité fournit les produits pour certains petits déjeuners du chef Fabrice Cherioux, comme une confiture de tomates acidulées.\n1 rue de l\'Ecluse, 011-33/2 -47 - 23 -63 -60, château-de-lisle. com, chambres à partir de 69\xa0$, petit-déjeuner 13\xa0$.\nCHENONCEAUX\nLa Roseraie\nCet hôtel de 18\xa0chambres à Chenonceaux était un séjour incontournable sur les itinéraires des notables politiques après la Seconde Guerre mondiale, lorsque les Alliés tentaient de comprendre comment reconstituer l\'Europe retour ensemble. Winston Churchill, Harry Truman et Eleanor Roosevelt ont tous, à un moment ou à un autre, posé leur tête à La Roseraie. -LRB- Une lettre de Roosevelt est toujours accrochée dans la salle du petit-déjeuner. -RRB-\nEn 2000, Sophie et Laurent Fiorito acquièrent la propriété et réalisent une rénovation de fond en comble. Il n\'y a pas de chambre standard ici - les minuscules célibataires ont juste assez d\'espace pour un lit jumeau, tandis qu\'une suite convient à une famille de cinq personnes. Le décor est tout aussi varié. Une chambre a une commode de style Louis Philippe et un lustre en cristal; un autre présente un canapé moderne à carreaux roses.\nLe meilleur atout de l\'hôtel est sa proximité -LRB- à cinq minutes à pied -RRB- du Château de Chenonceau, qu\'Henri II offrit à sa bien-aimée, Diane de Poitiers\xa0; l\'imposante bâtisse du XVIe siècle surplombe le Cher.\n7 rue du Docteur Bretonneau, 011-33/2 -47 -23 -90 -09, hôtel-chenonceau. com, fermé nov. -- janv. et mars, chambres à partir de 85 $.\nAMBOISE\nLe Clos d\'Amboise\nEntouré de murs en pierre calcaire de 3 mètres de haut, au cœur du vieux centre commercial d\'Amboise, ce 17 -room mansion inn offre un refuge inattendu dans les rues étroites et animées de la ville.\nLes chambres sont équipées d\'éléments décoratifs tels que des lits à baldaquin en acajou et des cheminées d\'origine, tandis que la salle commune présente des parquets inspirés de ceux du Château de Versailles. La plupart des chambres donnent sur le parc privé de l\'hôtel, agrémenté de magnolias centenaires et d\'un pin qui a survécu à la Révolution française.\nUne piscine au bord de la roseraie est idéale pour se rafraîchir après un moment de détente dans le sauna en bois , aménagé dans une ancienne écurie. À seulement 10 minutes à pied de l\'hôtel se dresse le château au bord de la rivière qui appartenait au roi François Ier - en partie forteresse médiévale, en partie maison royale de la Renaissance. Le roi est devenu captivé par Léonard de Vinci lors d\'une excursion en Italie en 1515, et il a installé le peintre dans un château à pignons en briques rouges -LRB- maintenant un musée -RRB-, juste en bas de la rue de son château.\n27 rue Rabelais, 011-33/2 -47 -30 -10 -20, leclosamboise.com, fermé déc. -- janv., chambres à partir de 98$.\nLIMERAY\nAuberge de Launay\nÀ eux deux, François et Hélène Thévard ont peaufiné leur compétences hôtelières dans certains des hôtels les plus respectés au monde, dont Le Meurice à Paris et le Savoy à Londres.\nAprès des années passées à gérer les propriétés d\'autres personnes, le couple - avec leurs deux jeunes enfants, Emma et Alexandre - ont emballé leur appartement parisien et acheté une ferme sans prétention dans le hameau isolé de Limeray pour la convertir en une auberge de 15 chambres.\n J\'ai grandi dans la région, alors quand nous avons pris la décision d\'ouvrir notre propre maison, Je savais qu\'il fallait que je revienne dans la Loire, raconte François. Conformément au design épuré de la maison, la décoration de l\'Auberge est simple\xa0: les chambres sont équipées de canapés en daim neutres, de parquets en bois blond et de photographies minimalistes de marguerites et de paysages.\nLe duo supervise le service quotidien du déjeuner et du dîner à le restaurant sur place, qui sert des spécialités locales telles que l\'anguille de Loire pochée au beurre et le poulet à la crème de moutarde. Le dessert maison, une tarte aux fruits au caramel salé, est cuit avec des pommes cueillies dans le verger de l\'hôtel. 15 -- 15 janv., chambres à partir de 72$, dîner deux services à partir de 26$.\nONZAIN\nChâteau des Tertres\nCe manoir au coeur de la Loire a appartenu à la mère de la romancière Marguerite Duras -LRB- qui mentionne le château dans sa célèbre œuvre The Lover -RRB-. Plus tard, dans les années 1960, ses couloirs de marbre sont remplis d\'étudiants d\'une école d\'ingénieurs voisine, qui utilise le bâtiment comme dortoir.\nEn 1992, un autre propriétaire curieux reprend la direction du château de 14 pièces : Bernard Valois, un artiste qui a conservé les détails du 19ème siècle du manoir -LRB- hauts plafonds, miroirs dorés -RRB- mais a ajouté des éléments de design contemporains, comme une photographie d\'une mystérieuse paire d\'yeux dans le hall.\nL\'année dernière, Valois et sa femme, Christine, a redécoré une ancienne maison de gardien en un cottage contemporain de quatre pièces avec imitation Jackson Pol verrouillez des peintures et des sculptures qui ressemblent à d\'énormes œufs.\n Mon mari aime transformer des choses qui semblent ordinaires en extraordinaires, dit Christine. Il a conçu un Spoutnikbidet qui a des robots japonais stationnés autour du réservoir. \nValois s\'occupe également d\'un jardin de roses et de clématites de 5 hectares, où vous trouverez un abri rempli de vélos gratuits pour explorer les 300\xa0km de sentiers équestres de la région.\n11 rue de Meuves, 011-33/2 -54 -20 -83 -88, château-tertres. com, fermé du 19 oct. au avr. 1, chambres à partir de 94\xa0$.\nCHITENAY\nAuberge du Centre\nLorsque la grand-mère de Gilles Martinet s\'est retrouvée veuve à la fin de la Première Guerre mondiale, elle a commencé à vendre du lait et des œufs pour joindre les deux bouts. De fil en aiguille, sa maison est finalement devenue une maison d\'hôtes et un restaurant spécialisé dans les plats de la campagne française, comme le poulet sauté au cidre frais.\nTrois générations plus tard, Martinet est maintenant propriétaire et chef de cette auberge, qui compte 26\xa0chambres joliment décorées dans nuances de sauge et de rose, ainsi qu\'une terrasse de jardin fleurie à l\'arrière. Il a conservé la salle de petit-déjeuner à peu près comme sa grand-mère l\'a conçue, avec des murs en pierre rustiques, une grande cheminée et un mobilier de campagne simple. , à environ huit kilomètres.\n34 Grande Rue, 011-33/2 -54 -70 -42 -11, auberge-du-centre. com, fermé en février, chambres à partir de 78\xa0$, entrées à partir de 17\xa0$.\nBases de la Loire\nÀ deux\xa0heures de route au sud-ouest de Paris, la vallée de la Loire est connue pour ses vins terreux de cabernet franc et ses centaines de châteaux majestueux.\nC\'est impossible pour les voir tous -LRB- la visite d\'un château peut prendre une demi-journée -RRB-, il est donc sage de se concentrer sur les plus dignes : Chenonceau, le gracieux palais fluvial ayant appartenu à Henri II -LRB- chenonceau .com, 13 $ -RRB-\xa0; Chambord, qui possède un escalier à double hélice inspiré des dessins de Léonard de Vinci -LRB- chambord.org, 12,50 $ -RRB- ; et Cheverny, réputé pour ses intérieurs flamboyants et dorés -LRB- chateau-cheverny. com, 9,75 $ -RRB-.\nRecevez les meilleures offres de voyage et des conseils par e-mail GRATUITEMENT - CLIQUEZ ICI\xa0!\nCopyright © 2009 Newsweek Budget Travel, Inc., tous droits réservés.\nRemarque\xa0: cette histoire était exacte au moment de sa publication. Assurez-vous de confirmer tous les tarifs et détails directement avec les entreprises en question avant de planifier votre voyage.\n,,,Les Rangers ont marqué trois buts en deuxième mi-temps pour battre le Celtic 4-2 lors du Old Firm Derby de Glasgow dimanche et aller quatre points d\'avance en Premier League écossaise.\nEn forme, Steven Naismith a renvoyé les Rangers en tête avec une frappe retentissante après la 22e minute, mais le Celtic n\'a pas tardé à égaliser grâce au but bien travaillé de Gary Hooper 12 minutes plus tard.\nCeltic a pris les devants juste avant la mi-temps alors que le gardien des Rangers Allan McGregor a permis à un tir faible du Marocain Badr El Kaddouri de lui échapper et de se glisser dans le filet. \nMulgrew, qui a eu la chance de s\'échapper avec juste un carton jaune pour son défi en première mi-temps, a ensuite vu rouge à la 75e minute et Naismith a profité de 10 hommes pour marquer le quatrième et son deuxième dans le temps additionnel.\n J\'ai fait n\\ ne pense pas il y avait énormément de choses entre les équipes en première mi-temps. Je ne suis pas sûr que nous méritions de perdre 2-1, mais c\'est ce à quoi nous étions confrontés, a déclaré le manager des Rangers Ally McCoist à Sky Sports.\n Nous avons demandé aux garçons une réaction en seconde période et garçon avons-nous compris. \nLe manager du Celtic, Neil Lennon, a été déçu par la performance de son équipe en seconde période\xa0: nous n\'avons pas concouru et c\'était presque comme si les joueurs étaient sortis en seconde période en pensant que le match était gagné, a-t-il déclaré.\\ nLes matchs entre les deux géants du football écossais s\'avèrent généralement cruciaux dans la course au titre national et la victoire a laissé les Rangers à 19 points après sept matchs, avec le Celtic à 15 en deuxième.\n,,,Saint Patrick\'s Day, 17 mars, est une fête religieuse chrétienne qui célèbre le saint patron de l\'Irlande. Selon la légende, Saint Patrick a chassé tous les serpents d\'Irlande. La Saint-Patrick est également devenue, notamment aux États-Unis, une journée de célébration de la culture irlandaise. Dans les grandes villes du pays comme à New York et Boston, il y a des défilés et des festivals. Les gens portent du vert et décorent avec des trèfles, un type de trèfle à trois feuilles. Certaines personnes portent des boutons ou des tee-shirts qui disent "Embrasse-moi, je suis irlandais". Aux États-Unis, la Saint-Patrick n\'est pas un jour férié, ce qui signifie que les bureaux du gouvernement et les écoles sont ouverts.\n \nLe plus grand et le plus ancien défilé a lieu à New\xa0York. Cela a en fait commencé il y a environ 250 ans avant même que les États-Unis ne deviennent leur propre pays. Chaque année, des milliers de personnes se rassemblent pour regarder défiler des centaines de danseurs irlandais et de joueurs de cornemuse. La gigue irlandaise se pratique en groupe ou en solo. Les danseurs exécutent des mouvements compliqués avec leurs pieds et leurs jambes tout en gardant le haut du corps raide. Le défilé a également été une source de controverse parce que le défilé est assolié à l\'Église catholique de New York; les organisations gaies et lesbiennes n\'ont pas été autorisées à manifester. De nombreuses personnes protestent contre le défilé ou le boycottent.\nSt. La Saint-Patrick est également devenue associée à la consommation de beaucoup d\'alcool, en particulier de bière. Une boisson que l\'on voit souvent pendant les vacances est la "bière verte". C\'est vraiment juste de la bière ordinaire avec du colorant alimentaire vert. Beaucoup de gens ont l\'impression que la fête célèbre les stéréotypes sur les Irlandais, comme le mythe selon lequel ils aiment tous boire et se saouler. D\'autres se plaignent que les vacances sont devenues trop axées sur la consommation d\'alcool, l\'ivresse publique étant un problème dans les grandes villes autour des vacances. ,,, Sean Edwards, l\'un des pilotes de course les plus prometteurs de Grande-Bretagne, a été tué dans un accident au Queensland Raceway en Australie.\nEdwards, 26\xa0ans, était le fils de l\'ancien pilote de Formule\xa01 Guy Edwards et a joué le rôle de son père dans le film récemment sorti Rush.\nEdwards menait le championnat des pilotes lors de la saison 2013 de la Porsche Mobil 1 Supercup et participait à une session de formation privée pour jeunes conducteurs.\nL\'accident s\'est produit lorsque la Porsche 966 dans laquelle il voyageait en tant que passager a percuté une barrière de pneus.\nGuy Edwards était l\'un des trois conducteurs qui ont sorti Niki Lauda de sa voiture avant qu\'elle ne s\'enflamme à la suite d\'un accident au Nurburgring lors du Grand Prix d\'Allemagne de 1976.\nSon fils l\'avait suivi dans la course et était très bien noté.\nCondoléances\n Nos pensées vont à sa famille et nous voulons ex présenter nos plus sincères condoléances à ses proches, a déclaré Hartmut Kristen, responsable du sport automobile chez Porsche.\nL\'inspecteur Dave Preston, de la police du Queensland, a déclaré que les services d\'urgence avaient eu du mal à sortir Edwards des véhicules endommagés.\n Les pompiers et les secours devaient faire travail considérable pour extraire et ouvrir suffisamment le véhicule pour le sortir, il a été piégé pendant un certain temps, a-t-il déclaré aux journalistes. éveillé et conscient de ce qui se passait autour de lui. \nEdwards a commencé sa carrière en Formule Ford il y a 10\xa0ans avant de concourir en Formule Renault UK et en British GT.\nIl a remporté le championnat FIA GT3 en 2006 avant de passer à la compétition Porsche Supercup en 2008.\nChoquant\nDavid Coulthard, l\'ancien pilote de Formule 1, s\'est adressé à Twitter pour présenter ses condoléances.\nIl a écrit\xa0: Des nouvelles choquantes concernant le décès de Sean Edwards, l\'homme le plus talentueux et le pilote super talentueux, condoléances à ses proches. \nLa Motor Sports Association -LRB- MSA -RRB- a également offert ses meilleurs vœux à la famille, se disant choquée et attristée par la nouvelle.\n` Sean était un jeune coureur extrêmement prometteur qui a traversé les formules juniors en Grande-Bretagne, avant de faire carrière dans la course automobile internationale, lisez la déclaration.\n Il est en tête du championnat Porsche Supercup 2013 et a remporté les 24 Heures du Nurburgring et de Dubaï cette année.\n La MSA présente ses plus sincères condoléances à Sean\'s famille, amis et fans en cette période difficile. \n,,,Memorial Day est un jour férié aux États-Unis pour se souvenir des hommes et des femmes qui sont morts alors qu\'ils servaient dans les forces armées. Les forces armées comprennent toutes les branches de l\'armée américaine. Ces branches sont l\'armée, la marine, les marines, l\'armée de l\'air et la garde côtière. Le Memorial Day est un jour férié fédéral, ce qui signifie que les bureaux du gouvernement, les banques, les bureaux de poste et les écoles sont fermés. Le Memorial Day est toujours observé le dernier lundi de mai. Le Memorial Day a ses racines dans la guerre civile américaine lorsqu\'un jour férié, le jour de la décoration, a été créé pour honorer les soldats de l\'Union et des Confédérés décédés. Le jour de la décoration, les gens avaient l\'habitude de "décorer" les tombes des soldats décédés avec des drapeaux et des fleurs. Le nom Memorial Day est devenu plus largement utilisé après la Seconde Guerre mondiale à la fin des années 1940. Cependant, la fête n\'a été officiellement appelée Memorial Day qu\'en 1967.\n \nLe Memorial Day est commémoré par des cérémonies dans les cimetières du pays. Le plus connu se tient au cimetière national d\'Arlington près de Washington D.C. Arlington est un cimetière militaire, ce qui signifie que seuls des militaires y sont enterrés. Des volontaires placent des drapeaux américains sur des tombes militaires à Arlington et dans d\'autres cimetières nationaux. Il y a aussi des défilés en l\'honneur des militaires tombés dans différentes villes. Une autre tradition du Memorial Day consiste à faire flotter le drapeau américain à mi-personnel jusqu\'à midi, ce qui signifie qu\'il flotte à la moitié de sa hauteur normale. Ceci est considéré comme un signe de deuil. Le drapeau est hissé sur toute sa hauteur après midi, signe que l\'armée se lèvera malgré la défaite. Il y a aussi un concert national du Memorial Day qui a lieu sur la pelouse de la capitale américaine.\nLe Memorial Day est également connu comme le début officieux de la saison estivale. De nombreuses plages et piscines ouvrent pour le week-end du Memorial Day. De nombreuses personnes organisent également des barbecues le week-end du Memorial Day.,,,Il n\'y a peut-être aucun autre réalisateur dans l\'industrie cinématographique américaine qui soit plus controversé que Michael Moore. de Moore la spécialité est l\'exposition. Il aime exposer les problèmes sociaux. Il est militant politique et critique social. Ses films suscitent de fortes réactions émotionnelles.\n \nMoore est né à Flint, dans le Michigan, en 1954. Son père était ouvrier à la chaîne chez General Motors, une usine de fabrication automobile\xa0; sa mère était secrétaire. Il a fréquenté l\'Université du Michigan, campus Flint, où il a abandonné après sa première année. Il a trouvé du travail en tant que journaliste dans quelques publications locales différentes, mais a été licencié de l\'un de ses emplois en raison d\'une controverse. Moore voulait publier un article sur l\'implication américaine au Nicaragua, mais ses éditeurs ont refusé, ce qui a conduit à son licenciement. Il a reçu 58\xa0000\xa0$ dans le cadre d\'un règlement à l\'amiable après avoir poursuivi ses employeurs.\nMoore a utilisé l\'argent pour commencer à réaliser des documentaires. Son premier était "Roger & Me", qui était une pièce conçue pour exposer la fermeture de l\'usine de Flint par GM et sa décision d\'ouvrir une usine au Mexique, où les salaires étaient beaucoup plus bas. Son film dénonce les sociétés cupides comme GM coupant la bouée de sauvetage de nombreux travailleurs américains afin que GM puisse faire plus de profit. GM avait rapporté des bénéfices records avant la fermeture, ce qui a beaucoup irrité Moore.\nIl a suivi ce film avec une série de documentaires qui ont abordé de nombreux problèmes sociaux et politiques auxquels les Américains sont confrontés quotidiennement. Son film «\xa0Fahrenheit 9/11\xa0» examine les conséquences des attentats terroristes du 11\xa0septembre aux États-Unis en 2001. Il n\'est pas tendre avec l\'administration George W. Bush pour son manque de réponse aux attentats, et avec l\'administration Bush. Les liens présumés de la famille avec Oussama Ben Laden, le cerveau de l\'attaque.,,,Le golf est l\'un des jeux les plus populaires aux États-Unis aujourd\'hui. Il y a eu de nombreux grands champions au fil des ans comme Arnold Palmer et Jack Nicklaus, mais aucun n\'a connu le succès comme Tiger Woods. Son père était noir; sa mère était thaïlandaise. Il est né et a vécu sa jeunesse en Thaïlande.\n \nWoods a rejoint le circuit professionnel de golf à l\'âge de 20\xa0ans en 1996, où il a commencé à établir des records de victoires aux championnats à un jeune âge. Il était l\'une des jeunes stars les plus prometteuses à avoir participé à la tournée depuis des années, ce qui a conduit Woods à acquérir plusieurs contrats lucratifs avec Nike et Titleist. Woods est devenu un très jeune millionnaire. En 2001, Woods est devenu le premier joueur de l\'histoire à détenir le titre dans les quatre tournois de golf majeurs. Il semblait que ce jeune homme ne pouvait rien accomplir.\nWoods a grandi dans le comté d\'Orange, en Californie, et a étudié pendant deux ans à la prestigieuse université de Stanford. Woods a commencé à jouer au golf à un âge précoce et, peu de temps après, il remportait régulièrement des championnats amateurs. Il a été la recrue de l\'année de la Professional Golfers Association (PGA) en 1996. Il a été nommé PGA Player of the Year et PGA Tour Player of the Year, 11 fois chacun. Il a également dirigé la tournée en tant que plus grand gagnant d\'argent plusieurs années. Il était marié et avait deux enfants. Sa vie était présumée idéale, mais cela a changé en 2009.\nWoods a été dénoncé comme un coureur de jupons par plusieurs femmes, à la suite d\'une prétendue liaison avec une femme de New York. Woods a nié tout acte répréhensible, mais a finalement avoué plusieurs "indiscrétions". La controverse a continué à prendre de l\'ampleur et a finalement conduit à son divorce et à la perte de l\'argent des sponsors. Woods est de retour sur la tournée, mais n\'a pas retrouvé ses compétences comme avant.,,,Les responsables de l\'espace russes disent que l\'équipage du vaisseau spatial Soyouz se repose après un retour difficile sur Terre.\nUn bio-ingénieur sud-coréen était l\'une des trois personnes à bord de la capsule Soyouz.\nL\'engin transportant le premier astronaute sud-coréen a atterri dans le nord du Kazakhstan samedi, à 260 miles -LRB- 418 kilomètres -RRB- de sa marque, ont-ils déclaré.\nLe porte-parole du contrôle de la mission, Valery Lyndin, a déclaré l\'état de l\'équipage - le bio-ingénieur sud-coréen Yi So-yeon, l\'astronaute américaine Peggy Whitson et l\'ingénieur de vol russe Yuri Malenchenko - était satisfaisant, bien que les trois aient été soumis à de graves forces G lors de la rentrée.\nLes hélicoptères de recherche en ont pris 25 minutes pour trouver la capsule et déterminer que l\'équipage était indemne.\nDes responsables ont déclaré que l\'engin suivait une trajectoire très abrupte qui soumet l\'équipage à des forces gravitationnelles pouvant atteindre 10\xa0fois celles de la Terre.\nInterfax a signalé que l\'atterrissage de l\'engin spatial était ro ugh.\nCe n\'est pas la première fois qu\'un vaisseau spatial dévie de sa trajectoire prévue lors de l\'atterrissage.\nEn octobre, la capsule Soyouz a atterri à 70\xa0kilomètres de la zone prévue à cause d\'un câble de commande endommagé. La capsule transportait deux cosmonautes russes et le premier astronaute malais. Envoyer un e-mail à un ami\n,,,Le bol de riz de la Thaïlande est désormais le cauchemar de la Thaïlande.\nLe delta du fleuve Chao Phraya alimente les vastes rizières du pays. Mais vendredi, le fleuve et ses nombreux affluents ont observé la capitale, Bangkok, comme un animal féroce traquant sa proie.\nLes pluies de la mousson ont inondé les plaines de la Thaïlande et maintenant la rivière en crue menace de noyer Bangkok.\nL\'eau est déjà devenue un parking beaucoup dans marin comme, les marchés dans les lacs et les maisons dans des maisons adaptées uniquement aux poissons. Il a jailli dans les maisons des saints - les temples bouddhistes se dressaient au milieu de la montée des eaux - et dans les sanctuaires des personnes vulnérables.\nQuinze éléphants, dont sept mères avec des bébés et un enfant de 9\xa0ans connu pour ses talents de peintre, sont bloqués sur sommet du Royal Elephant Kraal d\'Ayutthaya. Les éléphants ont grimpé au sommet du bâtiment la semaine dernière et ont faim maintenant que la nourriture ne peut être apportée qu\'en petites quantités sur des bateaux à rames. Les éléphants savent nager, mais les bébés peuvent se noyer en tentant de s\'échapper.\nÀ Bangkok et dans d\'autres villes et villages submergés, les forts portaient les faibles sur leur dos. Les jeunes ont aidé les vieux.\nTout le monde s\'est entraidé.\nIl était difficile d\'imaginer que l\'année dernière, les chemises rouges aient combattu les chemises jaunes lors de manifestations politiques meurtrières qui ont provoqué l\'état d\'urgence en Thaïlande et laissé une nation profondément divisée.\nLa catastrophe a apporté ils se sont réunis cette semaine.\nUne page Facebook en anglais créée pour aider les étrangers en Thaïlande a connu une frénésie d\'action vendredi.\n Les patients insuffisants rénaux qui souhaitent suivre un processus de dialyse peuvent contacter HSRI co. avec NE Kidney Medico \n Navette gratuite de l\'hôtel Dusit Thani au TU DOME disponible pour la logistique des dons demain \n Chaussons -LRB- type de mousse -RRB- nécessaires pour toutes les tailles au Don Muang Evacuation Cent. \n Les deux voies de Bangkruy-Sainoi Rd... inondées \nAvec 283\xa0morts et 8\xa0millions de personnes affectées par les inondations, les craintes étaient réelles à Bangkok.\nLes ouvriers bourraient furieusement des sacs de sable et étayaient les barrières. Les centres d\'évacuation ont commencé à se remplir de monde.\nBanjong Palim, 43\xa0ans, a déclaré qu\'il n\'avait jamais rien vu de tel de sa vie. Il a été forcé de fuir lorsque sa maison de la banlieue de Bangkok s\'est noyée.\nCertains se souviennent du tsunami dévastateur de 2004 qui a tout anéanti avec des murs d\'eau. Et paniqué.\nEd White a regardé la rivière couler par-dessus les murs de confinement à Ayutthaya. Il a empilé les meubles aussi haut que possible dans sa maison et s\'est senti chanceux qu\'ils soient faits de béton et de briques, et non de bois, comme tant de maisons thaïlandaises traditionnelles.\nWhite a grandi à Ocean City, dans le Maryland. Il était habitué aux ouragans et à la fureur de l\'Atlantique. Mais c\'était différent.\nL\'eau ne faisait pas rage. Il n\'y avait pas de torrents.\nIl n\'arrêtait pas de monter. Lentement, mais sûrement.\nAu marché où White se fait couper les cheveux, les gens se sont précipités, répandant des rumeurs nées de la peur. Dans deux heures, disaient-ils, tout serait sous l\'eau. Le déluge arrivait ! L\'inondation arrivait\xa0!\nWhite a déménagé en Thaïlande en 2003 pour travailler pour une entreprise de caoutchouc. Vendredi, il a déclaré qu\'il avait emménagé dans un appartement de l\'entreprise au deuxième étage à Bangkok, confiant qu\'il y serait en sécurité. Pendant ce temps, son employeur avait fait de l\'usine de caoutchouc une forteresse, érigeant des barrières en béton autour de tout comme s\'il s\'agissait d\'une zone de guerre.\nPour beaucoup, c\'était une bataille perdue d\'avance.\nL\'armée a aidé à les évacuer, les emmenant en bateau ou en transportant dans l\'eau jusqu\'à la poitrine.\nPresque tous les jours maintenant, la pluie tombe fort, exacerbant les inquiétudes. C\'est déjà la pire inondation que la Thaïlande ait connue depuis des décennies. Certains se sont demandés vendredi jusqu\'à quel point cela pourrait empirer.\nLe chef du Centre gouvernemental de secours aux inondations a déclaré que Bangkok serait épargnée. Mais avec un risque d\'orages dans les prévisions pour les prochains jours, les assurances vendredi ont été difficiles à avaler. l\'East River à New York. Techniquement, quiconque visite le siège de l\'ONU n\'est plus à New York. Ils ne sont même pas aux États-Unis. En effet, le terrain et les bâtiments sont considérés comme un territoire international. L\'ONU a son propre drapeau, son propre bureau de poste et ses propres timbres-poste. Il n\'est cependant pas nécessaire de changer de devise. L\'ONU utilise le dollar américain. Six langues officielles sont utilisées aux Nations Unies. Il s\'agit de l\'arabe, du chinois, de l\'anglais, du français, du russe et de l\'espagnol. Les règles de l\'ONU l\'emportent sur les lois de la ville de New York. Cela ne veut pas dire que les gens peuvent y commettre des crimes et s\'en tirer. Il n\'y a pas d\'immunité pour ceux qui y commettent des crimes.\n \nLa propriété où se trouvent maintenant les Nations Unies appartenait autrefois à un abattoir. Les bâtiments actuels ont été achevés en 1952. Le périmètre de l\'ONU est bordé de mâts avec les drapeaux des 193 États membres de l\'ONU et le drapeau de l\'ONU. Ils sont classés par ordre alphabétique en anglais.\nCertains peuvent penser que c\'est formidable d\'être le voisin d\'une agence de maintien de la paix comme l\'ONU. Cependant, de nombreux New-Yorkais ne le pensent pas toujours. Chaque fois qu\'un dignitaire comme un président ou un premier ministre visite, les rues sont fermées, ce qui rend difficile de se déplacer et de trouver un parking.\nVous n\'avez pas besoin d\'être diplomate pour visiter l\'ONU. Il y a des visites du lundi au vendredi et pas seulement en anglais. Les visites guidées sont disponibles en anglais, français, allemand, italien, japonais, coréen, mandarin, portugais, russe, espagnol et suédois h. Les visiteurs peuvent prendre un repas dans la salle à manger du délégué avec un buffet de plats du monde entier. ,,, Des manifestants anti-chinois se sont réunis samedi à Tokyo et dans six autres grandes villes du Japon pour se rassembler contre ce qu\'il appelle une invasion d\'îles contestées qui prétendent toutes deux faire partie de leurs territoires.\nLes manifestants ont brandi des drapeaux japonais et scandé, Nous ne permettrons pas à la Chine communiste d\'envahir notre territoire. \nPékin affirme que les îles Diaoyu et la majeure partie de la mer de Chine méridionale appartiennent à la Chine, contestant les revendications des pays voisins. Au Japon, les îles sont connues sous le nom de Senkaku.\nLe rassemblement a été organisé par Toshio Tamogami, un ancien chef d\'état-major japonais de la Force d\'autodéfense.\nLe Japon rejette la demande d\'excuses de la Chine\n Les îles Senkaku sont le Japon\\ du territoire traditionnel de, a déclaré Tamogami. Si nous ne le protégeons pas, la Chine prendra des mesures pour le prendre. \nTamogami a traité la Chine de voleur et a juré de protéger les îles.\nPékin et Tokyo se sont affrontés à propos de l\'arrestation d\'un capitaine de pêche chinois par le Japon au large des îles contestées.\nLe capitaine a été accusé d\'avoir heurté deux navires des garde-côtes japonais et d\'avoir fait obstruction au public officiers pendant qu\'ils accomplissaient leurs tâches.\n Je suis très en colère contre la Chine, a déclaré l\'étudiant Shohei Fukumoto.\nLa Chine devrait s\'excuser et payer pour la réparation du navire car le capitaine se trouvait sur le territoire japonais lorsque l\'accident s\'est produit, selon Fukumoto.\nLe Japon a depuis libéré le capitaine de pêche, qui est revenu en héros en Chine.\nYoko Sakamoto, 55\xa0ans, a déclaré que le Japon avait été trop indulgent avec la Chine.\n Je voudrais critiquer le gouvernement japonais\\ la faible attitude de la Chine envers la question de la Chine et des îles Senkaku, plutôt que la Chine elle-même, a déclaré Sakamoto. Nous devrions retrouver notre fierté nationale. \nL\'arrestation du capitaine a entraîné la suspension des pourparlers diplomatiques et l\'annulation des voyages entre les nations.\nPékin a demandé des excuses et une indemnisation pour l\'arrestation, ce que Tokyo a rejeté.\nLe Japon a également exigé une indemnisation pour les dommages causés aux navires de la garde côtière .\nLe journaliste Chie Kobayashi a contribué à ce rapport.\n,,,Depuis 2008, Barack Obama est président des États-Unis d\'Amérique. Il est un personnage historique principalement parce qu\'il est le premier Afro-Américain à occuper le poste de président. Sa politique lui a valu de nombreux partisans, ainsi que de nombreux détracteurs. Il est peut-être l\'un des présidents les plus controversés à gouverner le pays dans l\'histoire des États-Unis. Obama est un représentant du Parti démocrate et sa présidence vient après le départ controversé de George W. Bush.\n \nL\'un des aspects déterminants de sa présidence est la loi sur la protection des patients et les soins abordables, communément et à tort appelée Obamacare. Cette loi, promulguée par le Congrès américain et signée par le président en mars 2010, est entrée en vigueur le 1er janvier 2014. Il s\'agit du changement le plus important dans les soins de santé gouvernementaux depuis la mise en œuvre de Medicare en 1965. Cette loi visait à augmenter l\'abordabilité des soins pour les Américains, étendre la couverture d\'assurance et réduire le taux de non-assurés dans le pays. C\'est un acte controversé qui a de nombreux détracteurs, mais la plupart des Américains ont maintenant une assurance maladie, ce qui n\'était pas le cas avant 2014.\nObama est né à Honolulu, Hawaï en 1961. Il est diplômé de l\'Université Columbia à New York et Faculté de droit de Harvard. Avant de devenir président, il a été sénateur de l\'État de l\'Illinois. Il est considéré comme un libéral par de nombreux conservateurs du sud, mais cela peut être une étiquette injuste. La perception est largement due à la loi sur les soins abordables et à ses tentatives de réforme de l\'immigration. De plus, Obama a mis fin à l\'implication des États-Unis dans la guerre en Irak. Il a également ordonné l\'opération militaire qui a finalement abouti à la mort d\'Oussama Ben Laden. Obama a été réélu président en novembre 2012.,,,On l\'a surnommé "l\'hôtel de maman et papa", mais peu de maisons d\'hôtes ont des conditions aussi favorables. Alors que la crise du logement sévit, un cinquième des jeunes adultes restent dans la maison familiale jusqu\'à ce qu\'ils aient au moins 26 ans et la même proportion ne paie pas un sou pour leur entretien. Une enquête récente a révélé que la proportion d\'adultes vivant à la maison variait à travers le pays, allant d\'un peu moins de 9% dans les East Midlands à plus du double à Londres, où les prix des logements et les loyers sont les plus élevés. Alors que de nombreuses personnes à travers le pays ont contribué financièrement, l\'enquête a révélé que 20% ne payaient rien du tout. \nLes jeunes adultes sont pressés par les bas salaires et les loyers, qui ont atteint des niveaux record, tandis que ceux qui veulent acheter une propriété constatent que le coût mensuel de la location les empêche d\'économiser suffisamment pour monter sur l\'échelle du logement. Une étude publiée par l\'association caritative pour sans-abri Shelter a montré que la moitié des locataires n\'étaient pas en mesure d\'économiser un sou pour un dépôt, tandis qu\'un quart ne pouvait mettre que 100 £ ou moins chaque mois. Les prêts hypothécaires sont moins chers que jamais grâce à des taux d\'intérêt historiquement bas, mais les meilleures offres sont toujours réservées aux emprunteurs disposant de dépôts importants. \nFace avec cela, les jeunes adultes retournent de plus en plus au domicile familial pour économiser de l\'argent et les parents qui n\'ont pas les moyens d\'offrir une somme forfaitaire à leur progéniture semblent prêts à aider. L\'enquête a révélé que 28% des adultes vivaient à la maison parce qu\'ils essayaient d\'épargner pour un dépôt. Cependant, il a également constaté que 30% n\'économisaient pas d\'argent. \nUn porte-parole de l\'entreprise qui a mené l\'enquête a déclaré\xa0: "L\'hôtel de maman et papa reste souvent ouvert plus longtemps que prévu, selon nos dernières recherches. Les frais de location et les dépôts ou la nécessité d\'épargner pour un dépôt hypothécaire signifient que certains enfants doivent naturellement attendre avant de voler le nid. Et, pour certains, déménager ne sera peut-être jamais une option. \nMichael Day, 30 ans, qui vit avec ses parents à Bristol, dit qu\'il a été pris entre payer des loyers élevés et économiser pour un dépôt hypothécaire. Les loyers d\'une maison d\'une chambre dans la ville se situent entre 500 et 800 £ par mois, tandis que l\'achat d\'une propriété similaire coûterait environ 130 000 £. "Je ne veux pas vraiment déménager pour louer car c\'est plus qu\'une hypothèque, mais vous avez besoin d\'un si gros dépôt pour obtenir une hypothèque, donc ça a été un peu un cercle vicieux." \nDay ne veut pas partager avec des inconnus donc ses options sont limitées. À la maison, il paie un loyer nominal pour couvrir ses factures et peut conserver le reste de ses revenus de son travail chez un détaillant de bougies. Il joue au golf au niveau du comté et il admet qu\'au lieu d\'économiser, il dépense son argent de rechange pour le golf et les vacances. "Vous avez besoin de tant d\'argent que je devrai économiser dans un avenir prévisible", a-t-il déclaré. "Parce que ça a été si difficile, je suis parti en vacances et j\'en ai profité." \nSue Green, de Saga, une entreprise qui vend des assurances et des produits aux plus de 50 ans, a déclaré que la majorité des parents n\'avaient peut-être pas prévu que leurs enfants vivent avec eux jusque dans la vingtaine ou la trentaine. "La plupart seront plus qu\'heureuses de les loger gratuitement dans la maison familiale, car cela pourrait aider leurs enfants à accéder plus rapidement à l\'échelle de la propriété", a-t-elle déclaré. "Les enfants qui ne paient pas de loyer peuvent contribuer d\'autres manières, comme acheter des produits d\'épicerie, des plats à emporter pour la famille ou faire des petits boulots à la maison." \nAngus Hanton, co-fondateur d\'un groupe de réflexion appelé la Fondation intergénérationnelle, a déclaré que les générations plus âgées étaient "les architectes de la crise du logement" et que les enfants ne devraient pas être blâmés de rester à la maison. « Les moins de 30 ans ont subi une baisse de leurs revenus moyens d\'environ 20 % depuis la crise de 2008. Les loyers et l\'assurance automobile n\'ont jamais été aussi élevés et les règles de prêt hypothécaire ont été durcies pour les jeunes mais pas pour les investisseurs plus âgés qui achètent pour louer, qui évincent les jeunes », a-t-il déclaré. « La dette liée aux frais de scolarité augmente rapidement, mais de nombreux emplois proposés – des contrats zéro heure et à court terme – transforment les jeunes travailleurs en citoyens de seconde zone. Plutôt que de blâmer les jeunes, nous devrions défendre leurs intérêts afin qu\'ils puissent se permettre de construire leur propre vie. \nJenna Gavin, 29 ans, vit à Southport, Merseyside, dans la maison familiale où elle a grandi. Elle a déménagé pendant un an pour aller à l\'université, mais vit depuis chez ses parents. Elle travaille comme réceptionniste médicale à proximité et souhaite donc rester dans la région. Mais la location d\'un appartement d\'une chambre coûterait plus de 420 £ par mois avant les factures, ce qui absorberait une grande partie de ses revenus. "Je ne veux pas louer - je ne veux pas dépenser tout cet argent et ne rien avoir à la fin", a-t-elle déclaré. "J\'ai envisagé d\'acheter et j\'ai vu des conseillers en prêts hypothécaires, mais je ne peux tout simplement pas emprunter suffisamment pour monter sur l\'échelle de la propriété." \nGavin essaie d\'économiser mais a du mal à amasser les fonds nécessaires. "Vous ne le voyez pas vraiment s\'accumuler autant que vous en avez besoin - même un dépôt de 5% représente une telle somme d\'argent et j\'aimerais en déposer davantage", a-t-elle déclaré. Ses parents sont heureux de ne pas lui faire payer de loyer. "Ils veulent que j\'essaye d\'économiser et je contribue d\'autres manières - j\'apporte de la nourriture et je fais des choses dans la maison." \nGavin s\'entend bien avec ses parents et a son propre espace dans une chambre dans laquelle elle a emménagé à l\'âge de 14\xa0ans, mais elle a toujours imaginé qu\'elle aurait son propre logement à l\'âge de 30\xa0ans. "Je ne vois pas ça se passe comme c\'est l\'année prochaine. Mais j\'espère que dans quelques années, j\'aurai déménagé. \n,,,Lorsque les parents envoient leurs enfants à l\'école, souvent ils ne savent pas ce qui se passe au jour le jour. Les parents se fient à ce que leurs enfants leur disent sur la façon dont ils vont et ce qu\'ils apprennent. Généralement, les écoles organisent des réunions parents-enseignants au moins deux fois par an. Les conférences parents-enseignants sont de courtes rencontres entre les parents et les enseignants de leurs enfants. Habituellement, des conférences parents-enseignants ont lieu lorsque les enseignants donnent les notes d\'un élève pour le trimestre. Les conférences parents-enseignants donnent aux enseignants l\'occasion de faire savoir aux parents comment va leur enfant. Un enseignant informera un parent des points forts de l\'élève et portera à son attention tout problème de notes ou de comportement. Les réunions offrent également aux parents la possibilité de poser des questions et de voir ce que leur enfant apprend.\n \nQuand il est presque l\'heure de Lors des réunions parents-enseignants, l\'école enverra une note aux parents et leur donnera généralement une heure de rendez-vous. Il y a des rendez-vous en journée et en soirée. Les rendez-vous du soir sont utilisés pour les parents qui travaillent pendant la journée. Les réunions parents-professeurs ne durent pas très longtemps. Normalement, ils ne durent pas plus de 10 minutes. C\'est pourquoi il est important que les parents s\'assurent d\'arriver à l\'heure à leurs rendez-vous et de se préparer à poser des questions. Si un parent a besoin de plus de 10 minutes, il ou elle devrait essayer de planifier une autre rencontre avec l\'enseignant. Garder un enseignant plus de 10\xa0minutes lorsque des parents attendent est irrespectueux.\nLa plupart des enfants n\'accompagnent pas leurs parents aux réunions. Cela permet au parent et à l\'enseignant de parler honnêtement des progrès de l\'enfant sans que l\'enfant se sente mal. Habituellement, un enseignant offrira des conseils aux parents sur la façon de soutenir l\'éducation de leur enfant. ,,, y compris une salle de sport privée. Le superyacht à sept ponts comprend également deux piscines, de nombreux jacuzzis, un coiffeur et un barbier.\nPourtant, dans un climat mondial de nantis et de démunis, est-il moralement justifiable de dépenser 1\xa0milliard de dollars pour un yacht privé pouvant accueillir 36\xa0invités\xa0? ?\n Pour être honnête, ce que les mortels ordinaires pensent être moral, n\'est pas réellement pertinent pour ces personnes, a déclaré Kampfner.\n Vous devez comprendre que nous ne parlons pas des 1\xa0% les plus riches -- dans ce cas, nous \\ \'re parler de la 0,01 %. C\'est un tout petit groupe qui vit dans sa propre bulle, ils ne se mélangent qu\'entre eux. \nTimm voit les choses différemment, soulignant les opportunités d\'emploi créées lors de la construction et de l\'exploitation du superyacht.\n Le bateau nécessitera environ deux à trois millions d\'heures humaines pour être construit, sans compter toutes les heures de sous-traitance mondiales, ou le prêt- a fabriqué des articles déjà disponibles sur le marché prêts à être installés sur le yacht, a-t-il déclaré.\n La construction du yacht au chantier naval emploiera directement, au minimum, 500 personnes pendant une période de trois ans. Ces personnes, qui ont des familles à faire vivre et à nourrir, auront un emploi rémunéré.\n Et cela n\'inclut pas les 50 à 70 personnes qui travailleront sur le yacht et seront employées à plein temps par le propriétaire du yacht, ainsi que d\'autres qui fournir le yacht. \nIl étend ce raisonnement aux médias\xa0: vous, le journaliste, bénéficiez indirectement de cette construction en étant employé de manière rémunérée par CNN pour avoir écrit cette histoire. \nEt vous, la personne qui lit cette histoire\xa0? Kampfer voit l\'intérêt du public pour les modes de vie des super riches comme un type de porno de luxe.\n Il existe un énorme sentiment d\'injustice qui est partagé non seulement par les pauvres, mais aussi par la classe moyenne, envers le monde super riche, dit-il.\n Mais avec cela cependant, il y a un sentiment d\'envie et de reluquage. Donc, que vous appeliez cela confusion, ou double standard, ou hypocrisie, ou quoi que ce soit d\'autre, nous abhorrons tous les deux les modes de vie des super riches et en même temps nous sommes infiniment fascinés par eux. \nGalerie de photos\xa0: Superyachts spectaculaires du futur\nExpérience\xa0: La vie secrète d\'une hôtesse de superyacht\n,,,John Lennon est l\'un des individus les plus complexes à avoir jamais parcouru la surface de la terre. Le musicien d\'origine britannique est une véritable icône internationale qui a influencé le monde avec son message de paix et d\'amour. En Amérique, Lennon est l\'une des personnes les plus aimées à avoir élu domicile aux États-Unis.\n \nLennon était l\'un des membres fondateurs du super groupe international, les Beatles. Avec Paul McCartney, il a formé les premiers cerveaux créatifs d\'un groupe qui a changé la musique dans le monde entier. Le succès du groupe n\'a jamais été égalé, et sa musique est aussi populaire aujourd\'hui qu\'elle l\'était dans les années 1960 et 1970.\nLennon a écrit ou co-écrit certains des morceaux les plus mémorables jamais écrits dans le genre Rock \'n\' Roll. Il a connu un grand succès en tant que membre des Beatles, qui est le groupe le plus prospère de l\'histoire sur le plan commercial, et en tant qu\'artiste solo. Lennon était également un activiste politique. Ses opinions sur la religion et la politique ont provoqué une grande controverse aux États-Unis. Il était un critique virulent de l\'implication du pays dans le conflit du Vietnam. Sa popularité a suscité une grande inquiétude auprès des responsables gouvernementaux car il avait l\'attention des jeunes américains à l\'époque.\nLennon vivait à New York depuis plusieurs années, où il était surveillé par le FBI afin de trouver des motifs d\'expulsion lui des États-Unis. Il a utilisé des gadgets flamboyants pour répandre sa parole de paix. Dans cette manifestation passive, Lennon et sa femme Ono se sont adressés aux médias depuis leur lit en affirmant que les gens devraient "faire l\'amour, pas la guerre". Lennon a été tué le 8 décembre 1980 par un tireur fou devant son domicile à New York. Une plaque commémorative sur le site de sa mort est l\'un des sites les plus visités du pays.,,,Aujourd\'hui a été une journée difficile remplie d\'arguments.\nJ\'ai quitté l\'université après avoir terminé mon cours de design graphique en\nété. Jusque-là, j\'avais vécu dans divers logements universitaires\nsubventionnés quatre années. Alors quitter une vie protégée et\navoir à se loger sur la place du marché a été un vrai\nchoc. Nous avons décidé, deux filles et deux garçons, de trouver un appartement, et je me suis\n porté volontaire pour parler aux propriétaires.\nA première vue, on aurait pu penser que notre ville universitaire serait\nan un endroit facile pour trouver un logement. Toutes les industries lourdes et\nmanufacturières ont fermé il y a quelques décennies\net de nombreuses personnes ont déménagé pour trouver du travail, donc par rapport au\npays dans son ensemble, les prix des maisons sont bas ici. Des appartements de différentes\ntailles sont disponibles, mais trouver un appartement bon marché de taille raisonnable\nabordable pour quatre jeunes était un défi.\nComme toute ville importante, la nôtre a des quartiers avec des logements différents. Goldmark à l\'ouest contient la plupart des logements riches et nous n\'avons pas pris la peine de chercher là-bas. La sortie de Newtown vers le sud\nest principalement des blocs de logements préfabriqués avec des taux de chômage élevés mais très peu de choses\nsur le plan culturel. Après avoir éliminé ces zones, nous avons limité notre recherche à Babylone, à l\'est\navec son contexte multiculturel.\nSur les quatre, trois vivaient d\'épargne et de prestations sociales. Paul, l\'un des garçons,\navait un travail de nuit dans un mini-supermarché ouvert 24h/24 dans le centre-ville. Notre profil \néconomique médiocre signifiait que les propriétaires d\'appartements étaient réticents à nous louer, craignant que nous ne parvenions\nà payer nos loyers. En fin de compte, nous avons prétendu avoir des emplois que nous n\'avions pas. Si nous ne l\'avions pas\nfait, nous n\'aurions trouvé nulle part.\nL\'aspect de loin le plus troublant lors de notre recherche était le caractère délabré d\'une grande partie\ndes biens proposés. Une grande partie était bien en deçà des normes légales\xa0: les systèmes de gaz et d\'électricité\nétaient mal entretenus et les systèmes d\'égouts étaient en deçà des normes. Habituellement, ces propriétés\nétaient mal meublées et extrêmement sales. Nous avons dû choisir l\'option la moins pire\nqui était un appartement de trois pièces au quatrième étage d\'un immeuble.\nNous avons emménagé aujourd\'hui. Le propriétaire a demandé une caution de mille euros et trois mois de loyer à l\'avance. Avec beaucoup de difficulté, nous nous sommes conformés. L\'endroit est crasseux et j\'ai dressé une liste des travaux à effectuer\npour rendre l\'endroit vivable. Comme nous n\'avons que trois pièces, nous avons tiré au sort\npour voir qui finirait par la cuisine\xa0: l\'autre fille, Dora, a choisi la paille courte. Eh bien, elle\npeut faire des tasses de thé pendant la nuit.\nQuoi qu\'il en soit, je dois signer maintenant car j\'ai du travail à faire.\nJ,,,Soutenir une campagne du Congrès signifie généralement donner de l\'argent, frapper aux portes et passer des appels téléphoniques. Cela signifie rarement agir comme un nudiste dans une publicité télévisée qui sera diffusée dans toute votre communauté.\nC\'est jusqu\'à ce que la campagne Kelly Kultala pour le Congrès demande exactement cela à ses partisans.\nKultala, un démocrate candidat à l\'éviction du représentant républicain du Kansas. Kevin Yoder, a publié jeudi une publicité mettant en vedette une poignée de personnes apparemment nues exhortant les téléspectateurs à soutenir le démocrate lors des élections de novembre.\nLes personnes dans la publicité, cependant, ne sont pas de véritables nudistes. Ce sont des partisans de la campagne Kultala qui ont accepté d\'apparaître à la télévision avec différents objets devant leurs morceaux nus.\n Ce sont des bénévoles qui ont accepté de le faire parce qu\'ils soutiennent notre campagne et notre message, a déclaré Jacob Becklund, le directeur de campagne de Kultala . Il convient également de noter que personne n\'était réellement nu pendant le tournage. \nSelon Becklund, c\'était une vente assez facile à tous nos bénévoles, même si l\'annonce -- qui vient avec un coût élevé à cinq chiffres -LRB- derrière - sera vue sur le câble et diffusée dans le Marché de Kansas City à partir de vendredi.\nLa nudité -LRB- fausse -RRB- dans l\'annonce - intitulée Naked Truth - a frappé Yoder, qui en 2011 a plongé maigre dans la mer de Galilée en Israël, une décision pour laquelle il a été réprimandé par les républicains de la Chambre en 2012.\n Il y a quelque temps, le membre du Congrès Yoder a fait l\'actualité en se baignant au travail, dit Kultala dans l\'annonce. Mais c\'est plus éhonté ce qu\'il fait au Kansas. \nLa publicité présente ensuite les volontaires apparemment nus citant différents aspects du bilan de Yoder, notamment le fait que Yoder remplit les poches de ses donateurs millionnaires avec d\'importantes réductions d\'impôts. Et ils tapissent le sien, quand il a des poches. \n Nous avons pensé que c\'était une accroche humoristique, a déclaré Becklund, pour amener les gens à prêter attention au message. \n,,,L\'ère du Vietnam est l\'une des époques les plus conflictuelles de l\'histoire américaine. L\'ère du Vietnam pour les États-Unis s\'est déroulée approximativement de 1960 à 1975. Du milieu à la fin des années 60, l\'implication de l\'Amérique est devenue un énorme problème. Le conflit au Vietnam a été source de division pour de nombreuses raisons, mais principalement à cause de la perception du public qu\'il s\'agissait d\'une opération immorale et illégale. L\'autre facteur majeur était que les États-Unis étaient toujours engagés dans l\'enrôlement de jeunes hommes au service du pays contre leur gré.\n \nL\'une des principales stars du cinéma hollywoodien de l\'époque était Jane Fonda. Fonda est issue de la royauté du cinéma hollywoodien, étant la fille du grand acteur Henry Fonda. Elle a remporté deux Oscars, sept Golden Globe Awards et un Emmy award tout au long de sa longue carrière. Dans les années 60, elle était une militante politique qui était contre la guerre au Vietnam. Elle était très franche contre le conflit, comme l\'étaient de nombreuses célébrités à l\'époque.\nEn 1970, Fonda a fait la une des journaux pour une raison très différente. Elle et deux autres acteurs ont joué dans un film anti-guerre qui a suscité beaucoup de controverse. En juillet 1972, Fonda a attiré la colère de nombreux Américains lorsqu\'elle a été photographiée assise avec des membres de l\'armée nord-vietnamienne sur un canon anti-aérien. Cette image a entaché sa réputation d\'actrice et de véritable Américaine. Beaucoup ont qualifié Fonda de traître pour avoir interagi avec l\'ennemi. Des années plus tard, Fonda a déclaré qu\'elle avait été manipulée pour prendre cette photo, mais de nombreux Américains ont encore des doutes.\nAu cours de son voyage au Nord-Vietnam, qui était en guerre avec le Sud-Vietnam soutenu par les États-Unis, Fonda a fait un certain nombre d\'émissions de radio dénonçant l\'Amérique. L\'implication de \\ dans la guerre. Cela a encore renforcé son image de traître. Cependant, de nombreux Américains pensent qu\'elle était une vraie patriote parce qu\'elle a refusé de rester immobile et s\'est prononcée contre la guerre.,,,Les autorités maliennes ont trouvé une fosse commune contenant 21 corps qui seraient des soldats tués alors qu\'un chef de la junte était en pouvoir après un coup d\'État militaire en 2012, a déclaré mercredi une porte-parole du ministère de la Défense. Amadou Sanogo, qui a dirigé le coup d\'État de mars\xa02012, a été arrêté, accusé de complicité d\'enlèvement.\nUn contre-coup d\'État manqué peu après sa prise du pouvoir a opposé deux factions de l\'armée\xa0: les bérets verts fidèles à Sanogo et les bérets rouges fidèles à le président déchu Amadou Toumani Touré.\n Au camp militaire de Diago, il y avait à la fois des soldats bérets verts et des soldats bérets rouges. Nous ne savons pas à quel groupe appartenaient les soldats dans la tombe, a déclaré Diarra. travaille sur l\'affaire depuis avril, donc juste après le coup d\'État. Ils connaissaient la tombe depuis un certain temps, mais quand ils y sont finalement allés et ont ouvert la tombe, il y avait plus de corps qu\'ils ne le pensaient auparavant, a déclaré Diarra.\nLes corps sont tous des hommes et de différents groupes ethniques, a-t-elle déclaré.\nComme les corps sont retirés de la tombe, les enquêteurs médico-légaux commenceront le processus d\'identification des hommes.\n Les familles des soldats disparus lors du coup d\'État seront invitées à venir identifier les membres de leur famille, a déclaré Diarra.\n,,,1 nuages \u200b\u200b\nL\'un des plaisirs du vol est de voir les nuages \u200b\u200bde près. Même s\'ils semblent inconsistants, ils transportent un poids d\'eau considérable – environ 500 tonnes dans un petit cumulus. Et l\'eau est plus dense que l\'air. Alors pourquoi les nuages \u200b\u200bne tombent-ils pas du ciel comme la pluie ? Ils font. Mais les gouttelettes mettent beaucoup de temps à couler. Un nuage moyen mettrait un an à tomber d\'un mètre. \n2 Sur le nuage\xa09 \nLa plupart d\'entre nous sont heureux d\'appeler les nuages \u200b\u200b"duveteux" ou "méchants noirs", mais les météorologues identifient plus de 50\xa0types de nuages \u200b\u200ben fonction de leur forme et de leur altitude. Ceux-ci s\'inscrivent dans des catégories numérotées de un à neuf. Le nuage neuf est le vaste et imposant cumulonimbus, donc être « sur le nuage neuf » implique d\'être au sommet du monde. \n3 Autour de l\'arc-en-ciel \nIl n\'y a pas de meilleur endroit pour voir un arc-en-ciel que d\'un avion. Les arcs-en-ciel sont produits lorsque la lumière du soleil frappe les gouttes de pluie. Nous voyons un arc parce que la Terre gêne, mais, d\'un avion, un arc-en-ciel est un cercle complet. Lors du passage au-dessus des nuages, l\'ombre de l\'avion apparaît nettement au centre de l\'effet. \n4 Monsieur ciel bleu \nLa lumière du soleil est blanche, contenant toutes les couleurs du spectre mais, lorsqu\'elle traverse l\'air, une partie de la lumière est diffusée lorsqu\'elle interagit avec les molécules de gaz. La lumière bleue diffuse plus que les couleurs à faible énergie, de sorte que le bleu semble provenir du ciel. \n5 Il y a de la vie là-bas \nEn dehors des nuages \u200b\u200bet des autres avions, nous ne nous attendons pas à voir grand-chose directement à l\'extérieur du hublot d\'un avion en vol, mais l\'air bouillonne de vie bactérienne\xa0: jusqu\'à 1\xa0800\xa0types différents de bactéries ont été détectés villes et ils peuvent atteindre deux fois la hauteur de croisière d\'un avion. \n6 Terreur de la turbulence \nMême le pilote le plus expérimenté peut devenir vert à cause de la turbulence. Le résultat peut être n\'importe quoi, des chocs répétés aux plongeons soudains et dramatiques. La bonne nouvelle pour les voyageurs nerveux est qu\'aucun avion de ligne moderne n\'a jamais été abattu par des turbulences. Des personnes ont été blessées et parfois tuées lorsqu\'elles ne sont pas attachées ou sont heurtées par des bagages mal sécurisés - mais l\'avion ne sera pas renversé du ciel. \n7 Rayonnement en vol \nLorsque les scanners corporels ont été introduits dans les aéroports, il y a eu des alertes de rayonnement, mais le niveau produit est le même que celui que les passagers reçoivent en une minute de vol. La Terre est constamment bombardée par les rayons cosmiques, radiat naturel ion de l\'espace qui a plus d\'impact en altitude. \n8 Vous ne pouvez pas guérir du décalage horaire \nLe monde est divisé en fuseaux horaires. Le résultat est que les voyages long-courriers entraînent une différence entre l\'heure locale et l\'heure de votre corps, ce qui provoque un décalage horaire. Cependant, ses effets peuvent être minimisés en gardant les aliments fades pendant 24 heures avant le voyage, en buvant beaucoup de liquides et en vivant à l\'heure de votre destination à partir du moment où vous atteignez l\'avion. \n9\xa0747 supersoniques \nBeaucoup d\'entre nous ont voyagé plus vite que le son. Il existe un certain nombre de courants-jets dans la haute atmosphère, notamment lors du voyage des États-Unis vers l\'Europe, où une inversion de température fait qu\'un couloir d\'air se déplace à une vitesse pouvant atteindre 250 mph. Si un avion de ligne avec une vitesse de 550 mph entre dans un jet stream, le résultat peut être de voler à 800 mph, au-dessus des 740 mph du son. \n10 Voler dans le temps \nLes fuseaux horaires offrent un voyage artificiel dans le temps, mais la relativité restreinte signifie qu\'un vol implique un véritable voyage dans le temps. C\'est si minime, cependant, que traverser l\'Atlantique chaque semaine pendant 40 ans ne vous ferait avancer que de 1/1 000e de seconde dans le futur. \n11 Thé horrible \nNe blâmez pas le personnel de cabine si votre thé n\'est pas bon. L\'eau doit être légèrement inférieure à 100 °C lorsqu\'elle est versée sur des feuilles de thé, mais ce n\'est pas possible dans un avion. Il est impossible d\'avoir de l\'eau à plus de 90°C pendant le vol - alors choisissez le café. \n12 Je n\'entends pas ma nourriture \nLa nourriture des compagnies aériennes a la réputation d\'être fade et insipide. Une partie du problème peut ne pas être une mauvaise restauration, cependant. Un avion est un environnement bruyant et il est prouvé que la nourriture perd une partie de sa saveur lorsque nous sommes exposés à des bruits forts. \n13 Aiguille dans une botte de foin \nAvec la technologie moderne, il semble étrange que le vol malaisien MH370 puisse disparaître sans laisser de trace. Pourtant, retrouver un avion disparu est un problème d\'aiguille dans une botte de foin. L\'avion connaît sa position, à la fois grâce au GPS et au suivi inertiel, mais cette information n\'est pas relayée ailleurs en temps réel. Ce serait parfaitement possible. Les navires de haute mer sont suivis depuis les années 1980 – la limite n\'est pas la technologie mais l\'absence de législation l\'exigeant. \n14 Retombées volcaniques \nLes voyages aériens peuvent être annulés par l\'activité volcanique. Les particules de cendres semblables à du verre fondent dans la chaleur du moteur, puis se solidifient sur les rotors. Une politique de ciel clair dans un nuage de cendres peut être gênante - mais les risques d\'ignorer les cendres sont clairs. \n15 Le mythe des ailes \nPendant de nombreuses années, nous avons enseigné la mauvaise explication de la façon dont les ailes maintiennent les avions en l\'air. En fait, presque toute la portance d\'un avion provient de la troisième loi du mouvement de Newton. L\'aile est formée pour pousser l\'air vers le bas. Lorsque l\'air est poussé vers le bas, l\'aile reçoit une poussée égale et opposée vers le haut, soulevant l\'avion. \n16 Oubliez les avions électriques \nLorsque nous voyons des avions électriques ultra-légers expérimentaux, il est facile de supposer qu\'il y aura bientôt des avions de ligne propres, verts et électriques, mais cela n\'arrivera pas de si tôt. Le carburant des avions contient une quantité d\'énergie remarquable. Les batteries sont beaucoup moins efficaces. Pour fournir la même énergie qu\'une tonne de carburant, il faudrait 100 tonnes de batteries – et un 747 consomme 150 à 200 tonnes de carburant. À moins que la technologie des batteries ne soit rendue beaucoup plus efficace, les avions de ligne électriques ne décolleront pas. \n17 Attention au vortex \nLes pilotes attendent souvent longtemps avant d\'obtenir une autorisation. Il s\'agit de permettre à l\'air de se stabiliser après un décollage précédent, car les bouts d\'ailes d\'un avion génèrent des tourbillons dans l\'air, qui peuvent prendre deux ou trois minutes à se disperser. Si l\'avion suivant décollait immédiatement, l\'air en mouvement rapide rendrait l\'avion difficile à piloter. Le retard donne à l\'air le temps de se remettre des tourbillons miniatures causés par l\'avion précédent. \n18 Les portes ne sont pas verrouillées \nEn pratique, les portes d\'un avion n\'ont pas besoin d\'être verrouillées. Si vous regardez une porte d\'avion s\'ouvrir, elle oscille d\'une manière inhabituelle. Il faut d\'abord l\'ouvrir vers l\'intérieur avant de l\'écarter. Une fois que l\'avion a décollé, une différence de pression importante se crée rapidement entre l\'intérieur de l\'avion et l\'extérieur. Ce différentiel force la porte en place. Pour l\'ouvrir, il faudrait tirer contre la pression de l\'air, bien au-delà des capacités des muscles humains.\n,,,L\'actrice de Transformers Megan Fox a dit oui à la demande en mariage de l\'ex-fiancé Brian Austin Green, quatre mois seulement après annulant un engagement de deux ans avec Green, a confirmé son publiciste. \nFox, 24 ans, était tellement excitée quand Green, 36 ans, a posé la question sur une plage d\'Hawaï qu\'elle a laissé tomber la bague de fiançailles en diamant de 2 carats dans le sable, Us Weekly a rapporté le magazine.\nMalgré une longue recherche par le personnel de l\'hôtel, la bague n\'a jamais été retrouvée, a rapporté le magazine.\nFox et Green ont commencé à se fréquenter il y a six ans et se sont fiancés pour la première fois en 2007. Ils ont rompu en février dernier.\nMariage peut être plus facile pour Fox maintenant qu\'elle n\'est plus dans la franchise Transformers. Le réalisateur Michael Bay a annoncé ce mois-ci qu\'elle était remplacée en tant qu\'intérêt amoureux de Shia LaBeouf par l\'actrice-modèle Rosie Huntington-Whiteley pour Transformers 3. \nFox dépeint Mikaela Banes dans le premier Transformers et sa suite Transformers: Revenge of the Fallen. \nGreen a passé une décennie à jouer David Silver dans la série télévisée à succès Beverly Hills, 90210 jusqu\'à ce qu\'elle soit annulée en 2000.\nBretagne Kaplan de CNN a contribué à ce rapport\n,,,La fête du travail est célébrée le premier lundi de septembre . C\'est une fête destinée à honorer les contributions, à la fois économiques et sociales, des travailleurs américains. C\'est un jour férié fédéral officiel depuis 1894, ce qui signifie que les bureaux du gouvernement, y compris les bureaux de poste et les écoles, sont fermés.\n \nIl est également largement admis que la fête du Travail a été créée comme un jour férié alternatif au 1er\xa0mai. Le 1er mai, en particulier dans les années 1800, était devenu un jour de protestation, lorsque les travailleurs défilaient pour exiger de meilleurs droits et conditions de travail. Il y avait souvent des grèves qui se terminaient par la violence. À l\'époque, le gouvernement américain avait peur que le 1er mai soit dirigé par des communistes et ait une mauvaise influence sur les travailleurs. Certaines personnes pensent que septembre a également été choisi car il n\'y a pas d\'autres jours fériés entre le 4 juillet et Thanksgiving en novembre.\nLe jour de la fête du Travail, il y a généralement des défilés dans les grandes villes du pays. Les politiciens et les dirigeants syndicaux prononcent des discours et tiennent des conférences de presse. La fête du Travail est également considérée comme la fin non officielle de l\'été. Beaucoup de gens célèbrent la fête du Travail avec des excursions sur les plages, des pique-niques et des barbecues. De nombreuses villes organisent des feux d\'artifice ce jour-là. Beaucoup de gens prennent également leurs dernières vacances d\'été à cette époque. La fête du Travail marque également le début de la saison de football professionnel et universitaire aux États-Unis.\nLa fête du Travail est également devenue une importante fête du commerce de détail. De nombreux magasins organisent des ventes pour la fête du Travail qui coïncident avec le début de l\'année scolaire pour de nombreux enfants aux États-Unis. Cela signifie que de nombreux employés de magasin, pendant la célébration de leurs vacances, doivent travailler de longues heures. ,,, C\'est bien connu l\'industrie de la mode est l\'une des plus difficiles à percer. Pour se conformer au stéréotype traditionnel d\'un modèle, vous devez être mince, magnifique et avoir des jambes qui durent éternellement, n\'est-ce pas\xa0? Le premier mannequin albinos.\n J\'étais tenace et je disais non, attends une minute. C\'est une représentation différente de ce qu\'est la beauté africaine, raconte Modiselle à CNN.\n On vous dit qu\'un enfant noir doit avoir la peau brune, mais comment appelle-t-on alors une fille comme moi ? Je ne suis pas grand non plus. Mais là, je faisais du mannequinat avec des gens comme vos Adiambo\'s, vos célèbres mannequins qui ont vraiment bien marché. \nEn 1999, Modiselle n\'avait que 13 ans lorsqu\'elle a été approchée pour la première fois pour faire un shooting de cinq pages pour un magazine jeunesse visant à montrer comment le paysage de la mode africaine allait changer avec le tournant du millénaire. C\'était son premier goût pour la mode, mais elle a attrapé le virus.\nPourtant, ce n\'est qu\'après avoir terminé ses études secondaires et être entrée au collège -LRB- à la demande de sa mère -RRB- que Modiselle s\'est retrouvée dans à nouveau devant une caméra.\nRedéfinir la beauté\nCette décision a lancé sa carrière et bientôt l\'étudiante sud-africaine s\'est regardée à partir des pages de magazines sur papier glacé et a glissé sur les podiums lors d\'événements de grande envergure, comme la Mercedes Benz Africa Fashion Week.\n I défiait tout sur ce que la société dit définit un modèle. En gros, je dis OK, enseignons à nos enfants une définition différente de la beauté, ou une définition qui a toujours été là mais qui a toujours été ignorée. \nL\'albinisme est une maladie héréditaire causée par un manque de mélanine, un pigment présent dans la peau, les cheveux et les yeux. Bien qu\'il soit assez courant dans le monde - une personne sur 17\xa0000 naît avec une forme de maladie génétique - les albinos en Afrique sont souvent victimes de discrimination et même, dans certains pays, chassés et tués en raison de leur apparence.\n les personnes atteintes d\'albinisme n\'ont souvent pas la possibilité d\'entrer dans de telles industries parce que nous ne sommes pas connus comme des extravertis, nous n\'avons pas la chance d\'être identifiés dans la société comme des personnes qui ont le potentiel de représenter quelque chose, dit-elle. nCrise d\'identité\nMais Modiselle n\'a pas seulement à vivre avec la menace tacite de la discrimination qui plane dans l\'air. Née à Soweto, la jeune mannequin fougueuse a parfois lutté avec son identité.\nLa jeune femme de 28\xa0ans explique\xa0: L\'Afrique du Sud a une histoire d\'apartheid, où la race est une si grande ségrégation. Comment vous placez-vous alors ? Tu es une fille qui est née dans une société noire : le township. Et maintenant, il faut s\'intégrer dans un monde où, on vous dit : ceci est noir, ceci est blanc et vous êtes dans un pays qui combat constamment ces aspects. C\'est quelque chose qui n\'est pas facile parce que les gens essaient de vous situer et de vous identifier d\'une manière spécifique. \nConfirmant son statut de nom familier, elle passe également une journée par semaine à filmer un talk-show d\'actualité avec d\'autres personnalités de la télévision sud-africaine Masechaba Lekalake et le comédien Tol Ass Mo..\nLa participation à l\'émission permet à Modis elle pour révéler plus de sa personnalité au pays et pour montrer qu\'elle est plus qu\'un simple étendoir. Bien qu\'elle n\'ait même pas 30 ans, Modiselle pense déjà à son héritage.\n J\'enseignerai à travers les choses que je fais. -LRB- Mon plan -RRB- pour l\'avenir est simplement de me développer en tant que marque et en tant qu\'entité\xa0: toucher le monde à différents endroits et être en mesure d\'être un énorme représentant pour l\'Afrique. \n,,,Montant dans le métro de Londres il y a cinq ans, la trentenaire responsable marketing n\'aurait jamais imaginé que son trajet matinal pourrait éventuellement mener aux Jeux Paralympiques. Maintenant, c\'est exactement là que la Britannique Martine Wright vise.\nWright a perdu ses deux jambes lorsqu\'un kamikaze s\'est fait exploser dans une rame de métro près de la station Aldgate de Londres le matin du 7 juillet 2005 . Elle a également subi une fracture du crâne et de graves blessures au bras.\nBien qu\'elle ait été l\'une des plus gravement blessées ce jour-là, Wright vise maintenant une place dans l\'équipe paralympique britannique de volley-ball en 2012.\n J\'adorerais, j\'adorerais, participer aux Jeux paralympiques de 2012, a déclaré Wright à ITN, affilié à CNN, le mois dernier. Je veux dire, si tu m\'avais dit ça il y a cinq ans, j\'aurais pensé que tu étais complètement fou. Mais retourner à Londres, où je suis né, et faire quelque chose d\'aussi positif – après être parti sur un tel négatif – serait absolument incroyable. \nQuatre bombes ont été lancées ce jour-là en juillet, sur trois rames de métro et un bus. Cinquante-six personnes ont été tuées, dont les quatre auteurs de l\'attentat à la bombe, et plus de 770 ont été blessées.\nLes attentats ont eu lieu le lendemain de l\'annonce par le Comité international olympique que Londres accueillerait les Jeux olympiques et paralympiques en 2012.\nWright a déclaré qu\'elle avait fait une promesse à elle-même et à sa famille après les attentats pour saisir toutes les opportunités qu\'elle pouvait.\n Je pense que j\'ai fait plus maintenant que ce que j\'ai fait auparavant, a-t-elle déclaré à ITN. J\'ai appris à piloter des avions, et j\'ai sauté d\'un avion, j\'ai réappris à skier. \nElle a concouru pour - et a gagné - une place dans l\'équipe britannique de volleyball assis. La Fédération britannique de volley-ball a annoncé sa place dans l\'équipe le mois dernier, et cette semaine, l\'équipe participe aux Championnats du monde de volley-ball assis 2010 dans l\'État américain de l\'Oklahoma.\nQuelque 640\xa0athlètes de 20\xa0pays participent à l\'événement, le plus grand compétition internationale majeure avant Londres 2012, selon Lisa Wainwright, directrice générale de Volleyball England, l\'instance dirigeante nationale du sport.\nJohn Bestebroer, l\'entraîneur-chef de l\'équipe féminine, considère les championnats comme une excellente opportunité d\'entraînement pour 2012. \nLe volley-ball assis est une version du sport qui permet à tout le monde de participer, y compris les personnes handicapées. L\'épreuve masculine fait partie des Paralympiques depuis 1980 et l\'épreuve féminine depuis 2004, selon British Volleyball.\nLes règles internationales du volley-ball s\'appliquent dans la version assise, avec certaines modifications pour les joueurs handicapés\xa0: Pendant le jeu, un joueur doivent toucher le terrain avec une partie du corps entre les fesses et les épaules, et les joueurs sont autorisés à bloquer les services. De plus, le volley-ball assis se joue sur un terrain plus petit avec un filet plus bas.\nWright a déclaré à l\'Evening Standard de Londres que les Jeux de 2012 auront une émotion supplémentaire parce que son dernier vrai souvenir avant les attentats était de célébrer avec des collègues lorsque Londres a été choisie pour accueillir les Jeux olympiques.\nElle a été la dernière personne retirée de la station ce jour-là, et Wright a dit qu\'elle se sentait l\'une des plus chanceuses.\n Même les jours les plus sombres, il y aura toujours quelqu\'un qui sera pire que vous ne l\'êtes, malheureusement, a déclaré Wright à ITN. Il y avait 52 personnes ce jour-là qui étaient définitivement pires que moi, parce que je suis toujours là. \n,,,Une fillette de 4 ans de l\'Ohio qui a disparu il y a plus de trois semaines a été retrouvée vivante et en bon état, à l\'autre bout du pays, dans le sud de la Californie, ont annoncé les autorités.\nHaylee Donathan a été retrouvée avec sa mère et sa mère Le petit ami de \\, le délinquant sexuel fugitif Robbie Potter.\nHaylee Donathan, sa mère Candace Watson et Robbie Potter ont été découverts cachés la semaine dernière au Morning Star Ranch, une retraite à Valley Center, près de San Diego, a déclaré Peter Elliott, United Maréchal des États pour le district nord de l\'Ohio.\nPotter est un délinquant sexuel enregistré, ont déclaré des responsables. Il était recherché par le U.S. Marshal\'s Service.\nTard mardi, Haylee était sous la garde d\'une agence de services de protection pour enfants dans la région du comté de San Diego, a déclaré Elliott. Elle allait bien mais a peut-être la varicelle, a-t-il ajouté.\n Nous comprenons qu\'elle est en bonne santé et heureuse et je crois qu\'elle attend de revenir chez sa grand-mère ici, a-t-il dit. Les autorités de surveillance annoncent leur découverte \nLe voyage vers l\'ouest les a emmenés à plus de trois mille kilomètres de Mansfield, la ville du centre-nord de l\'Ohio où vivent la fille et sa mère.\n Je suis très heureuse que ma petite-fille rentre à la maison, a déclaré Mary Watson, la grand-mère de Haylee. Et je veux juste vous remercier tous du fond du cœur. \nWatson, 24\xa0ans, et Potter, 27\xa0ans, nous re arrêté sans incident et détenu par le département du shérif du comté de San Diego. Lorsqu\'on leur a demandé leur réaction, Elliott a dit, je crois un peu choqué que nous ayons pu les trouver. \nPotter et Watson utilisaient toujours le même véhicule, une camionnette Chevrolet bleue des années 1980 qu\'ils conduisaient depuis l\'Ohio, a déclaré Elliott.\nLes cheveux de Watson étaient un peu plus foncés et Potter a également légèrement changé son apparence.\nLes autorités craignaient que Haylee ne soit en danger. . Potter, un délinquant sexuel, s\'était échappé d\'une maison de transition le 28 mai, le jour où la fille et sa mère sont tombées hors de vue. a déclaré sans donner plus de détails.\nMais un conseil des membres du ranch aurait pu conduire au raid et à l\'arrestation, a déclaré un membre du ranch à CNN.\nKevin Carlin, membre du ranch dans le quartier de Valley Center, a déclaré à CNN lors d\'un entretien téléphonique que Les membres du ranch sont devenus méfiants lorsqu\'un ancien invité du ranch a vu des panneaux d\'affichage demandant de l\'aide pour retrouver Haylee et les deux adultes. les médias locaux et Clear Channel, qui ont mis en place des panneaux d\'affichage d\'enfants disparus à travers le pays.\nQuand elle sera renvoyée dans l\'Ohio, Haylee sera placée sous la garde de sa grand-mère, a déclaré Elliott. Le U.S. Marshals Northern Ohio Violent Fugitive Task Force avait offert une récompense de 10\xa0000\xa0$ pour toute information menant à l\'arrestation de Potter, âgé de 27\xa0ans.\nPotter était recherché en vertu d\'un mandat d\'État pour violation de la libération conditionnelle et d\'un mandat fédéral pour violation de la probation. Selon le U.S. Marshals Service, une unité du ministère de la Justice, il est un délinquant sexuel reconnu coupable de niveau\xa03.\nAlors qu\'ils étaient en fuite, Potter et Watson auraient été vus sur vidéosurveillance le 31\xa0mai dans un magasin Wal-Mart à Marion, Ohio, à 40 milles de Mansfield. Haylee n\'était pas sur cette vidéo. \nEn plus des inquiétudes, la colocataire de Watson, Samantha Covert, a déclaré la semaine dernière dans une interview sur Nancy Grace qu\'elle avait vu la petite fille au lit avec sa mère et Potter le matin de leur disparition. Covert a déclaré que Watson transportait un panier de vêtements, d\'oreillers et de couvertures.\nWatson devra probablement faire face à des accusations d\'État, y compris l\'hébergement d\'un fugitif, selon Elliott.\nEn plus d\'autres accusations, il a déclaré que Potter pourrait être poursuivi en vertu de la protection de l\'enfance d\'Adam Walsh et Safety Act, une loi fédérale sur l\'enregistrement et l\'auto-déclaration des délinquants sexuels qui entraîne une peine de plusieurs années pour crime en cas de violation.\nSur son site Web, le Morning Star Ranch se décrit comme une communauté de producteurs d\'avocats, de pamplemousses et de , kakis, noix et autres produits. \nElliot a déclaré qu\'il pensait qu\'il s\'agissait d\'un centre de désintoxication et l\'appelait également une retraite chrétienne.\n,,,Deux des principaux partis politiques turcs font pression pour un amendement constitutionnel visant à lever l\'interdiction du port du foulard dans les universités publiques, une décision qui a suscité des inquiétudes parmi la population laïque de Turquie.\nLa levée de l\'interdiction du foulard a suscité des inquiétudes parmi la population laïque de Turquie.\nLe Premier ministre Recep Tayyip Erdogan a initié cette décision, affirmant que cela créerait l\'égalité en Turquie l\'enseignement supérieur.\nLa commission constitutionnelle discutera de la proposition - soumise par les partis AKP et MHP - dans les prochains jours avant de la soumettre au vote.\nSi elle est approuvée, elle nécessiterait l\'approbation du président Abdullah Gul , ce qui est attendu.\nEn vertu de la proposition, les voiles, les burqas ou les tchadors - qui couvrent tous le visage d\'une femme - ne seraient pas autorisés.\nLes interdictions de se couvrir la tête ont été imposées au début des années 1980 par les universités turques. parce qu\'ils étaient considérés comme des symboles politiques et des conflits ted avec le système de gouvernement laïque de la Turquie.\nLa proposition de changer la constitution de la Turquie a envoyé des frissons à travers la population laïque de la Turquie. Des groupes de femmes se sont rendus au Parlement mardi pour exprimer leur rejet.\n Il s\'agit d\'une menace directe pour la république et ses fondements, a déclaré Deniz Baykal, chef du principal parti laïc de Turquie, le CHP.\nUne autre députée du CHP a déclaré qu\'elle craint que si la proposition est adoptée, les parents se sentiront obligés de faire porter le foulard à leurs filles, même à l\'école élémentaire.\nMustafa Akaydin, chef de la Commission de l\'enseignement supérieur de Turquie, est contre la proposition. Il a déclaré qu\'autoriser le port du foulard serait un rejet du système de gouvernement laïc de la Turquie.\n C\'est une tentative de créer une contre-révolution, a déclaré Akaydin. Ce sera un point de rupture. \nIl a déclaré que la majorité des lycéennes d\'une école portaient le foulard lors des examens d\'entrée du week-end dernier, une rareté dans les écoles turques.\nLa Commission de l\'enseignement supérieur se réunira vendredi à Ankara pour discuter des changements proposés. E-mail à un ami\n,,,Bien avant le mouvement féministe de la fin des années 1960 et 1970, vivait une femme du 19ème siècle du nom de Susan B. Anthony. Anthony se battait pour les droits des femmes bien avant la révolution sociale il y a 50 ans. Elle est née en 1820 dans une famille quaker. Quak Les ers sont un groupe de personnes étroitement liées, basées sur la foi, qui vivent une vie très simple et stricte. C\'est un mode de vie traditionnel où les rôles de genre sont clairement définis.\n \nElle a passé une grande partie de sa jeunesse à se battre pour le changement social. Elle était une abolitionniste anti-esclavagiste, une enseignante, une auteure et une figure de proue pour obtenir le droit de vote et d\'exercer des fonctions pour les femmes. Le mouvement pour le suffrage a été conçu pour obtenir le droit de vote des femmes qui n\'avaient auparavant aucune voix au gouvernement. Une autre cause sociale dans laquelle Anthony était fortement impliqué était le mouvement anti-esclavagiste. Anthony a mis sa ferme familiale à la disposition de personnes partageant les mêmes idées pour organiser des réunions anti-esclavagistes. Elle a été impliquée dans le mouvement abolitionniste jusqu\'à la fin de la guerre civile américaine, lorsque l\'esclavage a été aboli. Elle a ensuite tourné son attention vers les droits des femmes.\nAnthony a fondé une publication hebdomadaire appelée "The Revolution", qu\'elle a utilisée pour publier de la littérature sur les droits des femmes. La devise du journal était "Les hommes ont leurs droits, et rien de plus\xa0; les femmes leurs droits, et rien de moins".\nEn 1872, après avoir parcouru le pays en essayant de créer une dynamique pour les droits des femmes, elle vote illégalement à l\'élection présidentielle élection. Elle a été arrêtée et condamnée à une amende, même si elle s\'est battue. Anthony n\'a jamais payé l\'amende. L\'incident a servi à sensibiliser davantage à la lutte que les femmes ont dû endurer. Anthony a combattu le bon combat jusqu\'à sa mort en 1905 à l\'âge de 84 ans. Elle est honorée aujourd\'hui par la Monnaie des États-Unis avec la pièce de 1 $ Susan B. Anthony.,,,Thanksgiving est une fête célébrée par de nombreuses personnes aux États-Unis et Canada. Aux États-Unis, il est célébré le quatrième jeudi de novembre et marque le début de la saison de Noël. Certaines personnes célèbrent Thanksgiving pour se souvenir de la première récolte des pèlerins et des puritains, des groupes de personnes d\'Angleterre qui ont immigré en Amérique du Nord dans les années 1600. Certaines personnes recréent le premier Thanksgiving et se déguisent en pèlerins, en puritains et en Amérindiens qui auraient aidé les nouveaux arrivants à trouver de la nourriture pour survivre à leur premier hiver.\n \nLa plupart des gens, cependant, voient Thanksgiving comme un jour par an. réfléchir à ce pour quoi ils sont reconnaissants. Certaines familles s\'assoient ensemble et disent à tour de rôle à haute voix ce pour quoi elles sont reconnaissantes. Une partie importante de Thanksgiving est le rassemblement des familles. Les membres de la famille élargie parcourent souvent de longues distances pour partager la journée avec leurs proches. Aux États-Unis, Thanksgiving est le jour de voyage le plus chargé de l\'année. Les gens voyagent en avion, en train, en bus et en voiture pour partager un repas très spécial\xa0: le dîner de Thanksgiving.\nCe que la plupart des célébrations de Thanksgiving ont en commun, c\'est la nourriture. À Thanksgiving, de nombreuses familles préparent de grands dîners élaborés à partager avec leurs amis et leurs proches. Beaucoup de ces dîners utilisent des ingrédients associés à la récolte d\'automne, comme les canneberges, les pommes de terre, les patates douces, le maïs et les bleuets. La dinde rôtie est la viande traditionnelle servie. C\'est tellement courant que certaines personnes appellent le jour de Thanksgiving "le jour de la Turquie". La dinde est généralement servie avec de la sauce et de la farce. Les tartes sont généralement servies en dessert. Les tartes les plus couramment servies sont les patates douces, les pommes, les noix de pécan et la citrouille.\nL\'Action de grâces n\'est pas seulement célébrée dans l\'intimité des foyers. Chaque année, le président des États-Unis gracie une dinde, l\'empêchant de devenir le repas de quelqu\'un. À New York, Macy \\ \'s, un grand magasin, organise un grand défilé avec d\'énormes ballons de divers personnages de dessins animés flottant dans la rue. ,,, La fête des pères est une fête observée aux États-Unis pour honorer les pères, la paternité et d\'autres figures paternelles comme les grands-pères, les beaux-pères et les oncles. Elle est célébrée le troisième dimanche de juin.\n \nAu début du XXe\xa0siècle, un accident minier en Virginie-Occidentale a tué près de 400\xa0hommes. Une journée pour se souvenir des hommes tués, qui étaient pour la plupart des maris et des pères, a été le début d\'une campagne pour une fête nationale. C\'était en 1908. Ce n\'est cependant qu\'en 1972 que la fête des pères fut reconnue par la loi comme fête nationale. Cela est dû en partie au fait que les gens étaient réticents à ce qu\'ils considéraient comme une autre fête trop inventée destinée à faire dépenser de l\'argent. Ce n\'était pas considéré comme un moyen de se souvenir des morts ou d\'honorer les pères vivants et les figures paternelles. Les gens n\'avaient pas tout à fait tort. Les organisations représentant les fabricants de vêtements pour hommes ont investi beaucoup d\'argent pour que la fête devienne populaire.\nComme la fête des mères, c\'est en effet une fête qui est devenue très commerciale. Il y a des cartes de vœux pour marquer la journée et les magasins annoncent des cadeaux pour papa. Ces cadeaux comprennent traditionnellement des appareils électroniques, des cravates et des outils. Dans les écoles, les enfants fabriquent souvent des cartes et autres cadeaux. La fête des pères n\'est pas une fête fédérale même si elle est largement célébrée. Certaines des façons dont la fête des pères est célébrée sont avec des repas, en particulier des barbecues puisque les vacances tombent au début de l\'été. ,,, Alors que les familles se préparent à voyager en avion ce week-end du Memorial Day, United Airlines  n\'offrira pas d\'embarquement anticipé aux familles en autocar avec de jeunes enfants.\nRejoignant une poignée d\'autres transporteurs, United a fait le changement de politique fin avril avec peu de fanfare.\nAuparavant, les familles avec de jeunes enfants voyageant en autocar étaient autorisées à bord des avions avant l\'embarquement général, une commodité pour les mamans et les papas qui essaient de faire passer leurs enfants dans le ciel amical.\n Nous avons pensé qu\'il serait préférable de simplifier ce processus et de réduire le nombre de groupes d\'embarquement, a déclaré le porte-parole de United, Charles Hobart. La compagnie aérienne permet aux passagers avec enfants voyageant en première classe ou en classe affaires d\'embarquer plus tôt. crankyflier.com.\nKate Hanni de flyersrights.org qualifie cette décision de très anti-familiale. \n Il y a très peu de choses sur lesquelles un parent peut compter lorsqu\'il s\'agit de voyager en avion de nos jours, mais l\'une de ces choses a toujours été la possibilité d\'embarquer en premier pour que vos enfants s\'installent et que tous leurs besoins soient satisfaits avant la foule de gens monter à bord de l\'avion, a déclaré Hanni dans un e-mail à CNN. J\'espère que United changera d\'avis. \n Si les familles ont besoin d\'une assistance supplémentaire, notre agent d\'embarquement s\'efforcera de les accueillir, a déclaré Hobart. Ce message a été communiqué. \n Nous n\'allons pas voler United en sachant que nous allons devoir supporter ce tracas supplémentaire, a écrit Steven Manson, un Brooklyn, New York, père de jumeaux de 4 ans. Franchement, nous attendons toujours avec impatience le préembarquement lorsque nous volons et essayons de nous positionner près de la porte d\'embarquement lorsque l\'embarquement commence. Il est tellement utile d\'attacher le siège auto et que tout le monde s\'installe avant que l\'avion ne soit bondé de passagers, a déclaré Manson dans un e-mail à CNN.\nUne approche de guérilla pour voler avec des enfants\nLe changement de politique a également des racines dans le fusion annoncée en 2010 entre United et Continental.\nNous sommes passés à un processus d\'embarquement commun pour tous les avions, a déclaré Hobart.\nEn tant que politique, American Airlines n\'autorise pas non plus spécifiquement les familles avec enfants à monter dans l\'avion avant l\'embarquement général. Cependant, la compagnie aérienne a déclaré qu\'elle faisait une annonce à tous les passagers munis d\'un billet avant l\'embarquement général demandant à toute personne ayant besoin de plus de temps ou d\'assistance de se manifester.\n Il ne serait pas réaliste de préembarquer que de nombreuses familles uniquement sur cette seule base, a écrit un Américain Porte-parole des compagnies aériennes dans un e-mail à CNN. Cela pourrait facilement se traduire par la plupart des passagers du vol. \nJetBlue propose aux familles voyageant en autocar avec des enfants de moins de 2 ans d\'embarquer tôt, a indiqué la compagnie aérienne. Delta Air Lines a déclaré qu\'elle offrait également aux passagers avec enfants la possibilité de préembarquer, y compris ceux avec des billets de première classe et de classe affaires. Il en va de même pour Virgin America, qui a déclaré qu\'elle autorisait les enfants de moins de 5\xa0ans à embarquer plus tôt.\nUS\xa0Airways a modifié ses règles de préembarquement pour les enfants au cours de l\'année écoulée, permettant aux familles avec enfants de monter dans l\'avion après l\'appel pour la zone\xa02, mais avant le général annonce d\'embarquement. Auparavant, la compagnie aérienne autorisait l\'embarquement anticipé. Nous cherchons toujours des moyens de rendre le processus d\'embarquement un peu plus efficace et plus rapide pour nos passagers, a déclaré la porte-parole Liz Landau.\nAvez-vous eu une bonne ou une mauvaise expérience en essayant de préembarquer un vol\xa0? Veuillez partager vos expériences dans la section des commentaires ci-dessous.\n,,,Miami est bien connue pour son climat tropical, sa communauté de retraités et sa population cubano-américaine, mais pas tant qu\'un centre d\'art international. Heureusement pour le centre-ville de Miami, le promoteur immobilier milliardaire Jorge Pérez veut changer cela.\nLe promoteur immobilier et amateur d\'art a fait don de 40 millions de dollars, dont la moitié provient de sa collection d\'art privée, au Pérez Art Museum Miami, connu sous le nom de PAMM, un des quelques grandes institutions aux États-Unis avec un nom hispanique.\nC\'est certainement une façon de mettre Miami sur la carte.\nAncien village trouvé à Miami\nLe centre-ville de Miami est en pleine revitalisation\xa0: David Beckham est dans l\'espoir d\'y construire un stade pour son équipe de football de Miami récemment annoncée. Dans le monde de l\'art, l\'extravagance internationale d\'art et de mode Art Basel, qui se déroule à Miami Beach depuis plus d\'une décennie, est considérée comme la principale foire d\'art du pays avec 130 musées et institutions internationaux attirant des dizaines de milliers de les gens à l\'événement de cinq jours.\nJorge Pérez espère que son musée aidera à faire passer Miami au niveau supérieur.\nLes institutions culturelles ont joué un rôle essentiel dans la croissance de Miami, avec ce musée comme exemple, a déclaré Pérez dans un entretien avec CNN.\nAvant que Pérez ne soit connu sous le nom de Condo King of South Florida, il a commencé comme urbaniste, se concentrant sur les développements de logements à faible revenu.\nEnsuite, il a cofondé une société de développement immobilier, The Related Group of Florida , travaillant dans la construction de condos haut de gamme dans les années 1980.\nEn 2005, le magazine TIME l\'a nommé l\'un des 25 hispaniques les plus influents des États-Unis.\nLe milliardaire de 64\xa0ans est également foue Liste Forbes des 400 Américains les plus riches à plusieurs reprises, le seul immigrant latino à le faire.\nNé en Argentine de parents cubains, Pérez a grandi en Colombie avant de migrer à Miami à l\'adolescence. Sa passion de toujours pour l\'art latino-américain découle de son éducation.\nAu début de la vingtaine, il a commencé à collectionner des peintures de Roberto Matta, Diego Rivera, Wifredo Lam et Joaquin Torres-Garcia. Désormais, 110\xa0œuvres sélectionnées par le musée dans la collection privée de Pérez sont à la disposition de tous à l\'intérieur du musée de trois étages et de 200\xa0000\xa0pieds carrés, conçu par les architectes suisses Herzog & de Meuron, lauréats du prix Pritzker.\nPromenade à l\'intérieur du bâtiment de 220 millions de dollars et vous serez entouré de grandes baies vitrées qui seraient les plus grandes résistantes aux ouragans au monde. Suspendus de chaque côté du musée se trouvent 70\xa0jardins hydroponiques à arrosage automatique, conçus par le paysagiste français Patrick Blanc. Selon quoi ? collection.\nPAMM est l\'une des principales attractions de ce qui était connu sous le nom de parc du bicentenaire, aujourd\'hui parc du musée, qui abritera également le musée des sciences Patricia et Phillip\xa0Frost, dont l\'ouverture est prévue en 2015.\nLe musée d\'art de Pérez s\'est bien déroulé ouverture l\'année dernière, à l\'exception d\'un léger accroc\xa0: le nommer.\nAlors que la majorité des membres du conseil d\'administration ont voté en faveur de nommer le musée après Pérez, quatre ont démissionné en signe de protestation. Il semble qu\'ils aient jeté leur dévolu sur le Miami Art Museum. \nC\'est parce que 100 millions de dollars sur les 167 millions de dollars collectés provenaient de l\'argent des contribuables de la ville et du comté et que la ville de Miami a fait don du terrain riverain où se trouve PAMM. Certains des membres du conseil ont fait valoir que le nom du musée devrait représenter le musée d\'art central de la ville.\n Je ne peux pas parler pour les autres. Cependant, la grande majorité du conseil d\'administration était favorable. Sur un conseil de 47, 43 ont voté en faveur du changement de nom, a déclaré Pérez.\nPérez est impliqué dans le musée depuis bien avant le début du projet de construction et il s\'est dit honoré que le conseil ait choisi de le reconnaître de cette manière.\n Je pense que la plupart des gens veulent qu\'on se souvienne des causes qu\'ils jugent utiles et qu\'ils ont aidé à promouvoir, a déclaré Pérez. le nom a été changé pour le Patricia and Phillip Frost Museum of Science après qu\'un généreux donateur a donné 35 millions de dollars pour construire le nouveau musée à Museum Park.\n Le changement de nom a été extrêmement bien accueilli par la communauté. Patricia et Phillip Frost sont des gens très appréciés et ils symbolisent ce que nous sommes, a déclaré Gillian Thomas, présidente et chef de la direction du Patricia and Phillip Frost Museum of Science.\nEn ce qui concerne les droits de dénomination, Miami prend peut-être davantage son art au sérieux que la science.\n Les gens ont des opinions sur l\'art, alors que la science est un peu plus concrète, donc il n\'y aurait pas de contrecoup pour nommer un bâtiment après un donateur, a déclaré Thomas.\nMais, souligne-t-elle, nommer Miami\\ Le musée d\'art d\'après Pérez porte un sentiment de fierté pour la communauté hispanique, même si son don d\'art ne représente pas une collection. \n Jorge Pérez est le premier grand donateur de la communauté hispanique, a déclaré Thomas.\nD\'autres musées d\'art américains ont été nommés d\'après des donateurs, comme le J. Paul Getty Museum de Los Angeles, et n\'ont suscité aucune controverse.\\ n Bien que les gens ne soient pas d\'accord sur un nom, a déclaré Thomas, tout le monde peut voir que le centre-ville de Miami devient rapidement une plaque tournante pour la science et l\'art, ce qui signifie que des dons sont nécessaires. \nJusqu\'à présent, cela ne semble pas être un problème pour le musée d\'art Pérez\xa0; il a atteint la barre des 90 % dans sa campagne de financement de 220 millions de dollars.\n,,,En avril dernier, John a fait un voyage à Las Vegas, Nevada. Las Vegas est une destination populaire dans la partie ouest des États-Unis. La ville est surtout connue pour ses casinos, ses hôtels et sa vie nocturne animée.\nDans le centre-ville de Las\xa0Vegas, John a passé beaucoup de temps sur le Strip, qui s\'étend sur 4\xa0km de boutiques, de lieux de divertissement, d\'hôtels de luxe et de restaurants gastronomiques. . C\'est probablement la zone touristique la plus visitée de la ville. Le Strip la nuit est particulièrement beau. Tous les bâtiments s\'illuminent grâce à des enseignes au néon lumineuses et accrocheuses pour attirer l\'attention des visiteurs.\nUn séjour à Las\xa0Vegas peut ressembler à une visite dans de nombreuses villes populaires du monde entier. De nombreux hôtels ont des versions miniatures d\'importants sites et monuments internationaux. Ces monuments célèbres incluent la tour Eiffel, Venise et même la Rome antique.\nUn jour, John a fait un détour à l\'extérieur de la ville pour visiter le Grand Canyon, l\'une des sept merveilles du monde naturel. Le canyon offre une vue imprenable sur les crêtes et le paysage naturel du Nevada. John a particulièrement aimé le canyon car il était à l\'écart de tout le bruit et de l\'agitation du centre-ville de Las\xa0Vegas.\nJohn s\'est bien amusé lors de son voyage à Las\xa0Vegas. Il n\'a pas gagné beaucoup d\'argent dans les casinos. Cependant, il a réussi de voir beaucoup de sites étonnants lors de sa visite dans cette ville qui ne dort jamais.,,,Canapé à vendre\nCe magnifique canapé marron fabriqué par grandsofas.com est d\'excellente qualité et extrêmement confortable. Il est dans un état impeccable (voir photo) et bénéficie d\'une garantie de 25 ans. Le matériau est en coton et il a une sensation moderne et fraîche. Les housses de coussin sont toutes amovibles et lavables en machine, avec un rembourrage en mousse de qualité, et c\'est super confortable. Conserve sa forme lorsque vous vous asseyez dessus. Les pieds en bois de couleur foncée peuvent être facilement retirés pour permettre au canapé de passer par une porte standard.\nPas d\'usure car nous ne l\'avons que depuis deux ans et l\'avons gardé en bon état. Le canapé provient d\'une maison sans fumée et sans animaux. Je ne le vends qu\'en raison d\'un déménagement car le nouvel endroit n\'est pas assez grand pour le canapé.\nDimensions du canapé\xa0:\nLargeur\xa0: 182\xa0cm\nHauteur totale\xa0: 81\xa0cm\nHauteur sans pieds\xa0: 63\xa0cm\nPour toute question, n\'hésitez pas à interroger. Prix \u200b\u200bdemandé : 250€. **Pour la collection UNIQUEMENT**,,,Si nous regardons autour de nous les choses que nous avons achetées à un moment de notre vie, nous remarquerons sans doute que tout ce que nous possédons n\'est pas utilisé à bon escient\xa0: l\'épais manteau de laine qui nous pensions avoir l\'air à la mode malgré le fait que nous vivions dans un pays tropical, le smartphone qui a été rangé lorsque nous nous sommes achetés le dernier modèle, la voiture qui n\'est utilisée que le week-end, ou même la chambre d\'amis de notre maison qui s\'est en quelque sorte transformé en réserve.\nCes objets sous-utilisés peuvent sembler inutiles pour certains, mais peuvent être un atout pour d\'autres. Avec l\'avènement d\'Internet, les communautés en ligne ont trouvé un moyen de générer des bénéfices à partir du partage de ces actifs sous-utilisés. En utilisant des sites Web et des groupes de médias sociaux qui facilitent l\'achat et la vente de biens d\'occasion, il est désormais plus facile que jamais d\'organiser des activités de partage entre pairs. Et c\'est ce qu\'on appelle l\'économie du partage.\nCes plates-formes en ligne démocratisées offrent aux gens la possibilité de gagner rapidement de l\'argent ou deux. Pour donner un exemple, les parents occupés n\'auraient peut-être pas pris la peine d\'installer un stand au marché local ou à la vente de bottes de voiture pour vendre l\'ancien équipement de leurs enfants, mais avec les marchés en ligne, les parents peuvent désormais vendre ceux à peine portés. les vêtements pour bébés devenus trop grands pour leurs enfants et les poussettes et équipements pour bébés coûteux dans lesquels ils ont investi, afin de se remettre un peu d\'argent dans leurs poches.\nLes entreprises ont également compris la rentabilité de l\'économie du partage et cherchent à tirer profit de la fabrication l\'utilisation de ces ressources sous-utilisées. Un modèle commercial qui a rapidement gagné en popularité voit les entreprises fournir une plate-forme en ligne qui met les clients en contact avec ceux qui peuvent fournir un produit ou un service particulier. Des entreprises comme Airbnb agissent comme intermédiaires pour que les gens profitent de leurs chambres et maisons inutilisées et les louent comme hébergement lucratif. Un autre exemple est Uber, qui encourage les gens à utiliser leur propre voiture personnelle comme taxi pour gagner un peu d\'argent supplémentaire pendant leur temps libre.\nCette évolution vers une économie de partage n\'est pas sans critiques. Contrairement aux entreprises, les personnes non réglementées ne sont pas tenues de respecter certaines réglementations, ce qui peut entraîner une qualité inférieure et incohérente des biens et services et un risque de fraude plus élevé. Néanmoins, dans la société de consommation dans laquelle nous vivons aujourd\'hui, les opportunités accrues de vendre nos produits indésirables et sous-utilisés peuvent avoir un impact moindre sur notre environnement.,,,Rosa Parks est un excellent exemple de la façon dont un Américain ordinaire peut changer le monde. autour d\'elle. Elle est née en 1913, dans l\'État méridional de l\'Alabama. Parks est devenu célèbre pour avoir contesté une politique discriminatoire dans la ville de Montgomery, en Alabama. Au cours des années 1950, les Noirs américains du sud n\'étaient pas autorisés à monter sur les sièges avant des bus publics si une personne blanche avait besoin de ce siège. Les Noirs devaient se lever de leur siège et se rendre à l\'arrière du bus, qui était désigné comme «\xa0de couleur uniquement\xa0».\n \nLe 1er\xa0décembre\xa01955, Parks a refusé d\'obéir à l\'ordre des chauffeurs de bus de céder son siège à un passager blanc qui était monté dans le bus. Même si Parks était assise dans la section Colored Only, le bus était rempli et elle était tenue par la loi de donner sa place au cavalier blanc. Parks a été arrêté et accusé de désobéissance civile. Elle a dit plus tard qu\'elle en avait "fatiguée de céder".\nSon arrestation et son procès ont fait de Parks un symbole national du mouvement des droits civiques. Son arrestation a également conduit au Montgomery Bus Boycott, une campagne de protestation politique et sociale contre la ségrégation raciale. Bien que le boycott n\'ait duré que 21 jours, il a joué un rôle déterminant dans la décision de la Cour suprême de déclarer inconstitutionnelle la pratique de la ségrégation en Alabama.\nParks est resté un symbole du mouvement des droits civiques même après sa mort en 2005 à l\'âge de 92 ans. Elle a été surnommée la «\xa0première dame des droits civiques\xa0» par le Congrès des États-Unis et «\xa0la mère du mouvement pour la liberté\xa0». Elle a été honorée par le gouvernement américain avec des prix prestigieux such en tant que médaille présidentielle de la liberté et médaille d\'or du Congrès. Elle a été honorée avec son propre musée dans le Michigan et a reçu plusieurs autres prix pour les droits civiques avant et après sa mort.,,,Samuel Langhorne Clemens, mieux connu sous le nom de Mark Twain, est un auteur et humoriste américain. Il a écrit deux des romans les plus importants, Les Aventures de Tom Sawyer et Les Aventures de Huckleberry Finn. Il a grandi à Hannibal, dans le Missouri, qui a servi de cadre à ses romans.\n \nClemens a occupé divers petits boulots dans sa jeunesse. Il était journaliste, mineur, apprenti imprimeur et typographe, mais le journalisme était ce qu\'il aimait le plus faire. Son style d\'écriture lui a valu les éloges des critiques du monde entier. William Faulkner, un autre grand auteur américain, l\'appelait "Le père de la littérature américaine".\nTwain était déjà un écrivain à succès lorsqu\'il a écrit Les Aventures de Tom Sawyer. Le personnage principal Tom Sawyer était basé sur ses propres expériences d\'enfant. Ses meilleurs amis étaient également basés sur des amis réels avec lesquels Twain a grandi. Huckleberry Finn, le personnage principal de son meilleur travail, était également basé sur un ami d\'enfance.\nTwain était un écrivain prolifique qui a écrit plusieurs livres et d\'autres publications. La plupart de ses écrits ont été tirés de sa vie dans la partie ouest des États-Unis lors d\'un voyage avec son frère Orion. Il est né chrétien, mais plus tard dans la vie, il a remis en question sa foi et a développé des sentiments négatifs envers la religion organisée. Il a été cité une fois comme disant: "Si le Christ était ici, il y a maintenant une chose qu\'il ne serait pas - chrétien." Twain est mort en 1910 d\'une crise cardiaque à l\'âge de 74 ans.\n,,,De nos jours, de nombreuses personnes fréquentent des collèges communautaires avec l\'intention de passer dans un collège ou une université de quatre ans pour obtenir un baccalauréat. C\'est un peu comme un tremplin. Il y a plusieurs raisons pour lesquelles les gens font cela. L\'une des raisons est que certains collèges communautaires ont des accords de transfert avec des collèges et universités privés et publics qui garantissent l\'admission. Même si un collège communautaire n\'a pas cet accord d\'une place garantie dans un collège de quatre ans, il a des accords d\'articulation avec des écoles de quatre ans. Ces accords vous indiquent exactement les cours qu\'un étudiant d\'un collège communautaire doit suivre pour pouvoir être transférés. Ces accords garantissent que les étudiants ne perdent pas de temps à suivre des cours qui ne seront pas transférés. La plupart de ces cours qu\'il faut suivre avant le transfert sont des cours d\'enseignement général, comme les mathématiques et l\'anglais.\n \nUne autre raison pour laquelle de nombreux étudiants commencent leur diplôme de premier cycle dans un collège communautaire est d\'ordre financier. Un collège ou une université de quatre ans coûte beaucoup plus cher qu\'un collège de deux ans. Cela est particulièrement vrai pour les étudiants immigrants, qui ne sont pas admissibles à une aide financière, à des prêts ou à des bourses. De plus, les collèges communautaires ont tendance à offrir davantage de cours du soir afin de pouvoir accueillir les personnes qui doivent travailler tout en fréquentant l\'école. Ils sont également une bonne option pour les étudiants plus âgés avec des familles, qui ont besoin d\'un horaire plus flexible tout en s\'occupant des enfants. Les collèges communautaires ont également tendance à être des écoles de banlieue, ce qui signifie que les gens n\'ont pas à vivre sur le campus dans des dortoirs. Fréquenter un collège communautaire signifie que vous pouvez toujours vivre à la maison avec vos parents, ce qui peut faire économiser beaucoup d\'argent à la famille.\nSi un élève n\'a pas bien réussi au lycée, un collège communautaire lui en fournirait un autre possibilité d\'entrer dans une université de quatre ans. Les collèges communautaires offrent de nombreux cours pour aider les élèves à développer leurs compétences en mathématiques et en écriture. Lorsque vous fréquentez un collège de quatre ans, vous devez déjà posséder ces compétences. Le collège communautaire préparera les étudiants à réussir leur diplôme d\'une école de quatre ans.,,, Aux États-Unis, 43 États, Washington D.C., Porto Rico et les îles Vierges américaines ont tous leur propre loterie. Une loterie est utilisée par les États pour collecter des fonds. Les magasins agréés vendent des billets numérotés, ou les gens peuvent payer pour choisir leurs propres numéros. Lorsque vous laissez une machine choisir vos numéros, cela s\'appelle généralement un Quick Pick. Beaucoup de gens jouent des numéros porte-bonheur, des numéros associés à des dates de naissance ou d\'autres numéros importants pour eux. L\'argent gagné en vendant des billets de loterie sert à payer des écoles, des routes, des ponts et d\'autres services publics. Une fois par semaine ou plus, un tirage a lieu où les numéros sont choisis au hasard. Si votre billet comporte ces numéros, vous pourriez gagner un prix en espèces.\n \nIl existe différents types de loteries, telles que Powerball, Mega Millions et Lotto. Powerball et Mega Millions sont connus pour leurs gros gains. Il existe également des billets de loterie instantanée qui sont des cartes à gratter. Les gains sont généralement moins élevés, mais ils sont extrêmement populaires.\nAux États-Unis, vous pouvez jouer à la loterie ou acheter un billet si vous avez 18\xa0ans ou plus. Si vous gagnez, vous avez généralement la possibilité de recevoir la totalité du montant en un seul versement ou de recevoir des versements plus modestes sur plusieurs années. Les gains de loterie sont imposables, ce qui signifie que le montant que vous obtenez réellement est plus petit que le numéro du jackpot. Les gens peuvent acheter des billets de loterie ou jouer à la loterie dans de nombreux dépanneurs. Les loteries sont également très populaires car elles font de la publicité. Il existe de nombreuses publicités télévisées, radiophoniques et des panneaux d\'affichage incitant les gens à jouer.\nLes loteries ont été controversées car elles sont essentiellement une forme de jeu légalisé. On craint que cela ne rende les gens dépendants au jeu. Il existe également des jeux de loterie illégaux, généralement gérés par certaines formes de crime organisé.,,,La carrière de Chris Pratt est sur le point de sortir de ce monde - ou, plus précisément, de cette galaxie.\nEW a confirmé que l\'humoriste des parcs et loisirs jouera le personnage principal dans les prochains Gardiens de la Galaxie de Marvel, un élément clé du plan de la phase deux de Marvel. Deadline a annoncé la nouvelle pour la première fois.\nEW.com\xa0: Marvel Studios passe au micro et au macro avec les révélations du Comic-Con\nPratt, qui a récemment maigri -LRB- et gonflé -RRB- pour son rôle dans Zero Dark Thirty, jouera Star Lord , le chef mi-humain, mi-extraterrestre d\'une équipe bio-conçue de héros voyageant dans l\'espace. Cette équipe comprend un guerrier à la peau verte, un raton laveur lanceur d\'armes, une assassin extraterrestre et un redoutable arbre.\nAvec le réalisateur James Gunn, l\'homme derrière le chef-d\'œuvre shlocky qu\'est Slither, et l\'homme nouvellement couronné Pratt, y a-t-il une chance que Guardians of the Galaxy soit une pure comédie de super-héros\xa0?\nEW.com\xa0: James Gunn confirme qu\'il dirige Guardians of the Galaxy \'\nLe matériel source suggère certainement que le film pourrait fonctionner comme une comédie totalement ridicule aventure, mais nous devrons évidemment attendre et voir quel ton choisiront les cinéastes.\nVoir l\'histoire originale sur EW.com.\nAnthony Breznican d\'Entertainment Weekly a contribué à ce rapport.\nCLIQUEZ ICI pour essayer 2 SANS RISQUE numéros d\'Entertainment Weekly\n© 2011 Entertainment Weekly et Time Inc.. Tous droits réservés.\n,,,Deux jours après que l\'enlèvement d\'une femme de Philadelphie a été enregistré sur vidéo, la police affirme avoir une piste au-delà de la ligne d\'état\xa0: Quelqu\'un a utilisé sa carte bancaire mardi matin dans le Maryland.\nUn usage masculin d la carte ATM de Carlesha Freeland-Gaither à 6 h 01 une banque à Aberdeen, Maryland, à environ 75 miles de route au sud-ouest de la rue de Philadelphie où elle a été forcée de monter dans une voiture dimanche soir, l\'inspecteur en chef de la police de Philadelphie Dennis Wilson a déclaré aux journalistes.\nWilson n\'a pas dit comment la carte a été utilisée ni si de l\'argent a été retiré. Mais il a déclaré qu\'une vidéo de surveillance montrait un homme semblable à l\'homme qui, selon la police, aurait traîné et enlevé la femme de 22 ans dimanche soir. \n Veuillez la ramener. Nous voulons juste que Carlesha nous revienne en toute sécurité, a déclaré sa grand-mère Ana Mulero lors d\'une conférence de presse avec Wilson à Philadelphie. Il suffit de nous la rendre. Nous sommes sa famille. Elle a une famille ici. Nous avons besoin d\'elle. \nLa police n\'a pas immédiatement diffusé la vidéo de la banque. Mais les autorités ont publié plus tôt une vidéo qui, selon elles, montre Freeland-Gaither abordé par un inconnu dans le quartier Germantown-Penn Knox de Philadelphie vers 21h40. dimanche.\nCette vidéo, publiée lundi, montre un homme s\'approchant d\'une femme près d\'une intersection, puis l\'attrapant et la traînant dans une rue, vers la caméra.\nLa séquence publiée n\'inclut pas l\'audio, mais un témoin a rapporté que Freeland-Gaither a crié à l\'aide, a déclaré la police.\nLa vidéo montre Freeland-Gaither se débattant au sol avec son ravisseur et forcée de monter dans un véhicule à quatre portes, qui, selon la police, pourrait être une Ford Taurus grise.\nSes lunettes et son portable téléphone ont été jetés sur la route pendant la lutte. Juste avant que le véhicule ne s\'éloigne, Freeland-Gaither a cassé l\'une des vitres côté passager, a indiqué la police.\nLa police a décrit le ravisseur comme un homme noir inconnu de corpulence moyenne, âgé de 25 à 30 ans. La vidéo le montre portant une veste sombre avec une capuche, un chapeau sombre et un pantalon sombre.\nLa famille de Freeland-Gaither a déclaré à l\'affilié de CNN, WPVI, qu\'elle travaillait comme infirmière et qu\'elle se trouvait à quelques rues de chez elle lorsqu\'elle a été enlevée.\\ Les enquêteurs ne savent pas si elle connaissait son ravisseur, bien qu\'il n\'y ait aucune indication sur la vidéo de surveillance de Philadelphie qu\'elle l\'ait fait, a déclaré Wilson. La police n\'a pas parlé publiquement d\'un motif possible.\nLes autorités ont initialement déclaré que la voiture du ravisseur avait une plaque d\'immatriculation de Pennsylvanie, mais Wilson a déclaré mardi que les enquêteurs cherchaient à savoir si une plaque du Maryland se trouvait à l\'avant de la voiture. . La plaque n\'est pas facilement lisible dans la vidéo, et la police a déclaré qu\'aucun témoin ne leur avait donné de numéro de plaque. poursuivre une carrière.\nLes fonctionnaires offrent des récompenses de 42\xa0000\xa0$ pour les informations menant à la résolution de l\'affaire, dont 25\xa0000\xa0$ du FBI, qui aide à l\'enquête, a déclaré Wilson.\nDana\xa0Ford de CNN a contribué à ce rapport. \n,,,Chanteur pop-country P atti Page est décédé le 1er janvier à Encinitas, en Californie. Elle avait 85 ans.\nNée Clara Ann Fowler, Page était l\'artiste féminine la plus vendue des années 1950 et avait 19 singles d\'or et 14 de platine. Page a atterri 111 fois dans les charts et son mélange des genres a fait d\'elle une icône de la pop traditionnelle. Elle a marqué son premier morceau vendu à un million en 1950 avec With My Eyes Wide Open, I \'m Dreaming, et a chanté des tubes comme -LRB- How Much Is That -RRB- Doggie in the Window et Old Cape Cod. Sa chanson phare, Tennessee Waltz, est devenue plus tard une chanson officielle de l\'État du Tennessee, et sa popularité a conduit à ses propres émissions de télévision sur CBS, NBC et ABC dans les années 50 et à des rôles au cinéma dans les années 60.\nRavi Shankar, Temptations to get Grammy Lifetime Achievement Awards\nLa page a reçu des étoiles sur le Hollywood Walk of Fame et le Country Walk of Fame. Elle a également remporté un Grammy en 1999 pour son premier album live, Live at Carnegie Hall: The 50th Anniversary Concert, et a continué à tourner jusqu\'à sa mort. Les White Stripes ont repris son titre de 1948 Conquest sur leur album de 2007 Icky Thump.\nAvant sa mort, Page a été annoncée comme l\'une des récipiendaires de cette année des Lifetime Achievement Awards de la Recording Academy lors de la cérémonie des Grammy de cette année . Elle laisse dans le deuil ses deux enfants, Daniel O\'Curran et Kathleen Ginn, et sa soeur Peggy Layton. Les services commémoratifs n\'ont pas encore eu lieu.\nVoir l\'histoire originale sur RollingStone.com.\n,,,Madonna Louise Ciccone est une chanteuse, compositrice et actrice populaire aux États-Unis. Madonna, comme on l\'appelle communément, a fait irruption sur la scène musicale au début des années 1980 avec un style jamais vu auparavant. Ses costumes et performances sexy et légèrement vêtus ont à la fois ravi et exaspéré le public américain pendant des années. Elle excellait à repousser les limites de ce que la plupart des Américains percevaient comme acceptable dans ses vidéoclips et ses performances en direct. Madonna n\'a pas non plus hésité à aborder des sujets controversés.\n \nMadonna est née à Bay City, Michigan en 1958. Elle a déménagé à New York pour poursuivre une carrière dans la danse en tant que jeune femme. Elle a commencé sa carrière musicale avec quelques groupes de clubs à New York, mais a rapidement été remarquée par les dirigeants de l\'industrie musicale. En 1983, elle sort son premier album intitulé simplement "Madonna". Ce n\'était que le début pour la superstar. Elle a suivi son succès initial avec plusieurs albums et singles à succès tels que Like a Virgin et Into the Groove. Elle a associé son succès commercial dans la musique à des performances réussies dans une série de films tels que Desperately Seeking Susan et Evita. Il semblait que Madonna ne pouvait rien faire.\nL\'une des caractéristiques déterminantes de Madonna est sa capacité à se réinventer. De nombreux artistes réussissent dans l\'industrie très compétitive du divertissement, mais la plupart ne réussissent que pendant une courte période, mais pas Madonna. Elle a vendu plus de 300 millions de disques dans le monde à ce jour. Elle s\'est classée deuxième derrière les Beatles sur la liste Billboard des plus grands actes du 20e siècle.\nLa carrière de Madonna n\'a cependant pas été sans controverse. Elle a fait l\'objet de plusieurs poursuites judiciaires pour ses actes, mais elle a survécu et a prospéré à travers tout cela. Elle est toujours l\'une des meilleures artistes féminines aux États-Unis aujourd\'hui.,,,Sam plissa les yeux contre le soleil à la lointaine traînée de poussière ratissée par la voiture en route vers la Grande Maison. Les chevaux donnaient des coups de pied et agitaient la queue contre les mouches, sans se soucier de la première visite de leur propriétaire en dix mois. Sam a attendu. M. Carter n\'est pas venu ici à moins qu\'il n\'y soit obligé, ce qui convenait parfaitement à Sam. Plus il restait à l\'écart de son patron, plus longtemps il aurait un travail.\nCarter est venu plus tard pendant que Sam coupait du bois. Carter souleva son chapeau comme s\'il attendait un rendez-vous avec le curé de la ville, puis le retira complètement comme s\'il parlait à sa mère. Il sortit une pile de papiers de sa poche arrière et la lui tendit.\n\'Tu ne prends pas souvent ton courrier, n\'est-ce pas ?\'\nSam le prit sans un regard et laissa tomber les enveloppes sur le banc. \n\'Jamais,\' répondit-il et attendit que Carter dise pourquoi il était ici. Le fait que ce soit la maison de Carter n\'était pas une explication et ils le savaient tous les deux. Carter tordit son chapeau, se léchant les lèvres et s\'éclaircissant la gorge.\n\'Bon travail de réparation de ces clôtures,\' dit-il finalement.\n\'Je serai bientôt de retour au début,\' Sam a dit. Ce n\'était pas une plainte. Une clôture qui a pris un an à réparer signifiait une autre année de travail pour l\'homme qui l\'a bien fait.\n\'Vous n\'avez jamais envie de prendre des vacances\xa0?\'\n\'Et aller où\xa0?\\ \' Des vacances signifiaient être de retour dans le monde réel, un endroit d\'où même des gens comme Carter se sont rendus pour s\'échapper. L\'évasion de Sam était sa réalité et il ne reviendrait pas.\nMr Carter essuya la sueur de sa nuque. Les taches humides de sa chemise se rejoignaient comme des formes dans un atlas. Sa peau rougissait déjà sous le soleil de juin. Sinon, il avait le bronzage intérieur d\'un homme qui gagnait de l\'argent pendant que d\'autres travaillaient.\n\'J\'ai amené mon fils avec moi lors de ce voyage. Il a eu des problèmes à l\'école.es s\'est relevé, a cligné des yeux rapidement puis s\'est déplacé vers le chapeau qui occupait ses mains. \'Pas beaucoup de problèmes ici pour un jeune garçon.\' Il tenta de rire mais cela sortit comme un aboiement de chien.\nLes deux hommes regardèrent vers l\'extrémité nord de la propriété. Il s\'étendait à perte de vue. Même les clôtures étaient à peine visibles d\'où elles se trouvaient. Aussi ennuyé et rebelle qu\'un adolescent puisse être, il n\'était pas possible de s\'échapper à pied. Sam regarda le plus grand des chevaux, frappant le sol avec ses lourds sabots. Le garçon pouvait-il monter ? se demanda-t-il. Il y avait tout un tas d\'ennuis qu\'un bon cavalier pouvait avoir ici, à des kilomètres de n\'importe qui. Mais peut-être qu\'il y avait encore plus de problèmes pour quelqu\'un qui ne connaissait rien aux chevaux et voulait s\'éloigner de son père.,,,\ufeffDe tout le Rwanda, et même des régions du Burundi voisin, les gens affluent vers la ville méridionale de Butare pour un petit magasin appelé Inzozi Nziza (Sweet Dreams). Ils viennent goûter l\'inconnu, quelque chose que la plupart n\'ont jamais goûté - l\'étreinte douce, froide et veloutée de la crème glacée. \nIci, dans le premier glacier du pays d\'Afrique centrale, les clients peuvent acheter des boules aux saveurs de crème douce, de fruit de la passion, de fraise et d\'ananas. Les garnitures comprennent des fruits frais, du miel, des pépites de chocolat et du granola. Du thé noir et du café sont également en vente. \nLa boutique, qui affiche «\xa0des glaces, du café, des rêves\xa0» sur sa signalétique, attise la curiosité locale pour le dessert - et « change des vies\xa0» dans le processus, déclare la directrice d\'Inzozi Nziza, Louise Ingabire. \n"La crème glacée est importante", dit-elle entre deux bouchées d\'une offrande au goût de miel. "Certains Rwandais aiment les glaces, mais c\'est nouveau. Nous avons encore du travail à faire, pour dire aux autres qu\'ils apprécieront. \nFidèle à la devise d\'Inzozi Nziza, la boutique peut certainement faire des rêves une réalité. « Avant, je n\'avais pas de travail : je restais à la maison. Maintenant, j\'ai une vision pour l\'avenir. Je gagne de l\'argent et je peux en donner une partie à ma famille », déclare le jeune homme de 27 ans. \nButare, qui compte 89 600 habitants et est située à 135 km au sud de la capitale, Kigali, abrite l\'Université nationale du Rwanda. Inzozi Nziza est devenu une plaque tournante pour les étudiants fatigués qui cherchent à s\'offrir quelque chose de cool et de différent. \n«\xa0C\'est quelque chose qui unit les gens ici\xa0», déclare Kalisa Migendo, une étudiante en agriculture de 24\xa0ans. "Si vous avez besoin de sortir et de parler à un ami, une fille ou un garçon, vous venez chez Inzozi Nziza pour une glace." \nLa plupart des ingrédients sont d\'origine locale et le lait provient d\'un dépôt à proximité de Nyanza. Les gousses de vanille et le cacao sont importés. \nInzozi Nziza a été inauguré par la directrice de théâtre Odile Gakire Katese. Elle a rencontré Alexis Miesen et Jennie Dundas, co-fondateurs de Blue Marble Ice Cream à Brooklyn, New York, et a formé un partenariat pour ouvrir le magasin en 2010. \n"Un magasin de crème glacée, a proposé Katese, pourrait aider à mettre le les pièces humaines se reconstituent en reconstruisant les esprits, les espoirs et les traditions familiales », déclare Miesen. \nAu départ, Miesen et Dundas possédaient la boutique en partenariat avec son personnel et détenaient des parts dans l\'entreprise, qui est une coopérative à but non lucratif. Ils n\'ont pas fixé d\'objectifs financiers, mais ont attendu 18 mois avant de transférer leurs parts aux femmes, qui avaient alors prouvé leurs compétences commerciales. \nLe succès de la boutique ne fait pas exception, déclare Fatuma Ndangiza, directrice générale adjointe du Rwanda Governance Board. «\xa0Les petites entreprises sont principalement dirigées par des femmes, mais lorsqu\'il s\'agit de grandes entreprises où vous devez concourir pour de gros appels d\'offres, il y a très peu de femmes. Les femmes sont des nouvelles venues dans les grandes entreprises », souligne-t-elle. « Nous avons plus de femmes entrepreneures. C\'est un domaine où les femmes s\'intéressent, tant à l\'intérieur qu\'à l\'extérieur de Kigali. \nBien que la crème glacée soit quelque peu nouvelle au Rwanda, Ndangiza est enthousiasmée par l\'idée de la boutique. "Je pense que c\'est super. Cela demande beaucoup de compétences et fait évoluer les mentalités car vendre et manger des glaces ne fait pas partie de notre culture. Je pense que pouvoir innover et introduire cela sur le marché, et le processus de fabrication, est assez intéressant. \nLa boutique de Butare emploie neuf femmes, qui passent leur temps libre à s\'entraîner avec Ingoma Nshya, la première et unique troupe de percussionnistes féminines du Rwanda, créée par Katese il y a dix ans. \nLes musiciens sont des femmes Hutu et Tutsi, certaines rescapées du génocide de 1994, au cours duquel près d\'un million de Tutsis et de Hutus modérés ont été tués. \nCertains membres d\'Ingoma Nshya sont veuves, d\'autres orphelines. D\'autres ont été affectés par le massacre de différentes manières. \nHistoriquement, dit Ingabire, il était interdit aux femmes rwandaises de jouer du tambour et de nombreuses personnes considéraient que les tambours étaient trop lourds à porter pour les femmes. "Mais c\'est quelque chose qui apporte l\'unité." \nLe père d\'Ingabire, ses deux frères et sœurs et de nombreux cousins \u200b\u200bont été tués pendant le génocide. « Certains d\'entre nous sont des survivants ; certains connaissent quelqu\'un qui a été tué », dit-elle. "Quand je joue de la batterie avec eux, cela me donne du pouvoir parce que nous sommes toujours en vie et survivants." \nLe restaurant populaire est présenté dans un documentaire de Rob et Lisa Fruchtman , frères cinéastes. Sweet Dreams, qui raconte comment les femmes se sont forgé un avenir post-génocide prometteur, comprend également les batteuses. \nLe film, qui a été projeté dans plus d\'une douzaine de pays, dont les États-Unis, le Royaume-Uni et plusieurs États africains, sera présenté en première au Rwanda en 2014. "Nous pensons que le film parle de résilience, d\'espoir, de bravoure, d\'ingéniosité et de capacité à changer le cours de votre propre vie », déclare Lisa Fruchtman, qui a remporté un Oscar du montage en 1984.\n,,,-LRB- CNN -RRB- La famille de James McNair, le regretté comédien connu sous le nom de Jimmy Mack, a réglé un procès avec Walmart pour l\'accident de voiture qui l\'a tué et blessé sa collègue comédienne Tracy Morgan, a déclaré la famille de McNair.\nLa sœur de McNair, Pamela McNair, a déclaré que le montant convenu était confidentiel, mais qu\'il était satisfaisant pour tous parties.\n Il n\'y a aucune valeur, aucun montant en dollars, sur la vie de mon frère, dit-elle. Mais en termes de finances, cela aidera beaucoup la famille à continuer à vivre. \nMcNair était le père de Jamel, 26\xa0ans, et de Danita, 19\xa0ans, et était l\'aîné de sept frères et sœurs dont il tenait beaucoup, a déclaré Pamela.\n Nous ne pouvons rien faire pour changer ce qui s\'est passé, Walmart a dit. Walmart et la famille McNair ont travaillé en étroite collaboration pour parvenir à un accord. Walmart continue de travailler avec d\'autres personnes impliquées dans l\'accident et nous continuerons à faire ce qui est juste. \nEn juin dernier, un bus limousine transportant Morgan, McNair et leurs compagnons a été embouti par un camion Walmart sur l\'autoroute à péage du New Jersey. L\'accident a tué McNair et a laissé Morgan avec une grave lésion cérébrale et leurs collègues avec d\'autres blessures.\nUn rapport préliminaire du National Transportation Safety Board a déclaré que le camion Walmart roulait à 20 mph au-dessus de la limite de vitesse, et que son conducteur était presque à son limite de temps de conduite.\nEn juillet, quatre parties ont intenté une action contre Walmart, déclarant dans une plainte que la négligence des défendeurs était un facteur contributif important aux blessures des plaignants. \nPamela McNair a déclaré que la famille continuerait à poursuivre l\'héritage de James en tant qu\'humanitaire, et que les finances de la colonie aideraient.\n Mon frère a beaucoup fait pour la communauté. Il a encadré les jeunes pour la communauté, il a travaillé au refuge pour sans-abri. Les petites choses qu\'il a faites lui manquent vraiment, a déclaré McNair.\n C\'était un grand comédien, mais c\'était avant tout un bon membre de la famille. Il aimait la vie et voulait que les autres aiment la vie aussi. C\'était un humanitaire. \nLe maire de la ville natale de McNair, Peekskill, New York, a nommé le 12 juin en son honneur, a-t-elle déclaré.\nL\'avocat de Tracy Morgan a déclaré à CNN en novembre qu\'il se remettait toujours de l\'accident de voiture qui l\'avait laissé avec une grave lésion cérébrale et se bat pour redevenir la personne qu\'il était autrefois.\nMcNair a déclaré que sa famille n\'avait pas été en contact avec la famille Morgan.\nC\'est un peu déchirant, a déclaré McNair. Je comprends qu\'il a sa propre situation médicale, je veux le meilleur pour lui, je prie pour son rétablissement, mais j\'ai senti que quelqu\'un aurait pu tendre la main à la famille... Mon frère et Tracy avaient une très longue relation en tant qu\'amis et c\'est déchirant, nous n\'avons entendu parler de personne. \nChuck Johnston et Alexandra Field de CNN ont contribué à ce rapport.\n,,, Affectueusement connu dans sa ville natale de Madrid comme le sage d\'Hortaleza, Luis Aragones a laissé l\'héritage d\'aider l\'ascension de l\'Espagne vers le sommet du football mondial.\nAragones, dont le décès à l\'âge de 75\xa0ans a été annoncé samedi, a entraîné son équipe nationale jusqu\'au titre de champion d\'Europe en 2008, le premier succès du pays dans un tournoi majeur en plus de 40\xa0ans.\nCela La percée a été poursuivie par son successeur Vicente del Bosque, qui a continué avec le style de passe tiki taka introduit par Aragones, alors que La Roja a remporté la Coupe du monde 2010 et la couronne européenne 2012. Sans aucun doute, il a marqué la route dans cette phase finale réussie . J\'ai ressenti beaucoup d\'appréciation envers lui, a déclaré Del Bosque sur le site officiel de l\'équipe nationale.\n Je savais qu\'il avait des problèmes de santé, mais je n\'aurais jamais imaginé que cela conduirait à cela. \n Toujours avec nous, Luis, a dirigé le site Web hommage à l\'Atletico Madrid, le club où Aragones a joué pendant une décennie entre 1964 et 1974 et a été entraîneur-chef à quatre reprises, la dernière en 2001-03.\nIl a guidé l\'Atletico vers la Liga titre en 1977 et la Coupe d\'Espagne à trois reprises.\n Luis Aragones était un grand joueur et entraîneur, mais avant tout cela une grande personne et un ami, a déclaré le président de l\'équipe Enrique Cerezo.\n Il était avant tout Atlético. Nous tenons à exprimer au nom de tout le club nos condoléances à sa famille. \nLe dernier poste d\'entraîneur d\'Aragones était avec l\'équipe turque de Fenerbahce, qui s\'est terminé à la mi-2009, et à la fin de l\'année dernière, il a annoncé sa retraite du jeu - dans lequel il a débuté à Getafe en 1957.\nIl aurait été emmené dans un hôpital de Madrid la dernière semaine avec une maladie grave, dont il ne s\'est pas remis.\nAragones sera enterré lors d\'une cérémonie privée dimanche. L\'Atlético obtiendra observer une minute de silence avant le match à domicile contre la Real Sociedad, au cours duquel l\'équipe cherchera à se hisser en tête du classement au-dessus d\'une autre de ses anciennes équipes, Barcelone.\n Je souhaite exprimer notre chagrin face à la perte d\'un des plus grands hommes du football et l\'un des managers les plus charismatiques et les plus sympathiques dont nous nous souvenions, a déclaré le président du Barça, Josep Maria Bartomeu, avant la défaite 3-2 de son équipe contre Valence - sa première à domicile en championnat depuis avril 2012. \n En tant que club, nous avons eu l\'honneur de ses services, bien que pour une très courte période, lors de la saison 1987-88, lorsque nous avons remporté la Coupe d\'Espagne.\n Nous apprécions beaucoup son respect pour le Barça, son modèle et nos joueurs . L\'Euro 2008, avec Puyol, Xavi, Iniesta et tant d\'autres, en est un bel exemple. \nL\'appréciation d\'Aragon pour le style tiki taka du Barça - commencé par Johann Cruyff et développé par Pep Guardiola - a été confirmée par le nombre de joueurs du club catalan qu\'il a sélectionné pour l\'équipe nationale.\nMais son cœur était dans le capitale nationale, où il figurait dans les livres du grand rival de l\'Atletico, le Real Madrid, de 1958 à 1960 en tant que joueur - bien qu\'il ait passé la majeure partie de ce temps en prêt à d\'autres clubs.\n La perte de Luis Aragones attriste tous ceux d\'entre nous qui aiment le football, a déclaré le président du Real Florentino Perez.\n Aujourd\'hui est un jour de deuil pour ce sport, mais ce devrait aussi être un jour de reconnaissance pour une figure légendaire qui a été essentielle pour nous donner une période glorieuse avec notre équipe nationale espagnole.\n Il a anobli ce sport et tous les supporters espagnols lui doivent gratitude et respect. Sa carrière personnelle et professionnelle a toujours été caractérisée par l\'honnêteté et le travail acharné. \nLa seule tache sur sa carrière internationale a été un incident en 2004, lorsqu\'il a été accusé d\'avoir fait un commentaire racial -- qui a été filmé par des caméras de télévision -- à propos de l\'attaquant français Thierry Henry alors qu\'il essayait de motiver ses joueurs à l\'entraînement avant un match de qualification pour la Coupe du monde.\nAragones a été condamné à une amende par l\'instance dirigeante du football européen, l\'UEFA, et a ensuite présenté ses excuses à Henry, insistant sur le fait qu\'il n\'avait pas l\'intention de faire une insulte raciale.\n,,,Il y a cinq ans, dire à un ami que vous interagir régulièrement avec un cône de signalisation orange verbeux pourrait vous valoir un câlin de pitié et un voyage rapide au sanatorium.\nMais ces jours-ci, votre ami pourrait répondre qu\'il échange des barbes avec le gilet pull de Rick Santorum.\nBienvenue à l\'âge de de nouveaux comptes Twitter, où des animaux, des objets inanimés et d\'autres entités non humaines s\'expriment régulièrement\xa0: un pigeon avec une gueule grossière et un comportement plus grossier. Le célèbre brouillard de San Francisco. Même l\'univers. Sur la plate-forme de médias sociaux, les utilisateurs peuvent s\'exprimer de manière créative et faire rire en donnant la parole à ces choses qui ne peuvent pas parler d\'elles-mêmes.\nLes nouveaux comptes non humains sont un incontournable de Twitter depuis que Sockington, un chat domestique de la région de Boston, a rejoint le service en mars 2007 en route vers 1,4 million de followers. Mais ils ont de nouveau fait les manchettes récemment après qu\'un utilisateur a créé @olympicseat, une réponse ironique aux sièges curieusement vacants sur les sites olympiques de Londres. En l\'espace de quelques jours, il a rassemblé plus de 21\xa0000\xa0abonnés.\nUn autre flux lié aux Jeux olympiques célèbre la serviette étoilée que le gymnaste américain Danell Leyva se drape entre les événements. Il compte plus de 12\xa0000\xa0abonnés, ce qui n\'est pas mal si l\'on considère que l\'utilisateur moyen de Twitter en compte 126.\nLa plupart des membres de ce groupe d\'internautes remarquables affirment que leur principale motivation pour tweeter est de s\'amuser. Pour d\'autres, c\'est une chance de se faire une place dans le monde encombré du Web.\nCependant, tous s\'accordent à dire que leurs réflexions de 140\xa0caractères sont plus réfléchies qu\'il n\'y paraît.\nCat Food Breath\nQuand la femme derrière l\'animal -compte centré @CatFoodBreath a commencé son compte Twitter de nouveauté en août 2010, elle n\'avait pas l\'intention de rester.\n J\'ai pensé que la meilleure façon de tester les eaux était avec un pseudonyme, a déclaré la résidente de Burlington, Vermont, qui a demandé que son nom ne soit pas publié pour préserver l\'anonymat de son personnage Twitter. Je n\'offenserais pas mes amis et collègues si je ne les suivais pas, et je pouvais faire une sortie polie de Twitter quand j\'avais fini de fouiner. \nDeux ans et près de 19\xa0000 abonnés plus tard, elle y est toujours. Au début, cependant, elle et de nombreux autres succès de compte de nouveauté admettent qu\'ils ne savaient pas vraiment ce que c\'était.\n Au début, je ne savais pas que ce compte parlait d\'un chat, -LSB- mais - RSB- en explorant le Twitterverse, j\'ai vite réalisé que j\'avais un personnage auquel les gens s\'identifiaient, a-t-elle dit.\nVrai ou pas, Cat Food Breath est définitivement un personnage. Le chat -LRB- dont le sexe est ambigu -RRB- écrit des odes au canapé, donne des ordres à son propriétaire avec une acerbe à la Garfield et se plaint du chien de la famille - affectueusement appelé Labradum. \nLe brouillard de San Francisco\nD\'autres célébrités anonymes sur Twitter ont commencé leurs comptes pour éduquer et sensibiliser -LRB- hilarant -RRB- à des choses méconnues qui ne peuvent pas parler d\'elles-mêmes, comme le brouillard de San Francisco qui refroidit les étés dans cette ville.\n j\'adore tout sur le brouillard, a déclaré l\'homme qui a créé @KarlTheFog en août 2010. Je pense que la façon dont il se faufile sur les collines est belle, mystérieuse et même un peu romantique. J\'aime la façon dont il s\'arrête à certains points au-dessus de la ville, créant un mur de nuages. Et ayant déménagé ici d\'une ville où il faisait plus de 100 degrés en été, j\'étais surtout excité de ne pas dégouliner de sueur pendant trois mois consécutifs. \nL\'homme, qui a également requis l\'anonymat, s\'est inspiré de @BPGlobalPR, une réponse parodique populaire à la marée noire de BP dans le golfe du Mexique. À travers l\'objectif de Karl le brouillard, il rend compte des sentiments du public envers le brouillard et agit comme un représentant des relations publiques pour la forme de temps décriée – avec un penchant spirituel. Karl compte déjà plus de 5\xa0000\xa0abonnés.\nDonner une identité à Karl a suscité des réponses emphatiques et parfois superficielles, en particulier lorsque le brouillard a bloqué les feux d\'artifice du 4\xa0juillet, mais l\'univers Twitter a été globalement réceptif, a-t-il déclaré.\n -LSB- Je reçois -RSB- beaucoup de jeux de mots et de haïkus sur la météo. Mes tweets préférés sont lorsque les gens prennent des photos de Karl et les publient sur Twitter ou Instagram, a déclaré l\'homme. Voir autant de gens s\'en emparer et taguer Karl sur leurs photos a été extrêmement cool. \nThe Orange Cone\nComme Karl le brouillard, @TheOrangeCone adore quand ses abonnés interagissent avec lui en ligne.\nThe Orange Cone était à l\'origine un compte MySpace mais est venu sur Twitter après que son créateur a découvert les tweets pleins d\'esprit de Cobra du zoo du Bronx, le dernier le célèbre serpent échappé de l\'année et le satiriste du sport automobile @nascarcasm. Aujourd\'hui, l\'Orange\xa0Cone tient au courant plus de 20\xa0000\xa0utilisateurs des événements NASCAR.\nEt ses abonnés répondent en masse.\nLes gens remarquent les cônes dans leur vie quotidienne et m\'envoient des photos par centaines, a déclaré l\'homme derrière le Cone, qui a demandé que son nom ne soit pas utilisé. Je trouve ça hilarant - les gens arrêtent ce qu\'ils font dans le monde réel pour prendre une photo d\'un cône orange. Je l\'aime. \nIl a ajouté\xa0: "Chaque fois que le cône est touché au cours d\'une course NASCAR, mon flux explose. \nLorsque le pilote Kasey Kahne a frappé le cône lors d\'un entraînement en février à Daytona, le Twitterverse est venu en aide à Orange Cone.\n Kasey a présenté des excuses sur Twitter et j\'ai ajouté environ 1 200 abonnés dans l\'heure suivante, a-t-il déclaré. C\'était fou. \nRester dans le personnage\nPour certains, la perspective d\'interagir en vrai avec des abonnés n\'est pas envisageable.\n Je suis un peu comme Banksy. Personne ne sait qui je suis, a déclaré l\'homme derrière le personnage d\'oiseau colérique @PigeonJon, faisant référence à l\'artiste anonyme basé en Grande-Bretagne. Si possible, j\'aimerais que cela reste ainsi\xa0? a-t-il écrit dans un e-mail à CNN.\nEn dehors de cette déclaration, Jon n\'a jamais rompu son personnage. Au lieu de cela, il a offert des indices sur sa personnalité.\n J\'aime les biscuits. Je n\'aime pas le velcro, a-t-il écrit.\nLa personne derrière le très populaire @BronxZoosCobra, qui relate les exploits d\'un serpent réel qui s\'est brièvement échappé de son enclos au zoo de New York, a également refusé de briser le personnage dans un e- échange de courrier avec CNN. Le compte compte plus de 200\xa0000\xa0abonnés.\nLa chose la plus étrange qui a été tweetée contre le serpent, a déclaré son créateur, était une demande de soutien d\'un lycéen candidat au gouvernement étudiant. Je l\'ai soutenu, a-t-elle écrit à CNN. Il a perdu. Le sectarisme des serpents à son meilleur. \nAlter ego en ligne\nBeaucoup de nouveaux utilisateurs de Twitter très suivis sont déconcertés par leur popularité, ce qui les a amenés à réfléchir à la relation entre Internet et le monde réel.\n J\'ai l\'impression, et je peux me tromper, que les gens qui suivent The Orange Cone le trouvent plus réel que certains des pilotes sur la piste, a déclaré son créateur. Ils peuvent tendre la main et, dans la plupart des cas, obtenir une réponse. Et le fait que personne ne sache de qui il s\'agit - ce qui signifie que cela pourrait être n\'importe qui - ne fait qu\'ajouter au mystère et au plaisir. \nAjouté de l\'homme derrière KarlTheFog, je pense que les gens de la Bay Area aiment Karl parce que le brouillard était déjà un personnage dans leur vie. Dans un monde de plus en plus personnalisé, vous pouvez vous entourer d\'un groupe d\'amis sur Internet qui correspondent exactement à vos centres d\'intérêt et à votre sens de l\'humour. \nÊtre Karl l\'a mis au défi de penser au sport, à la culture pop et à la politique de manière ridicule, a-t-il déclaré.\nCela m\'oblige à être discipliné en m\'en tenant à un thème. Étant donné que mon travail n\'est pas créatif par nature, il me donne la chance d\'être aussi spirituel que je le souhaite. \nD\'autres utilisateurs de Twitter disent qu\'incarner des entités non humaines leur donne la possibilité de créer des alter ego. Le cône peut dire et faire les choses que je pense que nous souhaitons tous pouvoir faire, a déclaré son créateur. son humour décalé.\n C\'est une bouffée d\'air frais car ce n\'est pas malveillant, dit-il. Ce n\'est pas forcé par une marque avec un agenda ou un motif politique et c\'est fantaisiste, jetable et facile à digérer. Les gens réagissent bien à ce genre de choses. \nFantaisiste, oui. Mais soyons honnêtes\xa0: @WeirdH orse peut avoir un petit agenda.\n La récente demande de T-shirts et de tasses Weird Horse sur mon site... a été un autre avantage, a-t-il dit.\nQuel est votre compte Twitter non humain préféré\xa0? Exprimez-vous dans les commentaires.\nLes noms des utilisateurs de Twitter ont été modifiés afin qu\'ils restent anonymes.\n,,,\ufeffIl y a pire dans la vie que de flâner le long de la plage de Copacabana à Rio, au soleil, sur le chemin de regarder un match de Coupe du monde, il n\'était donc peut-être pas surprenant que le fan anglais Anthony McDowell de Liverpool n\'ait rien de la tristesse et de la tristesse qui ont précédé une partie de la préparation du tournoi au Brésil. « L\'endroit est charmant. Les gens sont super. Il y a une atmosphère de fête », a déclaré McDowell. "La seule chose qui pourrait être meilleure, c\'est l\'équipe d\'Angleterre."\nLui et une demi-douzaine d\'amis font partie des multitudes de supporters du monde entier qui ont fait du front de mer une zone de fête aux couleurs et aux chants nationaux. Certains ont dansé, d\'autres ont posé pour des photos, d\'autres ont bu, mais la plupart du temps, ils ont juste marché et parlé de football, attendant que le prochain match commence sur le grand écran du FanFest à proximité. La dernière fois qu\'il y avait autant de monde ici pendant la journée, le pape était en visite.\nL\'ambiance joyeuse et largement pacifique était loin des manifestations, du chaos des transports et des problèmes de stade qui ont tourmenté les préparatifs de la Coupe du monde. Mais, maintenant que le football est bel et bien lancé, les supporters visiteurs sont déterminés à profiter de l\'expérience. « Si j\'avais su, quand j\'ai commencé à planifier, à quel point ce serait compliqué et coûteux, je ne serais pas venu. Mais, maintenant que nous sommes ici, c\'est génial », a déclaré Brian Hill, un directeur général à la retraite de Sunderland.\nLe voyage n\'a pas été sans problème. Hill a voyagé pendant plus de 20 heures pour se rendre à Rio, puis a connu de longs retards sur la route depuis l\'aéroport à 6h30. Son fils, Andrew, s\'est fait voler ses lunettes de soleil presque dès qu\'il s\'est assis sur la plage. Et, ils ont été surpris que de nombreux bars ne soient pas équipés de grands écrans pour les jeux. Mais, comme beaucoup de supporters, ils ont déclaré avoir adoré l\'ambiance, sinon la logistique, de ce tournoi qui démarre de manière spectaculaire sur le terrain. tête des Pays-Bas face à l\'Espagne. Ailleurs, il y a eu une surabondance de buts\xa0: 28 lors des huit premiers matchs, soit près de trois fois plus qu\'au même stade en Afrique du Sud en 2010. Ajoutant à l\'ambiance carnavalesque dans les rues, où la majorité des fans viennent des pays voisins. nations, les équipes latino-américaines ont gagné dans chaque match qu\'elles ont disputé jusqu\'au moment de la rédaction.\nJusqu\'à présent, le tournoi a évité les pires scénarios Doomsday, bien qu\'il soit loin d\'être sans problème. Les stades ont peut-être été livrés en retard et - dans certains cas - pas complètement terminés, mais aucun problème structurel ou difficulté d\'accès au terrain n\'a été signalé depuis le coup d\'envoi.\nComme lors des Coupes du monde précédentes, la billetterie a posé problème, avec de nombreux sièges vides lors de plusieurs matchs, dont les Pays-Bas contre l\'Espagne. Le porte-parole de la FIFA, Saint-Clair Milesi, a confirmé que seuls 48 000 des 51 900 sièges au sol étaient pourvus. La FIFA enquête également sur les failles de sécurité qui ont permis aux supporters chiliens de tirer des feux d\'artifice lors de leur victoire sur l\'Australie.\nLe journal Globo a répertorié un certain nombre de lacunes dans les 12 villes hôtes. Presque tous ont souffert d\'embouteillages pires que d\'habitude. Les pires problèmes de transport se sont produits à Natal, où les chauffeurs de bus sont en grève depuis le 12 juin. À Salvador, certains temps de trajet étaient cinq fois plus longs que d\'habitude. "La circulation était déjà mauvaise mais cette semaine, c\'est chaotique", a déclaré au journal Jecilda Mello, présidente du groupe de résidents l\'Association des amis du centre historique.\nMais, les protestations ont diminué depuis le jour de l\'ouverture, lorsque de petites manifestations ont eu lieu dans plusieurs villes, incitant la police à utiliser des gaz lacrymogènes et du gaz poivré. Depuis lors, la seule menace à la sécurité a été le petit vol et les fans surexcités. Une fête de rue spontanée de supporters argentins a été dispersée avec du gaz poivré après que les supporters ont bloqué les routes.\nLes énormes distances ont conduit à des expériences de Coupe du monde très contrastées. Le tournoi n\'a fait qu\'une petite marque sur São Paulo, la ville la plus peuplée d\'Amérique du Sud, mais, loin à Manaus – la ville amazonienne éloignée où l\'Angleterre a joué contre l\'Italie – les visiteurs ont déclaré que la fièvre de la Coupe du monde battait son plein avec des rues décorées de couleurs vives et des drapeaux flottant. sur de nombreuses voitures.\nLe président de la FA, Greg Dyke, a déclaré que le changement d\'humeur était palpable. « Nous avons eu un accueil vraiment chaleureux à Manaus. C\'est une grande chose pour eux, même si c\'est un peu étrange de dépenser autant pour un stade sans personne pour y jouer. Mais nous étions à São Paulo pendant quatre ou cinq jours à l\'approche du match d\'ouverture et vous n\'auriez pas su jusqu\'au dernier jour qu\'il y avait même une Coupe du monde. C\'était bizarre.",,,Le touriste britannique qui a attrapé un enfant d\'un an qui est tombé du balcon du quatrième étage d\'un hôtel d\'Orlando, en Floride, a été honoré par le gouvernement un nd chefs d\'entreprise vendredi. Beard était dans la piscine de l\'Econo Lodge Inn and Suites vers 21h15 jeudi soir lorsque, après avoir entendu des cris horribles, elle a levé les yeux pour voir un petit enfant suspendu au balcon d\'un hôtel.\n Nous avons entendu des cris et je me suis retourné, a déclaré Beard, 44 ans, une mère qui était en vacances avec son partenaire, Philip Charlesworth, et son fils de 10 ans, Ben. J\'ai pu courir et attraper Jah-Nea, a-t-elle dit.\nBeard a dit qu\'il semblait que la fille avait glissé à travers une balustrade le long de la passerelle du balcon, ont déclaré les députés à l\'affilié de CNN WKMG.\nVoir l\'hôtel et jusqu\'où l\'enfant est tombé\nQuelques heures plus tard, Beard, analyste des services de santé, a été honorée pour ses actions héroïques. Elle a reçu une médaille du mérite du département du shérif du comté d\'Orange, de la chambre de commerce internationale Drive et du département d\'État des enfants et des familles pour avoir fait preuve d\'un instinct et d\'une initiative inhabituels lorsqu\'elle est passée à l\'action.\n Tout s\'est passé si vite , a déclaré Beard, qui était encore choqué par toute l\'attention. Je me suis juste assis avec elle et j\'ai essayé de la calmer pendant que tout le monde paniquait autour de moi. \nHelena Myles, la mère de l\'enfant, a déclaré avoir laissé sa fille avec un ami dans une chambre de motel, selon WKMG. L\'ami, Dominique Holt, a déclaré qu\'il était allé aux toilettes, puis avait trouvé la porte de la chambre d\'hôtel ouverte après avoir entendu des cris, a déclaré le bureau du shérif.\nUne femme qui se trouvait dans la chambre d\'hôtel avec Holt a déclaré qu\'elle n\'avait pas Je ne sais pas que l\'enfant était là, selon les députés.\nAucune accusation n\'a été déposée dans cette affaire.\n,,,L\'aide a afflué jeudi, trois jours après qu\'un tremblement de terre de magnitude 7,7 a frappé l\'Indonésie, déclenchant un tsunami qui a tué à moins 311\xa0personnes et plus de 400 disparues.\nUne équipe d\'évaluation du groupe à but non lucratif SurfAid International avait atteint la zone sinistrée, mais l\'éloignement de la région empêchait tout contact ultérieur avec le siège de Sumatera Barat, en Indonésie.\nSurfAid avait expédié 1\xa0000\xa0abris kits aux victimes, et se prépare à envoyer plus de fournitures une fois la communication établie, a déclaré jeudi la porte-parole Jossi Syahrial.\nLe groupe essayait de travailler avec des responsables gouvernementaux pour établir un centre d\'information et de coordination pour toutes les organisations non gouvernementales s, dit-elle.\nLa région durement touchée des îles Mentawai fait partie de la myriade d\'îles indonésiennes.\nIl est très difficile d\'atteindre la région par bateau ou par la route, a déclaré Andrew Judge, PDG de SurfAid.\nLe le voyage prend au moins 10 heures dans de bonnes conditions, selon les agences d\'aide.\nLa Croix-Rouge indonésienne avait également envoyé au moins deux équipes d\'aide. Une cargaison d\'aide comprenait environ 400 sacs mortuaires. Les besoins urgents devraient inclure de l\'eau potable, de la nourriture, des couvertures, des vêtements, des soins médicaux, des abris d\'urgence et des kits d\'hygiène, a déclaré Ita Balanda, responsable de programme pour l\'organisation d\'aide World Vision à Padang, Indonésie.\nOn estime que 7\xa0900\xa0ménages ont été touchés, a-t-elle déclaré.\n La zone la plus touchée est le nord et le sud de l\'île de Pagai. Des rapports faisant état de villages rasés viennent de là, a déclaré Antorizon de l\'agence de gestion des catastrophes de l\'ouest de Sumatra, qui, comme de nombreux Indonésiens, porte un seul nom.\nLes premières images de l\'île de North Pagai montraient des structures endommagées et rasées.\nLe séisme de lundi a généré important tsunami, a déclaré le Pacific Tsunami Warning Center. Certaines des personnes portées disparues pourraient inclure des personnes portées disparues après avoir fui vers les hauteurs, a déclaré Henri Dori Satoko, chef du parlement des îles Mentawai. mètres -LRB- près de 20 pieds -RRB- de haut. D\'autres rapports ont décrit le tsunami comme étant d\'environ 3 mètres -LRB- presque 10 pieds -RRB- de haut.\nAu moins un village avec une population d\'environ 200 personnes a été emporté, avec seulement 40 personnes récupérées, a déclaré Satoko.\nLe tremblement de terre a frappé à 21h42 Lundi, déclenchant une alerte au tsunami. Son épicentre était à 240 kilomètres -LRB- 150 miles -RRB- au sud de Padang, à une profondeur de 20,6 kilomètres -LRB- 12,8 miles -RRB-, selon le U.S. Geological Survey. La magnitude a été révisée à la hausse à partir d\'une magnitude préliminaire de 7,5.\n Sur Mentawai, une alerte -LSB- tsunami -RSB- a été émise sept minutes après la détection du tremblement de terre, ce que le système existant est capable de faire. Un rapport préliminaire indique que les vagues destructrices ont frappé les îles Mentawai après cinq minutes... Pour les villageois de Mentawai, l\'avertissement est tardif, car ils sont très proches de la source du tsunami, a déclaré Sanny Jegillos du Programme de développement des Nations Unies. Il supervise le système d\'alerte aux tsunamis dans l\'océan Indien, y compris en Indonésie.\nLe gouvernement indonésien a depuis averti les habitants de la province de Sumatra occidental de rester vigilants et de rester à l\'écart du littoral jusqu\'à cinq jours en raison de la possibilité d\'autres tremblements de terre, a déclaré Balanda. .\nLa ville de Padang et les îles Mentawai se trouvent au point de rencontre de deux plaques tectoniques, ce qui les rend vulnérables aux tremblements de terre et aux tsunamis.\nOn D Le 26 décembre 2004, un séisme de magnitude 9,1 frappe le nord de Sumatra. Un tsunami généré par ce tremblement de terre a tué plus de 225 000 personnes dans 14 pays, principalement l\'Inde, l\'Indonésie, les Maldives, le Sri Lanka et la Thaïlande. La région indonésienne de Banda Aceh a été durement touchée\xa0: environ 150\xa0000 personnes y sont mortes.\nKathy Quiano et Stan Grant de CNN ont contribué à ce reportage.\n,,,La révolution égyptienne a rendu la vie encore plus difficile au Caire\' s 80 000 chauffeurs de taxi, luttant pour gagner leur vie dans les rues chaotiques, encombrées et défoncées de la ville, dit l\'auteur qui a écrit un best-seller à leur sujet.\nLe roman Taxi de Khaled AlKhamissi, publié pour la première fois en 2006 , a été traduit en 10 langues. Il présente 58 monologues fictifs avec des chauffeurs de taxi recréés à partir de sa propre expérience. Une nouvelle édition en anglais post-révolutionnaire vient d\'être publiée.\nMalgré l\'optimisme qui a balayé l\'Égypte après le renversement du président Hosni Moubarak en février, AlKhamissi a déclaré que les chauffeurs de taxi, et le reste des pauvres du pays, ont en fait davantage souffert depuis puis.\n Mes chauffeurs de taxi font partie des 55 % de la population vivant sous le seuil de pauvreté de 2 dollars par jour, a déclaré AlKhamissi. Ces 55\xa0% n\'étaient pas dans la rue pendant la révolution et sont maintenant sans voix réelle.\n Ils sont sages, ils observent, ils analysent, mais ils n\'agissent pas. \nIl a ajouté\xa0: la vie a changé pour eux dans la mauvaise direction. C\'est pire pour eux car nous n\'avons aucune sécurité et vivre dans la rue sans sécurité est dangereux.\n Il y a moins de tourisme et plus de chaos dans les rues. \nAlKhamissi a déclaré qu\'il était trop tôt pour dire si la vie s\'améliorait pour les classes moyennes qui sont descendues dans la rue pendant la révolution.\n Nous sommes au milieu d\'une bataille, et pendant la bataille, nous ne pouvons pas voir si la vie s\'améliore ou pas, dit-il.\nAlKhamissi, dont le deuxième roman L\'Arche de Noé - également un best-seller - a été publié en 2009, a déclaré qu\'il avait arrêté d\'écrire son troisième roman au début du soulèvement le 25\xa0janvier de cette année et n\'a pas pu se concentrer sur l\'écriture depuis.\n Impossible d\'écrire car mon esprit est pris par ce qui se passe jour après jour, heure après heure, dit-il.\nDepuis janvier, AlKhamissi occupe son temps avec un hebdomadaire colonne de journal, conférences et événements parlants. Mais l\'auteur veut se remettre à écrire de la fiction.\n Tous les soirs j\'ai envie d\'écrire car c\'est mon seul travail, mon seul bonheur, dit-il. Je dois trouver une solution rapidement ou je vais mourir. \n Taxi a été crédité d\'avoir ravivé à lui seul l\'intérêt pour la lecture en Égypte, selon son éditeur Bloomsbury Qatar Foundation, mais AlKhamissi nie que son livre soit responsable.\nIl a déclaré\xa0:\xa0Certains journalistes ont dit cela à propos de mon livre, mais il\' C\'est absurde de dire qu\'un livre ou 10 livres peuvent changer les habitudes de lecture.\n La lecture a augmenté au cours des six dernières années, mais il a fallu de nombreux facteurs sociaux. Nous avons connu une révolution culturelle au cours des six dernières années dans tous les domaines\xa0: théâtre, musique, fiction et édition.\n Cette révolution culturelle est liée à la jeunesse en quête de rêve. \nAl Khamissi a déclaré qu\'il rêvait d\'une Égypte où les chauffeurs de taxi pourraient avoir le temps et l\'éducation nécessaires pour lire son livre - ou tout autre livre.\n Ils n\'ont pas le temps de lire car ils luttent pour survivre, a-t-il déclaré . Lorsque nous pourrons améliorer la qualité de l\'éducation et réduire la pauvreté, alors ils pourront lire. Je ne suis pas intéressé à savoir s\'ils lisent mon livre, j\'espère juste qu\'ils liront n\'importe quoi. \nUn autre romancier à qui l\'on attribue l\'évolution des habitudes de lecture en Égypte est Ahmed Khaled Towfik, un professeur de médecine de 49 ans décrit par la Bloomsbury Qatar Foundation comme l\'auteur à succès de fantasy et d\'horreur le plus éminent du monde arabe. Il a écrit plus de 200 livres et s\'est vendu à des millions d\'exemplaires.\nIl a déclaré qu\'il était l\'un des premiers auteurs égyptiens à écrire pour les jeunes.\n Il y a 20\xa0ans, il n\'y avait rien à lire pour les jeunes de 16, 17 et 18\xa0ans. dit Towfik. Nous avons inventé la littérature populaire. Je pense que j\'ai rendu les jeunes passionnés de lecture.\n Je pense que la plupart de mes lecteurs étaient sur la place Tahrir. J\'ai changé une partie de leur façon de penser, même si on ne peut pas comparer cela à leur déception économique ou au fléau du système policier. \nLe livre de Towfik, Utopia, qui vient d\'être publié en anglais, est un sombre récit futuriste de la société égyptienne en 2023 avec de riches communautés fermées isolées de la pauvreté à l\'extérieur.\nTowfik a déclaré que malgré la révolution égyptienne, il craint toujours que les scènes de son livre, publié pour la première fois en arabe en 2009, ne se réalisent.\n Le premier soulèvement est venu de la classe moyenne, mais je pense qu\'un autre plus sanglant pourrait venir des classes inférieures, comme décrit dans mon livre Utopia. \n Je pense toujours qu\'il y a un danger que mon livre devienne réalité. \nIl a ajouté\xa0: Après neuf mois, rien n\'a été accompli à part se débarrasser de Moubarak. Nous nous sommes débarrassés de la tête du serpent, mais le corps du serpent gouverne toujours tout. \n,,,Après environ 30 heures de délibération, un jury a condamné lundi le producteur de musique Phil Speacteur du meurtre au deuxième degré dans la mort de l\'actrice Lana Clarkson il y a plus de six ans.\nPhil Spector ne restera pas libre sous caution en attendant sa condamnation le 29\xa0mai.\nVêtu d\'un costume noir avec une cravate rouge et une pochette de costume, Spector n\'a montré aucun réaction à l\'annonce du verdict. Aujourd\'hui âgé de 69\xa0ans, il risque une peine de 18\xa0ans de prison à perpétuité lorsqu\'il sera condamné le 29\xa0mai. \nLe procureur Alan Jackson a déclaré qu\'il croyait que le jury avait rendu le verdict correct et il a reconnu la force et le soutien de la famille de Clarkson. Mais l\'avocat de la défense Doron Weinberg a déclaré que l\'équipe de défense de Spector n\'était pas d\'accord et prévoyait de faire appel. en attente de condamnation, citant le schéma de violence impliquant des armes à feu de Spector depuis des années. Regardez les avocats donner leur avis sur le verdict \n Ce n\'était pas un incident isolé, a déclaré Fidler, notant les deux précédentes condamnations liées aux armes à feu de Spector dans les années 1970. La prise d\'une vie humaine innocente, ça ne devient pas plus grave que ça. \nL\'épouse de Spector, Rachelle, était dans la salle d\'audience pour entendre le verdict lundi, tout comme la mère de Clarkson, Donna.\nClarkson, 40 ans, a été retrouvé mort, affalé sur une chaise dans le hall du Spector\'s Alhambra, Californie, manoir avec une blessure par balle à travers le toit de sa bouche en février 2003.\nUne annulation du procès a été déclarée lors du premier procès de Spector en septembre 2007. Après avoir délibéré 15 jours, les jurés ont déclaré à Fidler qu\'ils n\'étaient pas en mesure d\'atteindre un verdict. Spector a également été accusé de meurtre au deuxième degré lors de ce procès. Les jurés se sont retrouvés dans l\'impasse 10-2 en faveur de la condamnation.\nDans les plaidoiries finales du nouveau procès, le procureur Truc Do a qualifié Spector d\'homme très dangereux qui a l\'habitude de jouer à la roulette russe avec des femmes - six femmes. Il se trouve que Lana était la sixième. \nWeinberg a fait valoir que le dossier de l\'accusation reposait sur des preuves circonstancielles. Il a déclaré que la possibilité que Clarkson se soit suicidée ne pouvait être exclue.\nDo a toutefois souligné aux jurés que Clarkson avait acheté de nouvelles chaussures le jour de sa mort, ce qu\'une femme suicidaire n\'aurait pas fait, a déclaré le procureur.\\ nUne jurée qui a refusé d\'être identifiée a déclaré aux journalistes que les jurés avaient examiné toutes les preuves et tous les témoignages pour parvenir à leur verdict.\n Tout ce jury a pris cela très au sérieux, a-t-elle déclaré les larmes aux yeux, avant d\'ajouter qu\'il être dans un jury, car la vie d\'une autre personne est entre les mains du jury.\nClarkson a joué dans le film B de 1985 Barbarian Queen et est apparu dans de nombreux autres films, dont DeathStalker, Blind Date, Scarface, Fast Times à Ridgemont High et la parodie Amazon Women on the Moon. Elle travaillait comme hôtesse VIP à la House of Blues d\'Hollywood au moment de sa mort.\nDans le procès de 2007, les avocats de Spector ont fait valoir que Clarkson était déprimé par une récente rupture et ont saisi un pistolet de calibre .38 de se suicider chez Spector.\nMais les témoins à charge ont décrit Spector comme une menace armée. Cinq femmes ont pris la parole et ont affirmé qu\'il les avait menacées avec des armes à feu. Son chauffeur a témoigné qu\'il a entendu un grand bruit et a vu le producteur quitter la maison, pistolet à la main, en disant, je pense que j\'ai tué quelqu\'un. \nLe nouveau procès de Spector a commencé en octobre. Fidler a statué que les jurés pouvaient envisager la moindre accusation d\'homicide involontaire contre Spector. La marque professionnelle de Spector était le mur du son, la superposition de pistes instrumentales et de percussions qui sous-tendait une série de succès sur son label Philles - du nom de Spector et son associé, Lester Sill - au début des années 1960. iReport.com\xa0: Que pensez-vous de ce verdict\xa0?\nLes arrangements rugissants étaient au cœur de ce qu\'il appelait les petites symphonies pour les enfants - parmi lesquels les succès n°\xa01 comme les Ronettes Be My Baby et les Righteous Brothers You \'ve Perdu ce LovinFeelin. \nSpector a coproduit le dernier album des Beatles, Let It Be, et a travaillé avec les ex-Beatles George Harrison et John Lennon sur des projets solo après la séparation du groupe. Son enregistrement du concert-bénéfice de Harrison en 1971 pour le soulagement de la guerre au Bangladesh a remporté le prix Grammy 1972 de l\'album de l\'année.\nSpector a remporté deux Grammy Awards et a été intronisé au Rock and Roll Hall of Fame en 1989, mais il est resté hors de la vue du public pendant deux décennies avant son arrestation en 2003 dans la mort de Clarkson.\n,,,L\'entraîneur de football légendaire et diffuseur John Madden prend sa retraite, a-t-il annoncé jeudi. Beverly Hills, Californie, en 2008.\n Ça a été une si belle aventure... la NFL a été ma vie pendant plus de 40 ans, c\'est ma passion - ça l\'est toujours, a-t-il déclaré dans un communiqué publié par NBC Sports.\nMadden, 73\xa0ans, était un entraîneur du Temple de la renommée des Raiders d\'Oakland, mais il est surtout connu de millions de personnes comme un commentateur de football exubérant.\nIl a remporté 16\xa0prix Emmy en tant qu\'analyste sportif exceptionnel/ personnalité, a déclaré NBC.\n,,,Avoir votre enfant physiquement attaché à vous ne semble probablement pas amusant, mais un pédiatre américain, William Sears, promeut un style de parentalité qui maintient les petits enfants aussi physiquement attachés que possible. C\'est ce qu\'on appelle l\'attachement parental.\n \nL\'idée derrière l\'attachement parental est que si un petit bébé se sent protégé et en sécurité par le contact physique avec ses parents, en particulier sa mère, cet enfant grandira pour devenir un adulte plus indépendant. Une façon d\'encourager les parents à être près de leur bébé consiste à utiliser une écharpe ou une écharpe pour transporter le bébé au lieu de le mettre dans un landau. C\'est ce qu\'on appelle souvent "porter" votre bébé. Les parents sont également encouragés à dormir avec leur bébé. Les parents sont trop grands pour tenir dans un lit d\'enfant, alors on dit plutôt aux parents d\'amener le bébé dans le lit d\'adulte. Ceci est censé enseigner au bébé la régulation du sommeil, car le bébé apprendra à imiter les habitudes de sommeil des adultes. Ce co-sommeil est cependant controversé. On craint que les bébés qui dorment avec des adultes puissent suffoquer, se faire écraser accidentellement ou tomber du lit. On s\'inquiète également du syndrome de mort subite du nourrisson, ou SMSN, où les bébés arrêtent soudainement de respirer.\nUn autre marqueur de l\'attachement parental est de ne pas se soucier de gâter un bébé. Souvent, les parents ont l\'impression que prêter attention à chaque cri d\'un bébé le rend trop nécessiteux ou gâté. L\'attachement parental estime qu\'il faut répondre aux pleurs de chaque bébé et qu\'il ne faut jamais laisser le bébé « crier ». Au lieu de cela, le bébé doit être réconforté et les pleurs doivent être considérés comme un signal que le bébé a besoin de quelque chose. La plupart des parents attachés allaitent également à la demande, estimant que non seulement cela est physiquement plus sain, mais favorise également un lien entre la mère et l\'enfant. plus esquivé qu\'en tant que personne réelle, s\'est finalement exprimé.\nDans une interview accordée à un magazine du Brésil natal de sa famille, Saverin, un nouveau milliardaire après l\'offre publique d\'achat de Facebook, a parlé de ses impôts, sa relation avec le co-fondateur Mark Zuckerberg et la vie après The Social Network. \nDe nombreux observateurs pensent que Saverin, qui, selon l\'histoire d\'origine de Facebook, a été méprisé par Zuckerberg au fur et à mesure de la croissance du site, était une source principale du livre The Accidental Billionaires, sur lequel Social Network est basé.\nSurprise\xa0: Internet déteste les riches\\ nMais le résultat final était fantastique, dit-il. Et, non, il n\'a jamais jeté avec colère un ordinateur portable sur Zuckerberg comme son homologue hollywoodien, Andrew Garfield, l\'a fait dans le film. C\'est une déclaration que Fabio Altman, du magazine Veja, écrit est immédiatement crédible compte tenu de la nature réservée de Saverin.\n C\'est Hollywood, pas un documentaire, déclare Saverin, 30 ans, dans l\'interview réalisée dans son résidence de plusieurs millions de dollars à Singapour.\nSaverin a intenté une action en justice contre Facebook pour sa participation réduite dans l\'entreprise, et le litige a été réglé à l\'amiable. Mais malgré des frictions bien documentées au fil des ans, Saverin dit qu\'il n\'a aucune rancune envers Zuckerberg.\n Je ne peux que bien parler de Mark\xa0; Je ne lui en veux pas, dit Saverin. Sa concentration depuis le premier jour jusqu\'à aujourd\'hui est admirable. Il était un visionnaire et a toujours su que Facebook ne se développerait que s\'il restait fidèle à son idée centrale de personnes se présentant honnêtement et sans pseudonymes.\n C\'est la grande puissance de Facebook qui lui a permis de se transformer en un instrument de protestation, comme dans L\'Egypte, mais aussi comme instrument d\'affaires et, au-delà, comme moyen naturel d\'entrer en contact avec des amis. \nSaverin a eu peu à voir avec Facebook ces dernières années. Mais il est récemment revenu dans la conscience publique lorsque, quelques jours avant que les actions de Facebook n\'atteignent Wall Street, il a renoncé à sa citoyenneté américaine en faveur de Singapour, où il vit depuis 2009.\nBeaucoup y ont vu une esquive -- un moyen pour éviter de payer des impôts américains sur sa part de l\'entreprise, évaluée à environ 2 milliards de dollars. Il dit que ce n\'est pas le cas et a noté que, comme cela a été rapporté, il paie ses impôts.\n La décision était strictement basée sur mon intérêt de vivre et de travailler à Singapour, a-t-il dit. Je suis obligé et je paierai des centaines de millions de dollars d\'impôts au gouvernement américain. J\'ai déjà payé et je continuerai à payer les impôts que je dois en fonction de mon temps en tant que citoyen américain. \nLe père de Saverin, Roberto, a fait écho à ces pensées dans l\'article.\n C\'était difficile pour moi, à cause de la vie que j\'ai construite aux États-Unis, d\'apprendre d\'Eduardo qu\'il devait renoncer à sa citoyenneté, a-t-il dit. Mais il l\'a fait non pas parce qu\'il le voulait, mais parce qu\'il n\'avait pas d\'autre choix en tant qu\'étranger vivant à Singapour, où les transactions financières sont plus restreintes et bureaucratiques pour ceux qui détiennent un passeport américain. Il n\'y avait pas d\'autre moyen. \nRoberto et sa famille ont émigré aux États-Unis depuis le Brésil au milieu des troubles politiques, et le lieu de naissance enregistré de Saverin est Mia mi.\nSaverin continue de travailler en tant qu\'investisseur et partenaire dans des startups Internet depuis son appartement de Singapour, où un bureau contient trois moniteurs Mac de 27\xa0pouces sur lesquels il surveille en permanence tout, des cours des actions à la météo, écrit Altman.\nEn ce qui concerne Facebook lui-même ? Saverin a toujours son propre profil, avec plus d\'un million de personnes abonnées. Mais il l\'utilise presque exclusivement pour les nouvelles de ses nouvelles startups, gardant les détails personnels pour lui.\n Je n\'aime pas montrer ma vie privée en ligne, a-t-il dit.\n,,,Il a remporté six médailles aux Jeux olympiques d\'hiver, dont une médaille d\'or en 2008 et une de bronze il y a quelques mois à Sotchi. Les méthodes de Bode Miller, cependant, n\'ont jamais été orthodoxes.\n Je n\'ai jamais vraiment travaillé de manière très ciblée, admet-il. Je pense que j\'ai eu d\'excellents résultats et que j\'ai eu une longue carrière, donc, je veux dire, j\'en suis content. Mais je n\'ai jamais vraiment eu d\'objectifs, contrairement à la plupart des gens. \nMiller a toujours intrigué les médias et les fans. Une éducation non conventionnelle dans une cabane en rondins du New Hampshire sans électricité ni eau courante a cédé la place à une carrière de ski dans laquelle, pour Miller, son talent naturel et son amour du sport l\'ont toujours emporté, enfin, gagnant.\nEn effet, son objectif, comme indiqué dans son livre Bode: Go fast, Be Good, Have Fun était de skier aussi vite que l\'univers naturel le permettait. \nDe retour chez lui après des Jeux olympiques émouvants à Sotchi, où Miller a pleuré lors d\'une interview en direct avec Christin Cooper de NBC lorsqu\'il a été interrogé sur la mort récente de son frère, Miller se concentre sur un autre projet\xa0: sa Turtle Ridge Foundation.\nMiller a lancé la fondation après qu\'un ami proche, Cameron Shaw-Doran, ait subi une blessure traumatique qui l\'a laissé paralysé de la poitrine aux pieds.\n J\'essayais de l\'aider à se réimpliquer dans le sport. C\'était un grand athlète avant sa blessure et il avait vraiment du mal à s\'adapter, ce qui est un processus vraiment difficile quand on est blessé comme ça, dit Miller. Rien qu\'en le regardant traverser cela, j\'ai vu à quel point c\'était difficile et à quel point il y avait peu de soutien pour lui. \nL\'expérience a motivé Miller à trouver des moyens d\'aider les athlètes handicapés et blessés à s\'impliquer davantage dans le sport. Nous leur fournissons l\'équipement sportif et l\'environnement qui leur permet de participer à n\'importe quel sport, explique Miller.\nShaw-Doran est directeur du développement de l\'équipement pour Turtle Ridge, et Miller a fait appel à des amis et associés de tous les sports monde pour créer et adapter des équipements pour aider les skieurs adaptés et les athlètes adaptés dans d\'autres sports. complet avec des autographes, une vente aux enchères et une chance pour les enfants de faire la course avec leur idole. Je ne finirai pas par une boule de neige humaine géante.\n Tous les enfants veulent essayer de prendre l\'ascenseur avec moi pendant au moins une course et il y a environ 400 enfants. Cela se transforme en un peu de pandémonium en descendant, dit Miller.\nBodefest n\'est pas seulement une collecte de fonds, mais c\'est aussi une opportunité pour les athlètes adaptés d\'essayer certains des nouveaux équipements sur la neige de fin de saison.\n Le programme vient vraiment de changer ma vie, déclare le skieur adaptatif Owen Anketell, qui souffre d\'une maladie qui affecte les muscles du bas des jambes. Je n\'aurais jamais pensé que je serais capable de skier même si je suis en fauteuil roulant - ou à vélo - mais ce programme a vraiment changé mon opinion sur les sports adaptés. \n La clé est l\'autonomisation, dit Miller. Lorsque nous construisons une partie de notre équipement de ski et que vous le donnez à un enfant qui n\'a jamais eu la chance de monter et de découvrir ce que c\'est que de skier sur une montagne géante et que vous voyez à quel point cela peut changer sa vie, je pense que c\'est C\'est vraiment - c\'est assez incroyable. \nLe sentiment semble correspondre à la philosophie générale de Miller qui consiste à se concentrer sur ce que le sport vous fait ressentir, plutôt que sur la fin du jeu, un sentiment auquel tout le monde a droit, quelle que soit sa capacité.\nCe n\'est peut-être pas de la compétition, mais c\'est un objectif qu\'il peut prendre à la retraite - et quand même gagner.\n,,,Lorsque Lukas Hartmann, 29 ans, s\'est inscrit au service de dépistage génétique à domicile de 23andMe, il n\'y a eu aucune surprise dans ses résultats.\nLe Berliner a appris qu\'il mourrait probablement d\'un mélange de crise cardiaque et de cancer de la prostate, a-t-il écrit sur le blog d\'un ami, mais rien de spécial là-dedans. \nPuis il y a quelques mois, il a reçu une mise à jour de la société.\nLe code génétique de Hartmann a montré deux mutations liées à la dystrophie musculaire des ceintures, dit-il sur le site. Certaines personnes atteintes de dystrophie musculaire des ceintures perdent la capacité de marcher et souffrent d\'un handicap grave, lit-on sur sa page de résultats.\nCela ne peut pas être vrai, pensait-il. Ce doit être une erreur.\nLes tests génétiques peuvent être un outil puissant. Il peut fournir des informations sur vos antécédents familiaux, vous dire comment votre corps pourrait réagir à différents médicaments et vous identifier. vos facteurs de risque de maladie.\nCela peut également être trompeur.\nLa Food and Drug Administration a ordonné lundi à 23andMe d\'arrêter les ventes de ses kits de test génétique à domicile à 99\xa0$, affirmant que la société soutenue par Google n\'avait pas prouvé la validité de son produit.\\ nLa FDA a averti que les clients qui recevaient des résultats inexacts pourraient souffrir d\'angoisse mentale excessive ou subir des procédures médicales inutiles. L\'agence gouvernementale a demandé à 23andMe de revoir sa stratégie marketing pour se conformer aux réglementations fédérales.\nL\'action a provoqué l\'indignation de certains partisans de 23andMe.\n Donc OGM, aspartame, arômes et colorants artificiels, bave rose... pas de problème ! une affiche nommée Laura Ann a écrit sur la page Facebook de la société. Mais nous ne devrions en aucun cas avoir le droit de connaître notre propre matériel génétique comme moyen de prendre de meilleures décisions concernant notre santé. \nDans un communiqué, 23andMe a déclaré avoir reçu la lettre de la FDA et s\'efforcera de répondre aux préoccupations de l\'agence.\nIl y a plus de 3 milliards de lettres dans notre code génétique. Bien que 10 ans se soient écoulés depuis l\'achèvement du projet du génome humain, les scientifiques ne comprennent toujours pas ce que fait chaque gène et ce qu\'une mutation pourrait signifier pour la santé de quelqu\'un.\nC\'est l\'un des problèmes que les opposants aux kits de dépistage génétique à domicile soulèvent le plus souvent. Si les experts en ADN ne comprennent pas à 100\xa0% ce que signifie une mutation génétique, comment pouvons-nous attendre des consommateurs\xa0?\n5\xa0choses intéressantes que les tests ADN peuvent faire\nLes scientifiques ont identifié plus de 2\xa0000\xa0maladies monogéniques, déclare Rebecca Nagy, présidente de la Société nationale des conseillers en génétique. Ce sont des troubles qui peuvent être diagnostiqués sur la base de la mutation d\'un seul gène dans le corps ; certains exemples incluent la maladie de Huntington ou la fibrose kystique.\nMais des conditions telles que le diabète de type 2 ou la maladie d\'Alzheimer sont plus complexes. Des centaines de gènes peuvent contribuer au développement de ces maladies, dit Nagy, et le risque d\'une personne est également influencé par son environnement. Tester les marqueurs de risque disséminés dans le génome d\'une personne n\'est en réalité que tester la partie émergée de l\'iceberg. \n C\'est là que ça devient un peu effrayant, parce que si une personne a un résultat de test normal sur 23andMe, elle part en pensant qu\'elle n\'a pas de risque, dit-elle. Ce n\'est pas que la science derrière -LRB- ce genre de test génétique -RRB- n\'est pas bonne. C\'est que la science derrière eux n\'est pas complète. \nL\'avantage de ces kits de test génétique à domicile - et 23andMe n\'est pas la seule entreprise à les vendre - est qu\'ils ont généré beaucoup de buzz sur la génétique, dit Nagy. Elle dit qu\'elle pense que la plupart des premiers utilisateurs des tests sont informés et comprennent que les résultats ne sont pas gravés dans le marbre.\n Je ne pense pas que quiconque pense que ce kit remplace les tests formels par un médecin, Heather Armstrong a posté sur la page Facebook de 23andMe. Je doute sérieusement qu\'un médecin accepte d\'effectuer, disons, une double mastectomie, sur la base de ces résultats. \nJoseph Stolarski a posté, Si vous trouvez quelque chose d\'intéressant, vous l\'apportez ensuite à un professionnel de la santé pour une évaluation plus approfondie. Personne ne s\'y fie ou ne devrait s\'y fier pour un diagnostic médical. C\'est juste un outil, comme vérifier personnellement votre fréquence cardiaque avec une montre ou vérifier votre poids sur une balance. \nMais Nagy dit qu\'elle s\'inquiète de ce qui se passera lorsque les tests deviendront plus courants.\nDonner ce genre d\'informations à quelqu\'un qui ne connaît pas les limites des tests génétiques pourrait s\'avérer dangereux.\nNagy recommande à quiconque souhaite faire des tests génétiques de parler d\'abord à un conseiller en génétique. Les conseillers ont des diplômes d\'études supérieures spécialisées et une connaissance approfondie du génome humain. Ils sont également formés pour expliquer les résultats des tests aux profanes.\n Notre message est d\'être un consommateur avisé, dit Nagy. Sachez ce que vous faites afin que lorsque vous obtenez les résultats, vous puissiez vraiment les utiliser à votre meilleur avantage. \nHartmann a mené une enquête approfondie, affirmant qu\'il avait examiné les données brutes que 23andMe lui avait fournies. Il a finalement découvert qu\'il avait effectivement deux mutations, dit-il, mais qu\'elles n\'étaient pas sur le même gène. Il dit qu\'il a soumis son rapport de bogue à l\'entreprise et qu\'elle s\'est excusée.\nEn général, Hartmann pense que c\'est une bonne chose pour chaque personne d\'avoir un accès abordable à ses données génétiques. Mais il a parcouru les forums de la communauté sur 23andMe. com et a vu à quel point certaines personnes prennent leurs résultats au sérieux - sans les comprendre pleinement.\nHartmann dit qu\'il croit maintenant qu\'il ne souffrira pas de dystrophie musculaire des ceintures, bien qu\'il affirme qu\'il existe un risque qu\'il puisse transmettre les mutations génétiques à ses enfants .\n Je peux vivre avec ça, écrivit-il. Pour un bon bout de temps, j\'espère. \nLes parents font pression pour un dépistage standardisé des maladies génétiques juives\nDorrine Mendoza de CNN et James O\'Toole et Aaron Smith de CNNMoney ont contribué à ce rapport.\n,,,Dans le monde d\'aujourd\'hui, ce n\'est pas inhabituel pour les gens de monter à bord d\'un avion et d\'être à l\'autre bout du monde en quelques heures. Les avions de ligne gros porteurs utilisés aujourd\'hui sont monnaie courante dans la plupart des pays développés, mais il fut un temps, il n\'y a pas si longtemps, avant même que le transport aérien ne soit possible. C\'était avant Orville, et Wilbur Wright construisit et pilota son premier avion au début du XXe\xa0siècle.\n \nLe 17\xa0décembre\xa01903, les frères effectuèrent leur vol historique à Kitty Hawk, en Caroline du Nord, ouvrant officiellement une nouvelle ère dans le voyage. Leurs contributions à la nouvelle science de l\'aéronautique se sont poursuivies pendant au moins 10 ans après ce jour historique. Ce qui a séparé les frères Wright des autres pionniers de l\'aviation, c\'est que les frères ont développé un mécanisme efficace pour contrôler leur avion. Leur système de contrôle à trois axes est toujours utilisé aujourd\'hui, plus de 100 ans après son développement.\nLe premier vol n\'a pas été très long. Il ne faisait que 120 pieds de long, 10 pieds de haut et seulement 12 secondes de durée, mais c\'était du vol. Orville a piloté l\'avion, tandis que Wilbur courait à côté. Il n\'y avait que cinq témoins oculaires du vol, mais un photographe a capturé l\'image sur film, et le reste appartenait à l\'histoire de l\'aviation.\nLes frères étaient très protecteurs de leur nouvelle invention et il était difficile pour les journalistes de faire la chronique de leurs efforts. Cela a conduit à des rumeurs répandues selon lesquelles tout cela était un canular. Au cours des années suivantes, les frères ont travaillé pour affiner leur machine volante en secret, ce qui a suscité davantage de doutes. Cependant, en 1908, les frères apportèrent leur invention au Mans, en France, pour organiser une exposition. Wilbur a guidé son avion en effectuant des virages et d\'autres manœuvres, ce qui a étonné la foule. Bientôt, les frères Wright étaient célèbres. Ils ont changé la façon dont les gens voyagent.,,,L\'Opéra de New\xa0York a besoin de 7\xa0millions de dollars de collecte de fonds d\'ici lundi, sinon il pourrait fermer définitivement ses rideaux.\nEn plus de cet argent, la compagnie d\'opéra souhaite lever 13\xa0millions de dollars supplémentaires d\'ici la fin de 2013 vers les saisons futures, selon un communiqué de presse. Risa Heller a déclaré vendredi à CNN.\nLa société a lancé une campagne pour collecter 1 million de dollars sur ces 7 millions de dollars via un site de collecte de fonds en ligne -- Kickstarter.com -- qui se terminera le 30 septembre. Vendredi, la campagne en ligne n\'avait recueilli que 202 223 dollars de dons de plus d\'un millier de supporters.\nL\'organisation a collecté 1,5\xa0million de dollars en dehors de la campagne Kickstarter, selon Heller.\n Les chances sont contre nous depuis longtemps, a déclaré George\xa0Steel, directeur général et directeur artistique de l\'entreprise, dans un vidéo en ligne, mais face à cette difficulté ty nous avons fait d\'énormes progrès. \nLa société, surnommée The People\'s Opera par l\'ancien maire de New York Fiorello LaGuardia, a été fondée sur le principe que chaque New-Yorkais devrait pouvoir se permettre d\'aller à l\'opéra, a déclaré Steel.\nSteel explique que pour la société pour atteindre le seuil de rentabilité, il faudrait vendre chaque billet pour 600\xa0$, mais le prix de départ du billet est en fait de 25\xa0$ pour le rendre plus abordable.\nLe New York City Opera a reçu des éloges de la critique dans le passé pour ses premières mondiales d\'œuvres, dont Robert Ward \'s The Crucible and Anthony Davis The Life and Times of Malcolm X. \nLa compagnie a ouvert sa saison en cours le 17 septembre avec l\'opéra Anna Nicole, et si elle ne parvient pas à réunir les fonds nécessaires d\'ici lundi, cela pourrait être la Compagnie\\ Nous avons besoin de l\'aide des personnes que nous avons fondées pour organiser notre saison cette année, a déclaré Steel, s\'adressant aux habitants de New York et aux fans d\'opéra du monde entier. Nous avons besoin que vous vous réunissiez et que vous le portiez dans le futur. J\'espère que nous pourrons compter sur vous. \nCette année marque le 70e anniversaire du New York City Opera, qui a ouvert ses portes en 1943, ce qui en fait le deuxième opéra de New York. Le Metropolitan Opera a été fondé plus de 60 ans plus tôt, en 1880. Brésil. Le gouvernement américain est d\'accord.\nDavid Goldman se bat pour la garde de son fils après la mort de Bruna Bianchi Goldman lors d\'un autre accouchement.\nGoldman a déclaré à Larry King de CNN qu\'il n\'avait aucune idée du moment où il a déposé sa femme et son fils de 4 ans à l\'aéroport international de Newark en 2004 pour des vacances de deux semaines à Rio de Janeiro, au Brésil, qu\'il était sur le point d\'être impliqué dans une bataille internationale pour la garde.\nPeu de temps après l\'arrivée de Bruna Bianchi Goldman dans son pays natal, elle a appelé pour dire qu\'elle voulait divorcer , qu\'elle a obtenu au Brésil, et y resterait avec leur fils, Sean.\nLa bataille en cours pour la garde a pris de nombreux rebondissements et a attiré l\'attention du Département d\'État et de la Secrétaire d\'État Hillary Clinton.\n Un enfant appartient avec sa famille, et il n\'y a aucune raison pour que David Goldman ne récupère pas son enfant, a déclaré Clinton dans une récente interview à l\'émission Today de NBC. Et nous espérons que cela sera résolu très bientôt. Évidemment, sinon, nous continuerons à le soulever avec le gouvernement brésilien. \nPendant près de cinq ans, Goldman s\'est battu pour récupérer la garde de son fils. C\'est une affaire qui a été compliquée par le remariage de Bianchi Goldman, enceinte et mourante lors de l\'accouchement l\'été dernier. On dit que Sean Goldman vit avec son beau-père, qui s\'est vu accorder la tutelle provisoire. la mort de avait rendu la question de la garde un point discutable, et nous découvrons que cet homme ne dépose pas la garde, mais il dépose pour supprimer mon nom d\'un certificat de naissance brésilien qu\'ils avaient délivré pour mon fils, qui est né à Red Bank, New Jersey. \nGoldman continue de faire pression sur son cas auprès du gouvernement américain.\n Le Département d\'État travaille avec diligence pour s\'assurer que David et Sean Goldman se voient accorder leurs droits en vertu de la Convention de La Haye de 1980 sur l\'aspect droit civil de l\'enlèvement international d\'enfants, l\'agence dit dans un communiqué. Nous continuerons d\'insister pour que le gouvernement brésilien remplisse ses obligations en vertu du traité et du droit international. \nLa convention, dont le Brésil est signataire, est un traité multilatéral, qui vise à protéger les enfants contre les effets néfastes de l\'enlèvement et de la rétention à travers les frontières internationales en prévoyant une procédure pour provoquer leur retour rapide, selon le site Web de la Convention de La Haye .\nLes tribunaux brésiliens ont refusé de renvoyer Sean Goldman aux États-Unis pour toute audience de garde à vue ordonnée par les tribunaux du New Jersey et le gouvernement brésilien n\'est pas intervenu. En conséquence, l\'affaire reste dans l\'impasse. Je lui dirais qu\'il a été très courageux, car il s\'est battu pour que son fils lui soit rendu, a déclaré Clinton dans l\'interview de NBC. Son fils est la personne la plus précieuse au monde pour lui et il a fait non seulement un effort supplémentaire, mais mile après mile, d\'avant en arrière, essayant de faire comprendre que son fils Sean mérite de lui être rendu. \nMais sur Larry King Live, Helvecio Ribeiro, un porte-parole de la famille Bianchi, a remis en question l\'aptitude de Goldman en tant que parent.\n Je ne remets pas en question le droit biologique, a-t-il déclaré. Le fait est que pour être parent, il faut être plus qu\'un simple donneur d\'ADN, monsieur King. La paternité ne consiste pas à faire des films à la maison et à prendre des photos, c\'est une question de sacrifice. Il s\'agit d\'apporter un soutien à votre enfant. Il s\'agit d\'être là même quand vous n\'êtes pas là.\n Et M. Goldman, alors que Bruna était encore en vie, ne l\'a pas fait, a déclaré Ribeiro.\nGoldman a rendu visite à son fils pour la première fois en plus de quatre ans le mois dernier - une expérience douce-amère qui a été déchirante quand il était temps de retourner aux États-Unis.\n Il m\'a demandé pourquoi je n\'étais pas venu le voir depuis tout ce temps, a dit Goldman à Larry King, et que était très, très douloureux - et l\'angoisse sur son visage quand il m\'a posé cette question. Je ne voulais pas lui dire que... ils le retiennent. \nLes combats internationaux pour la garde des enfants sont courants et devaient être simplifiés par l\'accord de La Haye de 1980, mais des systèmes judiciaires contradictoires et des récits conflictuels de relations peuvent l\'emporter sur le droit international. La prochaine étape dans ce cas est incertaine.\n,,,\ufeffJunior Smart en sait long sur les gangs. Il a maintenant 36 ans et sa vie peut être divisée en deux phases distinctes. Il raconte comment, à la fin de son adolescence, après la mort de sa mère, il a été entraîné dans un gang du sud de Londres, ce qui, à l\'époque, a contribué à combler un vide énorme. \n"Ils sont devenus mon nouveau groupe de soutien", dit-il. "Au début, c\'était juste un peu amusant, mais ensuite c\'est devenu plus sérieux, de plus en plus pour gagner de l\'argent. Ils se sont impliqués dans la criminalité. C\'était comme ça. À l\'école, Smart a échoué à ses GCSE, puis les a repris à l\'université et a réussi le lot. Il a obtenu un emploi à temps plein dans l\'administration et a travaillé comme DJ. Mais, à côté, il gagnait de l\'argent illégalement dans le cadre du gang. \nFinalement, il a été arrêté pour de graves délits liés à la drogue et envoyé en prison pour 12 ans. Instantanément, dit-il, son sentiment d\'invincibilité a été brisé. "La première nuit après mon arrestation a été le plus grand réveil de ma vie", dit-il. « Je vivais une double vie. Je vivais comme une personne pour mes pairs et une autre personne pour les ennemis de mes pairs. J\'ai mis beaucoup de temps à me débrouiller. » \nAujourd\'hui, Junior Smart dirige une équipe de 12 travailleurs à temps plein et de six bénévoles, qui vise à détourner les jeunes criminels et les membres de gangs du crime. La plupart de ceux qui y travaillent sont, comme Smart lui-même, d\'anciens délinquants. Quelques-uns purgent encore leur peine mais sont considérés comme suffisamment réformés pour être autorisés à sortir pendant la journée pour aider. Ils travaillent avec la police, le service de probation et d\'autres organisations bénévoles pour aider ceux qui se sentent piégés et effrayés dans les gangs criminels violents qui opèrent à travers Londres. \nPour Smart, le voyage extraordinaire de membre de gang à mentor a commencé lorsqu\'il a été témoin, de l\'intérieur, d\'un système carcéral qui laissait si manifestement ses détenus tomber. Il se souvient d\'un toxicomane avec qui il s\'était lié d\'amitié et qui, à sa grande consternation, revenait sans cesse. « J\'ai été touché par les gens qui revenaient sans cesse, dit-il. «Je ne pouvais pas croire que rien n\'était fait à ce sujet. je parlais au détenus et ils savaient ce qu\'il fallait changer dans leur vie, mais le problème est que le système pénitentiaire ne s\'occupe que de l\'infraction "index". \n"Un type avait une dépendance à la cocaïne de 300\xa0£ par semaine, qu\'il a financée par un cambriolage. Il me racontait des trucs sur la façon dont il entrait dans une maison, même s\'il savait qu\'il y avait des gens. Donc, bien qu\'il ait eu une dépendance à la drogue, il n\'a jamais été traité. Ce qui m\'a attiré à ce sujet, c\'est qu\'il s\'agit simplement d\'une porte tournante. \nCette affaire et des dizaines d\'autres ont persuadé Smart de commencer à travailler comme "auditeur" de la prison - un détenu qui aide à rassurer les nouveaux arrivants et leur parle de leurs premiers jours à l\'intérieur. À partir de là, il a élaboré ce qu\'il appelle son propre "petit plan" pour gérer son propre programme une fois sorti de prison - en utilisant l\'expérience d\'anciens délinquants pour aider les autres à rejeter la porte tournante de la vie carcérale. Il a été libéré tôt, après cinq ans. « À la fin de ma peine, j\'ai eu l\'occasion de mettre mon petit modèle en pratique », dit-il. \nAlors qu\'en pense-t-il maintenant\xa0? Croit-il qu\'après les émeutes d\'août dernier et la promesse du gouvernement de sévir contre les gangs, les choses s\'améliorent ? Il prend soin de faire l\'éloge de la police et dit qu\'une grande partie de leur travail de recrutement de membres de gangs a été bon. Mais dans l\'ensemble, il est très critique à l\'égard d\'une approche gouvernementale décousue qui estime qu\'une fois le chef d\'un gang arrêté, le problème est résolu. Il est d\'accord avec les conclusions d\'un rapport selon lequel l\'arrestation de chefs de gangs peut même aggraver les choses. Il dit que l\'effet de retirer le chef est souvent de déstabiliser l\'ensemble du gang. \nIl dessine un schéma de la hiérarchie sur un bloc. «Lorsque vous arrêtez le meilleur gars, les gens commencent à se battre pour une position tout en haut. Qui était le plus fidèle ? Qui avait le plus de respect ? C\'est un peu comme une famille. Ils sont plus susceptibles d\'agir, par la violence. Cela signifie que l\'arrestation des chefs de gangs a été annulée car elle n\'a pas eu d\'effet à long terme. Ils n\'ont même pas donné de répit à la communauté. \nCela peut-il réellement rendre les rues plus dangereuses\xa0? «Cela peut le faire, car dans le vide, le recrutement et la manipulation des jeunes deviennent encore plus répandus, créant de nouveaux niveaux inférieurs tout en bas… Les gens prennent parti. Si un gang ou un autre réseau de rue territorial sait qu\'un ancien [chef] a été éliminé, alors ils pensent soudainement que ce gang est faible ... Et nous avons donc des conflits inter-états en cours. Et que se passe-t-il lorsque cet ancien est en prison ? Il forme des alliances avec d\'autres membres de gangs, ou lorsqu\'il est libéré, il tente alors de reprendre le contrôle. C\'est à ce moment-là que la violence se produit. \nAfin de répartir les risques jusqu\'aux niveaux les plus bas, il affirme que les gangs recrutent désormais beaucoup plus dans les écoles primaires. Les membres les plus jeunes sont appelés "tinies". « Au cours des dernières années, nous en avons vu de plus en plus. Les tout-petits peuvent avoir entre huit et onze ans. Les jeunes sont amenés pour protéger leurs aînés du risque. Ce sont souvent eux qui sont accusés d\'avoir fait du trafic de rue ou même d\'avoir poignardé, dit-il. \nSmart dit qu\'en l\'absence de financement gouvernemental et d\'engagement en faveur d\'une réhabilitation à long terme, les défis sont immenses, en particulier dans le climat économique actuel. «\xa0J\'essaie d\'engager un jeune qui gagne 300\xa0£ par semaine grâce à des méthodes illégales. C\'était dur avant, d\'essayer de le convaincre. Mais avec un chômage élevé et des réductions d\'allocations, cela rend les choses plus difficiles. \nCela dit, son projet, qui compte plus de 1\xa0000\xa0clients, donne des résultats. Moins de 20 % de ceux qui viennent chercher de l\'aide récidivent. Par expérience personnelle, Smart refuse de considérer qu\'un seul individu est irrécupérable et c\'est ce qui le motive. "Je ne pense pas ça de personne", dit-il. Il a remporté deux prix du meilleur acteur, dont un Golden Globe Award et un Screen Actor\'s Guild Award. Depp a joué certains des personnages les plus attachants de l\'histoire du cinéma américain moderne, faisant de lui l\'un des acteurs les plus populaires d\'Amérique aujourd\'hui.\n \nDepp est né dans le Kentucky, mais a grandi en Floride. Il a abandonné l\'école à l\'âge de 15 ans pour poursuivre une carrière de musicien Rock \'n\' Roll. Il a obtenu très peu de succès en tant que musicien et a été initié au théâtre par son ex-femme lors de leur visite à Los Angeles. Sauter la rue. L\'émission a duré plusieurs années et Depp a commencé à se faire remarquer par les producteurs hollywoodiens. Son premier grand succès a été de jouer un personnage artificiel dans le film Edward aux mains d\'argent. Ce rôle était le premier de nombreux personnages étranges joués par Depp. Edward aux mains d\'argent a été acclamé par la critique et a remporté plusieurs prix de l\'industrie.\nDepp a suivi son succès dans Edward aux mains d\'argent avec une série de rôles importants. En 1999, il incarne Ichabod Crane dans le film Sleepy Hollow. Sa performance a été bien accueillie par la critique, un d publics semblables. Son rôle le plus réussi a été de jouer le capitaine Jack Sparrow dans la franchise The Pirates of the Caribbean, un autre personnage excentrique. Ce rôle a été décrit comme difficile que seul quelqu\'un comme Depp pouvait jouer.\nEn 2005, Depp a joué dans Charlie et la chocolaterie, jouant à nouveau un personnage très excentrique qui vivait dans une chocolaterie.,,,Il peut y avoir pas de plus grand artiste dans l\'histoire de la musique aux États-Unis que feu Michael Jackson. La liste des pairs de Jackson dans l\'industrie est très courte, c\'est le moins qu\'on puisse dire. Jackson a commencé sa carrière au début des années 1970 en tant que plus jeune membre du groupe musical The Jackson 5. Michael était le chanteur principal et sans doute le plus talentueux des cinq frères qui composaient le groupe. Il est né le 29 août 1958 en tant que huitième enfant de 10 ans dans une famille afro-américaine de Gary Indiana. Le moteur du succès du groupe était leur père, Joseph "Joe" Jackson. Le groupe était très populaire à son époque, mais le jeune Michael allait se lancer dans sa propre carrière peu de temps après. Il était un chanteur, un auteur-compositeur, un acteur, un danseur et un homme d\'affaires à succès, mais sa carrière n\'a pas été sans controverse.\n \n Son album Thriller est l\'album le plus vendu de tous les temps. Il est sorti en 1982 et a été suivi d\'une série d\'albums, de films, de vidéoclips et d\'entreprises commerciales à succès. En 1993, la première de plusieurs personnes s\'est manifestée pour accuser Jackson de conduite sexuellement inappropriée sur un enfant. Cette accusation et d\'autres hanteront Jackson tout au long de sa vie d\'adulte. Jackson a également acquis une réputation de comportement étrange. L\'une des images indélébiles souvent associées à Jackson est celle où il tenait son nouveau-né au-dessus d\'un balcon d\'hôtel pour le montrer à son public. il a modifié son apparence pour apparaître blanc. Ses photographies de son enfance et de jeune adulte montrent un Jackson très différent. Il s\'est marié deux fois dans sa vie, dont un bref mariage avec Lisa Marie Presley, la fille du tout aussi grand musicien Elvis Presley.\nJackson est décédé le 25 juin 2009, et même sa mort n\'a pas été sans controverse. Sa famille a intenté une action en justice pour mort injustifiée contre son médecin, qui a ensuite été reconnu coupable. ,,, La ville de Mexico voit sa part de protestations, mais celle-ci était inhabituelle.\nUne femme a pleuré. D\'autres manifestants ont crié à tue-tête, exigeant des réponses. D\'autres encore ont montré des photos de leurs proches à des passants perplexes.\nLes manifestants qui se sont rassemblés jeudi sont des parents de 11 fêtards qui ont disparu il y a plus d\'une semaine d\'un bar d\'un quartier chic de Mexico connu sous le nom de Zona Rosa, ou Pink Zone. Le quartier a une scène nocturne animée avec des bars, des discothèques et des restaurants haut de gamme dans chaque rue.\nLes manifestants disent que leurs proches ont été kidnappés le 26\xa0mai alors qu\'ils faisaient la fête au Heaven, un bar ouvert tard dans le quartier. Tous les 11 ont disparu entre 10h et midi, disent-ils.\nLe bar n\'est qu\'à quelques pas du Paseo de la Reforma, une avenue emblématique du centre de Mexico. Le monument emblématique de l\'Ange de l\'Indépendance se trouve à proximité, tout comme l\'ambassade des États-Unis et le quartier financier.\nGuadalupe Dominguez, un proche de l\'un des disparus, a déclaré qu\'un témoin lui a dit que les 11 personnes avaient été enlevées par des hommes armés qui se sont présentés dans des SUV, mais les autorités disent qu\'il n\'y a aucune preuve d\'un tel incident.\n Un jeune homme qui a réussi à s\'échapper est celui qui nous en a parlé, mais nous ne savons vraiment rien d\'autre, a déclaré Dominguez.\nMexico rapporte plus de 26\xa0000\xa0disparus\nLa plupart de ceux qui traînaient au bar ouvert en soirée avaient la vingtaine, à l\'exception de Yersi Ortiz, qui a 16 ans. Maria Teresa Ramos, la grand-mère d\'Ortiz, a déclaré que les proches étaient mystifiés et voulaient des réponses.\n Cela s\'est soi-disant produit dimanche en plein jour. Cela n\'aurait pas pu se produire pendant la journée et à quelques pas de l\'avenue Reforma sans que personne ne s\'en aperçoive. Il devrait y avoir des caméras de surveillance qui peuvent nous montrer exactement ce qui s\'est passé, a déclaré Ramos.\nMaria del Carmen Zamudio, une autre parente à la manifestation, a déclaré que le témoin leur avait dit que les jeunes avaient tous soudainement reçu l\'ordre de partir.\n Le bar -LRB- Le propriétaire de -RRB- leur a apparemment dit qu\'il y aurait une opération de police et a éteint les lumières. Il leur a dit de sortir, et des hommes armés dans des SUV noirs les attendaient déjà à l\'extérieur, a déclaré Zamudio.\nLa police a déclaré qu\'il n\'y avait pas eu une telle opération. Le maire de Mexico, Miguel Angel Mancera, a déclaré que les autorités traitent l\'incident comme un cas de personne disparue car jusqu\'à présent, il n\'y a aucune preuve que les jeunes ont été kidnappés.\n Pour l\'instant, nous n\'avons pas été en mesure de confirmer comment ce qui s\'est passé ou l\'endroit précis où -LRB- ces jeunes -RRB- ont été enlevés. Ce que nous avons, c\'est un rapport sur les personnes disparues et la connaissance qu\'elles ont disparu. Nous devons faire quelque chose pour les retrouver, a déclaré Mancera.\nÀ l\'intérieur de San Pedro Sula, la capitale mondiale du meurtre\nLe procureur du district de Mexico, Rodolfo Rios Garza, a déclaré sles caméras de sécurité du bar ne fonctionnaient pas, mais les autorités comptent sur les caméras de la ville dans le quartier.\nJusqu\'à présent, a-t-il dit, une analyse de la vidéo des caméras ne montre aucune violence dans ou autour du bar où l\'enlèvement aurait eu lieu .\nLes autorités ont fouillé le bar et trouvé des drogues illégales, mais aucun signe d\'entrée forcée ou quoi que ce soit qui indiquerait que les jeunes avaient été kidnappés, a déclaré Rios Garza.\nPlus de 26\xa0000 personnes ont disparu au Mexique au cours des six dernières années alors que la violence augmentait. et le gouvernement du pays a réprimé les cartels de la drogue, selon le ministère de l\'Intérieur du Mexique. Les autorités ne disposent pas de données sur le nombre de disparitions liées au crime organisé.\nLes 26\xa0121\xa0disparitions se sont produites au cours du mandat de six ans de l\'ancien président Felipe Calderon, qui s\'est terminé le 1er\xa0décembre lorsque Enrique Pena Nieto a pris la présidence. .\nLe bar a été fermé pendant que l\'enquête sur les disparitions se poursuit. Le bureau du procureur général mexicain s\'est également joint à l\'enquête.\nLe mois dernier, Malcolm Shabazz, le petit-fils du militant des droits civiques Malcolm X, a été battu à mort dans un autre établissement de Mexico.\nShabazz, 28 ans, a été retrouvé juste à l\'extérieur du bar par la police à 3h30 du matin à un pâté de maisons au sud de la Plaza Garibaldi.\nDeux barmans ont été arrêtés en lien avec le meurtre de Shabazz, a indiqué le bureau du procureur général. Rios Garza a déclaré que les hommes travaillaient au bar, The Palace Club, où Shabazz et trois personnes avaient pris un verre. Une dispute s\'est ensuivie lorsque le personnel a déclaré que la facture était de 1 200 $. Shabazz a été battu tandis qu\'un autre homme a été menacé et dépouillé de ses biens, a déclaré Rios.\nRéserviste de la marine enlevé au Mexique, le FBI demande l\'aide du public\nSteve Almasy et Catherine E. Shoichet de CNN ont contribué à ce rapport.\n ,,,Après un week-end d\'enquête intense, les autorités rassemblent plus de détails sur la fusillade mortelle de vendredi à l\'aéroport international de Los Angeles, y compris le comportement du suspect plus tôt dans la semaine et un avertissement de sa famille qui pourrait être venu minutes trop tard.\nLes agents envoyés pour vérifier le bien-être de Paul Ciancia sont arrivés à son appartement moins d\'une heure après le début de la fusillade, a annoncé la police lundi.\nVoici un récapitulatif pour vous mettre au courant\xa0:\nLe suspect\nCiancia , 23 ans, de Los Angeles, est accusé du meurtre d\'un officier fédéral et de la commission de violences dans un aéroport international. était incapable de parler à enquêteurs.\nDes indices sur un mobile\nLe procureur général Eric Holder a déclaré lundi qu\'une enquête plus approfondie était nécessaire pour découvrir le mobile de l\'attaque.\nMais une note trouvée sur Ciancia indiquait qu\'il voulait tuer des employés de la Transportation Security Administration pour instiller la peur dans ce que le Le suspect a qualifié les agents d\'esprits traîtres, a déclaré l\'agent spécial responsable du FBI, David Bowdich.\nSelon quelqu\'un qui connaissait bien Ciancia et ses trois colocataires, Ciancia a commencé à demander à être conduit à l\'aéroport quelques jours avant la fusillade. Il a affirmé qu\'il devait prendre l\'avion pour le New Jersey pour aider son père malade, mais il n\'a jamais dit quel jour il devait partir, a déclaré la source. immédiatement, a déclaré la source, qui a parlé à CNN sous couvert d\'anonymat.\nLe colocataire a accepté. Les enquêteurs ne pensent pas que le colocataire ait eu la moindre idée des plans de Ciancia.\nLa quasi-sauvegarde\nÀ peu près au même moment, Ciancia envoyait des SMS aux membres de sa famille à Pennsville, New Jersey.\nL\'un d\'eux a suggéré que quelque chose de mauvais se produirait.\nBien que Ciancia n\'ait aucun antécédent connu de maladie mentale, il a dit dans les textos qu\'il était mécontent, et les messages étaient suffisamment alarmants pour que le père de Ciancia ait décidé d\'appeler la police.\n J\'ai senti que c\'était assez grave . On aurait dit que Paul Ciancia en Californie pensait à se faire du mal, alors je savais évidemment que je devais téléphoner au LAPD, a déclaré lundi le chef de la police de Pennsville, Allen Cummings, à Jake Tapper de CNN.\nCummings a parlé avec un lieutenant là, qui lui a dit que le département était en train de répondre à une fusillade à LAX. \n À ce stade, nous ne faisions pas le lien, a-t-il déclaré. Ils l\'ont fait plus tard lorsqu\'un journaliste a appelé le chef de la police, lui demandant de commenter la fusillade.\nLos Angeles Police Department Cmdr. Andy Smith dit que la police a d\'abord été appelée pour vérifier Ciancia à 10 h 06. Les agents sont arrivés à son appartement six minutes plus tard, selon Smith.\nCiancia était déjà partie.\nLe calendrier fourni lundi par la police différait de celui proposé plus tôt par le représentant. Michael McCaul, R-Texas, président du House Homeland Security Committee. Il a déclaré que la police était arrivée à l\'appartement de Ciancia environ 45 minutes après le départ du suspect pour l\'aéroport.\nSelon le récit du LAPD, ils sont arrivés 52 minutes après la fusillade, qui a commencé vers 9h20, selon la police. Ce n\'était pas immédiat Ciancia est parti pour l\'aéroport.\nL\'attaque\nVendredi vers 9 h 20, Ciancia s\'est rendu à un point de contrôle de la Transportation Security Administration dans le terminal 3. Il a sorti un fusil d\'assaut de calibre .223 d\'un sac et a tiré sur l\'agent de la TSA Gerardo Hernandez à bout portant, selon un document judiciaire déposé par un agent du FBI.\nCiancia a ensuite monté un escalator mais est revenue pour tirer à nouveau sur Hernandez, apparemment après l\'avoir vu bouger.\nIl a continué à marcher et à tirer. Des témoins ont dit qu\'il allait de personne en personne, demandant, êtes-vous TSA ? \n Je viens de secouer la tête, a déclaré le voyageur Leon Saryan à Anderson Cooper de CNN. Et il a continué. \nChaos et terreur à l\'intérieur du terminal 3 de LAX\nLes victimes\nHernandez, 39\xa0ans, a été le premier officier de la TSA à mourir dans l\'exercice de ses fonctions depuis la création de l\'agence en 2001.\n Il était fier de son devoir envers le public américain et pour la mission TSA, a déclaré sa femme, Ana Hernandez.\nLe couple, qui s\'est marié en 1998, a deux enfants.\nDeux autres officiers de la TSA -- James Speer, 54 ans, et Tony Grigsby, 36 ans -- ont été blessés mais ont été libérés de la mission hôpital.\nGrigsby, qui a reçu une balle dans le pied, a déclaré aux journalistes lundi qu\'il avait été blessé alors qu\'il aidait un homme âgé à se déplacer vers une zone sûre.\n Je me suis retourné et il y avait un homme armé, a-t-il dit. M\'a tiré dessus deux fois. \nUn voyageur qui a reçu une balle dans la jambe, Brian Ludmer, 29 ans, de Lake Forest, Illinois, était dans un état correct dimanche.\nLa réponse de la police\nLes agents de la TSA ne sont pas armés. Ce sont donc des policiers de l\'aéroport qui ont finalement tiré plusieurs fois sur Ciancia dans la poitrine, le frappant également au visage et au cou.\nLe chef de la police de l\'aéroport, Patrick Gannon, a déclaré que le FBI lui avait dit que ses policiers avaient 60 secondes de retard sur Ciancia. Il a salué leur réponse, même s\'il a reconnu qu\'il avait éloigné ses officiers des positions à l\'intérieur des points de contrôle au cours de l\'année écoulée.\n La menace... à l\'aéroport n\'existe pas derrière la sécurité à ce podium\xa0; la menace existe à partir du trottoir, a déclaré Gannon. Donc... nos hommes sont stationnés dans tout l\'aéroport. \nHolder a déclaré lundi que l\'enquête comprendra un examen des mesures de sécurité à LAX et dans d\'autres aéroports.\n La responsabilité de protéger la sécurité de l\'aéroport n\'est pas une fonction de la TSA, mais quelque chose que je pense que nous devons certainement examiner, compte tenu de ce qui s\'est passé à Los Angeles , a-t-il déclaré.\nRetards de voyage\nL\'incident a forcé les autorités à fermer certaines parties de l\'aéroport, à évacuer les voyageurs et à suspendre temporairement certains départs et atterrissages.\nPlus de 167\xa0050\xa0passagers aériens ont été touchés par l\'incident vendredi à la suite d\'annulations , des retards ou des détournements vers d\'autres aéroports, selon LAX. Une compagnie aérienne, JetBlue, a temporairement déplacé ses opérations vers l\'aéroport de Long Beach.\nSamedi, 40\xa0vols supplémentaires ont été touchés, dont 30 annulés, impliquant environ 4\xa0000\xa0passagers, selon l\'aéroport international de Los\xa0Angeles.\nSelon FlightAware, un vol site Web de suivi, les compagnies aériennes ont annulé 236 vols à destination ou en provenance de LAX après l\'incident vendredi matin et 27 autres samedi. à part la fusillade, cependant.\nL\'aéroport fonctionnait normalement lundi matin.\nLa famille du suspect répond\nLa famille de Ciancia, dans une déclaration lue lundi après-midi par l\'avocat John Jordan dans le New Jersey, a déclaré qu\'elle était choquée et engourdie par les événements tragiques de vendredi dernier. \n Il est très important pour nous, en tant que famille, d\'exprimer notre profonde et sincère sympathie à la famille Hernandez, a déclaré la famille Ciancia. -LRB- Au dire de -RRB-, l\'agent Hernandez était un membre exemplaire de la communauté des forces de l\'ordre et un bon père de famille. Nos pensées vont à sa famille et aux nombreuses autres personnes qui pleurent sa disparition.\n Nous souhaitons également exprimer notre espoir que ceux qui ont été blessés lors de cet incident connaîtront un rétablissement rapide et complet. Nous regrettons également les désagréments subis par des milliers de voyageurs ainsi que l\'administration et les employés de l\'aéroport de Los Angeles. \nLa famille Ciancia a déclaré qu\'elle continuerait à aimer et à prendre soin de Paul.\n Nous le soutiendrons pendant les moments difficiles à venir. Bien que nous ne voulions pas minimiser le chagrin et la détresse vécus par de nombreuses autres familles, nous espérons que le public comprendra que c\'est aussi une période très difficile pour notre famille, a déclaré la famille.\nQuelle est la prochaine\xa0?\nSi condamnée, Ciancia pourrait encourir la peine de mort ou la prison à vie sans possibilité de libération conditionnelle. Le procureur général des États-Unis décidera s\'il convient de poursuivre une condamnation à mort.\nL\'administrateur de la TSA, John Pistole, a déclaré que la fusillade avait entraîné une révision du protocole de sécurité avec les agences partenaires.\nMcCaul a déclaré qu\'une meilleure coordination avec les forces de l\'ordre locales pourrait améliorer la sécurité aux points de contrôle.\nMais le Le membre du Congrès a reconnu qu\'il est très difficile d\'arrêter ces types d\'attaques. \n C\'est presque comme un centre commercial ouvert, dit-il.\n,,,Bonjour Linda,\nComment ça va\xa0?\nDésolé, je n\'ai pas bJe suis en contact depuis si longtemps mais j\'ai eu des examens donc j\'ai étudié chaque minute de libre. Quoi qu\'il en soit, j\'aimerais entendre toutes vos nouvelles et j\'espère que nous pourrons nous réunir bientôt pour nous rattraper. Nous venons d\'emménager dans un appartement plus grand, alors peut-être que vous pourrez venir nous rendre visite un week-end\xa0?\nComment se passe votre nouveau travail\xa0? \nAu plaisir d\'avoir de vos nouvelles\xa0!\nHelga\nSalut Helga,\nCela fait longtemps que je voulais vous écrire, alors ne vous inquiétez pas\xa0! Comment se sont passés vos examens ? Quand connaîtrez-vous vos résultats ? Je suis sûr que vous avez brillamment réussi, comme toujours\xa0!\nQuant à moi, j\'aurai occupé le nouveau poste pendant trois mois d\'ici la fin de la semaine prochaine, donc je me sens plus à l\'aise. Au début, j\'avais l\'impression que je Je n\'avais aucune idée de ce que je faisais, mais maintenant je réalise que c\'est normal de se sentir comme ça. Il y avait beaucoup à apprendre – il y en a encore d\'ailleurs – et j\'ai vite dû m\'habituer à l\'idée que je ne pouvais pas tout savoir. Avant, je travaillais beaucoup tard et le week-end, mais j\'entre lentement dans une routine normale.\nCe qui signifie que j\'aimerais venir vous rendre visite\xa0! On a vraiment besoin d\'un bon rattrapage ! Je ne peux pas croire que nous ne nous soyons pas vus depuis le mariage de Carl. À quoi ressemble le mois prochain\xa0?\nQuoi qu\'il en soit, je ferais mieux de me remettre au travail.\nFélicitations pour le nouvel appartement\xa0! J\'ai hâte de vous voir\xa0!\nAmour,\nLinda,,,L\'équipe nationale de football du Kenya n\'a peut-être pas atteint la finale de la Coupe du monde au Brésil, mais un homme sera là pour sa nation africaine.\\ nEntré dans le rôle de représentant du Kenya, Aden Marwa, le premier juge de ligne du pays à officier lors du plus grand événement du beau jeu.\n Le football est ma vie, le football a beaucoup changé ma vie, a déclaré Marwa à CNN .\nComme beaucoup de jeunes garçons, Marwa est devenu obsédé par le football, parcourant souvent de longues distances pour pouvoir se rendre à la télévision pour regarder un match. Pourtant, malgré sa passion pour le jeu, il ne voulait rien avoir à faire avec l\'arbitrage et pour cause.\n À cette époque, c\'était difficile car un match se terminait difficilement sans qu\'un arbitre ne soit battu, explique-t-il. Les fans étaient tapageurs et vous savez, les installations n\'étaient pas très bonnes. \n Tu jouais dans des champs ouverts pour que l\'arbitre puisse être attaqué à tout moment alors quand j\'ai pu voir ça en tant que jeune garçon, je n\'aimais pas ça. Alors j\'ai eu peur. \nCependant, au moment où Marwa est devenue juge de ligne en 1998, à l\'âge de 21\xa0ans, le jeu avait radicalement changé car davantage de ressources et d\'éducation avaient été consacrées au sport.\n Une fois que vous avez décidé de devenir arbitre de football, il y a trois piliers. Le premier est votre aptitude médicale, le deuxième est votre aptitude physique et le troisième, l\'aptitude technique, de sorte qu\'ils ne peuvent être compromis. \nEn 2006, l\'ambitieux arbitre a obtenu sa grande pause en arbitrant un match de la Premier League kenyane avant de faire ses débuts internationaux en tant qu\'arbitre assistant pour la FIFA deux ans plus tard.\nAujourd\'hui, il détient le titre de meilleur arbitre assistant de la Premier League kenyane. et son talent l\'a emmené sur les côtes ensoleillées de l\'Amérique du Sud pour la Coupe du monde 2014, où il a joué en tant qu\'arbitre assistant de réserve.\nLe joueur de 37 ans sait que ce sera le plus grand test de sa carrière jusqu\'à présent. et est prêt à voir des millions de fans de football scruter chacune de ses décisions.\n Les arbitres doivent prendre des décisions dans les plus brefs délais. Je le vois une fois mais vous, à la télévision, le verrez et verrez une rediffusion et vous pouvez même faire une pause avec la technologie et même revoir, dit-il.\n En tant qu\'arbitre, je n\'ai qu\'un seul angle pour regarder ça... alors le défi que nous avons en tant qu\'arbitres, le plus grand, est de savoir comment se placer à la bonne position et au bon moment pour prendre cette décision très importante et cruciale. \nRegardez la vidéo ci-dessous pour découvrir comment Marwa s\'est préparé pour son plus grand moment à ce jour et a obtenu le poste de ses rêves en tant qu\'arbitre lors de la Coupe du monde de cette année.\n,,,\ufeffLorsque Larry Pizzi, un cadre vétéran de l\'industrie du vélo, a été le premier entendu parler des vélos électriques il y a près de 20 ans, il a demandé: "Pourquoi quelqu\'un voudrait-il bousiller un vélo en y mettant un moteur et des batteries?" \nC\'est une question qui intrigue encore les traditionalistes. Les magasins de vélos ont mis du temps à stocker des vélos électriques, même s\'ils existent depuis la fin des années 1990. Les ventes aux États-Unis ont été modestes. \nPizzi, qui est maintenant PDG de Currie Technologies, le premier vendeur de vélos électriques aux États-Unis, pense que cela est sur le point de changer. D\'autres dans l\'industrie sont d\'accord. Des marques familières telles que Trek, Raleigh et Specialized proposent toutes des modèles électriques et parient que le marché est sur le point de décoller. \n"Nous sommes sur le point d\'être adoptés par le grand public", a déclaré Pizzi. "Il semble que de plus en plus de joueurs entrent dans la catégorie chaque mois qui passe." \nLes États-Unis sont une exception en matière de vélos électriques. Près de 32 millions de vélos électriques ont été vendus en 2014, la grande majorité en Chine, où ils sont principalement utilisés pour le transport. Ils sont également populaires dans une grande partie de l\'Europe. Ils sont courants aux Pays-Bas et en Suisse; Les postiers allemands les utilisent pour se déplacer et BMW en propose un pour environ 3 000 $. \nLes vélos électriques sont différents des motos ou des cyclomoteurs, qui dépendent de la puissance motorisée\xa0; ce sont des vélos qui peuvent être pédalé avec – ou sans – l\'assistance d\'un moteur électrique. Conduire un vélo électrique donne l\'impression de conduire un vélo conventionnel avec un vent arrière vif; le moteur vous aide à aller plus vite ou à gravir des collines, mais ce n\'est généralement pas la principale source de propulsion. Contrairement aux cyclomoteurs, les vélos électriques sont généralement autorisés sur les pistes cyclables et ils ne peuvent pas rouler à plus de 20 mph. \nIl y a un débat sur le nombre de vélos électriques vendus aux États-Unis et il n\'y a pas de décompte officiel. \nLes estimations des ventes annuelles varient d\'environ 50\xa0000 à 175\xa0000. C\'est comparable au nombre de voitures électriques vendues aux États-Unis - 118 000 en 2014. Pourtant, alors que beaucoup de gens connaissent la Nissan Leaf, Chevy Volt et Tesla, peu ont entendu parler des entreprises de vélos électriques Currie, Pedego ou ElectroBike. \nPour réussir, l\'industrie du vélo électrique aux États-Unis doit franchir des obstacles juridiques, culturels et financiers. Les vélos électriques sont interdits dans certains États, dont New York, bien que la loi ne soit pas strictement appliquée. Les traditionalistes qui possèdent et occupent des magasins de vélos n\'aiment pas mettre des moteurs sur les vélos, citant, entre autres, le poids supplémentaire. Certains vélos électriques pèsent près de 30 kg. \nLes vélos électriques sont également chers. Alors que les modèles bas de gamme se vendent aussi peu que 700 $, Court Rye, fondateur et éditeur d\'ElectricBikeReview.com, un site Web populaire, affirme que les cyclistes doivent s\'attendre à payer au moins 1 500 $ pour un vélo électrique de qualité avec une bonne batterie. Les modèles haut de gamme coûtent plus du double. \nLes entreprises qui fabriquent et vendent des vélos électriques disent qu\'elles peuvent surmonter ces obstacles. \nLa technologie des vélos électriques, en particulier les batteries, s\'améliore. «Les batteries deviennent plus petites, plus légères, plus fiables et durent plus longtemps», déclare Don DiCostanza, fondateur et PDG de Pedego, un fabricant et détaillant de vélos électriques. \nDes entreprises comme Bosch, le géant allemand de l\'électronique, et Shimano, le premier fabricant de vitesses pour vélos, font leur entrée sur le marché, ce qui devrait contribuer à éroder la résistance des magasins de vélos. "Cela a vraiment attiré l\'attention et l\'imagination des revendeurs de vélos", déclare Larry Pizzi de Currie. Pedego et la startup ElectroBike n\'attendent pas que les magasins arrivent\xa0; ils construisent leurs propres magasins. En attendant, des efforts de lobbying sont en cours pour permettre l\'utilisation des vélos électriques partout. \nPeut-être plus important encore, à mesure que de plus en plus de villes construisent des infrastructures cyclables, y compris des pistes cyclables dédiées, les déplacements à vélo sont devenus plus populaires. Comme l\'a rapporté le US Census Bureau en 2014, le nombre de navetteurs à vélo est passé d\'environ 488 000 en 2000 à 786 000 en 2012. C\'est "un pourcentage d\'augmentation plus important que celui de tout autre mode de transport", note le rapport. Les vélos électriques rendent les déplacements plus pratiques - et amusants - en soulageant les soucis liés aux collines, aux vents contraires, à la fatigue et à la transpiration. \nLa plupart de nos clients sont "des baby-boomers vieillissants qui veulent raviver l\'expérience qu\'ils ont eue quand ils étaient enfants", explique Don DiCostanza de Pedego. "La principale raison pour laquelle ils ont arrêté de faire du vélo était à cause des collines." Pedego a ouvert près de 60 magasins aux États-Unis et a vendu des vélos à des voyagistes à San Francisco et à Washington, DC. \nElectroBike, qui exploite 30 magasins au Mexique, a ouvert son premier magasin américain à Venice Beach, en Californie, à l\'automne 2014 et espère atteindre 25 magasins aux États-Unis en un an. Le PDG Craig Anderson déclare : « Nous voulons aider à réduire le trafic, aider à réduire notre empreinte carbone et promouvoir un mode de vie sain. Il dit aux clients : « Roulez une fois et essayez de ne pas sourire. \nDes startups comme Pedego et ElectroBike devront concurrencer de grandes entreprises comme Trek et Currie, qui, en 2012, ont été acquises par le groupe Accell, une société publique basée aux Pays-Bas qui est le leader européen du marché des vélos électriques. Accell possède la marque Raleigh, ainsi que Haibike, un vélo électrique allemand primé. \n"Accell a de grandes attentes concernant les vélos électriques en Amérique du Nord", déclare Larry Pizzi de Currie. "Alors que les baby-boomers sont toujours un segment très important, nous constatons que beaucoup de jeunes utilisent des vélos électriques pour le transport, au lieu de voitures." \nLa marque Yuba d\'Accell vend même un vélo cargo avec un moteur et un porte-bagages arrière plus puissants. « Vous pouvez transporter deux enfants », dit Pizzi. « Vous pouvez transporter 45 kg de courses. C\'est une alternative aux mini-fourgonnettes. \n,,,\ufeffLes boules sont tombées dans la goulotte et les six chiffres correspondent, il est donc temps d\'acheter cette Audi, de réserver les vacances aux États-Unis et de téléphoner à l\'agent immobilier. Du moins, c\'est ce que font la plupart des millionnaires de la loterie, selon une analyse des dépenses et des investissements des gagnants du jackpot. \nDepuis son lancement en 1994, la loterie a créé 3\xa0000\xa0millionnaires qui ont gagné plus de 8,5\xa0milliards de livres sterling au total, pour une moyenne de 2,8\xa0millions de livres sterling chacun. L\'effet d\'entraînement signifie qu\'à eux deux, ils ont créé 3 780 millionnaires supplémentaires parmi leurs enfants, leur famille et leurs amis, selon les auteurs de l\'étude, Oxford Economics. \nLa plupart des gagnants (59\xa0%) abandonnent immédiatement leur travail, mais 19\xa0% continuent à faire leur travail de jour et 31\xa0% font du travail bénévole non rémunéré. La bonne nouvelle pour l\'économie est que 98% des dépenses des gagnants sont restées au Royaume-Uni. À travers le En ce qui concerne les dépenses immobilières, les véhicules et les vacances, on estime que chaque gagnant maintient six personnes dans un emploi à temps plein pendant un an. \nLes gagnants ont contribué pour près de 750\xa0millions de livres sterling au produit intérieur brut (PIB) et généré plus de 500\xa0millions de livres sterling de recettes fiscales pour le Trésor. La majeure partie de l\'argent est allée à l\'immobilier, avec 2,72 milliards de livres sterling dépensés pour les principales propriétés des gagnants et 170 millions de livres sterling pour le remboursement de la dette et des hypothèques existantes. \nLe maintien des revenus était une priorité, avec 2,125\xa0milliards de livres sterling dépensés en investissements. Les cadeaux à la famille et aux amis ont représenté 1,17 milliard de livres sterling et 680 millions de livres sterling ont été dépensés en voitures et en vacances. \nL\'étude, commandée par Camelot, les opérateurs de la loterie nationale du Royaume-Uni, pour marquer le cap des 3\xa0000\xa0gagnants, était basée sur des recherches menées auprès de 100\xa0gagnants de plus de 1\xa0million\xa0£. Il a révélé qu\'au total, les 3 000 gagnants ont acheté 7 958 maisons ou appartements au Royaume-Uni, soit 2,7 chacun, dépensant 3,3 milliards de livres sterling. La plupart des gagnants (82 %) ont changé de résidence principale, dépensant en moyenne 900 000 £. \nLa nouvelle maison sera probablement équipée d\'un bain à remous, et près d\'un tiers (29\xa0%) l\'ont inscrit sur leur liste de courses. Un dressing est indispensable pour 28%, près d\'un quart (24%) ont opté pour un bien avec portail électrique, 22% ont une salle de jeux, 7% installent un billard. \nLes grandes propriétés doivent être entretenues, et 30\xa0% des gagnants ont employé un nettoyeur et 24\xa0% un jardinier. Une faible proportion (5 %) employait une esthéticienne. \nLes Audi étaient les voitures préférées de 16\xa0% des gagnants, les Range Rover et les BMW étant également des achats populaires (11\xa0% chacun), ainsi que les Mercedes (10\xa0%) et les Land Rover (5\xa0%). Les gagnants ont dépensé 463 millions de livres sterling pour 17 190 voitures, le prix moyen de leur favori étant de 46 116 livres sterling. \nLes vacances étaient également une priorité. La majorité (68 %) choisissent des hôtels cinq étoiles à l\'étranger. Les États-Unis étaient la destination préférée de 27 %, suivis des Caraïbes (9 %). Plus près de chez nous, cependant, les ventes de caravanes au Royaume-Uni en ont profité. Au cours des 18 dernières années, 10 % des millionnaires ont acheté une caravane, générant des ventes d\'environ 7,4 millions de livres sterling. \nCertains gagnants (15\xa0%) ont créé leur propre entreprise, 9\xa0% ont aidé d\'autres personnes à le faire et 6\xa0% ont investi ou acheté les entreprises d\'autres personnes. Les entreprises créées ou soutenues par les gagnants de la loterie emploient 3 195 personnes, selon l\'étude. \nAndy Logan, auteur du rapport, a déclaré\xa0: "L\'effet d\'une victoire se propage beaucoup plus loin et plus largement que nous ne l\'avions prévu. Non seulement cela transforme la vie des amis et de la famille, mais chaque gain a un effet mesurable sur l\'économie britannique, en particulier avec une grande partie dépensée au Royaume-Uni. L\'utilisation de chaque victoire crée un effet d\'entraînement sur cette génération et très souvent sur la suivante. \n,,,Les Bruins de Boston ont remporté leur première Coupe Stanley en 39 ans, battant les Canucks de Vancouver 4-0 mercredi soir lors du septième et décisif match du championnat annuel de la Ligue nationale de hockey.\nLa défaite n\'a fait n\' Des centaines de fans des Canucks, pour la plupart jeunes, sont descendus dans la rue et ont incendié plusieurs véhicules renversés à quelques pâtés de maisons de la Rogers Arena où le match a été joué.\nCertains fans se sont arrêtés pour poser devant les flammes. D\'autres ont dansé sur un autre véhicule renversé. Un nuage terne de fumée grise a étouffé certaines zones du centre-ville.\niRapport\xa0: Incendies, émeutes et pillages à Vancouver\nDes images aériennes ont montré la police de Vancouver pataugeant dans la foule indisciplinée qui se moquait et lançait continuellement des objets sur les agents. Des membres de la foule ont sauté par-dessus un incendie de rue et des agents ont fait tomber plusieurs fans au sol.\n Nous ne sommes pas en mesure de publier quoi que ce soit de formel pour le moment, mais les agents sont occupés à traiter avec quelques groupes de personnes déterminées à commettre des actes criminels comme des méfaits, du vandalisme et des bagarres, a déclaré Lindsey Houghton, agente des relations avec les médias de la police de Vancouver, dans un communiqué.\n Nous travaillons dur pour que tout le monde rentre à la maison en toute sécurité. \nLe journaliste indépendant Jorge Contreras a déclaré à CNN que 1 000 à 2 000 personnes étaient dans les rues du centre-ville de Vancouver et que six véhicules de police avaient été incendiés.\nAprès avoir perdu les deux premiers matchs à Vancouver, les Bruins ont remporté quatre des cinq derniers matchs. des Canucks, qui ont eu du mal à rester dans le septième match après avoir perdu 1-0 en première période. son deuxième but du match.\n Nous savions que le deuxième but allait être énorme, Marchand, a déclaré la recrue de 23 ans dans une interview d\'après-match. C\'était bien d\'avoir celui-là, et nous avons continué à avancer après cela. \nMarchand terminerait le match avec deux buts et une passe décisive.\nMais le match a vraiment été décidé par Thomas, qui a terminé sa spectaculaire performance en séries éliminatoires avec un blanchissage du septième match pour remporter le trophée Conn Smythe tant convoité. Tim Thomas - dont le gardien de but peu orthodoxe a mystifié les Canucks toute la nuit - a établi le record de la LNH pour le plus de tirs affrontés et le plus d\'arrêts dans l\'histoire des séries éliminatoires. nEn entrant dans la série, Vancouver était considérée comme l\'équipe supérieure des équipes spéciales, en particulier sur jeux de pouvoir. Mais ce n\'était pas le cas lors de la Coupe Stanley de cette année, alors que Boston a inscrit cinq buts en avantage numérique contre deux pour Vancouver. Vancouver a également été surclassé 10-0 en deuxième période. \n Nous avons finalement ramené notre match de chez nous sur la route, a déclaré Zdeno Chara à Team Radio Canada. Nous avons joué physiquement et pris soin de la rondelle, joué extrêmement fort et fait attention aux détails. \nLe but de Patrice Bergeron en première période aurait dû être suffisant pour que les fans connaissent le résultat du match, car l\'équipe qui a marqué le premier but a remporté tous les matchs de cette série.\nLes Bruins ont maintenant remporté la Coupe Stanley à six reprises. , quatrième parmi toutes les franchises de la LNH, mais n\'avait pas remporté le prix le plus convoité du sport depuis 1972. Ils avaient été en finale cinq autres fois depuis leur dernière victoire, mais cette fois marquait leur première apparition depuis 1990 .\nLes Canucks ont participé à la finale à trois reprises au cours de leurs 40 saisons dans la ligue, mais n\'ont jamais gagné. Détenant l\'avantage de la glace à domicile, ils ont remporté les deux premiers matchs de la série, mais ont perdu les trois matchs disputés à Boston avant de perdre à domicile mercredi.\nIls cherchaient à devenir la première équipe canadienne à remporter le titre de la LNH depuis Montréal Canadiens en 1993.\n,,,Il est dans ce schéma d\'atterrissage auquel les idoles d\'âge moyen arrivent enfin\xa0: du temps pour la maison et la famille et une femme qui mérite de garder son amour.\nL\'acteur oscarisé de plus de 50\xa0ans, producteur et l\'activiste politique George Clooney est apparemment fiancé. Il est à un âge particulièrement charnière. Oubliez que les années 50 sont les nouvelles années 40 - les années 50 sont vraiment très spécifiques. Même les plus jeunes commenceront à réfléchir au sens de la vie alors que les années ne s\'étirent plus dans un fantasme d\'infini. Ces années entre 50 et 60 semblent être un âge particulièrement poignant pour les hommes, en particulier le genre d\'homme qui a du mal à abandonner les motos. à cause de leur dernier accident de moto -RRB- et ils se tournent vers l\'amour avec un nouveau sens de sa gravité. Avec un clin d\'œil à la motivation que la plupart d\'entre nous, les mortels, avons, ils sont prêts pour un héritage émotionnel. -LRB- Indice\xa0: faites attention aux enfants -RRB-\nCela pourrait irriter Clooney d\'être perçu comme si prévisible. C\'est un type doué, idiosyncrasique, qui suit son propre chemin et qui est irrévérencieux. Mais il s\'est entraîné pour cette transition de vie en pleine vue. Dans ses films, il a joué des hommes d\'âge moyen perdus, perdus dans leur carrière, perdus dans des mariages négligés, perdus à cause d\'un manque de connexion avec quelque chose d\'important – Up in the Air, The Descendents, Michael Clayton, entre autres. Beaucoup de ces hommes remanient leur vie, réajustent leurs principes ou au moins trouvent une certaine connaissance de soi qui leur manquait cruellement.\nCe que nous savons d\'Amal Alamuddin\nSi les rapports sur ses fiançailles sont vrais, c\'est un peu ce que Clooney a fait, et ce que beaucoup d\'hommes célibataires professionnels finissent par le faire, en particulier à propos de l\'amour. Sa fiancée apparente, Amal Alamuddin, est une avocate accomplie et mondaine. Clooney fait ce choix d\'amour entre deux âges. Il choisit une femme de substance ainsi que de beauté - le genre de femme que vous voudriez comme amie et conseillère, le genre de femme qui aurait les gènes pour être la mère de vos enfants - et c\'est maintenant, parce qu\'il vous apparaît soudainement que vous n\'avez pas de temps à perdre.\nDans ce scénario du vieux Lothario châtié, le candidat est assez jeune pour être convoité et assez intelligent pour être fécondé. Ce n\'est pas que les femmes, surtout les stars, n\'arrivent pas au même point que Clooney et finissent dans la même position. Mais la biologie les oblige à faire le choix comparable au moins une décennie plus tôt. Contrairement à une femme de son âge, Clooney peut faire tout le pari de s\'installer à presque n\'importe quel âge. Mais il a probablement compris que certains âges sont meilleurs pour cela que d\'autres.\nJe sais que ce n\'est pas très romantique, mais le moment est - que le groupe de 50 ans le sache ou non - guidé par la Faucheuse. Si cela sonne mal, ce n\'est pas le cas.\nL\'âge moyen est une période importante pour la réévaluation et la correction de cap. Peu importe la brillance de votre étoile, vous savez qu\'il ne reste que peu de temps pour avoir une relation profonde et, peut-être, une famille. C\'est le véritable point d\'ancrage de l\'amour d\'âge moyen\xa0: il est temps d\'arrêter de faire l\'idiot.\n,,,Yao Ming est l\'une des personnes les plus influentes à avoir jamais vécu aux États-Unis d\'Amérique. Alors, comment un Chinois de 22 ans, né à Shanghai, devient-il une superstar internationale, qui change la façon dont les deux cultures se regardent ? Ming mesure 7\xa0\\\xa0\'6\xa0pouces, ce qui fait de lui l\'un des joueurs les plus grands à avoir jamais joué au jeu, mais ce n\'est pas l\'étendue de sa puissance. Ming était grand et talentueux.\n \nMing a commencé sa carrière en Chine, où il a joué pour les Sharks de Shanghai de l\'Association chinoise de basket-ball. C\'était la saison où les Sharks ont remporté un championnat de l\'ABC. Il a pu négocier une libération de son contrat avec les Sharks, ce qui le rend éligible pour la National Basketball Association dr à l\'arrière en 2002. Ming était l\'un des joueurs les plus recherchés jamais venus de Chine, et il a finalement été repêché par les Houston Rockets.\nMing a connu un succès modéré en NBA. Sa taille et sa force faisaient de lui une présence redoutable. Il était si grand et si talentueux qu\'il pouvait dominer n\'importe quel match rien qu\'en étant sur le terrain. Ming a mené les Rockets à la première apparition de l\'équipe en séries éliminatoires depuis plusieurs années, mais ses totaux statistiques n\'étaient pas si impressionnants. Pourtant, Ming était l\'un des joueurs les plus populaires de l\'histoire de la NBA. Les fans l\'aimaient et l\'ont élu huit fois pour le match des étoiles.\nMing a créé un certain ressentiment de la part d\'autres joueurs, notamment Shaquille O\'Neal des Lakers de Los Angeles. O\'Neal a été l\'un des joueurs les plus dominants du jeu pendant quelques années, et en 2003, Ming a battu O\'Neal lors d\'un vote des fans de la NBA par plus de 250 000 voix. Cela a mis en colère O \\ \'Neal, qui a juré de surpasser Ming chaque fois que les deux géants se rencontraient sur le terrain. ,,, Les États-Unis d\'Amérique sont le seul pays à avoir jamais fait atterrir un vaisseau spatial habité sur la surface de la lune. La première personne à avoir marché sur la lune était Neil Armstrong. Armstrong a posé le pied sur la surface de la lune le 20 juillet 1969, dans ce qui a été l\'une des émissions télévisées les plus regardées de l\'histoire du pays. Armstrong est devenu une célébrité instantanée, dont le nom est devenu un mot familier pour les générations à venir.\n \nLa mission d\'alunissage a été effectuée sur le vaisseau spatial Apollo 11. Armstrong était le commandant de mission pour le vol historique et était la deuxième et dernière mission qu\'il a effectuée. Il y a eu des complications impliquant le module d\'atterrissage qu\'Armstrong a traitées au moment de l\'atterrissage. Le module arrivait plus vite que prévu. Il était destiné à manquer le site d\'atterrissage de plusieurs kilomètres, mais une réflexion rapide d\'Armstrong a empêché un désastre potentiel.\nLorsqu\'Armstrong a marché sur la surface lunaire, ses paroles ont fait écho dans tout le pays et dans le monde\xa0: "C\'est un petit pas pour l\'homme, un pas de géant pour l\'humanité." Des années plus tard, Armstrong et la National Aeronautics and Space Administration (NASA) ont insisté sur le fait que la citation était incorrecte et était masquée par des interférences. Cependant, la citation reste à jamais dans sa forme originale.\nArmstrong a reçu la Médaille de la liberté du Congrès du président Richard Nixon, et a ensuite reçu la Médaille d\'honneur de l\'espace du Congrès par le président Jimmy Carter. Le programme a été interrompu au début des années 1970. D\'autres pays ont débarqué des missions sans pilote vers le plus grand satellite de la Terre, mais à ce jour, personne d\'autre n\'a marché sur la lune depuis lors. Neil Armstrong, un véritable héros américain, est décédé en 2012 des suites d\'une intervention chirurgicale.,,,Les enquêteurs ont élargi leur recherche au Texas lundi pour une fillette de 7 ans qui a été portée disparue après que sa mère a été retrouvée morte dans l\'Oklahoma dimanche soir .\nLes autorités tentent également de retrouver l\'ex-mari de la femme décédée, a déclaré le Bureau d\'enquête de l\'État de l\'Oklahoma dans un communiqué.\nLa fille - Aja Daniell Johnson - a été portée disparue à Geronimo, Oklahoma, dimanche soir, le a déclaré le bureau.\nSa mère, Tonya Hobbs, 37 ans, a été retrouvée morte à l\'intérieur d\'un camping-car garé dimanche soir, a indiqué le bureau. Hobbs et sa fille rendaient visite au mari séparé de Hobbs, Lester William Hobbs, au domicile de sa sœur, a indiqué le bureau. pour essayer de la retrouver. Lester Hobbs n\'est pas le père biologique d\'Aja, a indiqué la police. Les autorités pensent que Hobbs a des liens avec Rockport, au Texas, et l\'alerte a été émise de manière préventive, a-t-elle déclaré.\nTonya Hobbs et sa fille ont été vues pour la dernière fois samedi soir. Les autorités n\'ont pas dit comment Hobbs est morte, mais le bureau a déclaré que les enquêteurs pensaient que quelqu\'un l\'avait tuée. Oklahoma, bureau, a déclaré aux journalistes lundi. Les parents de Hobbs sont devenus méfiants dimanche après n\'avoir vu ni Tanya Hobbs ni Aja, et l\'un d\'eux a ouvert la porte et a découvert Tanya Hobbsbody, a-t-il déclaré. Lester Hobbs et Aja étaient partis, a déclaré Goss.\nLes Hobbs étaient séparés, mais apparemment Tanya Hobbs rendait visite à Lester Hobbs pour tenter de se réconcilier, a déclaré Goss. Le couple était ensemble depuis environ cinq ans.\nUn médecin légiste procédait à une autopsie, a indiqué le bureau, et la police veut interroger Lester Hobbs sur la mort.\nOn pense que lui et la fille voyagent dans sa voiture, un blanc 1992 Toyota Paseo avec numéro d\'étiquette Oklahoma 577-BPW. La voiture à deux portes n\'a pas d\'enjoliveurs et la vitre du passager arrière est recouverte de plastique, a indiqué le bureau. en ce moment est de trouver la petite fille, le shérif Kenny STradley a déclaré aux journalistes. Nous savons qu\'elle a besoin de médicaments, et les autorités vérifiaient les pistes à ce sujet, a-t-il déclaré. Il n\'a pas précisé l\'état de santé d\'Aja.\nGoss a déclaré que Lester Hobbs avait de longs antécédents criminels, mais n\'a pas donné de détails. Le bureau du shérif a déclaré qu\'il avait des condamnations antérieures pour crime. \nLe père biologique d\'Aja\\ a obtenu sa garde d\'urgence en novembre, selon des documents du tribunal de district du comté d\'Oklahoma obtenus par l\'affilié de CNN KWTV d\'Oklahoma City, Oklahoma. Lors d\'une audience, Tonya Hobbs - identifiée comme Tonya Dunkin dans les documents - et le père de la fille, John Johnson, ont convenu qu\'elle aurait supervisé les visites avec Aja et gardé la fille loin de Lester Hobbs, selon les documents. \nAja mesure 4 pieds et pèse 59 livres, a déclaré Goss. Elle a les yeux bruns et ses cheveux sont bruns avec les extrémités teintes en noir.\nLester Hobbs mesure environ 6 pieds 1 pouce et 190 livres, avec des yeux noisette et des cheveux bruns, selon l\'Oklahoma State Bureau of Investigation. L\'agence a publié une photo de lui et a déclaré qu\'il avait une moustache mais qu\'il n\'avait plus de bouc.\nGeronimo se trouve à environ 20\xa0km au sud de Lawton, dans l\'Oklahoma, dans le centre-sud de l\'État.\n,,,\ufeffIn Canada\'s Arctique, les étés sont marqués par une lumière vive qui baigne la toundra sans arbres pendant plus de 20 heures par jour. Pour certains, c\'est un changement bienvenu par rapport à l\'obscurité implacable de l\'hiver. Mais, pour la petite mais croissante communauté musulmane d\'Iqaluit, au Nunavut, la vie au pays du soleil de minuit représente un véritable défi pendant le mois de Ramadan, au cours duquel les musulmans jeûnent généralement du lever au coucher du soleil. \n«\xa0Je ne me suis pas évanoui une seule fois\xa0», a déclaré Abdul Karim, 29\xa0ans, l\'un des rares de la ville à avoir soigneusement chronométré son jeûne du Ramadan sous le soleil de l\'Arctique depuis son départ d\'Ottawa en 2011. Cette année, cela signifie manger vers 1 h 30 avant le lever du soleil et rompre son jeûne vers 23 heures lorsque le soleil se couche. "La seule raison d\'arrêter serait si cela nuit à ma santé", a déclaré Karim. Montrant sa silhouette imposante, il a ri en ajoutant : "Mais, vu mon état, je ne pense pas que le jeûne me fera du mal." \nAlors que la fin du Ramadan approche pour les musulmans du monde entier, une grande partie de l\'accent mis par le mois sacré sur le travail communautaire, la prière et la réflexion a été une constante dans les communautés du monde entier. Mais à Iqaluit et dans les autres communautés musulmanes de l\'Arctique, les longues journées ont forcé un changement dans la façon dont l\'élément du jeûne est abordé. \nLa plupart des habitants d\'Iqaluit adhèrent à l\'horaire suivi par les musulmans d\'Ottawa, à environ 1\xa0300 milles au sud de la ville – suivant les conseils d\'érudits musulmans qui ont déclaré que les musulmans de l\'extrême nord devraient observer le Ramadan en utilisant l\'horaire de La Mecque ou de la ville musulmane la plus proche. Cela signifie toujours jeûner pendant environ 18 heures par jour, a déclaré Atif Jilani, qui a déménagé à Iqaluit de Toronto en 2015. «Ce sont de longues journées, mais plus gérables.» \nDe nombreux membres de la communauté de 100 personnes rompent leur jeûne ensemble, se rassemblant dans la toute nouvelle mosquée de la ville - achevée en février avec des températures qui ont chuté jusqu\'à -50 ° C avec le refroidissement éolien - pour les soupers nocturnes. Alors qu\'ils dégustent des plats traditionnels tels que des dattes et des currys de chèvre ou d\'agneau, le soleil brille à travers les fenêtres. \nC\'est une scène qui se déroule dans les mosquées les plus au nord du Canada pendant le Ramadan, alors que les communautés musulmanes luttent contre la géographie unique du pays. Les quelque 300 musulmans de Yellowknife, dans les Territoires du Nord-Ouest, ont plusieurs options en matière de jeûne pendant le ramadan, a déclaré Nazim Awan, président du Centre islamique de Yellowknife, avec des exceptions pour les femmes enceintes ou malades. "Certains surhumains pourraient vouloir jeûner pendant 23 heures, mais l\'autre option est de suivre l\'intention et l\'esprit du jeûne en suivant les villes voisines ou ils peuvent suivre les heures de La Mecque et de Médine." Au cours des dernières années, une grande partie de la communauté a choisi de suivre l\'horaire du Ramadan d\'Edmonton, en Alberta. Certains, comme Awan – père de deux jeunes enfants, dont un de 12 ans qui a récemment commencé à jeûner – suivent les horaires de La Mecque. Il espère encourager son fils avec le calendrier plus gérable d\'environ 15 heures de jeûne par rapport à environ 18 heures à Edmonton. \n« Si je jeûne à Yellowknife ou à Edmonton, mon fils pourrait dire : \'Papa, tu es vraiment fou. Qu\'est-ce que tu fais ?\' » dit-il. Face à l\'impossibilité de suivre les mouvements locaux du soleil, la centaine de musulmans d\'Inuvik, une petite ville située à 200 km au nord du cercle polaire arctique, ont également suivi l\'horaire d\'Edmonton. "Nous avons actuellement 24 heures de soleil par jour", a déclaré Ahmad Alkhalaf. "Il n\'y a ni lever ni coucher de soleil." \nLe respect de l\'horaire d\'Edmonton était déjà en place en 2001 lorsqu\'il a quitté Toronto pour la petite communauté du Nord de 3 500 habitants. « Mon premier Ramadan ici a eu lieu en décembre. Il n\'y a pas de soleil à ce moment-là; il fait noir toute la journée et nicombat. Nous avons donc utilisé l\'heure d\'Edmonton. Parfois, il peut être psychologiquement difficile de suivre l\'horloge plutôt que ce qui se passe à l\'extérieur, a déclaré Alkhalaf. "Vous êtes censé rompre votre jeûne au crépuscule et nous mangeons quand le soleil est levé. Ce n\'est pas habituel d\'avoir l\'iftar [le repas de rupture du jeûne] quand le soleil est levé », a-t-il déclaré. À Inuvik, où une grande partie de la population est inuite, la communauté musulmane a cherché à trouver un équilibre entre le ramadan et la culture et les traditions locales. Le repas de l\'iftar comprend des dattes et des currys riches - ainsi que du gibier local comme le renne, préparé conformément à la loi islamique. "Nous faisons une soupe ou un curry… mais au lieu d\'utiliser du bœuf, nous utilisons du renne." \nÀ Iqaluit, alors que la communauté musulmane se prépare à marquer la fin du Ramadan, certains pensent que le calendrier de 2016 - qui s\'étend sur certains des jours les plus longs de l\'année - en a fait l\'un des plus difficiles de ces dernières années. C\'est particulièrement vrai pour ceux qui, comme Karim, ont suivi avec détermination le lever et le coucher du soleil locaux. Mais, ses efforts seront récompensés dans des années, a déclaré Karim, grâce au calendrier lunaire. Le ramadan finira par tomber pendant l\'hiver, qui, à Iqaluit, voit le soleil se lever et se coucher en quelques heures chaque jour. "Je suivrai ces heures aussi", a-t-il dit en riant. "Oh oui, certainement." \n,,, Le co-créateur des Berenstain Bears, Jan Berenstain, est décédé à l\'âge de 88 ans, selon une page Facebook dédiée aux personnages à fourrure créés par Berenstain et son mari.\n C\'est avec une grande tristesse que nous vous informons que notre cher Jan est décédé le vendredi 24 février. Nous vous sommes reconnaissants pour toutes vos pensées très aimables. S\'il vous plaît, souvenez-vous de sa famille en cette période très difficile, a déclaré une déclaration sur la page Facebook de Berenstain Bears.\nBerenstain et son mari, Stan, étaient déjà des dessinateurs à succès lorsqu\'ils ont décidé de passer à l\'écriture de livres pour enfants.\nSelon l\'éditeur Harper Collins, The Big Honey Hunt, publié en 1962, a été le premier livre mettant en vedette la famille des ours.\nDepuis lors, plus de 330 livres Berenstain Bears ont été publiés au cours des 50 dernières années, selon l\'éditeur.\nPlus de 260 millions d\'exemplaires ont été vendus, ce qui en fait l\'une des séries de livres pour enfants les plus vendues de l\'histoire, a déclaré Harper Collins. une série télévisée quotidienne à succès qui a débuté sur PBS en 2002.\nAprès la mort de Stan Berenstain à l\'âge de 82\xa0ans en 2005, Jan Berenstain et son fils Mike ont continué à écrire et à illustrer de nouvelles aventures pour les Berenstain Bears, selon le Berenstain Bear Facebook page.\nBerenstain vivait dans le comté de Bucks, en Pennsylvanie, selon son éditeur.\n,,,Le chef de la secte polygame Warren Steed Jeffs, qui attend son procès dans le sud de l\'Utah, a été accusé de huit chefs d\'accusation supplémentaires en Arizona dans deux affaires distinctes, ont annoncé jeudi les autorités.\nLe polygame Warren Jeffs fait face à huit chefs d\'accusation supplémentaires liés à des allégations d\'inceste et de contact sexuel avec un mineur.\nUn grand jury a inculpé Jeffs le 10\xa0mai de deux chefs de conduite sexuelle avec un mineur et de deux chefs d\'inceste pour des incidents présumés qui s\'est produit en 2002, selon une déclaration publiée par le procureur du comté de Mohave, Matthew J. Smith.\nJeudi, un autre grand jury l\'a inculpé de deux chefs supplémentaires de conduite sexuelle avec un mineur et de deux chefs supplémentaires d\'inceste. Ces infractions ont eu lieu à Colorado City, Arizona, en 2003, selon l\'acte d\'accusation.\nLes deux actes d\'accusation indiquent que Jeffs, chef de l\'Église fondamentaliste de Jésus-Christ des Saints des Derniers Jours -LRB- FLDS -RRB-, a commis les infractions en tant que un complice. \nSelon la loi de l\'Arizona, le comportement sexuel avec un mineur est un crime de classe 6 passible d\'une peine pouvant aller jusqu\'à un an de prison. L\'inceste est un crime de classe 4 passible d\'une peine pouvant aller jusqu\'à 2 ans et demi de prison.\n Les affaires impliquent deux victimes distinctes, a déclaré Smith dans le communiqué. M. Jeffs ne viendra pas dans le comté de Mohave pour faire face à ces accusations ou pour avoir une première comparution jusqu\'à ce que son cas dans l\'Utah soit résolu. \nJeffs, 51\xa0ans, attend son procès dans le comté de Washington, dans l\'Utah, pour avoir été complice de viol en arrangeant le mariage d\'une fille de 14\xa0ans avec sa cousine de 19\xa0ans.\nLe procès a initialement été fixé. pour commencer en avril, mais le juge du cinquième district James Shumate l\'a reporté plus tôt cette année, et une nouvelle date n\'a pas été fixée. À l\'époque, Shumate a déclaré que le retard était dû à des informations récentes dans l\'affaire, mais il n\'a pas précisé.\nLe juge en mars a refusé une demande de la défense de déplacer le procès hors du comté de Washington en raison de la couverture médiatique, mais il a dit reconsidérez si les avocats rencontrent des difficultés lors de la sélection du jury.\nLes procureurs de l\'Utah disent que parce que Jeffs a forcé la fille mineure à se marier, et l\'a donc forcée à avoir des relations sexuelles avec l\'homme adulte, et parce que Jeffs a insisté pour qu\'elle reste mariée, il est complice de viol. \nAu cours de son audience préliminaire en décembre, les avocats de Jeffsdefense ont déclaré qu\'il n\'avait jamais savoir si le mariage qu\'il a arrangé a été consommé et, si c\'était le cas, si le sexe était consensuel.\nLa jeune fille, identifiée uniquement comme Jane Doe, a témoigné lors de l\'audience préliminaire qu\'elle avait eu si peur pendant le mariage forcé.\n Tout le temps Je pleurais, dit-elle. Je voulais mourir. \nLe viol en tant que complice est un crime au premier degré dans l\'Utah, passible d\'une peine de cinq ans de prison à vie en cas de condamnation.\nJeffs a déjà été inculpé en Arizona de deux chefs de conduite sexuelle avec un mineur pour avoir prétendument présidé mariages, ainsi qu\'une accusation supplémentaire de complot.\nCes accusations, à partir de 2005, ont été suspendues pendant que Jeffs est jugé dans l\'Utah.\nJeffs a été capturé le 28\xa0août\xa02006 lors d\'un contrôle routier près de Las\xa0Vegas, au Nevada. À l\'époque, il figurait depuis des mois sur la liste des 10 personnes les plus recherchées par le FBI.\nLe FLDS pratique librement la polygamie dans les villes de Hildale, Utah, et de Colorado City, Arizona, qui chevauchent la frontière de l\'État. La secte maintient des groupes d\'adeptes au Texas, dans le Dakota du Sud, au Nevada, en Colombie-Britannique et au Mexique.\nLa secte dérivée s\'est séparée de l\'Église principale de Jésus-Christ des Saints des Derniers Jours, basée à Salt Lake City, dans l\'Utah, à cause de la pratique de la polygamie. .\nL\'église mormone, qui a abandonné le mariage plural il y a plus d\'un siècle, n\'a aucun lien avec Jeffsgroup. Envoyer un e-mail à un ami\n,,,Lorsqu\'il s\'agit de décider où va l\'argent, l\'équipe Lotus Formula One affirme que sa principale priorité est la voiture, et non le pilote vedette assis dans le cockpit.\nKimi Raikkonen a déclaré en dernier semaine qu\'il a décidé de la quitter et de rejoindre Ferrari pour la saison 2014 car Lotus n\'a pas mon salaire. \nMais l\'équipe, détenue par le groupe d\'investissement luxembourgeois Genii Capital, a maintenant réagi en expliquant qu\'elle préfère concentrer ses finances sur le développement de la voiture, même si cela signifie voir leur talent de pilote s\'éloigner.\nRaikkonen a affiché la victoire de Lotusonly cette année, avec lui-même et l\'équipe tous deux quatrièmes au classement respectif des pilotes et des constructeurs.\n Nos fournisseurs et les personnes clés qui développent la voiture étaient notre priorité - peut-être pas Kimi, a déclaré Eric Boullier, directeur de l\'équipe Lotus, sur le site officiel de la F1.\n Kimi était dans une position similaire l\'année dernière et tout était réglé à la fin de l\'année. Et Genii avait l\'intention de faire la même chose cette année.\n Il est maintenant de notoriété publique que nous avons tardé à le payer, et il s\'est fâché.\n Pour gérer les flux de trésorerie - et je le fais n\\ cela ne veut pas dire l\'argent lui-même ou le budget que nous avons, qui est garanti par Genii, ou du moins la majeure partie car nous n\'avons pas le flux de revenus pour nous permettre de vivre indépendamment de Genii -- ce flux de trésorerie est un problème si vous avez des coûts fixes et que vous souhaitez maintenir le niveau de développement.\n Vous devez décider où vous souhaitez dépenser votre argent. \nGenii Capital a pris le contrôle total de l\'équipe - qui a couru en F1 depuis 1981 sous ses anciennes formes Toleman, Benetton et Renault - de l\'ancien propriétaire Renault en 2011.\nPropriété par une société d\'investissement privée, au lieu de constructeurs automobiles tels que Mercedes et Fiat, qui possèdent Ferrari, ou une entreprise mondiale telle que Red Bull, peuvent fournir un contexte financier plus risqué pour une équipe de F1.\nMais Boullier dit que le modèle commercial alternatif de Lotus ne signifie pas qu\'il est embourbé dans une crise financière, car les rumeurs circulant dans le sport le suggèrent.\n Red Bull ou Mercedes, ces sociétés sponsorisent les équipes, a déclaré Boullier, dont la société de gestion de pilotes Gravity a également des liens avec Genii. Genii a une stratégie différente\xa0: ils prêtent de l\'argent.\n Cela fait partie de la stratégie que des partenaires rejoignent l\'équipe et Genii récupérera leur investissement.\n Soixante-quinze pour cent de la dette de Lotus provient de Genii. Ils pourraient l\'annuler demain en disant que cet argent est un parrainage - et alors notre dette serait considérablement réduite. Notre dette normale est similaire à la plupart des autres équipes. \nLa stratégie de Genii consistant à choisir de financer l\'équipe avant de chercher des accords de sponsoring pour récupérer son investissement est un pari.\nAvoir le champion du monde 2007 Raikkonen à bord aurait inévitablement aidé à attirer des sponsors, mais maintenant Boullier cherche des alternatives.\nIl dit que l\'équipe prévoit d\'imiter la stratégie commerciale de certains de ses rivaux, dont Red Bull et McLaren, en recherchant un sponsor titre.\n Ce n\'est un secret pour personne que depuis un certain temps, les Genii recherchent des partenaires pour s\'assurer apporter plus d\'argent à l\'équipe et avoir accès à un plus grand portefeuille de sponsors, a ajouté le Français.\n Nous devons trouver des sponsors, car c\'est la seule façon d\'intensifier.\n attirer les meilleures personnes dans une équipe dont vous avez besoin pour avoir de l\'argent, pour pouvoir construire quelque chose à long terme.\n Le plan de Genii était de faire monter l\'équipe d\'ici cinq ans -- ce que nous essayons de faire -- mais pour passer à l\'étape suivante, vous avez besoin d\'un autre plan quinquennal en place. Et c\'est sur cela que nous travaillons. \n,,,L\'un des jours les plus sombres de l\'histoire américaine a été le 11 septembre 2001. C\'est "un jour qui vivra dans l\'infamie" pour la plupart des américains. Ce fut le jour le plus tragique de l\'histoire du pays depuis le 7 décembre 1941, lorsque les Japonais ont attaqué Pearl Harbor. Le 11 septembre a été le jour où des extrémistes musulmans ont détourné quatre avions de ligne commerciaux dans le but de détruire l\'esprit américain en projetant ces jets contre quatre bâtiments. Trois des quatre avions ont atteint leurs objectifs, tandis que l\'un a été arrêté par un groupe de héros du quotidien qui ont donné leur vie pour leur pays.\n \nAu petit matin du 11\xa0septembre\xa02001, le vol\xa011 d\'American\xa0Airlines a percuté la tour nord. du World Trade Center à New York. Quelques minutes plus tard, le vol 175, également un vol d\'American Airlines, a percuté la tour sud. Les deux attentats-suicides ont fait tomber les tours, mais pas immédiatement. Il a fallu du temps pour que la chaleur extrême de la combustion du carburéacteur affaiblisse les tours jusqu\'à leur effondrement éventuel. À peu près au même moment, American Airlines, le vol 77 s\'est écrasé sur le Pentagone, qui est un bâtiment gouvernemental dans la capitale du pays, Washington D.C.\nLe quatrième vol, le vol 93 d\'United Airlines, se dirigeait également vers la capitale. La plupart des gens pensent que sa cible était soit le bâtiment du Capitole, soit la Maison Blanche, la résidence du président. Un groupe de passagers courageux, en apprenant la nouvelle de ce qui se passait dans le pays, a décidé de faire tomber l\'avion géant même s\'ils savaient qu\'ils ne survivraient pas. Ils ont pris d\'assaut le cockpit et ont pris les commandes des pirates de l\'air. L\'avion est devenu incontrôlable et s\'est écrasé dans un champ de l\'État de Pennsylvanie, où tous les passagers ont été tués.\nEn tout, environ 3\xa0000\xa0Américains ont perdu la vie ce jour-là, mais l\'esprit américain n\'a pas été supprimé. Les responsables des attentats ont été rapidement traités et le pays a rapidement entamé son chemin vers la reprise. Le 11 septembre 2001, un jour que l\'Amérique n\'oubliera jamais. ,,, Un enfant riche se fait arrêter au volant d\'une Lamborghini jaune à Miami Beach par la police et arrêté parce qu\'il est soupçonné de conduire sous l\'influence lors de courses de rue. Ça arrive, mais quand ça arrive à Justin Bieber, c\'est plus qu\'un gros problème.\nL\'idole des adolescents n\'est que la dernière jeune star à s\'être heurtée à la fois à la police et aux relations publiques. Mais faut-il s\'attendre à ce qu\'un adolescent ait à la fois la renommée et l\'accès à tant de richesses\xa0?\nDes tonnes de jeunes stars avec plus d\'argent que des années derrière elles ont déjà rencontré de tels problèmes auparavant. Pour beaucoup, se tenir au bord du précipice de l\'âge adulte ou juste au-dessus de cette ligne, a semblé déclencher une série de problèmes\nCes dernières années, Lindsay Lohan, Chris Brown et Amanda Bynes ont fait la une des journaux avec des problèmes juridiques et des séjours en cure de désintoxication.\n Ces enfants n\'ont pas leur identité définie et personne ne dit non, a déclaré Sherrie Campbell, psychologue et auteur du livre, Loving Yourself: The Mastery of Being Your Own Person. \n Ces enfants reçoivent tout cet argent et ils cessent de se sentir alors ils agissent et ils agissent jusqu\'à ce qu\'ils aient des ennuis, presque en ce sens qu\'ils doivent tout perdre pour vivre ce qu\'ils ont, a-t-elle dit.\nBieber a eu.\nBieber a eu. les deux.\nSelon Forbes, en juin\xa02013, le jeune homme de 19\xa0ans avait un revenu de 58\xa0millions de dollars. Depuis qu\'il a été découvert sur YouTube en 2008, le chanteur avait bénéficié d\'une image presque irréprochable jusqu\'à l\'année dernière.\nEnsuite, les chiens de l\'enfer de la mauvaise publicité se sont déchaînés.\nDe l\'accusation d\'uriner en public aux rumeurs d\'une visite dans un bordel brésilien et le démêlés avec la police -LRB- les autorités ont récemment envahi son manoir à Calabasas, en Californie, dans le cadre d\'une enquête sur le vandalisme -RRB-, Bieber a passé la majeure partie de 2013 dans l\'actualité pour mauvais comportement présumé.\nSon mentor musical Usher Raymond, un ancien star de l\'adolescence lui-même, a déclaré à MTV en décembre 2013 que son jeune protégé éprouvait des difficultés à grandir sous les projecteurs.\n Le succès s\'accompagne d\'une grande responsabilité. Et pouvoir grandir dedans et malheureusement être analysé devant le monde pour avoir simplement grandi, je pense qu\'ils pourront voir la vérité, a déclaré Usher. Et ils peuvent faire une évaluation basée sur ce à quoi ressemble sa réalité. \nDernièrement, la réalité de Bieber n\'est pas si belle.\nRencontrez le cercle intime de Justin Bieber\nL\'ancien enfant star Danny Bonaduce a déclaré jeudi dans son émission de radio de Seattle sur KZOK-FM que la récente arrestation de Bieber pourrait être juste ce dont la star adolescente assiégée a besoin en ce moment.\n C\'est la meilleure chose qui aurait pu lui arriver, a déclaré Bonaduce, qui dans le passé a lutté contre la toxicomanie et des problèmes juridiques. Parlant d\'expérience personnelle, cela pourrait lui sauver la vie. \nCertaines de ses collègues célébrités ont condamné son comportement tandis que d\'autres ont offert leur soutien. La psychologue Sherrie Campbell a souligné que la célébrité est sa propre dépendance et a déclaré que les célébrités sont souvent sous la pression des attentes du public.\n Je pense que parfois ces stars pensent qu\'elles doivent être à la hauteur de leur propre image au point qu\'elles ont presque devient fans de leur propre image, dit-elle. f passage à ce stade. Britney Spears est passée de princesse de la pop à mère enquêtée lorsqu\'en 2006, elle a été filmée en train de conduire avec son fils alors en bas âge, Sean, sur ses genoux. Un adjoint a été envoyé à Spearshome pour enquêter, bien qu\'aucune accusation n\'ait résulté de l\'incident. Depuis lors, Spears a réussi à changer sa vie, a décroché un concert en tant que juge sur The X Factor pendant une saison, ainsi qu\'un nouveau petit ami, un album et une résidence à Las Vegas.\nD\'autres célébrités n\'ont pas rebondi aussi bien que Spears .\nDécouvrez Britney Spears au fil des ans\nLindsay Lohan a récemment annoncé qu\'elle jouerait dans un nouveau film et qu\'elle en produirait, et qu\'une série documentaire en six épisodes est en préparation. Mais son dernier film a été bombardé et sa carrière est loin de ce qu\'elle était autrefois. De même, la réputation de Chris Brown ne s\'est jamais complètement remise de son arrestation en 2009 à la suite de l\'attaque contre sa petite amie d\'alors Rihanna juste avant les Grammys. En octobre 2013, Brown a été arrêté à Washington, soupçonné d\'avoir frappé un homme, puis il a été expulsé d\'un centre de désintoxication pour avoir agi avec violence. \nMaintenant, Bieber se retrouve membre du club des jeunes, riches et infâmes d\'Hollywood dont les mugshots rejoignent leurs clichés de paparazzi. Son accusation de course de dragsters dans une voiture de sport survient quelques mois seulement après une vague de chagrin pour la star de Fast & Furious, Paul Walker, décédé dans un accident où la vitesse était un facteur. Il reste à voir quelle est la réponse du chanteur. sera à ses derniers ennuis. Dans le passé, il a été moins que repentant, racontant au Hollywood Reporter en novembre à propos de ceux qui critiquent son comportement, je m\'en fous. \n Je m\'en fous d\'être imprudent et de faire n\'importe quoi, mais je m\'en fous de ce qu\'ils disent, aurait dit Bieber.\n,,,\ufeffEst-ce le moment quand le streaming devient vraiment grand public\xa0? \nSelon la Fédération internationale de l\'industrie phonographique (IFPI), seuls 41\xa0millions d\'abonnés utilisaient des services de diffusion de musique dans le monde en 2014. Il s\'agit peut-être du domaine de croissance des revenus le plus important pour l\'industrie du disque, mais il s\'agit toujours d\'une niche incroyable. Non seulement cela, mais un nombre important de ces abonnés proviennent d\'accords groupés avec des opérateurs de téléphonie mobile, il est donc discutable à quel point ses utilisateurs sont «actifs». Le plus grand tour de passe-passe d\'Apple est de prendre quelque chose qui existe déjà sur le marché - les téléchargements (iTunes), les lecteurs de musique numérique (l\'iPod) et les smartphones (l\'iPhone) - et de l\'adapter pour le rendre irrésistible pour le grand public. Des informations divulguées ont récemment suggéré qu\'Apple visait à atteindre 100 millions d\'abonnés, ce qui, sur la base d\'un abonnement de 120 dollars par an, générerait 12 milliards de dollars par an. Pour mettre cela en contexte, la valeur mondiale totale de la musique enregistrée en 2014 était d\'un peu moins de 15 milliards de dollars. Apple est bon pour intégrer les produits, mais ce n\'est pas si bon. \nEst-ce la fin du téléchargement\xa0? \nL\'iTunes Store est arrivé en 2003 (2004 en Europe) à une époque où le piratage MP3 semblait insurmontable. Apple a réussi à persuader les consommateurs de payer pour les téléchargements et a développé une énorme entreprise, qu\'elle dominait, avec une part de marché estimée à 70 %. Les téléchargements détiennent toujours la plus grande part des revenus numériques de la musique enregistrée, représentant 52\xa0% des revenus numériques totaux en 2014 selon les chiffres de l\'IFPI. Apple en détient la part du lion, ce qui en fait le plus grand détaillant de musique au monde. Mais les revenus de téléchargement ont culminé en 2013 au Royaume-Uni à 283 millions de livres sterling et sont tombés à 249 millions de livres sterling en 2014. La baisse des ventes de téléchargement a frappé les États-Unis en 2013. Apple a donc acheté Beats en 2014 pour 3 milliards de dollars, non seulement pour entrer sur le marché des écouteurs haut de gamme, mais aussi pour passer de la propriété de la musique (téléchargements) à l\'accès à la musique (streaming par abonnement). Apple et l\'industrie du disque ne peuvent pas se permettre de se débarrasser du marché du téléchargement pour l\'instant. Le streaming et le téléchargement devront donc coexister sous la marque Apple, l\'un représentant l\'avenir alors que l\'autre devient progressivement plus lent et plus malade. La grande majorité des gens aiment la musique mais ne l\'aiment pas assez pour payer 120 $ par an pour l\'écouter ; un achat de téléchargement sporadique ici et là leur conviendra parfaitement. La dépense moyenne d\'un acheteur de musique au Royaume-Uni en 2014, par exemple, n\'était que de 39,52 £, selon une étude. Même Apple aura du mal à s\'attendre à ce que la plupart d\'entre eux triplent leurs dépenses annuelles en musique enregistrée. \nApple Connect a-t-il fait d\'Apple le service le plus convivial pour les artistes\xa0? \nApple a déjà essayé de créer une plate-forme conviviale pour les artistes via iTunes. Ils l\'ont appelé Ping et ça n\'a pas marché. Apple Connect est quelque chose de très différent, quelque part au milieu de YouTube, Facebook et SoundCloud, permettant aux artistes de publier de la musique, des vidéos, des photos et plus encore sur leurs pages de profil. Ça ne pourrait plus être 2015 si ça essayait. Apple, en partie à cause de sa taille et aussi en partie parce qu\'elle traite la musique comme de l\'«\xa0art\xa0» plutôt que du «\xa0contenu\xa0», a généralement eu une forte relation relations non seulement avec l\'industrie de la musique, mais aussi avec les artistes eux-mêmes et, en général, elle jouit d\'une bonne réputation auprès des artistes. Comparez cela à Spotify, qui a été critiqué par des artistes de Thom Yorke de Radiohead à Taylor Swift. En revanche, Apple apparaît comme un oncle bienveillant. Il y a une odeur de révolution dans l\'air et Apple veille à ce qu\'elle soit du bon côté des lignes de bataille lorsqu\'elles sont tracées. \nOù sont les exclusivités des artistes\xa0? \nCe sera la partie intéressante lorsque le service sera réellement mis en ligne. Obtenir des exclusivités pour les gros albums va devenir crucial pour le streaming. Spotify a payé beaucoup d\'argent pour obtenir exclusivement Led Zeppelin et Metallica. Apple aura observé cela attentivement et pris ses propres notes. Il a déjà AC/DC et les catalogues des Beatles exclusivement à télécharger sur iTunes, mais il reste à voir s\'il peut convaincre ces deux-là de faire le saut dans le streaming. Il a également réussi à obtenir l\'album surprise de Beyoncé en 2013 avant tout le monde, il est donc inévitable qu\'il en veuille plus comme ça. Il y a beaucoup de spéculations sur Apple cherchant à obtenir les droits exclusifs sur le nouvel album d\'Adele. \nDonner à iTunes l\'exclusivité de téléchargement sur un album n\'était pas un si grand acte de foi pour les actes étant donné la part du marché du téléchargement qu\'il contrôle, mais essayer de le faire en streaming n\'est pas la même chose. Ajoutez à cela le fait que le streaming compte désormais dans le palmarès des albums sur des marchés comme le Royaume-Uni et les États-Unis et que les artistes, qui considèrent toujours le palmarès comme une mesure de succès, ne voudront pas limiter leur audience en se limitant à un seul service. \nCela va-t-il tuer Spotify\xa0? \nCertaines personnes disent déjà qu\'Apple Music va détruire des rivaux comme Spotify. Cependant, ce n\'est pas si simple. Apple entre sur un marché où d\'autres travaillent et acquièrent de l\'expérience depuis de nombreuses années. Il a beaucoup de rattrapage à faire. Le vainqueur de cette bataille ne sera pas l\'entreprise avec le meilleur service ; ce sera l\'entreprise avec le plus d\'argent. Les concurrents d\'Apple ont une longueur d\'avance sur le marché, mais ils perdent d\'énormes sommes d\'argent. Spotify, par exemple, a perdu 93,1 millions d\'euros en 2013. Apple, en revanche, a commencé 2015 en devenant l\'entreprise la plus rentable de l\'histoire de l\'entreprise, avec 178 milliards de dollars en banque. Si Apple Music perd de l\'argent Apple, l\'entreprise ne le poursuivra pas longtemps mais elle n\'arrêtera pas d\'investir sans au moins essayer de battre la concurrence.\n,,,À 16 ans, mon innocence a été brisée lorsque deux hommes armés ont assassiné 13 personnes à mon l\'école et blessé d\'innombrables autres personnes.\nColumbine High School promettait d\'être un lieu d\'apprentissage sûr et sécurisé. Et cette promesse a été rompue le 20\xa0avril\xa01999.\nCe matin-là, je suis allé à l\'école, inquiet à propos de mon test de mathématiques de 10e\xa0année et de ma prochaine compétition d\'athlétisme. Des soucis inutiles\xa0: le test n\'a jamais été donné et nous n\'avons jamais organisé la compétition.\nIl reste des cicatrices de ce jour que personne ne peut voir. Des cicatrices qui faisaient pâlir mes inquiétudes concernant les tests de mathématiques ou le suivi des performances par rapport à la question de savoir si mon partenaire scientifique survivrait ou si le discours de mon camarade de classe serait altéré par les éclats d\'obus logés dans son crâne. Aujourd\'hui, ces cicatrices mentales palpitent dans de grandes foules et m\'obligent à scruter la pièce pour trouver des issues. Ils font battre mon cœur plus vite lorsque j\'entends les pales d\'un hélicoptère au-dessus de ma tête.\nMaintenant, en tant qu\'enseignante en huitième année de classe, je considère chaque jour où je vais au travail comme un privilège. Je chéris la joie et l\'enthousiasme de mes élèves, et surtout, leur innocence.\nJe crois qu\'il est de notre devoir, en tant que société, de protéger ces vertus chez nos jeunes. Je veux qu\'ils s\'inquiètent des tests de mathématiques et des compétitions d\'athlétisme, des expo-sciences et des élections du conseil étudiant - le genre de trucs scolaires normaux qui forgent le caractère. Mais notre épidémie de violence armée crée une culture de la peur dans nos écoles, où les élèves s\'inquiètent de la sécurité et des intrus. Ce sont des soucis qu\'aucun élève ne devrait avoir.\nCela devient encore plus évident lorsque nous menons notre exercice d\'urgence mensuel dans notre école. C\'est une façon de se préparer au pire, alors nous pratiquons les confinements, les exercices d\'incendie et les évacuations.\nL\'autre jour, j\'expliquais à mes élèves de troisième que nous allions pratiquer un confinement au cas où un ours se trouverait sur le terrain de jeu - un vrai scénario pour notre école du Colorado. J\'ai utilisé cet exemple pendant toute ma carrière d\'enseignante, car c\'est une raison facile et non menaçante de pratiquer le confinement.\nUne fille a levé la main et a demandé\xa0: Est-ce ce que nous ferions si un méchant venait avec une arme à feu pour nous faire du mal\xa0? ? J\'ai eu un coup de pied dans l\'estomac quand j\'ai entendu cette question. Un ours n\'est plus la pire chose à anticiper pour les élèves\xa0: ils craignent ce qui se passe dans notre pays.\nC\'est dommage que mes élèves doivent apprendre ces exercices à un si jeune âge - une honte qu\'ils doivent les apprendre du tout. Je me suis dit, c\'est le résultat de la fusillade de Columbine, ainsi que de tant d\'autres actes de violence. C\'est ma réalité, et maintenant c\'est aussi la leur. \nC\'est une triste réalité, mais nous ne sommes pas impuissants. j\'enseigne à un wonderful school, qui a fait tout ce qui était en son pouvoir pour assurer la sécurité de nos élèves. Mais ce n\'est pas assez. S\'il y avait une solution simple, alors cette épidémie de violence aurait pris fin. La solution est complexe, multiforme et réalisable. Je crois qu\'une approche globale peut aider à garantir un environnement d\'apprentissage sûr.\nâ $cents Nos enfants méritent plus de ressources en santé mentale pour identifier, traiter et suivre les personnes qui ont besoin d\'aide.\nâ $cents Nos enfants méritent une sécurité scolaire renforcée\xa0: Cela inclut identifier qui se trouve dans l\'école et prendre des mesures sensées pour assurer la sécurité du bâtiment - sans apporter d\'armes à feu dans le lieu d\'apprentissage ni en faire une forteresse. sur les clips de magazine haute capacité. Et à tout le moins, il devrait inclure des vérifications complètes et exécutoires des antécédents pour aider à garder les armes hors de portée des personnes dangereuses. Malheureusement, notre Congrès a refusé de prendre cette mesure de bon sens et a rejeté cette législation.\nLes tireurs de mon école ont obtenu leurs armes illégalement, par le biais de ventes privées et d\'achats de paille. Aujourd\'hui, ils pourraient facilement aller en ligne et acheter ces mêmes armes sans vérification des antécédents. Qu\'est-ce qui empêchera la prochaine personne qui choisira de faire exactement cela\xa0?\nNous pouvons faire mieux.\nJe pense que je peux dire pour nous tous que les meurtres à Columbine High School, au cinéma Aurora, à Sandy Hook Elementary et tant d\'autres d\'autres nous ont profondément choqués et ont créé des sentiments durables de tristesse. Mais nous ne pouvons pas laisser ces événements créer des sentiments de vulnérabilité. Nous avons le pouvoir de travailler ensemble pour créer un monde plus sûr pour nos élèves.\nInstruisez-vous de mon expérience\xa0: n\'attendez pas d\'être victime de violence pour réaliser que nous devons agir. Nous devons agir maintenant.\nLes opinions exprimées dans ce commentaire sont uniquement celles de Katie Lyles.\n,,,Sept policiers ont été tués et neuf blessés mardi lors d\'une série d\'attaques à des points de contrôle de sécurité dans la province irakienne de Anbar, a indiqué la police.\nLes attentats ont eu lieu dans la ville d\'al-Rutba, une communauté musulmane majoritairement sunnite qui a été pendant quelques années un bastion d\'Al-Qaïda après l\'invasion du pays par les États-Unis en 2003.\nDans le premier , un attentat-suicide à la voiture piégée à un poste de contrôle de la police dans le nord d\'al-Rutba a tué quatre policiers et en a blessé trois autres, ont déclaré des responsables de la police de Ramadi, la capitale provinciale.\nDes hommes armés ont ensuite attaqué plusieurs postes de contrôle de sécurité dans la ville, tuant trois policiers et blessant six personnes, a indiqué la police.\nLes informations sur les victimes des assaillants, au-delà du kamikaze, n\'étaient pas disponibles. Al-Rutba est à environ 435\xa0kilomètres -LRB- 270\xa0miles -RRB- à l\'ouest de la capitale du pays, Bagdad. une personne et 11 blessés, a indiqué la police de Bagdad.\nMusayyib, une ville majoritairement chiite, se trouve à environ 70\xa0kilomètres -LRB- 43\xa0miles -RRB- au sud de Bagdad.\nL\'Irak a connu une forte augmentation des tensions entre ses populations chiites et sunnites depuis avril, lorsque les forces de sécurité ont effectué une descente sur un site utilisé par des manifestants sunnites pour manifester contre le gouvernement dirigé par les chiites.\nLes sunnites, qui représentent une minorité d\'Irakiens, se sentent politiquement marginalisés depuis le renversement de Saddam Hussein en 2003. Les chiites constituent une majorité d\'Irakiens.\nJusqu\'à présent en 2013, plus de 6\xa0000 personnes ont été tuées en Irak. Plus de 350 de ces décès sont survenus ce mois-ci, et près de 1\xa0000 sont survenus en septembre.\nLe journaliste Mohammed\xa0Adnan de Falluja a contribué à ce rapport.\n,,,\ufeffMaria est assise sur une chaise en plastique noir dans un centre communautaire par un mardi froid après-midi en attendant que son numéro soit appelé. Elle porte le numéro 34. \nQuand c\'est son tour, Maria est appelée pour ramasser un sac en papier brun rempli de produits de première nécessité, notamment des pâtes, des œufs et des cornflakes, et est invitée à choisir entre la courge musquée ou les carottes comme légumes de la semaine. \nMaria est la 34e "cliente" à ce jour à East Hampton Food Pantry, une initiative communautaire mise en place à quelques rues de certaines des propriétés les plus chères et les plus exclusives au monde. À la fin de la journée, les organisateurs du garde-manger s\'attendent à ce que plus de 400 familles aient suivi Maria à travers les portes pour récupérer leur colis alimentaire hebdomadaire afin de les aider à traverser l\'hiver froid et sombre de Long Island. \nEn été, les Hamptons, un ensemble de villes et de villages historiques en bord de mer à 160\xa0kilomètres de Manhattan, sont le terrain de jeu des milliardaires. Mais, à la fête du Travail début septembre, lorsque les riches et célèbres ferment leurs manoirs et retournent à Manhattan ou à Beverly Hills, le faste cède la place à la dure réalité de la vie pour la communauté majoritairement immigrée qui vit ici toute l\'année. "Les gens qui viennent ici sont riches et célèbres, mais nous qui vivons ici ne le sommes pas", explique Maria, qui travaille 14 heures par jour dans les manoirs d\'été mais passe des mois sans travailler à tout en hiver. \nMaria rit lorsqu\'on lui demande si elle a assez d\'argent. « Il n\'y a pas de travail en hiver, seulement en été », dit Maria, qui, comme beaucoup de travailleurs des Hamptons, est originaire d\'Amérique latine. "Ici, beaucoup de gens vivent dans une chambre individuelle parce qu\'ils ne peuvent pas payer le loyer." \nElle dit que certaines familles avec jusqu\'à cinq enfants sont entassées dans des sous-sols et paient encore plus de 1 000 $ par mois en loyer. "Les gens viennent ici pour chercher du travail mais, l\'hiver, il n\'y a rien." \nBeaucoup de ses amis ne peuvent pas payer le chauffage ou les médicaments et beaucoup auraient faim s\'il n\'y avait pas le East Hampton Food Pantry, dit-elle, qui n\'est qu\'un des nombreux garde-manger de la ville. \nVicki Littman, présidente du garde-manger d\'East Hampton, qui a fourni plus de 31\xa0000 colis alimentaires en 2015, affirme que le nombre de personnes recherchant le garde-manger ne cesse d\'augmenter. "Une fois que la fête du Travail arrive et que la saison est terminée et que les heures des gens commencent à être réduites, notre nombre passe à environ 400 familles par semaine", dit-elle. « Quand ils viennent chez nous le mardi, ils reçoivent deux à trois jours de nourriture. Sans nous, ils lutteraient encore plus. Littman dit qu\'il peut être difficile pour les étrangers de se rendre compte qu\'il y a des gens qui ont du mal à s\'en sortir dans un endroit connu dans le monde entier pour ses excès. \n"Quand je discute avec la communauté estivale qui vient ici des garde-manger, ils sont toujours choqués parce qu\'il y a ce côté glamour des Hamptons où il y a des galas et les plages et les manoirs qui sont ici. «Mais ce que les gens ne réalisent pas, c\'est qu\'il existe une industrie de services. Ce sont les paysagistes, les nounous, les serveuses - ils comptent tous sur ce revenu d\'été pour passer l\'hiver, mais les gens ne le voient pas lorsqu\'ils partent en vacances. \n"Il y a des personnes âgées qui doivent parfois choisir entre payer leurs médicaments, payer leurs factures ou acheter de la nourriture, et cela ne devrait pas être le cas." \nLittman dit que la ville a perdu trop de personnes occupant des emplois clés – comme des enseignants, des policiers et même des médecins et des dentistes – parce qu\'ils ne peuvent pas se permettre de vivre dans la communauté et que le conseil du garde-manger est déterminé à faire plus pour s\'assurer que les gens avoir une meilleure chance de rester sur place. Le logement est, de loin, le plus gros coût dans les Hamptons. À 147 millions de dollars, la propriété la plus chère du pays est le domaine en bord de mer de 18 acres du gestionnaire de fonds spéculatifs Barry Rosenstein au 67 Further Lane, à deux pas du Maidstone Golf Club, qui est considéré comme "le plus élitiste, le plus prestigieux et le plus difficile d\'accès" de tous. les clubs des Hamptons. \nLarry Cantwell, superviseur de la ville d\'East Hampton et résident de longue date, affirme que les maisons changent régulièrement de mains à plus de 25 millions de dollars et que l\'inflation rapide des prix à l\'extrémité supérieure s\'est répercutée même sur les appartements les plus modestes de la ville. \n"Trouver sa première maison est un défi dans une région comme celle-ci", déclare Cantwell. «Pas seulement les gens que vous qualifieriez de pauvres – les familles de la classe ouvrière et de la classe moyenne traversent également une période difficile. Si vous pouvez trouver une maison à acheter n\'importe où à East Hampton pour 500 000 $, vous avez beaucoup de chance. Cantwell dit que plus de la moitié des maisons de la ville sont vides pendant la majeure partie de l\'année, ce qui fait chuter la population à 10 000 habitants pendant les mois d\'hiver, contre 80 000 en août. « C\'est un peu l\'histoire de deux villes. Il y a certainement beaucoup de richesse ici, mais la quasi-totalité de cette richesse se trouve dans des résidences secondaires utilisées uniquement en été », explique Cantwell, fils d\'un père pêcheur et d\'une mère femme de ménage. «Mais, le reste d\'entre nous vivons ici toute l\'année. \n"Il y a des gens célèbres et très riches, mais il y a aussi des gens qui travaillent dur et qui ont du mal à s\'en sortir. Vous devez vous rappeler que cette communauté a été fondée comme une communauté d\'agriculteurs et de pêcheurs de personnes qui vivaient de la terre et de l\'eau - une véritable communauté ouvrière. \nCantwell dit qu\'économiser suffisamment d\'argent pour acheter votre première maison tout en travaillant comme agriculteur ou pêcheur est presque impossible à East Hampton aujourd\'hui "et il n\'y a pas que les pauvres - les policiers, les enseignants, les jeunes professionnels et d\'autres ont tous du mal à trouver une place vivre ici et beaucoup d\'entre eux n\'ont pas les moyens de posséder leur propre maison. \nÊtre sans abri dans les Hamptons signifie passer beaucoup de temps dans un bus. Divers lieux de culte se sont réunis pour s\'assurer qu\'il y a un endroit où les sans-abri peuvent passer la nuit pendant l\'hiver. Les églises de haut en bas de la fourche nord et sud de Long Island assument le fardeau une nuit à la fois et environ 50 sans-abri sont transportés entre eux chaque jour. Eddie Vallone, 22 ans, fait partie de ceux qui sont dans le bus tous les soirs. "Les gens considèrent les Hamptons comme une sorte de ville riche et il n\'y a aucun problème. Mais il y a beaucoup de problèmes ici, surtout la drogue. \n"C\'est difficile à comprendre -\'OK, l\'été touche à sa fin. Qu\'est-ce que je vais faire pour l\'hiver\xa0?\xa0»\xa0» Dit Vallone à Maureen\'s Haven, une organisation caritative qui coordonne le programme de refuges pour sans-abri. "Je veux travailler mais il n\'y a pas de travail à faire." \nVallone, who travaille à nettoyer des piscines et à faire des petits boulots dans des propriétés de luxe, dit que s\'il épargne bien et n\'achète pas d\'impulsion, il peut faire durer ses revenus d\'été jusqu\'en novembre. "Mais, les travaux ne reprennent qu\'en mai ou début juin."\n,,,\ufeffPourquoi faire\xa0? S\'adressant aux hommes désignés pour être les arbitres d\'élite du futur, la question suscite un sourire complice. Cette saison, les critiques des arbitres se sont multipliées au point que certains anciens arbitres se sont sentis obligés de se plaindre des normes. C\'est assez frappant car, plus vous parlez aux arbitres, plus il est évident que se soutenir mutuellement contre vents et marées est fondamental. \nAlors pourquoi le faire\xa0? Pourquoi passer d\'innombrables heures à conduire à travers le pays pour imposer des règles, dont certaines vont inévitablement contrarier les gens, essayant de gravir les échelons jusqu\'à ce que vous ayez la chance de prendre des décisions à la télévision devant des millions de personnes qui vous scrutent vous et votre capacité à l\'aide d\'innombrables angles différents et de rediffusions au ralenti\xa0? \nUn aperçu d\'une réponse apparaît sur le visage de Lee Swabey quelques instants après qu\'il ait donné le coup de sifflet final d\'une victoire 2-1 pour Grimsby sur Woking, un match au niveau 5 du système de la ligue anglaise. Il obtient ce que tous les arbitres espèrent chaque fois qu\'ils arbitrent un match. "Vingt-deux poignées de main", explique-t-il ensuite, fièrement. Symboliquement, une série complète de poignées de main, plus un « bravo » des deux managers, représente une satisfaction maximale. "Le buzz", comme il l\'appelle, d\'un jeu qui se passe bien, est quelque chose qu\'il aime. "Je ne passerais pas autant de temps loin de ma famille si cela ne signifiait pas le monde pour moi." \nEn tant que membre du groupe très apprécié par les Professional Game Match Officials Limited (PGMOL) pour son potentiel, Swabey savait qu\'il était surveillé lors de ce match. Le chef de PGMOL, Mike Riley, était présent, ainsi que Steve Dunn, qui est l\'entraîneur de ce niveau, armé de notes et d\'un chronomètre pour répertorier chaque mouvement important effectué par les officiels. \nQuelques semaines plus tôt, Riley, Dunn et un autre ancien arbitre, Peter Jones, se sont dirigés vers un autre match de niveau 5 pour surveiller un autre arbitre appelé à progresser - John Brooks. "J\'espère avoir l\'opportunité d\'être promu en Premier League et d\'arbitrer certains des meilleurs matchs de ce pays", a déclaré Brooks. \nCependant, le plus que la délégation PGMOL a pu observer, c\'est la façon dont Brooks a géré la situation quelque peu délicate d\'annuler le match en raison d\'un terrain gelé. Tout cela fait partie de l\'expérience que Brooks doit acquérir avant de se voir confier des matchs plus médiatisés, les différents problèmes à résoudre – souvent, les clubs sont très réticents à avoir un report tardif, en particulier lorsqu\'ils doivent payer tous les le personnel qui s\'est présenté mais qui ne recevra pas d\'argent d\'entrée. \nBrooks a téléphoné à son entraîneur pour obtenir des conseils et a pris la décision difficile mais correcte. Un peu plus tard, le secrétaire du club de football est arrivé avec des enveloppes pour payer les officiels pour leur temps – les frais de match au niveau 5 sont de 95 £, il est donc prudent de supposer que ces hommes ne le font pas pour l\'argent. \nBrooks, comme Swabey, a des ambitions claires de progresser. Il ne se fait aucune illusion sur le fait que développer une peau épaisse et perfectionner des stratégies pour faire face aux déceptions en est une grande partie. Comment se sent-il en regardant le football à la télévision lorsqu\'un arbitre est vilipendé ? "Euh… pas génial", admet-il. "J\'ai été dans cette situation une fois où j\'ai pris une mauvaise décision et c\'était un but décisif qui était juste hors-jeu. Votre cœur coule. Vous ne pouvez pas arrêter d\'y penser. \n"Je souhaite parfois que les gens comprennent le temps et les efforts que nous y consacrons. Il est très facile de critiquer une décision, mais nous faisons tout pour essayer de prendre ces décisions correctement. Dans certaines situations, vous allez être impopulaire, mais si vous n\'êtes pas à l\'aise avec cela, vous vous trompez probablement de travail. \nLes anciens arbitres conviennent que la sauvegarde, la formation et les outils dont disposent les arbitres d\'aujourd\'hui sont à mille lieues de ce qu\'ils ont connu à leur époque. Riley, en tant que jeune arbitre, est sorti et s\'est acheté des livres sur la psychologie et la nutrition car il n\'y avait aucune information à lui offrir. \nComparez cela avec Brooks, qui a un coach au bout du fil. Ils se consultent chaque semaine, discutent du déroulement de ses matchs, étudient des séquences de décisions clés et déterminent comment s\'améliorer. Il bénéficie également du soutien d\'un psychologue du sport, Liam Slack, pour des conseils réguliers et un régime d\'exercices pour l\'aider à gérer les 11 km qu\'il parcourt pendant un match. \nBrooks dit que la psychologie est essentielle à son développement. "L\'une des choses dont nous avons parlé est d\'oublier les décisions et de passer à autre chose", explique-t-il. «Il peut y avoir une grande décision à prendre dans les 30 premières secondes du match. Une fois que vous avez fait cela, vous devez rester concentré pendant les 89 prochaines minutes et ne pas vous demander si c\'était correct ou vous inquiéter de cette décision. Liam nous a enseigné quelques techniques pour libérer cette décision. Travailler avec le psychologue du sport est vraiment important pour la force mentale. \nJones croit que toute l\'application le cafard ne peut qu\'aider. "J\'ai arbitré dans le football professionnel mais, avec le recul, j\'étais un amateur", dit-il. «J\'allais travailler – je travaillais pour British Telecom – et je pouvais arbitrer à Newcastle un mercredi soir et, à 9 heures du matin le lendemain matin, j\'étais à Leicester pour essayer de parler aux clients. Je n\'avais peut-être pas dormi. La formation était ponctuelle. Nous étions des amateurs dans un milieu professionnel par rapport à maintenant. \nLorsque le sujet de la technologie est abordé, les trois anciens arbitres sont unanimes à le soutenir. "Nous sommes tous en faveur de tout ce qui améliore le travail des arbitres et les rend plus efficaces sur le terrain de jeu", déclare Riley. Minimiser les erreurs est le but. Après tout, une mauvaise décision peut rester avec vous pendant un certain temps. "Le reste de votre vie", note Jones avec un petit rire.\n,,,Le champion en titre Rafael Nadal a accédé au troisième tour de l\'événement sur terre battue des Masters de Rome avec une victoire 6-1 6-3 sur Philipp Kohlshreiber.\\ nLe numéro trois mondial, en quête de son cinquième titre en six ans dans la capitale italienne, n\'a commis que 11 fautes directes, contre 28 pour l\'Allemand.\nNadal, 23 ans, a cassé le service de son adversaire à deux reprises dans un premier set à sens unique .\nLe numéro 29 mondial allemand s\'est amélioré dans le deuxième set et a raté une occasion de prendre une avance de 3-2 en gâchant sa seule balle de break du match.\nNadal, qui n\'a perdu qu\'une seule fois au Masters de Rome, a fait irruption le match suivant et pressé pour remporter le match.\nAilleurs, la cinquième tête de série, Robin Soderling, de Suède, a battu le joker italien Paolo Lorenzi 6-1 7-5 pour atteindre le troisième tour.\nLe Suédois a battu 32 vainqueurs alors qu\'il remportait la victoire en moins d\'une heure et quart.\nGuillermo Garcia-Lopez d\'Espagne a également progressé avec une victoire 6-2 6-3 sur l\'ancien Wimbledon et U.S. Open ch ampion Lleyton Hewitt.\nLa deuxième tête de série Novak Djokovic et la quatrième tête de série Andy Murray ont tous deux atteint le troisième tour mardi tandis que le numéro un mondial Roger Federer s\'est écrasé contre le Letton Ernests Gulbis.\n,,,Hier, Stephen est revenu d\'un voyage à Washington, D.C. , la capitale des États-Unis. Sa visite a eu lieu au cours de la semaine précédant le 4 juillet. En toute logique, de nombreuses animations et festivités se sont déroulées en ville en préparation de la fête de l\'Indépendance. Pendant son séjour dans la ville, Stephen a visité de nombreux sites et monuments historiques importants, et il est reparti avec une compréhension plus approfondie de l\'histoire politique des États-Unis.\nStephen a passé beaucoup de temps à l\'extérieur à explorer les monuments importants entourant Capitol Hill. Bien sûr, il a vu la Maison Blanche depuis sa porte extérieure au 1600 Pennsylvania Avenue. Stephen a également visité le Washington Monument, le Jefferson Memorial et le Lincoln Memorial. Ces statues et pavillons sont dédiés aux anciens présidents américains. Ils commémorent les contributions que ces dirigeants ont apportées tout au long de l\'histoire américaine. Washington, D.C. possède également plusieurs monuments commémoratifs de guerre dédiés aux soldats tombés au combat pendant les grandes guerres du 20e\xa0siècle.\nLoin de la région de Capitol Hill, Washington, D.C. compte de nombreux musées et galeries d\'art qui valent la peine d\'être visités. Stephen a apprécié sa visite à Washington, DC parce qu\'il en a appris beaucoup plus sur l\'histoire américaine après avoir visité chaque point de repère important. ,,, L\'un des monuments les plus beaux et les plus reconnaissables de Californie est le Golden Gate Bridge dans la ville de San Francisco. Le pont enjambe la baie de San Francisco et fait partie d\'un réseau de cinq ponts principaux qui traversent la baie dans la région de San Francisco et d\'Oakland. La région avec sa baie offre l\'une des vues les plus époustouflantes au monde, et dont une partie est l\'historique Golden Gate Bridge.\n \nC\'est peut-être le monument le plus emblématique et le plus reconnaissable de tous les États-Unis. La construction du pont a commencé en 1933, alors que le pays était profondément embourbé dans la Grande Dépression. Beaucoup de gens pensent que le pont a été, en grande partie, responsable de la reprise économique du pays en raison du nombre de travailleurs employés. Le matériau utilisé pour construire le pont a également contribué à la fabrication, à la distribution et au transport.\nLe pont relie San Francisco à la péninsule de San Francisco dans le comté de Marin via la California State Route 1 et la U.S. Route 101. Avant la construction du pont, la seule façon de se rendre d\'un endroit à l\'autre était de faire le tour de la baie ou de prendre un ferry pour traverser. La construction du pont était controversée à l\'époque. Beaucoup pensaient que le projet était tout simplement trop complexe, coûteux et dangereux à construire. Le plan de construction approuvé se concentrait sur la sécurité des ouvriers qui seraient impliqués dans la construction, et il fonctionna bien jusqu\'au 17 février 1937. Fait remarquable, un seul ouvrier était décédé jusqu\'à cette date, mais à cette date, un échafaudage s\'est effondré tuant 10 personnes. hommes dans le tragique accident.\nLe pont a été achevé plus tard cette année-là et est utilisé depuis. C\'est l\'un des grands monuments de la Californie avec des gens du monde entier attirés pour le voir. ,,, L\'un des plus grands obstacles qui existait au début du transport aérien était la vitesse du son. C\'était contre Il était impossible de construire et de piloter un avion capable de dépasser cette vitesse, mais c\'était avant que le monde ne soit présenté à Chuck Yeager. Yeager était déjà un pilote de chasse accompli pendant la Seconde Guerre mondiale, mais sa plus grande réussite a été après la guerre lorsqu\'il a servi comme pilote d\'essai pour l\'US Air Force. On pensait qu\'il était impossible de voler plus vite que le son pendant cette période, et personne ne savait si un pilote pouvait survivre en volant à cette vitesse, ni comment cela l\'affecterait s\'il survivait. Cela n\'a cependant pas dissuadé Yeager.\n \nYeager est né le 13\xa0février\xa01923 à Myra, en Virginie-Occidentale. Il a servi pendant la Seconde Guerre mondiale en tant que pilote de chasse. Il n\'a cependant pas commencé comme pilote. Yeager s\'est enrôlé en tant que soldat qui a travaillé comme mécanicien d\'aéronefs dans l\'ancienne armée de l\'air, où Yeager s\'est inscrit à un programme spécial qui formait des hommes enrôlés à devenir pilotes. Yeager a grandement bénéficié d\'une vue supérieure. Il était considéré comme un pilote naturel au début de sa carrière de pilote.\nEn tant que pilote de chasse sur le théâtre européen, Yeager a été abattu en France par des avions de chasse allemands. Il a évité la capture et a aidé un autre pilote à se mettre en sécurité après que ce pilote eut perdu sa jambe dans l\'accident. Yeager a reçu une médaille pour ses actions. Il a fait preuve d\'un talent exceptionnel en tant que pilote et a déjà abattu cinq chasseurs ennemis en une seule journée. Après la guerre, Yeager a commencé une nouvelle carrière en tant que pilote d\'essai, où il a piloté l\'avion expérimental Bell X-1 dans l\'histoire de l\'aviation.\nLe 14 octobre 1947, Yeager a fait voler son X-1 plus vite que la vitesse du son. Il est devenu le premier être humain à voler plus vite que le son, et il a battu plus de records au cours de ses 60 ans d\'histoire de vol. Yeager a pris sa retraite de l\'armée avec le grade de général de brigade et a remporté de nombreux honneurs pour ses contributions à l\'aviation.,,,La Révolution américaine était une guerre d\'indépendance menée par ceux qui vivaient dans les 13 colonies d\'Amérique du Nord contre la Grande-Bretagne. Les habitants des colonies étaient particulièrement en colère contre les impôts que les Britanniques les obligeaient à payer. Les gens estimaient que les impôts étaient injustes parce qu\'il n\'y avait pas de représentation coloniale au parlement britannique. Une taxe était la taxe de timbre, qui taxait les imprimés tels que les journaux. Les colons appelaient cela "l\'imposition sans représentation". Les colons qui voulaient se libérer de la domination anglaise étaient connus sous le nom de Patriotes. Ceux qui soutenaient la domination britannique étaient connus sous le nom de loyalistes. Avant que la guerre à grande échelle n\'éclate, il y a eu de nombreux actes de protestation de la part des patriotes, comme le Boston Tea Party lorsque des Bostoniens sont montés à bord de navires britanniques et ont jeté 342 caisses de thé dans le port à cause d\'une taxe sur le thé. Le 19\xa0avril, les premiers coups de feu sont tirés à Lexington et Concord, dans le Massachusetts, ce qui déclenche officiellement la guerre.\n \nDe nombreuses personnes sont impliquées dans la lutte pour l\'indépendance. Benjamin Franklin, Thomas Jefferson et John Adams ont travaillé ensemble pour rédiger la déclaration d\'indépendance. Le 4 juillet 1776, les patriotes se déclarent libres. La plupart des gens savent que George Washington a été le premier président des États-Unis. Avant cela, il était le commandant en chef de l\'armée patriote. La plupart des batailles de la Révolution américaine se sont déroulées dans le nord-est, près de Boston et de New York.\nLes Britanniques, connus sous le nom de redcoats en raison de leurs vestes d\'uniformes rouges, ont reçu l\'aide de soldats mercenaires allemands appelés Hessians. Les patriotes avaient le soutien d\'autres Européens. La France est entrée dans la Révolution américaine aux côtés des colons en 1778. Cela a contribué à forcer la reddition britannique à Yorktown, en Virginie, en 1779. Cependant, les combats ne se sont officiellement terminés qu\'en 1783 avec le Traité de Paris, lorsque la Grande-Bretagne a reconnu les États-Unis comme une nation indépendante.,,,Vous pouviez le voir dans ses yeux.\nMême avant que le pape François ne prononce ses premiers mots devant la foule sur la place Saint-Pierre, vous pouviez le regarder dans les yeux et sentir l\'émerveillement.\nLe monde le regardait. Mais il regardait aussi quelque chose : tous ces visages renversés. À tous les yeux qui le fixent.\nRien ne peut préparer une personne à un tel spectacle.\nDans cette vie, il y a des moments, et puis il y a des moments. La première apparition d\'un nouveau pape au balcon de la Basilique Saint-Pierre a toujours été un Moment pas comme les autres.\nAvant la télégraphie, avant la radio, avant la télévision, avant Internet, avant il y avait Twitter, la vision - l\'orchestration solennelle - d\'un pape nouvellement élu montant sur le balcon était le moyen le plus dramatique possible d\'apporter d\'énormes nouvelles au monde.\nC\'est toujours le cas.\n Je voudrais vous remercier pour votre étreinte, dit le nouveau pape en regardant tous ces yeux.\nLorsque les auteurs et les orateurs ont cherché des moyens d\'exprimer l\'exemple ultime d\'exaltation révérencielle, la scène répétée périodiquement sur la place Saint-Pierre, à travers le siècles, a souvent été celui vers lequel ils se sont tournés.\nTom Wolfe, dans son livre The Right Stuff, sur les premiers astronautes de Mercury, a voulu décrire aux lecteurs à quel niveau dévotion que les astronautes nouvellement nommés avaient reçue du peuple américain. Les sept pilotes n\'avaient été que cela - des pilotes de chasse, des pilotes d\'essai - avant d\'être sélectionnés pour le programme spatial. Soudain, ils étaient autre chose. Les gens les voyaient et se mettaient à pleurer.\nWolfe a trouvé l\'analogie idéale pour décrire le phénomène\xa0:\nLes garçons n\'auraient pas été gênés par ce qui suit. Cela ne les aurait pas dérangés d\'apparaître une fois par an sur un balcon au-dessus d\'une immense place où la moitié du monde est assemblée. Ils ondulent. Le monde rugit son approbation, ses applaudissements, et éclate en une tempête de trente minutes d\'acclamations et de larmes... Une petite adulation sur l\'ordre du Pape ; c\'est tout ce que les Vrais Frères au sommet de la pyramide voulaient vraiment. \nMais l\'humilité n\'est pas requise des pilotes de chasse et des astronautes. Cela n\'est pas non plus exigé des rock stars ou des athlètes professionnels ou de l\'un des nombreux autres artistes publics, dans des lieux disparates, qui sont régulièrement présentés avec une majesté soigneusement planifiée et commercialisée conçue pour les élever aux yeux de leur public.\nUn pape est différent, et son introduction au monde ne ressemble à aucune autre. Lorsque le mercredi matin s\'est levé à Rome, un homme nommé Jorge Mario Bergoglio était l\'un des millions de personnes qui se préparaient à commencer la nouvelle journée.\nÀ la fin de la journée, il n\'était plus Jorge Mario Bergoglio, et il n\'était plus l\'un des millions.\nEn tant que pape François, il a marché sur ce balcon et a été accueilli par cette vue.\nDe derrière ses lunettes, il a tout compris.\nLes yeux du monde ont rencontré les siens.\nIl a demandé des prières.\nLes opinions exprimées dans ce commentaire sont uniquement ceux de Bob Greene.\n,,,\ufeffDes milliers de personnes ont manifesté sur les plages d\'Australie contre l\'abattage des requins en Australie-Occidentale. Ils ont demandé au Premier ministre de l\'État d\'arrêter l\'abattage, et le propriétaire de la RSPCA Australie et de Virgin Atlantic, Richard Branson, l\'a critiqué. La capture et la mise à mort de requins de plus de trois mètres ont commencé après un certain nombre d\'attaques de requins sur la côte ouest de l\'Australie. Un surfeur de 35\xa0ans, tué en novembre\xa02013, était la sixième personne à mourir d\'une attaque de requin en deux ans.\nMais l\'ensemble de l\'Australie n\'a enregistré, en moyenne, qu\'un seul décès lié à un requin par an depuis les 50 dernières années. Kate Faehrmann, de la Sea Shepherd Conservation Society, a déclaré lors d\'une manifestation à Perth, la capitale de l\'État d\'Australie-Occidentale : « Nous avons toujours dit que cette idée ne fonctionnerait pas. Les drumlines, qu\'ils utilisent pour attraper les requins, tuent les requins s\'ils mesurent un, deux, trois mètres ou plus, ainsi que des dauphins, des tortues et d\'autres choses. C\'est pourquoi nous ne voulons pas d\'abattage."\nDes milliers de personnes ont manifesté sur les plages des villes de Perth, Sydney et Adélaïde, ainsi que sur les plages de Victoria et du Queensland. Faehrmann a déclaré que les manifestations montraient que les Australiens voulaient que les requins soient protégés : « Ce qui est incroyable, c\'est que tant de gens en Australie aiment les requins. Cela a montré que les gens ont peur, mais des milliers de personnes sortent à travers le pays pour dire: "C\'est l\'océan des requins". Nous les respectons, nous les aimons et nous ne voulons pas que vous les tuiez." Anthony Joyce, un surfeur qui s\'est déjà pris le pied dans la bouche d\'un requin, a déclaré : "Le nombre de requins qu\'ils vont tuer va ne fait aucune différence."\nLe gouvernement de l\'État ne dira pas combien de requins ils ont tués, mais certaines personnes disent que des requins de moins de trois mètres ont été relâchés après avoir été capturés sur des drumlines, des tambours flottants fixés au fond de la mer avec des appâts sur des hameçons en dessous leur. Les écologistes disent qu\'il n\'y a aucune preuve que l\'abattage réduira le nombre d\'attaques de requins contre les humains, car aucun autre abattage n\'a utilisé que des drumlines. Des chercheurs de l\'Université d\'Australie-Occidentale affirment que l\'augmentation du nombre d\'attaques de requins dans l\'État peut être due au fait que l\'Australie-Occidentale a la population à la croissance la plus rapide d\'Australie, et non à cause d\'un nombre croissant de requins.\nRichard Peirce, de la conservation basée au Royaume-Uni organisme de bienfaisance, le Shark Trust, a déclaré que l\'abattage ne fonctionnerait pas et pourrait amener plus de prédateurs vers la côte. "L\'activité en Australie-Occidentale s\'ajoute à la tragédie humaine des attaques de requins. Il est très triste qu\'un gouvernement ait ignoré les meilleurs conseils et choisi une méthode inefficace et contre-productive », a-t-il déclaré,« et, même s\'ils surveillent les lignes de batterie tout au long de la journée, laisser les lignes en place la nuit pourrait attirer d\'autres prédateurs vers le zone, attirés par ces requins et d\'autres espèces accrochés et blessés.\xa0»\nDans le monde, en 2012, il y a eu 80 attaques de requins. Sept des attaques ont été mortelles. Cela se compare à près de 100 millions de requins tués par l\'homme chaque année. La RSPCA Australie dit qu\'elle pense que l\'abattage est une erreur. "Il n\'y a aucune preuve que l\'augmentation des attaques soit le résultat de l\'augmentation du nombre de requins. Nous pensons que c\'est le résultat d\'une population changeante et d\'un changement de comportement humain ; c\'est-à-dire qu\'il y a plus de gens dans l\'eau », a-t-il déclaré.\nRichard Branson a déclaré que l\'idée ne fonctionnait pas. "Je suis sûr que l\'une des raisons pour lesquelles le premier ministre de l\'Australie-Occidentale, Colin Barnett, l\'a fait, c\'est parce qu\'il pensait g cela encouragerait le tourisme. Ça va faire le contraire, je pense. Vous faites la publicité d\'un problème qui n\'existe pas de manière importante et vous allez empêcher les gens de vouloir venir à Perth et dans votre belle campagne qui l\'entoure. Tout ce que vous allez accomplir, je pense, c\'est d\'inquiéter les gens."\n,,,L\'une des femmes les plus riches des États-Unis est Meg Whitman, PDG de Hewlett-Packard. Whitman est une personne instruite de l\'Ivy League qui est la quatrième femme la plus riche de l\'État de Californie et figure au 20e rang sur la liste Forbes des femmes les plus riches des États-Unis. Elle est diplômée de l\'Université de Princeton, où elle a obtenu son baccalauréat en économie en 1977. Après Princeton, Whitman a obtenu son MBA à la Harvard Business School en 1979. Elle voulait à l\'origine étudier la médecine, mais a changé de spécialisation tôt à Princeton. \n \nWhitman a trouvé le succès partout où elle a travaillé. Elle a commencé sa carrière chez Proctor and Gamble avant de rejoindre la Walt Disney Corporation où elle a occupé le poste de vice-présidente de la planification stratégique en 1989. Elle a également travaillé pour Stride Rite et Hasbro Corporations. Son plus grand succès a été au cours d\'une période de 10 ans à travailler pour eBay.\nLorsque Whitman a repris eBay en 1998, les enchères en ligne et le commerce de détail réalisaient environ 4 millions de dollars de ventes par an. Lorsqu\'elle a quitté l\'entreprise en 2008, l\'entreprise était devenue un géant de 8 milliards de dollars. EBay est passé de 30 employés en 1998 à plus de 15 000 pendant son mandat de PDG de l\'entreprise. Whitman a remporté plusieurs prix pour son travail sur eBay, et le magazine Forbes l\'a classée parmi les cinq femmes les plus puissantes du pays.\nWhitman est une républicaine modérée, qui a soutenu l\'échec de la campagne présidentielle de Mitt Romney en 2008. Elle s\'est présentée au poste de gouverneur de Californie en 2010, mais a perdu contre Jerry Brown. Elle espère se présenter à la présidence à l\'avenir, et est répertoriée comme l\'une des femmes les plus susceptibles de devenir présidente des États-Unis.,,,Kit Skogsbergh peut le voir maintenant.\n Il serait un acolyte, m\'a-t-elle dit. Il aurait une voix douce et apaisante ... Il ressemblerait un peu à Porcinet dans les films Pooh parce qu\'il serait un peu prudent et effrayé par les choses, mais une fois qu\'il connaîtrait le plan et comment les choses fonctionnaient, il se réchaufferait pour et peut-être qu\'à la fin du film, il n\'aura peut-être pas à se rouler en boule -LRB- et à se cacher -RRB-. Il serait capable d\'affronter les choses de front. \nL\'homme de 32\xa0ans de Pasadena, en Californie, a récemment déposé une pétition sur Change.org demandant à Disney de créer un personnage animé à partir d\'un pangolin, un mammifère couvert d\'écailles peu connu qui est notoirement timide et, pour Le point de Kit, peut se protéger de presque tout dans la nature -- lions, tigres -- en se roulant en boule.\nLe pangolin fait l\'objet d\'un trafic vers l\'extinction -- vendu pour ses écailles, qui sont utilisées dans la tradition médecine, et pour sa viande, qui est un mets délicat au Vietnam et en Chine. Les pangolins sont échangés par tonnes.\nDisney pourrait donner au pangolin la célébrité dont il a besoin pour survivre.\nJe vous encourage à signer la pétition de Kit sur Change.org.\nSi les gens se soucient du pangolin comme ils se soucient sur les tigres ou les éléphants, il pourrait facilement être sauvé de l\'extinction. Le problème : peu de gens semblent réaliser que ces animaux incroyablement introvertis existent. Ils sont peu étudiés et à peine compris. Ils ne sont pas non plus conventionnellement mignons, ce qui permet aux organisations de conservation et au public de les considérer plus facilement comme consommables. Pourtant, ce sont des créatures essentielles et précieuses.\nJe suis récemment allé au Vietnam et en Indonésie dans le cadre du projet Change the List de CNN pour documenter le commerce illégal massif d\'espèces de pangolins en voie de disparition. Je n\'avais jamais entendu parler de pangolins avant que les lecteurs ne votent pour que je couvre le commerce illégal d\'animaux dans le cadre de ce projet\xa0; et Kit n\'en avait pas entendu parler jusqu\'à ce qu\'elle lise l\'histoire la semaine dernière.\n J\'étais à mi-chemin de -LRB- l\'histoire -RRB- et mon estomac me faisait si mal que je ne pouvais pas le terminer, a dit Kit moi la semaine dernière. J\'ai eu ce sentiment tenace, vous savez, de faire quelque chose à ce sujet. La pétition de \nKit -- intitulée Aidez les beaux et mystérieux pangolins en les présentant dans un film d\'animation ! – comptait 1 425 signatures lundi matin. Au moins 13 autres lecteurs ont également lancé des pétitions exhortant Disney à mettre un personnage de pangolin dans un film d\'animation ou un court métrage, selon Pulin Modi, un responsable de la campagne chez Change.org.\nDisney et Disney Pixar ont la capacité de rendre le pangolin populaire. Et ils n\'ont pas à le faire par bonté de cœur. Cela pourrait être un succès. Les pangolins sont les outsiders ultimes\xa0: adorables et secrets, curieux et peu connus.\nLes mères pangolins portent leurs bébés sur leur dos\xa0; les pangolins pendent aux arbres par la queue ; et leurs langues sont plus longues que leurs corps.\nDe plus, Disney a déjà un choix de noms. Les lecteurs de CNN votent sur un nom pour le pangolin P26, qui a été sauvé du commerce des espèces sauvages et a récemment été relâché dans la nature au Vietnam. Certains des noms seraient parfaits pour un c animé caractère : Pemalu, qui signifie timide en indonésien ; Bao, vietnamien pour la protection. Ou Percival, qui est allitératif et amusant.\n Lorsque vous introduisez un personnage dans un film Disney, l\'animal obtient soudainement plus de reconnaissance et les gens commencent à en parler, a déclaré Kit, qui est un fan de Disney.\n Cette éducation et cette prise de conscience sont ce qui compte \\ va aider à façonner l\'esprit des générations futures. Peut-être que nous pouvons commencer avec les plus jeunes et qu\'ils peuvent grandir en ne voulant pas manger le pangolin. \nIl peut sembler farfelu que n\'importe quel spectateur de Disney mange du pangolin. Mais pas pour Kit, qui est sino-américaine et qui m\'a dit que son père avait probablement consommé des produits d\'espèces sauvages en voie de disparition tout en faisant des affaires en Chine, y compris peut-être du pangolin.\nCela a été une source de conflit familial.\nC\'est comme parler à un morceau de fromage, vous n\'irez nulle part, a-t-elle déclaré à propos de ses conversations avec son père sur le bien-être des animaux en voie de disparition. C\'est drôle pour lui que les gens s\'inquiètent de ce qui arrive à un animal - parce que ce n\'est qu\'un animal, ce n\'est rien de plus que ça. Il n\'offre rien de plus que de la nourriture ou des propriétés médicinales. Il rit et dit que cela vous fait vivre plus longtemps ou que cela rend votre peau plus belle. Et il dira, T\'es fou ! Vous les Américains\xa0!\nElle ne pense pas qu\'elle pourra persuader son père de changer, mais les jeunes générations pourraient être celles qui sauveront des animaux comme les pangolins et les rhinocéros, a-t-elle dit.\nEt Disney pourrait les atteindre.\\ nLa propre histoire de Kit est la preuve que cela pouvait fonctionner.\nKit est née aux États-Unis, et elle a développé un penchant pour les animaux quand elle était jeune. Ses parents appréciaient les animaux pour leur fonction - la capacité de Doom le Doberman à protéger leur maison, leurs œufs de canard - mais elle les considérait comme des amis.\n Peut-être que je n\'avais pas assez d\'amis et que les animaux étaient toujours super amical tout le temps, dit-elle. J\'ai l\'impression qu\'il est assez facile de se lier d\'amitié avec des animaux. Elle a maintenant trois chats (Cheezburger, Monster et Ninja) et un lapin nommé Bunny.\nElle est devenue une défenseure des animaux parce qu\'elle a appris à les connaître.\n En tant qu\'Américaine d\'origine chinoise, je veux aussi m\'exprimer parce que le la demande en Chine contribue à stimuler la demande de pangolins à tuer pour la viande et les médicaments, écrit-elle dans la pétition Change.org. Je peux vous dire que de nombreux citoyens chinois ne soutiennent pas cette cruauté, et je pense que nous pouvons réunir des défenseurs de la faune du monde entier pour demander à Disney de prendre cette mesure simple, amusante et positive pour aider les pangolins. \n Cela ne m\'a coûté qu\'une demi-heure de mon temps, a-t-elle dit à propos de la pétition.\nEt cela prendrait beaucoup moins de temps pour la signer.\n,,,L\'une des séries télévisées les plus marquantes de tous les temps à courir à la télévision américaine était marié avec des enfants. L\'émission a été diffusée sur la chaîne FOX pendant 11\xa0ans, et beaucoup pensent qu\'elle était responsable de la survie de la station d\'information pendant les premières années.\n \nEn 1987, la télévision commerciale aux États-Unis était dominée par les trois\xa0grands\xa0: l\'American Broadcasting Company (ABC), la National Broadcasting Company (NBC) et la Columbia Broadcasting Company (CBS). Lorsque la chaîne FOX est apparue sur la scène, peu de gens lui ont donné beaucoup de chance de survivre face à sa concurrence féroce, mais Marié avec des enfants a changé tout cela.\nL\'émission a suivi les exploits d\'une famille dysfonctionnelle qui vivait dans une banlieue fictive de Chicago. Le personnage principal était le père, Al Bundy, qui était un perdant adorable vivant avec sa femme égocentrique et ses enfants bratty. Ensemble, la famille rend la vie d\'Al misérable alors qu\'il trébuche et se fraye un chemin dans la vie. Al travaille comme vendeur de chaussures qui paie très peu et a peu de chance de promotion.\nAl est un personnage attachant qui a gagné l\'adoration du public américain pendant de nombreuses années. L\'acteur Ed O\'Neil, qui jouait Al, a déclaré que les gens l\'approchent encore aujourd\'hui en l\'appelant Al, ou M. Bundy. O \\ \'Neil a également déclaré que jouer le personnage pendant tant d\'années lui avait rendu difficile de trouver du travail une fois la série terminée parce que les gens l\'associaient à Al. -limites aux écrivains. C\'était une émission controversée qui traitait de sujets controversés. Beaucoup de gens ont boycotté l\'émission à cause de son contenu racé, mais FOX a continué. L\'émission a cultivé un culte qui a commencé dans les années 80 et qui continue aujourd\'hui.,,,\ufeffLe problème avec les lunettes Google, dit Takahito Iguchi, c\'est qu\'elles ne sont pas cool. Il a un point. Il existe déjà un site Web dédié aux personnes qui les portent avec un air ridicule ou suffisant ou, plus souvent, les deux. Ce n\'était peut-être pas la décision la plus intelligente de Google de publier les 10 000 premières paires aux développeurs de logiciels plutôt que, disons, aux mannequins ou à Scarlett Johansson. Recherchez Google Images et l\'un des premiers résultats est une photo d\'un grand homme nu les portant sous la douche. Et c\'est ce qu\'Iguchi, un entrepreneur japonais, espère être le talon d\'Achille de Google. Il lance un concurrent un peu plus stylisé. Un peu plus de Blade Runner. Un peu plus e japonais. \nLes lunettes de réalité augmentée d\'Iguchi, qui ne sont pas tant des lunettes qu\'une seule pièce de métal avec une caméra et un micro-projecteur, s\'appellent Telepathy One, et, après les avoir dévoilées au festival South by Southwest à Austin, au Texas, ils ont attiré 5 millions de dollars de capital-risque. Comme Glass, Telepathy One devrait être lancé en 2014. \nIl s\'agit d\'une version allégée et simplifiée de Google Glass. Alors que Glass est, dit-il, "un appareil égoïste" avec une gamme d\'utilisations - vous pouvez surfer sur le net, lire des e-mails, prendre des photos, faire des choses non spécifiées avec des applications encore non spécifiées - Telepathy sera "plus un appareil de communication". Connecté via Bluetooth à votre téléphone, il se concentrera sur le partage visuel et audio en temps réel. Vous pourrez publier des photos et des vidéos de votre champ de vision sur Facebook ou les envoyer par e-mail. Ou regardez et parlez à une image vidéo flottante d\'un ami. \n"Cela vous aidera à vous rapprocher de vos amis et de votre famille. Nous sommes très concentrés sur les possibilités de communication et de partage », explique Iguchi, qui a travaillé dans l\'industrie technologique japonaise pendant 20 ans, développant plus récemment une application téléphonique basée sur la localisation appelée Sekai Camera. \nBien sûr, tout le monde ne veut pas se rapprocher de l\'homme au casque futuriste, je le précise. Iguchi secoue la tête. « Je suis un visionnaire. Je rêve que les gens comprennent les autres. Quand je vais à Londres, je suis un étranger. Parfois, je ressens de la peur. Mais je crois que tout le monde veut être compris et se comprendre. Et, avec cet appareil, vous pouvez obtenir plus d\'informations sur les gens avant même de leur parler. \nComparé aux goûts de Google, bien sûr, Telepathy est un vairon. Non pas que cela semble intimider Takahito Iguchi. Dans son espace de bureau partagé à San Francisco – un entrepôt converti et cool au cœur de la terre des startups, rempli de vingt ans – il cite The Art of War de Sun Tzu et souligne que même de petites armées peuvent parfois battre des forces puissantes. Quand il grandissait, la technologie japonaise dominait le monde : le Sony Walkman était l\'iPhone de son époque. Maintenant, pour concourir, il a dû quitter Tokyo pour la Silicon Valley. \n"Tokyo est très riche en mode et en culture mais c\'est quand même une île. C\'est isolé. Il n\'y a aucun moyen d\'agrandir. Alors que dans la Silicon Valley, tout le monde vient de partout. C\'est là que vous venez de vous connecter à l\'échelle mondiale. Le matériel sera fabriqué au Japon, tandis qu\'il rassemble une équipe d\'ingénieurs logiciels aux États-Unis pour développer ses applications. Le jour où je le rencontre, il est suivi par une équipe de journalistes japonais qui s\'intéresse à la nouvelle vague d\'entrepreneurs japonais contraints de quitter leur patrie. \n"Nous perdons confiance", me dit le correspondant, Takashi Yanagisawa. « Et nous devons trouver un moyen de retrouver notre pouvoir. Iguchi est un peu comme la nouvelle frontière. Nous espérons qu\'il pourrait être une nouvelle solution. Construire le prototype de Telepathy One était facile, dit Iguchi. « Nous avons toutes sortes de technologies à Tokyo. C\'est de le présenter au monde qui est le défi. Les principaux fabricants font la queue pour travailler avec lui, dit-il, parce qu\'ils ont la technologie, ils ont juste du mal à la vendre. « Il doit y avoir une histoire au produit. Comme Apple l\'a fait avec l\'iPod – 1 000 chansons dans votre poche. Et la façon dont ils se sont positionnés face à Microsoft et IBM, c\'était comme l\'histoire de David et Goliath. Et Steve Jobs a été inspiré par Akio Morita, le co-fondateur de Sony, et il m\'a inspiré, alors peut-être que ça va tourner en rond. \nPeut-être. Il a certainement la confiance de Jobs, même si, avec un fort accent japonais, il a parfois du mal à se faire comprendre, un fait qui a peut-être contribué à la conception de Telepathy One. Lorsqu\'il s\'est rendu à Londres pour présenter le casque au prestigieux Founders Forum, il a séjourné dans un Airbnb. « Le propriétaire de la maison n\'était pas mon ami mais j\'ai parlé avec lui pendant trois heures, et maintenant c\'est mon ami. C\'est le temps qu\'il faut pour se comprendre, pour partager nos sentiments, notre parcours et notre carrière. Peut-être que Telepathy rend cela plus court. Si vous obtenez des informations à partir du cloud et des réseaux sociaux, cela se fera plus facilement. Et cet homme est impliqué dans l\'obtention d\'investissements du Royaume-Uni vers l\'Afrique, et il était très enthousiasmé par la télépathie, que ce serait un moyen d\'éduquer les gens sur l\'Afrique, de leur montrer le point de vue des autres. \nC\'est l\'espoir le plus cher d\'Iguchi - que voir le point de vue littéral de quelqu\'un d\'autre vous aidera à voir son point de vue métaphorique. En tant qu\'étudiant, explique-t-il, il a étudié la philosophie le jour et appris seul à coder la nuit. "Et, un jour, j\'ai ouvert la porte de mon appartement et j\'ai soudain compris que tout était codé. C\'était mon illumination. Tout est codé et partageable entre humains. Et tout peut être encodé et décodé. Et, si le code est échangeable entre humains, cela mettra fin à toute guerre les uns contre les autres. \n,,,La génération Y sur le lieu de travail\nContexte\nLa génération Y (ceux nés entre le début des années 1980 et le début des années 1990) mak Nous formons une grande partie de nos effectifs, mais ils semblent manquer de loyauté envers les entreprises et les dirigeants pour lesquels ils travaillent. Les entreprises multinationales remarquent des taux de rotation plus élevés de la génération Y alors que les taux de rétention des employés diminuent. Ce rapport examine les résultats de deux enquêtes à grande échelle sur l\'état d\'esprit de la génération du millénaire et explore comment les organisations peuvent s\'efforcer de répondre à ces besoins, d\'accroître l\'engagement des employés et d\'encourager la rétention. \nRecherche\nDans le cadre d\'une enquête mondiale menée par PricewaterhouseCoopers (PwC), plus de 40\xa0000\xa0répondants de la génération Y (nés entre 1983 et 1993) et non milléniaux ont été recueillis sur les thèmes de la culture d\'entreprise, des styles de communication et de travail, de la structure de rémunération, de l\'évolution de carrière, équilibre travail-vie personnelle, etc.\nDans une enquête mondiale distincte menée par Deloitte, plus de 10\xa0000\xa0millennials ont participé à une étude sur leurs perceptions des menaces et des opportunités dans le monde complexe du travail.\nRésultats clés\nLes milléniaux sont aussi engagés envers leur travailler comme leurs collègues plus expérimentés.\nLa génération\xa0Y apprécie un travail intéressant et un bon équilibre entre vie professionnelle et vie privée. Ils ne croient pas que des exigences de travail excessives valent la peine d\'être sacrifiées dans leur vie personnelle.\nLa génération\xa0Y veut de la flexibilité dans ses horaires de travail et est prête à renoncer aux augmentations de salaire et aux promotions pour un horaire de travail flexible. Ils pensent que le succès doit être mesuré par la productivité et non par le nombre d\'heures passées dans un bureau.\nLes Millennials veulent se sentir soutenus et appréciés par leur entreprise et leurs supérieurs.\nLes Millennials veulent plus d\'opportunités pour développer leurs compétences. Il s\'agit notamment des compétences technologiques, du travail d\'équipe et des compétences interpersonnelles.\nLa génération Y pense que les entreprises et les chefs d\'entreprise doivent contribuer à l\'amélioration de la société et ils sont plus susceptibles d\'être fidèles à une entreprise avec une éthique forte.\nRecommandations\nOrganisations et managers souhaitant fidéliser la génération Y devraient envisager\xa0:\nde surveiller leur charge de travail et leur niveau de satisfaction à l\'égard de leur équilibre entre vie professionnelle et vie privée\nde créer une culture de travail flexible dans laquelle les employés ont plus de contrôle sur leurs heures de travail et leur lieu de travail\nd\'offrir un travail significatif et des opportunités intéressantes\nd\'offrir de l\'aide et du soutien dans la formation professionnelle continue développement\nfaire passer les objectifs de l\'organisation d\'être principalement à but lucratif à des motifs qui répondent aux préoccupations sociales et résolvent des problèmes sociétaux plus larges.,,,\ufeffRobert Mysłajek s\'arrête net. Entre deux empreintes de pattes sur une piste de montagne boueuse, le scientifique trouve ce qu\'il cherche. "Crottes!" s\'enthousiasme-t-il. Les observations de loups sont si rares que l\'observation de leurs excréments marque une bonne journée, même pour un traqueur chevronné. \nMais ça devient plus facile. Il y a maintenant environ 1 500 loups en Pologne. Le nombre a doublé en 15 ans. Les loups sont, avec l\'ours brun, le lynx et le carcajou, les derniers grands prédateurs carnivores d\'Europe. Des défenseurs de l\'environnement de Grande-Bretagne, d\'Allemagne et des Pays-Bas se fraient un chemin ici pour découvrir comment le pays a sauvé cette espèce protégée, calomniée même dans les contes de fées. \nDes morceaux d\'os et de cheveux dépassent des précieuses matières fécales noires. "Il a mangé un cerf rouge", explique le biologiste de l\'Université de Varsovie. "Dans mon laboratoire, je peux tout vous dire sur ce loup - non seulement son régime alimentaire, mais aussi son sexe, ses habitudes sexuelles, son âge, son état de santé et ses liens familiaux." \nDes tests ADN ont établi que les loups polonais sont des voyageurs. « Un loup a atteint les Pays-Bas, où il a malheureusement été renversé par une voiture. Ils ont une gamme énorme. Ils ont besoin d\'espace. Le territoire moyen requis par une meute polonaise est de 250 km² », a déclaré Mysłajek. \n«\xa0Y a-t-il une chance que nous puissions un jour réintroduire des loups en Écosse\xa0?\xa0» demande l\'étudiant Alex Entwisle, 23 ans, lors d\'une excursion dans le sud de la Pologne depuis son collège au Royaume-Uni. Les étudiants en sciences animales ont passé la journée à observer des excréments et des empreintes de pattes dans les Beskides couvertes d\'épicéas des Carpates polonaises. \nLeur sujet de discussion brûlant est de savoir s\'il faut réintroduire des loups dans les îles britanniques pour la première fois depuis le 18e siècle. \nEn tant qu\'invité d\'une organisation caritative britannique, la Wolves and Humans Foundation, Mysłajek a visité les Highlands écossais en 2015 et a répondu aux questions des villageois sur l\'expérience polonaise. « La grande différence entre l\'Ecosse et la Pologne, c\'est que nous mangeons du porc. Nous n\'avons pas beaucoup de moutons ici. \n"La similitude est que nous avons beaucoup d\'animaux\xa0: 300\xa0000\xa0cerfs et plus de 800\xa0000\xa0chevreuils. En Pologne, nous avons également une surpopulation massive de sangliers - environ 200 000 - et ceux-ci ravagent les cultures céréalières des agriculteurs. Ici, les loups font partie de la solution », dit-il. \nLe scientifique, qui est un visage familier de la télévision polonaise, affirme que les loups sont des animaux exceptionnels capables de se déplacer jusqu\'à 30 km lors d\'une seule chasse. « La meute des Beskides est une unité forte, huit ou neuf individus. Cette année, nous avons enregistré cinq oursons, deux yearlings et deux adultes. \n"Nous les suivons à l\'aide de caméras activées par le mouvement dans la forêt et en suivant leur empreintes dans la boue et la neige. Dans chaque groupe familial, un seul couple se reproduit, une fois par an. Tous les membres de la meute s\'occupent des jeunes avec solidarité et dévouement. \nMys\xa0łajek, le fils d\'un berger, est déconcerté par la mauvaise réputation des loups. "Pourquoi parle-t-on d\'un "Loup solitaire tireur\'\xa0? Pourquoi avons-nous dû avoir le Petit Chaperon Rouge\xa0?" \nIl est fasciné par ces chiens distants qui sont restés à l\'état sauvage il y a 33\xa0000\xa0ans lorsque d\'autres ont décidé d\'une existence beaucoup plus confortable en tant que chiens domestiques. \nLes loups ne sont pas des cabots. Mysłajek dit seulement Des arguments scientifiques - la nécessité de régénérer les forêts et de contrôler la population d\'animaux sauvages - peuvent sauver les carnivores sauvages d\'Europe, en particulier le loup impopulaire." Les prédateurs naturels équilibrent l\'écosystème. Ils contrôlent les herbivores, permettant ainsi aux arbres de grandir pour que les oiseaux puissent nicher. .” \nTirer sur le cerf\xa0? "Ce n\'est qu\'une solution partielle", dit-il. "Dans un environnement diversifié, vous avez ce qu\'on appelle le "paysage de la peur", où les herbivores ne passent plus toute la journée à brouter l\'herbe tendre des berges. Ils s\'éloignent, par précaution, pour éviter d\'être piégés par un prédateur, ce qui laisse une chance à la végétation. \nL\'interdiction de la chasse au loup dans les Carpates occidentales est entrée en vigueur en 1995 et dans tout le pays en Pologne en 1998. Il y a maintenant des meutes résidentes dans pratiquement toutes les grandes forêts du pays. Les prédateurs coexistent avec les humains plutôt que d\'être clôturés, comme ils le sont dans Parcs safari en Afrique. \nLe gouvernement polonais verse une indemnité pour le bétail tué par les loups. Mysłajek conseille aux agriculteurs d\'ériger des clôtures électriques. Il a contribué à relancer l\'utilisation de deux moyens de dissuasion que les loups trouvent particulièrement effrayants, pour des raisons que personne ne comprend très bien\xa0: le bruant rouge ( accroché autour des enclos à moutons) et l\'écorce du chien de berger blanc et pelucheux des Tatras. \nLa survie et la mobilité des loups de Pologne ont été favorisées par le développement tardif des infrastructures du pays. En 1989, lorsque les communistes ont quitté le pouvoir, la Pologne n\'avait qu\'une seule autoroute. les projets routiers - nécessitant des études d\'impact sur la faune - ont commencé après l\'adhésion de la Pologne à l\'Union européenne en 2004. Le pays a maintenant l\'une des densités les plus élevées au monde d de passages aériens et passages souterrains pour animaux sauvages. \nLes attitudes ont également changé. « Pendant de nombreuses années, la chasse était culturelle. En 1975, il y avait moins de 100 loups en Pologne. Dès les années 1950, la chasse au loup avait été encouragée par les autorités. Ils ont payé une récompense pour avoir tué un loup d\'une valeur d\'un mois de salaire. C\'était un carnage. » \nMysłajek dit que l\'amélioration des chances de survie des loups polonais a été considérable mais reste fragile. Les meutes sont mobiles au-delà des frontières et la chasse se poursuit toujours dans les pays voisins, la Russie, la Lituanie, la Biélorussie, l\'Ukraine et la Slovaquie. \nIl affirme que le nouveau gouvernement polonais, élu en octobre\xa02015, est hostile aux loups. « Le ministre de l\'Environnement, Jan Szyszko, ne cache pas qu\'il est un chasseur. Il y a 120 000 chasseurs autorisés en Pologne et ils sont influents au parlement. \n«\xa0Les chasseurs affirment que les loups sont nuisibles et qu\'il y en a 4\xa0000 en Pologne, ce qui est un chiffre erroné basé sur un décompte non scientifique. Ce gouvernement est capable de revenir en arrière. \nÊtre un défenseur des loups n\'est pas facile. "Ce n\'est pas comme si vous pouviez prétendre aux politiciens que les loups sont une grande attraction touristique. La plupart des touristes veulent voir les animaux, mais les loups restent à l\'écart des humains. Ils ont un odorat extrêmement sensible. \nLes 12\xa0étudiants britanniques en sciences animales quittent les Carpates polonaises sans apercevoir un loup\xa0; juste des photographies d\'empreintes de pattes et d\'excréments. Entwisle est convaincu que l\'Écosse ne pourra jamais égaler le succès de la Pologne. \n"Ce serait incroyable pour l\'environnement de les faire revenir à cause du problème du trop grand nombre de cerfs. Mais ce ne serait tout simplement pas viable à cause des routes et des moutons. \n"Il y aurait des problèmes avec les agriculteurs. Nous avons eu notre révolution industrielle il y a trop longtemps. Nous l\'avons ruiné pour nous-mêmes. En Grande-Bretagne, nous aimons que les prédateurs soient loin et les regardent à la télévision, a déclaré Entwisle. \n,,,Le troisième lundi de janvier est un jour férié fédéral américain, Martin Luther King Jr. Day. Il tombe près de l\'anniversaire du leader des droits civiques, assassiné en 1968. King, un ministre, est devenu connu pour avoir protesté de manière non violente contre le traitement des Afro-Américains aux États-Unis et contre les lois discriminatoires à l\'égard des Noirs. Plus précisément, King a protesté contre la ségrégation qui séparait les Noirs et les Blancs dans les toilettes publiques, les piscines publiques, les écoles publiques, les bus et les restaurants. King a dirigé de nombreuses marches aux États-Unis, en particulier dans le Sud où la ségrégation était particulièrement pratiquée et appliquée, et à Washington D.C. Sa marche sur Washington D.C. était peut-être sa plus célèbre, où il a prononcé le discours "J\'ai un rêve". n \nLe jour de Martin Luther King Jr., les bureaux fédéraux, étatiques et locaux sont fermés, y compris les écoles publiques et les bureaux de poste. Pendant les vacances, de nombreuses personnes participent à des marches et à des veillées en souvenir du Dr King. D\'autres utilisent la journée comme une journée de service et faites du bénévolat dans leur communauté en nettoyant les ordures dans un parc local ou en servant de la nourriture aux sans-abri. Cependant, la fête était controversée. Certains États n\'ont pas voulu honorer le Dr King et ont essayé de renommer le jour férié ou de combiner le jour avec un autre jour férié.\nSeules deux autres personnes ont un jour férié aux États-Unis, Christophe Colomb et George Washington. Martin Luther King Jr. Day est devenu un jour férié en 1983 après la pression des militants des droits civiques et des marches, similaires à celles que King dirigeait. Les efforts pour créer une fête du roi ont commencé la même année que King a été tué. Le premier État à reconnaître King Day comme un jour férié a été l\'Illinois. Le dernier État à reconnaître la fête était l\'Arizona.\n,,,La première fois que je suis allé en Libye, en 2010, j\'ai été arrêté deux jours seulement après le début de mon voyage. En train de filmer un documentaire pour VICE, j\'ai été arrêté pour avoir tourné là où les autorités pensaient que je ne devrais pas, commençant des séries interminables de questions, des cris emphatiques et une incrédulité à secouer la tête face à mes affirmations d\'innocence - et, bien sûr, les implications requises que j\'étais un espion. Quand j\'ai finalement été libéré, j\'ai juré que je ne retournerais jamais dans la Grande Jamahiriya arabe libyenne populaire et socialiste. Mais cette promesse a été rapidement rompue et je me suis retrouvé dans le pays presque exactement un an plus tard, au milieu d\'une révolution chaotique et violente.\nMon deuxième voyage en Libye a consisté à voyager de la frontière égyptienne à Benghazi puis sur la lignes de front à Misurata, s\'intégrant avec quelques groupes rebelles différents en cours de route. J\'ai été choqué de voir à quel point beaucoup d\'entre eux étaient jeunes. À peine la puberté passée et se battant avec tout ce qu\'ils pouvaient trouver -LRB- un gars avait un fusil à harpon -RRB-, ils ont fait preuve de tant d\'héroïsme et de courage que je pleurais en leur parlant. Un rebelle avec qui j\'ai parlé avait quitté l\'hôpital plus tôt dans la nuit - bien qu\'il ait perdu une jambe - afin de pouvoir retourner sur les lignes de front. Une ONG lui a proposé un vol pour l\'Allemagne et une nouvelle prothèse, mais il s\'est enfui de l\'hôpital pour rejoindre ses camarades.\nMais la grande question qui planait sur tout était\xa0: pourquoi se battent-ils\xa0? \nTous ceux à qui j\'ai posé la question - banquiers, employés de magasin, étudiants, ouvriers du bâtiment, ingénieurs pétroliers et anciens partisans de Kadhafi - m\'ont donné la même réponse\xa0: la liberté. C\'était comme la fin de Braveheart à chaque fois qu\'un rebelle me regardait dans les yeux et le disait. Un jeune de 16 ans m\'a dit, je vais mourir pour que les autres puissent au moins respirer de l\'air libre. Des trucs entêtants pour un ado, surtout quand la plupart des rebelles n\'ont pas l\'âge d\'avoir connu un autre système politique que le Kadafisme. Risquer sa vie pour la liberté est une chose. Mais le risquer pour le concept de liberté est tout autre chose.\nVoir le reste de The Rebels of Libya sur VICE.COM\nQuand nous sommes finalement arrivés à Misrata, elle était encerclée par les troupes de Kadhafi et accessible uniquement par voie maritime. Nous nous sommes lentement dirigés vers le front, nous arrêtant périodiquement pour parler aux rebelles. Un garçon de 15 ans que j\'ai rencontré préparait un camion-missile Grad pour la bataille. Rayonnant, il s\'est demandé si je pouvais demander à Clinton et Obama de nouvelles armes pour qu\'ils puissent battre Kadhafi et qu\'il puisse réaliser son rêve de jouer pour le Miami Heat ou les Dallas Mavericks. Pendant que nous parlions, j\'ai été frappé par tout ce qui avait changé en si peu de temps. C\'était une Libye différente de celle que j\'ai connue l\'année dernière -- un pays complètement nouveau.\n,,,"""

In [14]:
L1=ch2.split(",,,")

In [15]:
len(L1)

142

In [16]:
L3=[]
for i in range(143):
    L3.append(eng_data.loc[i,"label"])

****

In [17]:
new_data=pd.read_csv("../input/csvvvv/Classeur 1 (5).csv")

In [18]:
new_data.drop(["Y","Word Count","Language",' '], axis=1, inplace=True)
new_data.columns = ["text", "CEFR Review","CEFR Level_Samar"]

In [19]:
new_data.shape

(191, 3)

In [20]:
new_data.head(2)

,text,CEFR Review,CEFR Level_Samar
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",A2,C1
1,Quoique ce détail ne touche en aucune manière ...,C1,C2


In [21]:
mike_20_june_samar=new_data[pd.notnull(new_data['CEFR Level_Samar'])]
mike_20_june_samar.columns = ["text", "CEFR Review","label"]
mike_20_june_samar.drop(["CEFR Review"],axis=1, inplace=True)
mike_20_june_samar.to_csv("mike_20_june_samar.csv" , index=False)
mike_20_june_samar.head()

,text,label
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",C1
1,Quoique ce détail ne touche en aucune manière ...,C2
2,"En 1804, M. Myriel était curé de Brignolles. I...",C1
3,"Vers l'époque du couronnement, une petite affa...",C2
4,—Quel est ce bonhomme qui me regarde?,C1


In [22]:
mike_20_june_new=new_data[pd.notnull(new_data['CEFR Review'])]
mike_20_june_new.columns = ["text", "label","CEFR Level_Samar"]
mike_20_june_new.drop(["CEFR Level_Samar"],axis=1, inplace=True)
mike_20_june_new.to_csv("mike_20_june_new.csv" , index=False)
mike_20_june_new.head() 

,text,label
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",A2
1,Quoique ce détail ne touche en aucune manière ...,C1
2,"En 1804, M. Myriel était curé de Brignolles. I...",A2
3,"Vers l'époque du couronnement, une petite affa...",B2
4,—Quel est ce bonhomme qui me regarde?,A2


**!!!! samar annotation seems to be weird because there is just C1 and C2**

In [23]:
mike_20_june_samar["label"].unique(), mike_20_june_new["label"].unique()

(array(['C1', 'C2'], dtype=object),
 array(['A2', 'C1', 'B2', 'A1', 'B1', 'C2'], dtype=object))

In [24]:
mike_20_june_new.label.value_counts()

C1    58
A1    41
B2    33
B1    29
A2    24
C2     6
Name: label, dtype: int64

In [25]:
mike_20_june_samar.shape,mike_20_june_new.shape

((141, 2), (191, 2))

In [26]:
trainn=pd.read_csv("../input/frenchcefr/french dataset.csv")
trainn.shape

(315, 4)

****

In [27]:
model_name = "dbmdz/bert-base-french-europeana-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/227k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

In [28]:
lr = 2e-5
epochs =  6
batch_size = 5
max_seq_len = 75

test_frac = 0.1

In [29]:
import os

def set_seed(seed=106052):
    """Set seed for reproducibility.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [30]:
class CEFRDataset(Dataset):
    """Classification dataset, built on top of pytorch dataset object
    """
    
    def __init__(self, texts, labels):
        
        self.encoder = LabelEncoder()
        print(self.encoder.__dict__)
        self.texts = texts
        self.labels = self.encoder.fit_transform(labels)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoded_text = tokenizer(
            text,
            padding="max_length",
            max_length=max_seq_len,
            truncation=True,
            return_tensors="pt",
        )
        encoded_text["input_ids"] = encoded_text["input_ids"].squeeze()
        encoded_text["attention_mask"] = encoded_text["attention_mask"].squeeze()
        label = torch.tensor(label)

        return {
            "input_ids": encoded_text["input_ids"],
            "attention_mask": encoded_text["attention_mask"],
            "labels": label,
        }

    def get_labels(self):
        return self.labels

In [31]:
def train(train_set, valid_set, epochs=10, warmup_size=0.1, lr=1e-3, batch_size=16):
    model = get_model(model_name)
    optim = AdamW(model.parameters(), lr=lr)
    scheduler = get_scheduler(
        optim, warmup_size, round(len(train_set) / batch_size * epochs)
    )
    training_args = get_training_args(epochs, batch_size)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=valid_set,
        optimizers=[optim, scheduler],
        compute_metrics=compute_accuracy,
    )
    trainer.train()
    trainer.save_model()
    return trainer

In [32]:
def get_model(pretrained_checkpoint):
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_checkpoint, num_labels=6
    )
    return model.to(device)

In [33]:
os.environ["WANDB_DISABLED"] = "true"


def get_scheduler(optimizer, warmup_size, total_steps):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=round(total_steps * warmup_size),
        num_training_steps=total_steps,
    )
    return scheduler


def get_training_args(epochs, batch_size):
    return TrainingArguments(
        output_dir="./b",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        logging_steps=50,
        fp16=False,
        evaluation_strategy="epoch",
        eval_accumulation_steps=1,
        report_to=None,
        save_total_limit=1,
        load_best_model_at_end=True
    )


def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}



In [34]:
lr = 2e-5
epochs =  7
batch_size = 8
max_seq_len = 512

In [35]:
def split_valid(df, frac=0.01):
    
    val = pd.DataFrame()
    val["text"] = ""
    val["label"] = -1
    
    for i in df.label.unique():
        val = pd.concat([val, df[df.label == i].sample(frac=frac)])
        
    return df[~df.index.isin(val.index)].reset_index(drop=True) , val.reset_index(drop=True)

In [36]:
# train_set_df = pd.read_csv("../input/newdatasets/mike_20_june_new (1).csv")
# train_set_df

In [37]:
mike_20_june_new.head(2)

,text,label
0,"En 1815, M. Charles-François-Bienvenu Myriel é...",A2
1,Quoique ce détail ne touche en aucune manière ...,C1


In [38]:
mike_20_june_new.shape

(191, 2)

In [39]:
new=pd.read_csv("../input/lingua/fr_lingua.csv")

In [40]:
new.columns=["text","label"]

In [41]:
french=pd.read_csv("../input/frenchcefr/french dataset.csv")
french=french[["text","label"]]
french.shape,french.head(2)


((315, 2),
                                                 text label
 0  Il était une fois un prince qui voulait épouse...    A2
 1  Il fit le tour de la Terre pour en trouver une...    B1)

In [42]:
# train_set_df=pd.concat([french, mike_20_june_new,new], ignore_index=True)

In [43]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_set_df = pd.read_csv("../input/newdatasets/mike_20_june_new (1).csv")
train_set_df = train_set_df[train_set_df.label != "XX"]
train_set_df = train_set_df[["text", "label"]]

train_set_df.text = train_set_df.text.apply(lambda x: x.replace("\r", "").replace("\n", " "))

extra_df = pd.read_csv("../input/frenchcefr/french_mike_june.csv")
extra_df.columns = ["text", "label", "label_"]
extra_df = extra_df[["text", "label"]]
extra_df.text = extra_df.text.astype(str)
#train_set_df = pd.concat([train_set_df, extra_df]).reset_index(drop=True)

train_set_df, valid_set_df = split_valid(train_set_df)

In [44]:
train_set_df.label.unique()

array(['A2', 'C1', 'B2', 'A1', 'B1', 'C2'], dtype=object)

In [45]:
train_set_df.label = le.fit_transform(train_set_df.label)
valid_set_df.label = le.transform(valid_set_df.label)

In [46]:
train_set_df.label.nunique()

6

In [47]:
train_set_df.label.value_counts()

4    57
0    41
3    33
2    29
1    24
5     6
Name: label, dtype: int64

In [48]:
valid_set_df.label.nunique()

1

In [49]:
# train_set_df = train_set_df.sample(frac=1)

In [50]:
from tqdm import tqdm 

def predict(model, text):
    
    preds = []
    
    for i in tqdm(range(len(text))):
        tokenized = tokenizer(text[i:i+1], return_tensors="pt", truncation=True, max_length=512).to("cuda")
        pred = model(**tokenized)
        preds.append(pred.logits.argmax(-1).item())

    return preds

In [51]:
train_set = CEFRDataset(train_set_df["text"], train_set_df["label"])
valid_set = CEFRDataset(valid_set_df["text"], valid_set_df["label"])


trainer_second = train(train_set, valid_set, epochs=epochs, warmup_size=0.2, lr=lr, batch_size=batch_size)
model = trainer_second.model

{}
{}


Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-french-europeana-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificat

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,No log,2.087388,0.000000,0.049400,20.234000
2,No log,3.319314,0.000000,0.042800,23.361000
3,1.512000,4.162873,0.000000,0.042000,23.830000
4,1.512000,4.354156,0.000000,0.047500,21.042000
5,0.867000,4.624961,0.000000,0.046400,21.562000
6,0.867000,4.566848,0.000000,0.048200,20.727000
7,0.585500,4.568268,0.000000,0.062900,15.905000


In [52]:
valid_set_df["preds"] = train_set.encoder.inverse_transform(predict(model, valid_set_df.text.tolist()))
valid_set_df.columns = ["text", "cefr", "preds",] 

100%|██████████| 1/1 [00:00<00:00, 31.89it/s]


In [53]:
lingua = pd.read_csv("../input/lingua/fr_lingua.csv")
lingua["preds"] = train_set.encoder.inverse_transform(predict(model, lingua.text.tolist()))

100%|██████████| 111/111 [00:01<00:00, 57.38it/s]


In [54]:
# lingua.columns=["text","len","lang","cefr","preds"]
# lingua = lingua[lingua.cefr != "XX"]

In [55]:
lingua.cefr.value_counts()

B1    60
A2    26
A1    13
B2    12
Name: cefr, dtype: int64

In [56]:
lingua.preds.unique()

array([4])

In [57]:
def compute_average_distance(df, col_name="cefr") :
    
    labels = ["A1", "A2", "B1", "B2", "C1", "C2"]
    return (df[col_name].apply(lambda x: labels.index(x)) - df.preds.apply(lambda x: labels.index(x))).abs().mean()


In [58]:
lingua.preds.value_counts()

4    111
Name: preds, dtype: int64

In [59]:
lingua["preds"] = le.inverse_transform(lingua.preds)

In [60]:
lingua.preds.value_counts()

C1    111
Name: preds, dtype: int64

In [61]:
print("Distance: ")
print(compute_average_distance(lingua, "cefr"))

print(f"Lingua accuracy: {(lingua['preds'] == lingua['cefr']).mean()}")

Distance: 
2.3603603603603602
Lingua accuracy: 0.0


In [62]:
for i in range(lingua.shape[0]):
    if (lingua.loc[i,"preds"]=="C1") | (lingua.loc[i,"preds"]=="C2") :
        lingua.loc[i,"Predss"]="B2"
    else :
        lingua.loc[i,"Predss"]= lingua.loc[i,"preds"]
lingua["Predss"].value_counts()

B2    111
Name: Predss, dtype: int64

In [63]:
print("Distance: ")
print(compute_average_distance(lingua, "cefr"))

print(f"Lingua accuracy: {(lingua['Predss'] == lingua['cefr']).mean()}")

Distance: 
2.3603603603603602
Lingua accuracy: 0.10810810810810811


****

Distance: 
0.990990990990991
Lingua accuracy: 0.40540540540540543 <br>
2    82
3    15
5    10
4     2
0     1
1     1 <br> 
inokufu/flaubert-base-uncased-xnli-sts-finetuned-education

****


Distance: 
0.8648648648648649
Lingua accuracy: 0.450450450450450462 <br>
3     7
1     5
5     4
4     4
0     1 <br> inokufu/flaubert-base-uncased-xnli-sts-finetuned-education